A cute little demo showing the simplest usage of minGPT. Configured to run fine on Macbook Air in like a minute.

In [ ]:
testCD_CDKTB3MSpreadInverseFloater_CK1F_CK1F_20220620_tlDelta.m

clc
clear
valueDate = '2022-06-20';

maxIdx = 0;
zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;

callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% CD-USDLibor3M Spread Range Accrual Pricing start

%RA info
rangeAccrualParams.payCcy = 'KRW';
rangeAccrualParams.startDate = H_Date('2019-08-23');

rangeAccrualParams.maturity = 15;

valueDateH = H_Date(valueDate);

nominal = 10000;
% coupon = 0.033;
coupon = 0.052;

% couponFreq = 1;
couponFreq = 0.25;


% Long: KRW
% Short: USD
tenorL = 0.25;
tenorS = 0.25;

% lowerS = 0.0;

lowerS = -0.001;
upperS = 999;

% 
% lower = -1;
% upper = 0.05;

tenor = 0.25;

lower = -999;
upper = 999;

%resetDate startDate endDate paymentDate
couponScheduleInt = [20190823	20190823	20191123	20191123
20191123	20191123	20200223	20200223
20200223	20200223	20200523	20200523
20200523	20200523	20200823	20200823
20200823	20200823	20201123	20201123
20201123	20201123	20210223	20210223
20210223	20210223	20210523	20210523
20210523	20210523	20210823	20210823
20210823	20210823	20211123	20211123
20211123	20211123	20220223	20220223
20220223	20220223	20220523	20220523
20220523	20220523	20220823	20220823
20220823	20220823	20221123	20221123
20221123	20221123	20230223	20230223
20230223	20230223	20230523	20230523
20230523	20230523	20230823	20230823
20230823	20230823	20231123	20231123
20231123	20231123	20240223	20240223
20240223	20240223	20240523	20240523
20240523	20240523	20240823	20240823
20240823	20240823	20241123	20241123
20241123	20241123	20250223	20250223
20250223	20250223	20250523	20250523
20250523	20250523	20250823	20250823
20250823	20250823	20251123	20251123
20251123	20251123	20260223	20260223
20260223	20260223	20260523	20260523
20260523	20260523	20260823	20260823
20260823	20260823	20261123	20261123
20261123	20261123	20270223	20270223
20270223	20270223	20270523	20270523
20270523	20270523	20270823	20270823
20270823	20270823	20271123	20271123
20271123	20271123	20280223	20280223
20280223	20280223	20280523	20280523
20280523	20280523	20280823	20280823
20280823	20280823	20281123	20281123
20281123	20281123	20290223	20290223
20290223	20290223	20290523	20290523
20290523	20290523	20290823	20290823
20290823	20290823	20291123	20291123
20291123	20291123	20300223	20300223
20300223	20300223	20300523	20300523
20300523	20300523	20300823	20300823
20300823	20300823	20301123	20301123
20301123	20301123	20310223	20310223
20310223	20310223	20310523	20310523
20310523	20310523	20310823	20310823
20310823	20310823	20311123	20311123
20311123	20311123	20320223	20320223
20320223	20320223	20320523	20320523
20320523	20320523	20320823	20320823
20320823	20320823	20321123	20321123
20321123	20321123	20330223	20330223
20330223	20330223	20330523	20330523
20330523	20330523	20330823	20330823
20330823	20330823	20331123	20331123
20331123	20331123	20340223	20340223
20340223	20340223	20340523	20340523
20340523	20340523	20340823	20340823
];

endDate = H_Date(couponScheduleInt(size(couponScheduleInt,1),4));

%% CK1F model param matu setting start
% matu = 15 + 1.0/365.0;
matu = DateDiff(H_Date(couponScheduleInt(end,4)),H_Date(valueDate))/365.0;
% matu = matu -10.0;

%% end

 rangeDays = zeros(60,1);
%     rangeDays(1) = 1;

for i=1:length(rangeDays)
    if DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) >= 0
        rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) + 1;
    end
end


%resetDate startDate endDate paymentDate
callScheduleInt = [20190823	20190823	20190830	20190830
20191116	20191116	20191123	20191123
20200216	20200216	20200223	20200223
20200516	20200516	20200523	20200523
20200816	20200816	20200823	20200823
20201116	20201116	20201123	20201123
20210216	20210216	20210223	20210223
20210516	20210516	20210523	20210523
20210816	20210816	20210823	20210823
20211116	20211116	20211123	20211123
20220216	20220216	20220223	20220223
20220516	20220516	20220523	20220523
20220816	20220816	20220823	20220823
20221116	20221116	20221123	20221123
20230216	20230216	20230223	20230223
20230516	20230516	20230523	20230523
20230816	20230816	20230823	20230823
20231116	20231116	20231123	20231123
20240216	20240216	20240223	20240223
20240516	20240516	20240523	20240523
20240816	20240816	20240823	20240823
20241116	20241116	20241123	20241123
20250216	20250216	20250223	20250223
20250516	20250516	20250523	20250523
20250816	20250816	20250823	20250823
20251116	20251116	20251123	20251123
20260216	20260216	20260223	20260223
20260516	20260516	20260523	20260523
20260816	20260816	20260823	20260823
20261116	20261116	20261123	20261123
20270216	20270216	20270223	20270223
20270516	20270516	20270523	20270523
20270816	20270816	20270823	20270823
20271116	20271116	20271123	20271123
20280216	20280216	20280223	20280223
20280516	20280516	20280523	20280523
20280816	20280816	20280823	20280823
20281116	20281116	20281123	20281123
20290216	20290216	20290223	20290223
20290516	20290516	20290523	20290523
20290816	20290816	20290823	20290823
20291116	20291116	20291123	20291123
20300216	20300216	20300223	20300223
20300516	20300516	20300523	20300523
20300816	20300816	20300823	20300823
20301116	20301116	20301123	20301123
20310216	20310216	20310223	20310223
20310516	20310516	20310523	20310523
20310816	20310816	20310823	20310823
20311116	20311116	20311123	20311123
20320216	20320216	20320223	20320223
20320516	20320516	20320523	20320523
20320816	20320816	20320823	20320823
20321116	20321116	20321123	20321123
20330216	20330216	20330223	20330223
20330516	20330516	20330523	20330523
20330816	20330816	20330823	20330823
20331116	20331116	20331123	20331123
20340216	20340216	20340223	20340223
20340516	20340516	20340523	20340523
];

callYN = ones(60,1);
callYN(1) = 0;

manualScheduleYN = 1;

rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                'tenorS','lower','upper','rangeDays','callYN', ...
                                'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                        {nominal,coupon,couponFreq,tenorL,tenorS,lower,upper,...
                        rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});
% 
% tenorL = 0.25;
% tenorS = 0.25;
% 
% tenor = 0.25;
% 
% lowerS = 0.0;
% upperS = 999;
% 
% 
% lower = 0.0;
% upper = 0.05;

rangeAccrualInfo('lowerS') =lowerS;
rangeAccrualInfo('upperS') =upperS;
rangeAccrualInfo('tenor') = 0.25;


rangeAccrualInfo('couponLowerBound') = -0.05;


rangeAccrualParams.params = rangeAccrualInfo;

% rangeAccrual = SpreadRangeAccrual(rangeAccrualParams);
% rangeAccrual = CDCMSSpreadDualRangeAccrual(rangeAccrualParams);
rangeAccrual = CDCMSSpreadDualRangeAccrualInverseFloater(rangeAccrualParams);

% maxIdx =2.5;

maxIdx = 10;

rmseSimul = zeros(100,1);
for idx2=1:21
    %% KRW Model building

    %% import and construct a zeroCurve
    rawData = [0.002739726	0.999952523
0.25	0.9950684
0.5	0.988031579
0.75	0.979080331
1	0.968776974
1.25	0.959202682
1.5	0.948993277
1.75	0.939322512
2	0.929369492
2.25	0.920482183
2.5	0.911621558
2.75	0.902762032
3	0.893905238
3.25	0.885611153
3.5	0.877424651
3.75	0.869320968
4	0.861299205
4.25	0.853502656
4.5	0.845805585
4.75	0.838202041
5	0.830690747
5.25	0.823531148
5.5	0.816487433
5.75	0.809552209
6	0.802723731
6.25	0.796000289
6.5	0.789380204
6.75	0.782861832
7	0.776443559
7.25	0.770025284
7.5	0.763695404
7.75	0.757455789
8	0.751305075
8.25	0.74524192
8.5	0.73926501
8.75	0.73337305
9	0.727564772
9.25	0.721838929
9.5	0.716194297
9.75	0.710629672
10	0.705143874
10.25	0.699194275
10.5	0.693278485
10.75	0.687420297
11	0.681619095
11.25	0.675874273
11.5	0.670185229
11.75	0.664551369
12	0.658972106
12.25	0.654130635
12.5	0.649391356
12.75	0.644722276
13	0.640122375
13.25	0.635590647
13.5	0.631126108
13.75	0.626727791
14	0.622394747
14.25	0.618126045
14.5	0.613920769
14.75	0.609778022
15	0.605696922
15.25	0.60235689
15.5	0.599139226
15.75	0.595995535
16	0.592924834
16.25	0.589926161
16.5	0.586998583
16.75	0.58414119
17	0.581353097
17.25	0.578633444
17.5	0.575981393
17.75	0.573396131
18	0.570876865
18.25	0.568422827
18.5	0.566033269
18.75	0.563707467
19	0.561444716
19.25	0.559244332
19.5	0.557105652
19.75	0.555028035
20	0.553010856
20.25	0.550157221
20.5	0.547261794
20.75	0.544409494
21	0.541599805
21.25	0.538832219
21.5	0.536106239
21.75	0.533421375
22	0.530777148
22.25	0.528173085
22.5	0.525608723
22.75	0.523083608
23	0.520597293
23.25	0.518149339
23.5	0.515739316
23.75	0.513366801
24	0.511031378
24.25	0.508732641
24.5	0.506470189
24.75	0.50424363
25	0.502052578
25.25	0.498564056
25.5	0.494988089
25.75	0.491434812
26	0.487904104
26.25	0.484395847
26.5	0.48090992
26.75	0.477446204
27	0.474004582
27.25	0.470584935
27.5	0.467187146
27.75	0.463811098
28	0.460456674
28.25	0.457123757
28.5	0.453812233
28.75	0.450521986
29	0.447252902
29.25	0.444004865
29.5	0.440777762
29.75	0.43757148
30	0.434385905
30.25	0.431221981
30.5	0.428081088
30.75	0.424963073
31	0.421867769
    ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

    
    %simulation testCase
    for idx=1:size(rawData,1)
        rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    KRWAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];

% rawData = [0.002739726	0.999952523
% 0.25	0.9950684
% 0.5	0.988031579
% 0.75	0.979080331
% 1	0.968776974
% 1.25	0.959202682
% 1.5	0.948993277
% 1.75	0.939322512
% 2	0.929369492
% 2.25	0.920482183
% 2.5	0.911621558
% 2.75	0.902762032
% 3	0.893905238
% 3.25	0.885611153
% 3.5	0.877424651
% 3.75	0.869320968
% 4	0.861299205
% 4.25	0.853502656
% 4.5	0.845805585
% 4.75	0.838202041
% 5	0.830690747
% 5.25	0.823531148
% 5.5	0.816487433
% 5.75	0.809552209
% 6	0.802723731
% 6.25	0.796000289
% 6.5	0.789380204
% 6.75	0.782861832
% 7	0.776443559
% 7.25	0.770025284
% 7.5	0.763695404
% 7.75	0.757455789
% 8	0.751305075
% 8.25	0.74524192
% 8.5	0.73926501
% 8.75	0.73337305
% 9	0.727564772
% 9.25	0.721838929
% 9.5	0.716194297
% 9.75	0.710629672
% 10	0.705143874
% 10.25	0.699194275
% 10.5	0.693278485
% 10.75	0.687420297
% 11	0.681619095
% 11.25	0.675874273
% 11.5	0.670185229
% 11.75	0.664551369
% 12	0.658972106
% 12.25	0.654130635
% 12.5	0.649391356
% 12.75	0.644722276
% 13	0.640122375
% 13.25	0.635590647
% 13.5	0.631126108
% 13.75	0.626727791
% 14	0.622394747
% 14.25	0.618126045
% 14.5	0.613920769
% 14.75	0.609778022
% 15	0.605696922
% 15.25	0.60235689
% 15.5	0.599139226
% 15.75	0.595995535
% 16	0.592924834
% 16.25	0.589926161
% 16.5	0.586998583
% 16.75	0.58414119
% 17	0.581353097
% 17.25	0.578633444
% 17.5	0.575981393
% 17.75	0.573396131
% 18	0.570876865
% 18.25	0.568422827
% 18.5	0.566033269
% 18.75	0.563707467
% 19	0.561444716
% 19.25	0.559244332
% 19.5	0.557105652
% 19.75	0.555028035
% 20	0.553010856
% 20.25	0.550157221
% 20.5	0.547261794
% 20.75	0.544409494
% 21	0.541599805
% 21.25	0.538832219
% 21.5	0.536106239
% 21.75	0.533421375
% 22	0.530777148
% 22.25	0.528173085
% 22.5	0.525608723
% 22.75	0.523083608
% 23	0.520597293
% 23.25	0.518149339
% 23.5	0.515739316
% 23.75	0.513366801
% 24	0.511031378
% 24.25	0.508732641
% 24.5	0.506470189
% 24.75	0.50424363
% 25	0.502052578
% 25.25	0.498564056
% 25.5	0.494988089
% 25.75	0.491434812
% 26	0.487904104
% 26.25	0.484395847
% 26.5	0.48090992
% 26.75	0.477446204
% 27	0.474004582
% 27.25	0.470584935
% 27.5	0.467187146
% 27.75	0.463811098
% 28	0.460456674
% 28.25	0.457123757
% 28.5	0.453812233
% 28.75	0.450521986
% 29	0.447252902
% 29.25	0.444004865
% 29.5	0.440777762
% 29.75	0.43757148
% 30	0.434385905
% 30.25	0.431221981
% 30.5	0.428081088
% 30.75	0.424963073
% 31	0.421867769
%     ];


for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

 %simulation testCase
    for idx=1:size(rawData,1)
        rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
    end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);

    KRWAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurfaceNormal



rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.01025373	0.00996465	0.00939453	0.0088449	0.00838736	0.00835816	0.00827411	0.00844853	0.00844853
2	0.00987033	0.00941532	0.00896799	0.00850851	0.00803853	0.00809306	0.00814872	0.00832187	0.00832187
3	0.00965448	0.00917228	0.00868278	0.00815841	0.00766413	0.00768619	0.00766987	0.007827	0.007827
4	0.0086456	0.00835905	0.00803055	0.00773857	0.00743965	0.00747184	0.00750208	0.00760185	0.00760185
5	0.008024	0.00784518	0.00761802	0.00741163	0.00719132	0.00718529	0.00716174	0.0072174	0.0072174
7	0.00777954	0.0075868	0.00743466	0.00726955	0.00710966	0.00717731	0.00726665	0.00728989	0.00728989
10	0.00697384	0.00701303	0.00705259	0.00704171	0.0070555	0.00714878	0.00723997	0.00722954	0.00722954
15	0.00731508	0.00731858	0.00728587	0.00725674	0.0072209	0.00720153	0.00717821	0.0071639	0.0071639
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    KRWAllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a KRWBlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(KRWAllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = KRWBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= KRWBlack.NormalATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
   %% create a CK1FRisky Model with initial modelParams
%     vol_r.tenor = [1;2;3;4;5;7;10;15;20];
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;
%     meanR_r = 0.0;
    
    
    joint_swap_tenor = 1;
%     targetSize = 10;

    targetSize = 15;
    
    freq = 4;
    
    modelParams = containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'},...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
    
%     import irModels.*;
%     import targetFunctions.*;

 %% allFixings
    %% CD91
    tempDate = rangeAccrualParams.startDate;
    irIndexFixing3 = containers.Map({StrDate(tempDate,'str')},{0.025});

    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        irIndexFixing3(StrDate(tempDate,'str')) = 0.025;
    end

    %% fixing info -> let it be the modelParams !
    allFixings = containers.Map({'LIBOR3M'},{irIndexFixing3});
    modelParams('allFixings') = allFixings;
    
    KRWCK1FRisky = CK1FRisky(IRModelRisky(Model(KRWAllMktData, modelParams)));
    
    tic
%     out = KRWLGM2F.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
    % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
%     out = KRWCK1FRisky.CalibrateToATMDiagSwaption(KRWBlack,'global');
    out = KRWCK1FRisky.CalibrateToATMDiagSwaptionNormal(KRWBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        KRWVol_r(idx2,idx3) = out.vol_r.quote(idx3);
        KRWRe(idx2,idx3) = out.re(idx3);
    end
    KRWRMSE(idx2) = out.rmseTotal;
    
    toc
    %% CK1FRisky model building end
    %% KRW model building end

    %% USD model building start
    %% import and construct a zeroCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

 %simulation testCase
    for idx=1:size(rawData,1)
        rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
    end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','USDZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    USDAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];


% rawData = [0.002739726	0.999952523
% 0.25	0.9950684
% 0.5	0.988031579
% 0.75	0.979080331
% 1	0.968776974
% 1.25	0.959202682
% 1.5	0.948993277
% 1.75	0.939322512
% 2	0.929369492
% 2.25	0.920482183
% 2.5	0.911621558
% 2.75	0.902762032
% 3	0.893905238
% 3.25	0.885611153
% 3.5	0.877424651
% 3.75	0.869320968
% 4	0.861299205
% 4.25	0.853502656
% 4.5	0.845805585
% 4.75	0.838202041
% 5	0.830690747
% 5.25	0.823531148
% 5.5	0.816487433
% 5.75	0.809552209
% 6	0.802723731
% 6.25	0.796000289
% 6.5	0.789380204
% 6.75	0.782861832
% 7	0.776443559
% 7.25	0.770025284
% 7.5	0.763695404
% 7.75	0.757455789
% 8	0.751305075
% 8.25	0.74524192
% 8.5	0.73926501
% 8.75	0.73337305
% 9	0.727564772
% 9.25	0.721838929
% 9.5	0.716194297
% 9.75	0.710629672
% 10	0.705143874
% 10.25	0.699194275
% 10.5	0.693278485
% 10.75	0.687420297
% 11	0.681619095
% 11.25	0.675874273
% 11.5	0.670185229
% 11.75	0.664551369
% 12	0.658972106
% 12.25	0.654130635
% 12.5	0.649391356
% 12.75	0.644722276
% 13	0.640122375
% 13.25	0.635590647
% 13.5	0.631126108
% 13.75	0.626727791
% 14	0.622394747
% 14.25	0.618126045
% 14.5	0.613920769
% 14.75	0.609778022
% 15	0.605696922
% 15.25	0.60235689
% 15.5	0.599139226
% 15.75	0.595995535
% 16	0.592924834
% 16.25	0.589926161
% 16.5	0.586998583
% 16.75	0.58414119
% 17	0.581353097
% 17.25	0.578633444
% 17.5	0.575981393
% 17.75	0.573396131
% 18	0.570876865
% 18.25	0.568422827
% 18.5	0.566033269
% 18.75	0.563707467
% 19	0.561444716
% 19.25	0.559244332
% 19.5	0.557105652
% 19.75	0.555028035
% 20	0.553010856
% 20.25	0.550157221
% 20.5	0.547261794
% 20.75	0.544409494
% 21	0.541599805
% 21.25	0.538832219
% 21.5	0.536106239
% 21.75	0.533421375
% 22	0.530777148
% 22.25	0.528173085
% 22.5	0.525608723
% 22.75	0.523083608
% 23	0.520597293
% 23.25	0.518149339
% 23.5	0.515739316
% 23.75	0.513366801
% 24	0.511031378
% 24.25	0.508732641
% 24.5	0.506470189
% 24.75	0.50424363
% 25	0.502052578
% 25.25	0.498564056
% 25.5	0.494988089
% 25.75	0.491434812
% 26	0.487904104
% 26.25	0.484395847
% 26.5	0.48090992
% 26.75	0.477446204
% 27	0.474004582
% 27.25	0.470584935
% 27.5	0.467187146
% 27.75	0.463811098
% 28	0.460456674
% 28.25	0.457123757
% 28.5	0.453812233
% 28.75	0.450521986
% 29	0.447252902
% 29.25	0.444004865
% 29.5	0.440777762
% 29.75	0.43757148
% 30	0.434385905
% 30.25	0.431221981
% 30.5	0.428081088
% 30.75	0.424963073
% 31	0.421867769
%     ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

 %simulation testCase
    for idx=1:size(rawData,1)
        rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
    end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);
    USDAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
% GIROPremiumToNormalVol From LIB
% rawDataSwaption = [0	1	2	3	4	5	7	10	12	15	20
% 1	0.0041	0.0045	0.0052	0.0061	0.0066	0.0071	0.0077	0.0080	0.0081	0.0082
% 2	0.0050	0.0057	0.0064	0.0066	0.0068	0.0071	0.0074	0.0077	0.0077	0.0078
% 3	0.0064	0.0069	0.0068	0.0068	0.0068	0.0072	0.0076	0.0078	0.0079	0.0079
% 4	0.0076	0.0071	0.0071	0.0070	0.0071	0.0072	0.0077	0.0081	0.0081	0.0081
% 5	0.0069	0.0070	0.0070	0.0071	0.0071	0.0070	0.0076	0.0083	0.0084	0.0083
% 7	0.0073	0.0074	0.0075	0.0072	0.0070	0.0075	0.0087	0.0096	0.0096	0.0095
% 10	0.0066	0.0066	0.0071	0.0074	0.0077	0.0092	0.0101	0.0102	0.0102	0.0102
% 15	0.0103	0.0105	0.0107	0.0109	0.0111	0.0108	0.0103	0.0102	0.0102	0.0101
% 20	0.0088	0.0089	0.0090	0.0091	0.0091	0.0091	0.0090	0.0089	0.0089	0.0089
%     ];
% 
rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.01025373	0.00996465	0.00939453	0.0088449	0.00838736	0.00835816	0.00827411	0.00844853	0.00844853
2	0.00987033	0.00941532	0.00896799	0.00850851	0.00803853	0.00809306	0.00814872	0.00832187	0.00832187
3	0.00965448	0.00917228	0.00868278	0.00815841	0.00766413	0.00768619	0.00766987	0.007827	0.007827
4	0.0086456	0.00835905	0.00803055	0.00773857	0.00743965	0.00747184	0.00750208	0.00760185	0.00760185
5	0.008024	0.00784518	0.00761802	0.00741163	0.00719132	0.00718529	0.00716174	0.0072174	0.0072174
7	0.00777954	0.0075868	0.00743466	0.00726955	0.00710966	0.00717731	0.00726665	0.00728989	0.00728989
10	0.00697384	0.00701303	0.00705259	0.00704171	0.0070555	0.00714878	0.00723997	0.00722954	0.00722954
15	0.00731508	0.00731858	0.00728587	0.00725674	0.0072209	0.00720153	0.00717821	0.0071639	0.0071639
    ];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'},...
                            {rawDataSwaption,valueDate,'USD','USDSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    USDAllMktData('swaptionVol') = swaptionVolMktData;
    
   %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    USDBlack = IRBlack(IRModel(Model(USDAllMktData,modelParams))); 
    
    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);
    
    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = USDBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= USDBlack.BlackATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = USDBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
    %% create a CK1FModel with initial modelParams
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;

%     matu = 15 + 1.0/365.0;
    joint_swap_tenor = 1;
    targetSize = 15;
    freq = 4;
    
    modelParamsUSD= containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'}, ...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
                        
     %% allFixings
    %% CD91
    tempDate = rangeAccrualParams.startDate;
    irIndexFixing3USD = containers.Map({StrDate(tempDate,'str')},{0.025});

    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        irIndexFixing3USD(StrDate(tempDate,'str')) = 0.025;
    end

    %% fixing info -> let it be the modelParams !
    allFixingsUSD = containers.Map({'LIBOR3M'},{irIndexFixing3USD});
    modelParamsUSD('allFixings') = allFixingsUSD;
    
    USDCK1F = CK1F(IRModel(Model(USDAllMktData, modelParamsUSD)));
    
    tic
    outUSD = USDCK1F.CalibrateToATMDiagSwaptionNormal(USDBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        USDVol_r(idx2,idx3) = outUSD.vol_r.quote(idx3);
        USDRe(idx2,idx3) = outUSD.re(idx3);
    end
    USDRMSE(idx2) = outUSD.rmseTotal;
    
    toc
    %% USD model building end

    %% USDKRW fx model buidling start
    rawData = [0.083333333	0.0
0.25	0.0
0.5	0.0
1	0.0
2	0.0
3	0.0
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','USDKRWATMTermVol'});
    impliedTermVolMktData = MktData(mktData);
%     impliedTermVole = FXVolSurface(impliedTermVolMktData);
%     impliedVolSurface.params('moneyNessType') = 'deltaFwdMoneyNess';
    USDKRWAllMktData =containers.Map('impliedTermVol',impliedTermVolMktData);
%     AllMktData('impliedVolSurface') = impliedTermVolMktData;

    %% create  an FXBlackModel to calculate market PlainVanilla Options
%     modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});
%     USDKRWBlack = FXBlack(FXModel(Model(USDKRWAllMktData,modelParams))); 
    %% USDKRW fx model building end

    %% MultiAssetModel building start
    multiAssetMktData = containers.Map({'KRW','USD','USDKRW'}, ...
                            {KRWAllMktData, USDAllMktData, USDKRWAllMktData});

    % KRW,KTB                    
%     correlationMatrix = ones(2,2);
%     correlationMatrix(1,2) = 0.3;
%     correlationMatrix(2,1) = correlationMatrix(1,2);

    % KRW, USD & USDKRW
    correlationMatrix = ones(3, 3);
    correlationMatrix(1,2) = 0.845;
%     correlationMatrix(1,2) = 0.6;
    
%     correlationMatrix(1,2) = 0.7;
    
    correlationMatrix(1,3) = 0.0;
    correlationMatrix(2,3) = 0.0;
    correlationMatrix(2,1) = correlationMatrix(1,2);
    correlationMatrix(3,1) = correlationMatrix(1,3);
    correlationMatrix(3,2) = correlationMatrix(2,3);
    
    multiAssetMktData('correlationMatrix') = correlationMatrix;
    
    modelNameMap = containers.Map({'KRW','USD'},{KRWCK1FRisky, USDCK1F});
    modelParams = containers.Map({'modelNameMap'}, {modelNameMap});
    % correlationNames = {'KRW','USD'};
    modelParams('stochasticModelNames') = {'KRW','USD'};
    modelParams('correlationMatrix') = correlationMatrix;
    modelParams('refModelName') = 'KRW';
    % modelParams('useQuasiRandomYN') = 'true';
    modelParams('useQuasiRandomYN') = 'false';
    modelParams('fxVolCalibrated') = true;
    
    KRWCK1FRiskyUSDCK1F = MultiAssetModelCK1FCK1F(multiAssetMktData, modelParams);
    %% MultiAssetModel building end
   
    %% NumMethod Init Start
    numOfRandom = 40000;
    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                            {'AMC', 7, numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);
    %% NumMethod Init End
    
    % weekly step calculator
    % pricer = Pricer(rangeAccrual,KRWLGM,numMethodAMC);
    % pricingOut = pricer.computePrice(valueDate);
    
    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    % pricer = DailyWeeklyRiskyPricer_CDKTBSpreadRA(rangeAccrual,KRWLGM1FRiskyUSDLGM1F,numMethodAMC,false);
%     pricer = DailyWeeklyRiskyPricer_CDUSDLibor3MSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, true);    
    pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRAInverseFloater(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, true);    
     
    pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH, 'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH,'PSANo');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSANo');
%     pricingOutClosed = pricer.computePriceClosedSimple(valueDateH);
    
    callablePrice(idx2) = pricingOutMC1.callable.npv;
    nonCallPrice(idx2) = pricingOutMC1.nonCall.npv;
%     closedPrice(idx2) = pricingOutClosed.pricerInfo.nonCall.npv;
    
%     for idx3 = 1:length(pricingOutMC1.nonCall.cashflow_npv_e)
% %     for idx3 = 1:length(pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e)
%         nonCallPrice_cashflowNPV(idx2, idx3) = pricingOutMC1.nonCall.cashflow_npv_e(idx3);
% %         closedPrice_cashflowNPV(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
% %         
% %         closedPrice_fwdCD(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdCD(idx3);
% %         closedPrice_fwdKTB(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdKTB(idx3);
% %         closedPrice_fwdSpread(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSpread(idx3);
% %         closedPrice_fwdMu(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdMu(idx3);
% %         closedPrice_fwdSigma(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSigma(idx3);
% %                 
%     end
end

aaa =1.0;


In [ ]:

testCD_CDKTB3MSpreadInverseFloater_CK1F_CK1F_20220620_ktbDelta.m

clc
clear
valueDate = '2022-06-20';

maxIdx = 0;
zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;

callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% CD-USDLibor3M Spread Range Accrual Pricing start

%RA info
rangeAccrualParams.payCcy = 'KRW';
rangeAccrualParams.startDate = H_Date('2019-08-23');

rangeAccrualParams.maturity = 15;

valueDateH = H_Date(valueDate);

nominal = 10000;
% coupon = 0.033;
coupon = 0.052;

% couponFreq = 1;
couponFreq = 0.25;


% Long: KRW
% Short: USD
tenorL = 0.25;
tenorS = 0.25;

% lowerS = 0.0;

lowerS = -0.001;
upperS = 999;

% 
% lower = -1;
% upper = 0.05;

tenor = 0.25;

lower = -999;
upper = 999;

%resetDate startDate endDate paymentDate
couponScheduleInt = [20190823	20190823	20191123	20191123
20191123	20191123	20200223	20200223
20200223	20200223	20200523	20200523
20200523	20200523	20200823	20200823
20200823	20200823	20201123	20201123
20201123	20201123	20210223	20210223
20210223	20210223	20210523	20210523
20210523	20210523	20210823	20210823
20210823	20210823	20211123	20211123
20211123	20211123	20220223	20220223
20220223	20220223	20220523	20220523
20220523	20220523	20220823	20220823
20220823	20220823	20221123	20221123
20221123	20221123	20230223	20230223
20230223	20230223	20230523	20230523
20230523	20230523	20230823	20230823
20230823	20230823	20231123	20231123
20231123	20231123	20240223	20240223
20240223	20240223	20240523	20240523
20240523	20240523	20240823	20240823
20240823	20240823	20241123	20241123
20241123	20241123	20250223	20250223
20250223	20250223	20250523	20250523
20250523	20250523	20250823	20250823
20250823	20250823	20251123	20251123
20251123	20251123	20260223	20260223
20260223	20260223	20260523	20260523
20260523	20260523	20260823	20260823
20260823	20260823	20261123	20261123
20261123	20261123	20270223	20270223
20270223	20270223	20270523	20270523
20270523	20270523	20270823	20270823
20270823	20270823	20271123	20271123
20271123	20271123	20280223	20280223
20280223	20280223	20280523	20280523
20280523	20280523	20280823	20280823
20280823	20280823	20281123	20281123
20281123	20281123	20290223	20290223
20290223	20290223	20290523	20290523
20290523	20290523	20290823	20290823
20290823	20290823	20291123	20291123
20291123	20291123	20300223	20300223
20300223	20300223	20300523	20300523
20300523	20300523	20300823	20300823
20300823	20300823	20301123	20301123
20301123	20301123	20310223	20310223
20310223	20310223	20310523	20310523
20310523	20310523	20310823	20310823
20310823	20310823	20311123	20311123
20311123	20311123	20320223	20320223
20320223	20320223	20320523	20320523
20320523	20320523	20320823	20320823
20320823	20320823	20321123	20321123
20321123	20321123	20330223	20330223
20330223	20330223	20330523	20330523
20330523	20330523	20330823	20330823
20330823	20330823	20331123	20331123
20331123	20331123	20340223	20340223
20340223	20340223	20340523	20340523
20340523	20340523	20340823	20340823
];

endDate = H_Date(couponScheduleInt(size(couponScheduleInt,1),4));

%% CK1F model param matu setting start
% matu = 15 + 1.0/365.0;
matu = DateDiff(H_Date(couponScheduleInt(end,4)),H_Date(valueDate))/365.0;
% matu = matu -10.0;

%% end

 rangeDays = zeros(60,1);
%     rangeDays(1) = 1;

for i=1:length(rangeDays)
    if DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) >= 0
        rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) + 1;
    end
end


%resetDate startDate endDate paymentDate
callScheduleInt = [20190823	20190823	20190830	20190830
20191116	20191116	20191123	20191123
20200216	20200216	20200223	20200223
20200516	20200516	20200523	20200523
20200816	20200816	20200823	20200823
20201116	20201116	20201123	20201123
20210216	20210216	20210223	20210223
20210516	20210516	20210523	20210523
20210816	20210816	20210823	20210823
20211116	20211116	20211123	20211123
20220216	20220216	20220223	20220223
20220516	20220516	20220523	20220523
20220816	20220816	20220823	20220823
20221116	20221116	20221123	20221123
20230216	20230216	20230223	20230223
20230516	20230516	20230523	20230523
20230816	20230816	20230823	20230823
20231116	20231116	20231123	20231123
20240216	20240216	20240223	20240223
20240516	20240516	20240523	20240523
20240816	20240816	20240823	20240823
20241116	20241116	20241123	20241123
20250216	20250216	20250223	20250223
20250516	20250516	20250523	20250523
20250816	20250816	20250823	20250823
20251116	20251116	20251123	20251123
20260216	20260216	20260223	20260223
20260516	20260516	20260523	20260523
20260816	20260816	20260823	20260823
20261116	20261116	20261123	20261123
20270216	20270216	20270223	20270223
20270516	20270516	20270523	20270523
20270816	20270816	20270823	20270823
20271116	20271116	20271123	20271123
20280216	20280216	20280223	20280223
20280516	20280516	20280523	20280523
20280816	20280816	20280823	20280823
20281116	20281116	20281123	20281123
20290216	20290216	20290223	20290223
20290516	20290516	20290523	20290523
20290816	20290816	20290823	20290823
20291116	20291116	20291123	20291123
20300216	20300216	20300223	20300223
20300516	20300516	20300523	20300523
20300816	20300816	20300823	20300823
20301116	20301116	20301123	20301123
20310216	20310216	20310223	20310223
20310516	20310516	20310523	20310523
20310816	20310816	20310823	20310823
20311116	20311116	20311123	20311123
20320216	20320216	20320223	20320223
20320516	20320516	20320523	20320523
20320816	20320816	20320823	20320823
20321116	20321116	20321123	20321123
20330216	20330216	20330223	20330223
20330516	20330516	20330523	20330523
20330816	20330816	20330823	20330823
20331116	20331116	20331123	20331123
20340216	20340216	20340223	20340223
20340516	20340516	20340523	20340523
];

callYN = ones(60,1);
callYN(1) = 0;

manualScheduleYN = 1;

rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                'tenorS','lower','upper','rangeDays','callYN', ...
                                'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                        {nominal,coupon,couponFreq,tenorL,tenorS,lower,upper,...
                        rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});
% 
% tenorL = 0.25;
% tenorS = 0.25;
% 
% tenor = 0.25;
% 
% lowerS = 0.0;
% upperS = 999;
% 
% 
% lower = 0.0;
% upper = 0.05;

rangeAccrualInfo('lowerS') =lowerS;
rangeAccrualInfo('upperS') =upperS;
rangeAccrualInfo('tenor') = 0.25;


rangeAccrualInfo('couponLowerBound') = -0.05;


rangeAccrualParams.params = rangeAccrualInfo;

% rangeAccrual = SpreadRangeAccrual(rangeAccrualParams);
% rangeAccrual = CDCMSSpreadDualRangeAccrual(rangeAccrualParams);
rangeAccrual = CDCMSSpreadDualRangeAccrualInverseFloater(rangeAccrualParams);

% maxIdx =2.5;

maxIdx = 10;

rmseSimul = zeros(100,1);
for idx2=1:21
    %% KRW Model building

    %% import and construct a zeroCurve
    rawData = [0.002739726	0.999952523
0.25	0.9950684
0.5	0.988031579
0.75	0.979080331
1	0.968776974
1.25	0.959202682
1.5	0.948993277
1.75	0.939322512
2	0.929369492
2.25	0.920482183
2.5	0.911621558
2.75	0.902762032
3	0.893905238
3.25	0.885611153
3.5	0.877424651
3.75	0.869320968
4	0.861299205
4.25	0.853502656
4.5	0.845805585
4.75	0.838202041
5	0.830690747
5.25	0.823531148
5.5	0.816487433
5.75	0.809552209
6	0.802723731
6.25	0.796000289
6.5	0.789380204
6.75	0.782861832
7	0.776443559
7.25	0.770025284
7.5	0.763695404
7.75	0.757455789
8	0.751305075
8.25	0.74524192
8.5	0.73926501
8.75	0.73337305
9	0.727564772
9.25	0.721838929
9.5	0.716194297
9.75	0.710629672
10	0.705143874
10.25	0.699194275
10.5	0.693278485
10.75	0.687420297
11	0.681619095
11.25	0.675874273
11.5	0.670185229
11.75	0.664551369
12	0.658972106
12.25	0.654130635
12.5	0.649391356
12.75	0.644722276
13	0.640122375
13.25	0.635590647
13.5	0.631126108
13.75	0.626727791
14	0.622394747
14.25	0.618126045
14.5	0.613920769
14.75	0.609778022
15	0.605696922
15.25	0.60235689
15.5	0.599139226
15.75	0.595995535
16	0.592924834
16.25	0.589926161
16.5	0.586998583
16.75	0.58414119
17	0.581353097
17.25	0.578633444
17.5	0.575981393
17.75	0.573396131
18	0.570876865
18.25	0.568422827
18.5	0.566033269
18.75	0.563707467
19	0.561444716
19.25	0.559244332
19.5	0.557105652
19.75	0.555028035
20	0.553010856
20.25	0.550157221
20.5	0.547261794
20.75	0.544409494
21	0.541599805
21.25	0.538832219
21.5	0.536106239
21.75	0.533421375
22	0.530777148
22.25	0.528173085
22.5	0.525608723
22.75	0.523083608
23	0.520597293
23.25	0.518149339
23.5	0.515739316
23.75	0.513366801
24	0.511031378
24.25	0.508732641
24.5	0.506470189
24.75	0.50424363
25	0.502052578
25.25	0.498564056
25.5	0.494988089
25.75	0.491434812
26	0.487904104
26.25	0.484395847
26.5	0.48090992
26.75	0.477446204
27	0.474004582
27.25	0.470584935
27.5	0.467187146
27.75	0.463811098
28	0.460456674
28.25	0.457123757
28.5	0.453812233
28.75	0.450521986
29	0.447252902
29.25	0.444004865
29.5	0.440777762
29.75	0.43757148
30	0.434385905
30.25	0.431221981
30.5	0.428081088
30.75	0.424963073
31	0.421867769
    ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

    
%     %simulation testCase
%     for idx=1:size(rawData,1)
%         rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
%     end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    KRWAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];

% rawData = [0.002739726	0.999952523
% 0.25	0.9950684
% 0.5	0.988031579
% 0.75	0.979080331
% 1	0.968776974
% 1.25	0.959202682
% 1.5	0.948993277
% 1.75	0.939322512
% 2	0.929369492
% 2.25	0.920482183
% 2.5	0.911621558
% 2.75	0.902762032
% 3	0.893905238
% 3.25	0.885611153
% 3.5	0.877424651
% 3.75	0.869320968
% 4	0.861299205
% 4.25	0.853502656
% 4.5	0.845805585
% 4.75	0.838202041
% 5	0.830690747
% 5.25	0.823531148
% 5.5	0.816487433
% 5.75	0.809552209
% 6	0.802723731
% 6.25	0.796000289
% 6.5	0.789380204
% 6.75	0.782861832
% 7	0.776443559
% 7.25	0.770025284
% 7.5	0.763695404
% 7.75	0.757455789
% 8	0.751305075
% 8.25	0.74524192
% 8.5	0.73926501
% 8.75	0.73337305
% 9	0.727564772
% 9.25	0.721838929
% 9.5	0.716194297
% 9.75	0.710629672
% 10	0.705143874
% 10.25	0.699194275
% 10.5	0.693278485
% 10.75	0.687420297
% 11	0.681619095
% 11.25	0.675874273
% 11.5	0.670185229
% 11.75	0.664551369
% 12	0.658972106
% 12.25	0.654130635
% 12.5	0.649391356
% 12.75	0.644722276
% 13	0.640122375
% 13.25	0.635590647
% 13.5	0.631126108
% 13.75	0.626727791
% 14	0.622394747
% 14.25	0.618126045
% 14.5	0.613920769
% 14.75	0.609778022
% 15	0.605696922
% 15.25	0.60235689
% 15.5	0.599139226
% 15.75	0.595995535
% 16	0.592924834
% 16.25	0.589926161
% 16.5	0.586998583
% 16.75	0.58414119
% 17	0.581353097
% 17.25	0.578633444
% 17.5	0.575981393
% 17.75	0.573396131
% 18	0.570876865
% 18.25	0.568422827
% 18.5	0.566033269
% 18.75	0.563707467
% 19	0.561444716
% 19.25	0.559244332
% 19.5	0.557105652
% 19.75	0.555028035
% 20	0.553010856
% 20.25	0.550157221
% 20.5	0.547261794
% 20.75	0.544409494
% 21	0.541599805
% 21.25	0.538832219
% 21.5	0.536106239
% 21.75	0.533421375
% 22	0.530777148
% 22.25	0.528173085
% 22.5	0.525608723
% 22.75	0.523083608
% 23	0.520597293
% 23.25	0.518149339
% 23.5	0.515739316
% 23.75	0.513366801
% 24	0.511031378
% 24.25	0.508732641
% 24.5	0.506470189
% 24.75	0.50424363
% 25	0.502052578
% 25.25	0.498564056
% 25.5	0.494988089
% 25.75	0.491434812
% 26	0.487904104
% 26.25	0.484395847
% 26.5	0.48090992
% 26.75	0.477446204
% 27	0.474004582
% 27.25	0.470584935
% 27.5	0.467187146
% 27.75	0.463811098
% 28	0.460456674
% 28.25	0.457123757
% 28.5	0.453812233
% 28.75	0.450521986
% 29	0.447252902
% 29.25	0.444004865
% 29.5	0.440777762
% 29.75	0.43757148
% 30	0.434385905
% 30.25	0.431221981
% 30.5	0.428081088
% 30.75	0.424963073
% 31	0.421867769
%     ];


for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

 %simulation testCase
    for idx=1:size(rawData,1)
        rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
    end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);

    KRWAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurfaceNormal



rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.01025373	0.00996465	0.00939453	0.0088449	0.00838736	0.00835816	0.00827411	0.00844853	0.00844853
2	0.00987033	0.00941532	0.00896799	0.00850851	0.00803853	0.00809306	0.00814872	0.00832187	0.00832187
3	0.00965448	0.00917228	0.00868278	0.00815841	0.00766413	0.00768619	0.00766987	0.007827	0.007827
4	0.0086456	0.00835905	0.00803055	0.00773857	0.00743965	0.00747184	0.00750208	0.00760185	0.00760185
5	0.008024	0.00784518	0.00761802	0.00741163	0.00719132	0.00718529	0.00716174	0.0072174	0.0072174
7	0.00777954	0.0075868	0.00743466	0.00726955	0.00710966	0.00717731	0.00726665	0.00728989	0.00728989
10	0.00697384	0.00701303	0.00705259	0.00704171	0.0070555	0.00714878	0.00723997	0.00722954	0.00722954
15	0.00731508	0.00731858	0.00728587	0.00725674	0.0072209	0.00720153	0.00717821	0.0071639	0.0071639
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    KRWAllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a KRWBlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(KRWAllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = KRWBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= KRWBlack.NormalATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
   %% create a CK1FRisky Model with initial modelParams
%     vol_r.tenor = [1;2;3;4;5;7;10;15;20];
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;
%     meanR_r = 0.0;
    
    
    joint_swap_tenor = 1;
%     targetSize = 10;

    targetSize = 15;
    
    freq = 4;
    
    modelParams = containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'},...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
    
%     import irModels.*;
%     import targetFunctions.*;

 %% allFixings
    %% CD91
    tempDate = rangeAccrualParams.startDate;
    irIndexFixing3 = containers.Map({StrDate(tempDate,'str')},{0.025});

    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        irIndexFixing3(StrDate(tempDate,'str')) = 0.025;
    end

    %% fixing info -> let it be the modelParams !
    allFixings = containers.Map({'LIBOR3M'},{irIndexFixing3});
    modelParams('allFixings') = allFixings;
    
    KRWCK1FRisky = CK1FRisky(IRModelRisky(Model(KRWAllMktData, modelParams)));
    
    tic
%     out = KRWLGM2F.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
    % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
%     out = KRWCK1FRisky.CalibrateToATMDiagSwaption(KRWBlack,'global');
    out = KRWCK1FRisky.CalibrateToATMDiagSwaptionNormal(KRWBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        KRWVol_r(idx2,idx3) = out.vol_r.quote(idx3);
        KRWRe(idx2,idx3) = out.re(idx3);
    end
    KRWRMSE(idx2) = out.rmseTotal;
    
    toc
    %% CK1FRisky model building end
    %% KRW model building end

    %% USD model building start
    %% import and construct a zeroCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

 %simulation testCase
    for idx=1:size(rawData,1)
        rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
    end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','USDZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    USDAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];


% rawData = [0.002739726	0.999952523
% 0.25	0.9950684
% 0.5	0.988031579
% 0.75	0.979080331
% 1	0.968776974
% 1.25	0.959202682
% 1.5	0.948993277
% 1.75	0.939322512
% 2	0.929369492
% 2.25	0.920482183
% 2.5	0.911621558
% 2.75	0.902762032
% 3	0.893905238
% 3.25	0.885611153
% 3.5	0.877424651
% 3.75	0.869320968
% 4	0.861299205
% 4.25	0.853502656
% 4.5	0.845805585
% 4.75	0.838202041
% 5	0.830690747
% 5.25	0.823531148
% 5.5	0.816487433
% 5.75	0.809552209
% 6	0.802723731
% 6.25	0.796000289
% 6.5	0.789380204
% 6.75	0.782861832
% 7	0.776443559
% 7.25	0.770025284
% 7.5	0.763695404
% 7.75	0.757455789
% 8	0.751305075
% 8.25	0.74524192
% 8.5	0.73926501
% 8.75	0.73337305
% 9	0.727564772
% 9.25	0.721838929
% 9.5	0.716194297
% 9.75	0.710629672
% 10	0.705143874
% 10.25	0.699194275
% 10.5	0.693278485
% 10.75	0.687420297
% 11	0.681619095
% 11.25	0.675874273
% 11.5	0.670185229
% 11.75	0.664551369
% 12	0.658972106
% 12.25	0.654130635
% 12.5	0.649391356
% 12.75	0.644722276
% 13	0.640122375
% 13.25	0.635590647
% 13.5	0.631126108
% 13.75	0.626727791
% 14	0.622394747
% 14.25	0.618126045
% 14.5	0.613920769
% 14.75	0.609778022
% 15	0.605696922
% 15.25	0.60235689
% 15.5	0.599139226
% 15.75	0.595995535
% 16	0.592924834
% 16.25	0.589926161
% 16.5	0.586998583
% 16.75	0.58414119
% 17	0.581353097
% 17.25	0.578633444
% 17.5	0.575981393
% 17.75	0.573396131
% 18	0.570876865
% 18.25	0.568422827
% 18.5	0.566033269
% 18.75	0.563707467
% 19	0.561444716
% 19.25	0.559244332
% 19.5	0.557105652
% 19.75	0.555028035
% 20	0.553010856
% 20.25	0.550157221
% 20.5	0.547261794
% 20.75	0.544409494
% 21	0.541599805
% 21.25	0.538832219
% 21.5	0.536106239
% 21.75	0.533421375
% 22	0.530777148
% 22.25	0.528173085
% 22.5	0.525608723
% 22.75	0.523083608
% 23	0.520597293
% 23.25	0.518149339
% 23.5	0.515739316
% 23.75	0.513366801
% 24	0.511031378
% 24.25	0.508732641
% 24.5	0.506470189
% 24.75	0.50424363
% 25	0.502052578
% 25.25	0.498564056
% 25.5	0.494988089
% 25.75	0.491434812
% 26	0.487904104
% 26.25	0.484395847
% 26.5	0.48090992
% 26.75	0.477446204
% 27	0.474004582
% 27.25	0.470584935
% 27.5	0.467187146
% 27.75	0.463811098
% 28	0.460456674
% 28.25	0.457123757
% 28.5	0.453812233
% 28.75	0.450521986
% 29	0.447252902
% 29.25	0.444004865
% 29.5	0.440777762
% 29.75	0.43757148
% 30	0.434385905
% 30.25	0.431221981
% 30.5	0.428081088
% 30.75	0.424963073
% 31	0.421867769
%     ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

 %simulation testCase
    for idx=1:size(rawData,1)
        rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
    end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);
    USDAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
% GIROPremiumToNormalVol From LIB
% rawDataSwaption = [0	1	2	3	4	5	7	10	12	15	20
% 1	0.0041	0.0045	0.0052	0.0061	0.0066	0.0071	0.0077	0.0080	0.0081	0.0082
% 2	0.0050	0.0057	0.0064	0.0066	0.0068	0.0071	0.0074	0.0077	0.0077	0.0078
% 3	0.0064	0.0069	0.0068	0.0068	0.0068	0.0072	0.0076	0.0078	0.0079	0.0079
% 4	0.0076	0.0071	0.0071	0.0070	0.0071	0.0072	0.0077	0.0081	0.0081	0.0081
% 5	0.0069	0.0070	0.0070	0.0071	0.0071	0.0070	0.0076	0.0083	0.0084	0.0083
% 7	0.0073	0.0074	0.0075	0.0072	0.0070	0.0075	0.0087	0.0096	0.0096	0.0095
% 10	0.0066	0.0066	0.0071	0.0074	0.0077	0.0092	0.0101	0.0102	0.0102	0.0102
% 15	0.0103	0.0105	0.0107	0.0109	0.0111	0.0108	0.0103	0.0102	0.0102	0.0101
% 20	0.0088	0.0089	0.0090	0.0091	0.0091	0.0091	0.0090	0.0089	0.0089	0.0089
%     ];
% 
rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.01025373	0.00996465	0.00939453	0.0088449	0.00838736	0.00835816	0.00827411	0.00844853	0.00844853
2	0.00987033	0.00941532	0.00896799	0.00850851	0.00803853	0.00809306	0.00814872	0.00832187	0.00832187
3	0.00965448	0.00917228	0.00868278	0.00815841	0.00766413	0.00768619	0.00766987	0.007827	0.007827
4	0.0086456	0.00835905	0.00803055	0.00773857	0.00743965	0.00747184	0.00750208	0.00760185	0.00760185
5	0.008024	0.00784518	0.00761802	0.00741163	0.00719132	0.00718529	0.00716174	0.0072174	0.0072174
7	0.00777954	0.0075868	0.00743466	0.00726955	0.00710966	0.00717731	0.00726665	0.00728989	0.00728989
10	0.00697384	0.00701303	0.00705259	0.00704171	0.0070555	0.00714878	0.00723997	0.00722954	0.00722954
15	0.00731508	0.00731858	0.00728587	0.00725674	0.0072209	0.00720153	0.00717821	0.0071639	0.0071639
    ];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'},...
                            {rawDataSwaption,valueDate,'USD','USDSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    USDAllMktData('swaptionVol') = swaptionVolMktData;
    
   %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    USDBlack = IRBlack(IRModel(Model(USDAllMktData,modelParams))); 
    
    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);
    
    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = USDBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= USDBlack.BlackATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = USDBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
    %% create a CK1FModel with initial modelParams
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;

%     matu = 15 + 1.0/365.0;
    joint_swap_tenor = 1;
    targetSize = 15;
    freq = 4;
    
    modelParamsUSD= containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'}, ...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
                        
     %% allFixings
    %% CD91
    tempDate = rangeAccrualParams.startDate;
    irIndexFixing3USD = containers.Map({StrDate(tempDate,'str')},{0.025});

    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        irIndexFixing3USD(StrDate(tempDate,'str')) = 0.025;
    end

    %% fixing info -> let it be the modelParams !
    allFixingsUSD = containers.Map({'LIBOR3M'},{irIndexFixing3USD});
    modelParamsUSD('allFixings') = allFixingsUSD;
    
    USDCK1F = CK1F(IRModel(Model(USDAllMktData, modelParamsUSD)));
    
    tic
    outUSD = USDCK1F.CalibrateToATMDiagSwaptionNormal(USDBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        USDVol_r(idx2,idx3) = outUSD.vol_r.quote(idx3);
        USDRe(idx2,idx3) = outUSD.re(idx3);
    end
    USDRMSE(idx2) = outUSD.rmseTotal;
    
    toc
    %% USD model building end

    %% USDKRW fx model buidling start
    rawData = [0.083333333	0.0
0.25	0.0
0.5	0.0
1	0.0
2	0.0
3	0.0
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','USDKRWATMTermVol'});
    impliedTermVolMktData = MktData(mktData);
%     impliedTermVole = FXVolSurface(impliedTermVolMktData);
%     impliedVolSurface.params('moneyNessType') = 'deltaFwdMoneyNess';
    USDKRWAllMktData =containers.Map('impliedTermVol',impliedTermVolMktData);
%     AllMktData('impliedVolSurface') = impliedTermVolMktData;

    %% create  an FXBlackModel to calculate market PlainVanilla Options
%     modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});
%     USDKRWBlack = FXBlack(FXModel(Model(USDKRWAllMktData,modelParams))); 
    %% USDKRW fx model building end

    %% MultiAssetModel building start
    multiAssetMktData = containers.Map({'KRW','USD','USDKRW'}, ...
                            {KRWAllMktData, USDAllMktData, USDKRWAllMktData});

    % KRW,KTB                    
%     correlationMatrix = ones(2,2);
%     correlationMatrix(1,2) = 0.3;
%     correlationMatrix(2,1) = correlationMatrix(1,2);

    % KRW, USD & USDKRW
    correlationMatrix = ones(3, 3);
    correlationMatrix(1,2) = 0.845;
%     correlationMatrix(1,2) = 0.6;
    
%     correlationMatrix(1,2) = 0.7;
    
    correlationMatrix(1,3) = 0.0;
    correlationMatrix(2,3) = 0.0;
    correlationMatrix(2,1) = correlationMatrix(1,2);
    correlationMatrix(3,1) = correlationMatrix(1,3);
    correlationMatrix(3,2) = correlationMatrix(2,3);
    
    multiAssetMktData('correlationMatrix') = correlationMatrix;
    
    modelNameMap = containers.Map({'KRW','USD'},{KRWCK1FRisky, USDCK1F});
    modelParams = containers.Map({'modelNameMap'}, {modelNameMap});
    % correlationNames = {'KRW','USD'};
    modelParams('stochasticModelNames') = {'KRW','USD'};
    modelParams('correlationMatrix') = correlationMatrix;
    modelParams('refModelName') = 'KRW';
    % modelParams('useQuasiRandomYN') = 'true';
    modelParams('useQuasiRandomYN') = 'false';
    modelParams('fxVolCalibrated') = true;
    
    KRWCK1FRiskyUSDCK1F = MultiAssetModelCK1FCK1F(multiAssetMktData, modelParams);
    %% MultiAssetModel building end
   
    %% NumMethod Init Start
    numOfRandom = 40000;
    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                            {'AMC', 7, numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);
    %% NumMethod Init End
    
    % weekly step calculator
    % pricer = Pricer(rangeAccrual,KRWLGM,numMethodAMC);
    % pricingOut = pricer.computePrice(valueDate);
    
    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    % pricer = DailyWeeklyRiskyPricer_CDKTBSpreadRA(rangeAccrual,KRWLGM1FRiskyUSDLGM1F,numMethodAMC,false);
%     pricer = DailyWeeklyRiskyPricer_CDUSDLibor3MSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, true);    
    pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRAInverseFloater(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, true);    
     
    pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH, 'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH,'PSANo');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSANo');
%     pricingOutClosed = pricer.computePriceClosedSimple(valueDateH);
    
    callablePrice(idx2) = pricingOutMC1.callable.npv;
    nonCallPrice(idx2) = pricingOutMC1.nonCall.npv;
%     closedPrice(idx2) = pricingOutClosed.pricerInfo.nonCall.npv;
    
%     for idx3 = 1:length(pricingOutMC1.nonCall.cashflow_npv_e)
% %     for idx3 = 1:length(pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e)
%         nonCallPrice_cashflowNPV(idx2, idx3) = pricingOutMC1.nonCall.cashflow_npv_e(idx3);
% %         closedPrice_cashflowNPV(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
% %         
% %         closedPrice_fwdCD(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdCD(idx3);
% %         closedPrice_fwdKTB(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdKTB(idx3);
% %         closedPrice_fwdSpread(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSpread(idx3);
% %         closedPrice_fwdMu(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdMu(idx3);
% %         closedPrice_fwdSigma(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSigma(idx3);
% %                 
%     end
end

aaa =1.0;


In [ ]:
testCD_CDKTB3MSpreadInverseFloater_CK1F_CK1F_20220620_irsVega.m


clc
clear
valueDate = '2022-06-20';

maxIdx = 0;
zcStep = 0.001;
%shift size 10bp
% shiftAmount = 0.001;
shiftAmount = 0.0005;

callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% CD-USDLibor3M Spread Range Accrual Pricing start

%RA info
rangeAccrualParams.payCcy = 'KRW';
rangeAccrualParams.startDate = H_Date('2019-08-23');

rangeAccrualParams.maturity = 15;

valueDateH = H_Date(valueDate);

nominal = 10000;
% coupon = 0.033;
coupon = 0.052;

% couponFreq = 1;
couponFreq = 0.25;


% Long: KRW
% Short: USD
tenorL = 0.25;
tenorS = 0.25;

% lowerS = 0.0;

lowerS = -0.001;
upperS = 999;

% 
% lower = -1;
% upper = 0.05;

tenor = 0.25;

lower = -999;
upper = 999;

%resetDate startDate endDate paymentDate
couponScheduleInt = [20190823	20190823	20191123	20191123
20191123	20191123	20200223	20200223
20200223	20200223	20200523	20200523
20200523	20200523	20200823	20200823
20200823	20200823	20201123	20201123
20201123	20201123	20210223	20210223
20210223	20210223	20210523	20210523
20210523	20210523	20210823	20210823
20210823	20210823	20211123	20211123
20211123	20211123	20220223	20220223
20220223	20220223	20220523	20220523
20220523	20220523	20220823	20220823
20220823	20220823	20221123	20221123
20221123	20221123	20230223	20230223
20230223	20230223	20230523	20230523
20230523	20230523	20230823	20230823
20230823	20230823	20231123	20231123
20231123	20231123	20240223	20240223
20240223	20240223	20240523	20240523
20240523	20240523	20240823	20240823
20240823	20240823	20241123	20241123
20241123	20241123	20250223	20250223
20250223	20250223	20250523	20250523
20250523	20250523	20250823	20250823
20250823	20250823	20251123	20251123
20251123	20251123	20260223	20260223
20260223	20260223	20260523	20260523
20260523	20260523	20260823	20260823
20260823	20260823	20261123	20261123
20261123	20261123	20270223	20270223
20270223	20270223	20270523	20270523
20270523	20270523	20270823	20270823
20270823	20270823	20271123	20271123
20271123	20271123	20280223	20280223
20280223	20280223	20280523	20280523
20280523	20280523	20280823	20280823
20280823	20280823	20281123	20281123
20281123	20281123	20290223	20290223
20290223	20290223	20290523	20290523
20290523	20290523	20290823	20290823
20290823	20290823	20291123	20291123
20291123	20291123	20300223	20300223
20300223	20300223	20300523	20300523
20300523	20300523	20300823	20300823
20300823	20300823	20301123	20301123
20301123	20301123	20310223	20310223
20310223	20310223	20310523	20310523
20310523	20310523	20310823	20310823
20310823	20310823	20311123	20311123
20311123	20311123	20320223	20320223
20320223	20320223	20320523	20320523
20320523	20320523	20320823	20320823
20320823	20320823	20321123	20321123
20321123	20321123	20330223	20330223
20330223	20330223	20330523	20330523
20330523	20330523	20330823	20330823
20330823	20330823	20331123	20331123
20331123	20331123	20340223	20340223
20340223	20340223	20340523	20340523
20340523	20340523	20340823	20340823
];

endDate = H_Date(couponScheduleInt(size(couponScheduleInt,1),4));

%% CK1F model param matu setting start
% matu = 15 + 1.0/365.0;
matu = DateDiff(H_Date(couponScheduleInt(end,4)),H_Date(valueDate))/365.0;
% matu = matu -10.0;

%% end

 rangeDays = zeros(60,1);
%     rangeDays(1) = 1;

for i=1:length(rangeDays)
    if DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) >= 0
        rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) + 1;
    end
end


%resetDate startDate endDate paymentDate
callScheduleInt = [20190823	20190823	20190830	20190830
20191116	20191116	20191123	20191123
20200216	20200216	20200223	20200223
20200516	20200516	20200523	20200523
20200816	20200816	20200823	20200823
20201116	20201116	20201123	20201123
20210216	20210216	20210223	20210223
20210516	20210516	20210523	20210523
20210816	20210816	20210823	20210823
20211116	20211116	20211123	20211123
20220216	20220216	20220223	20220223
20220516	20220516	20220523	20220523
20220816	20220816	20220823	20220823
20221116	20221116	20221123	20221123
20230216	20230216	20230223	20230223
20230516	20230516	20230523	20230523
20230816	20230816	20230823	20230823
20231116	20231116	20231123	20231123
20240216	20240216	20240223	20240223
20240516	20240516	20240523	20240523
20240816	20240816	20240823	20240823
20241116	20241116	20241123	20241123
20250216	20250216	20250223	20250223
20250516	20250516	20250523	20250523
20250816	20250816	20250823	20250823
20251116	20251116	20251123	20251123
20260216	20260216	20260223	20260223
20260516	20260516	20260523	20260523
20260816	20260816	20260823	20260823
20261116	20261116	20261123	20261123
20270216	20270216	20270223	20270223
20270516	20270516	20270523	20270523
20270816	20270816	20270823	20270823
20271116	20271116	20271123	20271123
20280216	20280216	20280223	20280223
20280516	20280516	20280523	20280523
20280816	20280816	20280823	20280823
20281116	20281116	20281123	20281123
20290216	20290216	20290223	20290223
20290516	20290516	20290523	20290523
20290816	20290816	20290823	20290823
20291116	20291116	20291123	20291123
20300216	20300216	20300223	20300223
20300516	20300516	20300523	20300523
20300816	20300816	20300823	20300823
20301116	20301116	20301123	20301123
20310216	20310216	20310223	20310223
20310516	20310516	20310523	20310523
20310816	20310816	20310823	20310823
20311116	20311116	20311123	20311123
20320216	20320216	20320223	20320223
20320516	20320516	20320523	20320523
20320816	20320816	20320823	20320823
20321116	20321116	20321123	20321123
20330216	20330216	20330223	20330223
20330516	20330516	20330523	20330523
20330816	20330816	20330823	20330823
20331116	20331116	20331123	20331123
20340216	20340216	20340223	20340223
20340516	20340516	20340523	20340523
];

callYN = ones(60,1);
callYN(1) = 0;

manualScheduleYN = 1;

rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                'tenorS','lower','upper','rangeDays','callYN', ...
                                'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                        {nominal,coupon,couponFreq,tenorL,tenorS,lower,upper,...
                        rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});
% 
% tenorL = 0.25;
% tenorS = 0.25;
% 
% tenor = 0.25;
% 
% lowerS = 0.0;
% upperS = 999;
% 
% 
% lower = 0.0;
% upper = 0.05;

rangeAccrualInfo('lowerS') =lowerS;
rangeAccrualInfo('upperS') =upperS;
rangeAccrualInfo('tenor') = 0.25;


rangeAccrualInfo('couponLowerBound') = -0.05;


rangeAccrualParams.params = rangeAccrualInfo;

% rangeAccrual = SpreadRangeAccrual(rangeAccrualParams);
% rangeAccrual = CDCMSSpreadDualRangeAccrual(rangeAccrualParams);
rangeAccrual = CDCMSSpreadDualRangeAccrualInverseFloater(rangeAccrualParams);

% maxIdx =2.5;

maxIdx = 5;

rmseSimul = zeros(100,1);
for idx2=1:11
    %% KRW Model building

    %% import and construct a zeroCurve
    rawData = [0.002739726	0.999952523
0.25	0.9950684
0.5	0.988031579
0.75	0.979080331
1	0.968776974
1.25	0.959202682
1.5	0.948993277
1.75	0.939322512
2	0.929369492
2.25	0.920482183
2.5	0.911621558
2.75	0.902762032
3	0.893905238
3.25	0.885611153
3.5	0.877424651
3.75	0.869320968
4	0.861299205
4.25	0.853502656
4.5	0.845805585
4.75	0.838202041
5	0.830690747
5.25	0.823531148
5.5	0.816487433
5.75	0.809552209
6	0.802723731
6.25	0.796000289
6.5	0.789380204
6.75	0.782861832
7	0.776443559
7.25	0.770025284
7.5	0.763695404
7.75	0.757455789
8	0.751305075
8.25	0.74524192
8.5	0.73926501
8.75	0.73337305
9	0.727564772
9.25	0.721838929
9.5	0.716194297
9.75	0.710629672
10	0.705143874
10.25	0.699194275
10.5	0.693278485
10.75	0.687420297
11	0.681619095
11.25	0.675874273
11.5	0.670185229
11.75	0.664551369
12	0.658972106
12.25	0.654130635
12.5	0.649391356
12.75	0.644722276
13	0.640122375
13.25	0.635590647
13.5	0.631126108
13.75	0.626727791
14	0.622394747
14.25	0.618126045
14.5	0.613920769
14.75	0.609778022
15	0.605696922
15.25	0.60235689
15.5	0.599139226
15.75	0.595995535
16	0.592924834
16.25	0.589926161
16.5	0.586998583
16.75	0.58414119
17	0.581353097
17.25	0.578633444
17.5	0.575981393
17.75	0.573396131
18	0.570876865
18.25	0.568422827
18.5	0.566033269
18.75	0.563707467
19	0.561444716
19.25	0.559244332
19.5	0.557105652
19.75	0.555028035
20	0.553010856
20.25	0.550157221
20.5	0.547261794
20.75	0.544409494
21	0.541599805
21.25	0.538832219
21.5	0.536106239
21.75	0.533421375
22	0.530777148
22.25	0.528173085
22.5	0.525608723
22.75	0.523083608
23	0.520597293
23.25	0.518149339
23.5	0.515739316
23.75	0.513366801
24	0.511031378
24.25	0.508732641
24.5	0.506470189
24.75	0.50424363
25	0.502052578
25.25	0.498564056
25.5	0.494988089
25.75	0.491434812
26	0.487904104
26.25	0.484395847
26.5	0.48090992
26.75	0.477446204
27	0.474004582
27.25	0.470584935
27.5	0.467187146
27.75	0.463811098
28	0.460456674
28.25	0.457123757
28.5	0.453812233
28.75	0.450521986
29	0.447252902
29.25	0.444004865
29.5	0.440777762
29.75	0.43757148
30	0.434385905
30.25	0.431221981
30.5	0.428081088
30.75	0.424963073
31	0.421867769
    ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

    
%     %simulation testCase
%     for idx=1:size(rawData,1)
%         rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
%     end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    KRWAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];

% rawData = [0.002739726	0.999952523
% 0.25	0.9950684
% 0.5	0.988031579
% 0.75	0.979080331
% 1	0.968776974
% 1.25	0.959202682
% 1.5	0.948993277
% 1.75	0.939322512
% 2	0.929369492
% 2.25	0.920482183
% 2.5	0.911621558
% 2.75	0.902762032
% 3	0.893905238
% 3.25	0.885611153
% 3.5	0.877424651
% 3.75	0.869320968
% 4	0.861299205
% 4.25	0.853502656
% 4.5	0.845805585
% 4.75	0.838202041
% 5	0.830690747
% 5.25	0.823531148
% 5.5	0.816487433
% 5.75	0.809552209
% 6	0.802723731
% 6.25	0.796000289
% 6.5	0.789380204
% 6.75	0.782861832
% 7	0.776443559
% 7.25	0.770025284
% 7.5	0.763695404
% 7.75	0.757455789
% 8	0.751305075
% 8.25	0.74524192
% 8.5	0.73926501
% 8.75	0.73337305
% 9	0.727564772
% 9.25	0.721838929
% 9.5	0.716194297
% 9.75	0.710629672
% 10	0.705143874
% 10.25	0.699194275
% 10.5	0.693278485
% 10.75	0.687420297
% 11	0.681619095
% 11.25	0.675874273
% 11.5	0.670185229
% 11.75	0.664551369
% 12	0.658972106
% 12.25	0.654130635
% 12.5	0.649391356
% 12.75	0.644722276
% 13	0.640122375
% 13.25	0.635590647
% 13.5	0.631126108
% 13.75	0.626727791
% 14	0.622394747
% 14.25	0.618126045
% 14.5	0.613920769
% 14.75	0.609778022
% 15	0.605696922
% 15.25	0.60235689
% 15.5	0.599139226
% 15.75	0.595995535
% 16	0.592924834
% 16.25	0.589926161
% 16.5	0.586998583
% 16.75	0.58414119
% 17	0.581353097
% 17.25	0.578633444
% 17.5	0.575981393
% 17.75	0.573396131
% 18	0.570876865
% 18.25	0.568422827
% 18.5	0.566033269
% 18.75	0.563707467
% 19	0.561444716
% 19.25	0.559244332
% 19.5	0.557105652
% 19.75	0.555028035
% 20	0.553010856
% 20.25	0.550157221
% 20.5	0.547261794
% 20.75	0.544409494
% 21	0.541599805
% 21.25	0.538832219
% 21.5	0.536106239
% 21.75	0.533421375
% 22	0.530777148
% 22.25	0.528173085
% 22.5	0.525608723
% 22.75	0.523083608
% 23	0.520597293
% 23.25	0.518149339
% 23.5	0.515739316
% 23.75	0.513366801
% 24	0.511031378
% 24.25	0.508732641
% 24.5	0.506470189
% 24.75	0.50424363
% 25	0.502052578
% 25.25	0.498564056
% 25.5	0.494988089
% 25.75	0.491434812
% 26	0.487904104
% 26.25	0.484395847
% 26.5	0.48090992
% 26.75	0.477446204
% 27	0.474004582
% 27.25	0.470584935
% 27.5	0.467187146
% 27.75	0.463811098
% 28	0.460456674
% 28.25	0.457123757
% 28.5	0.453812233
% 28.75	0.450521986
% 29	0.447252902
% 29.25	0.444004865
% 29.5	0.440777762
% 29.75	0.43757148
% 30	0.434385905
% 30.25	0.431221981
% 30.5	0.428081088
% 30.75	0.424963073
% 31	0.421867769
%     ];


for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

%  %simulation testCase
%     for idx=1:size(rawData,1)
%         rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
%     end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);

    KRWAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurfaceNormal



rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.01025373	0.00996465	0.00939453	0.0088449	0.00838736	0.00835816	0.00827411	0.00844853	0.00844853
2	0.00987033	0.00941532	0.00896799	0.00850851	0.00803853	0.00809306	0.00814872	0.00832187	0.00832187
3	0.00965448	0.00917228	0.00868278	0.00815841	0.00766413	0.00768619	0.00766987	0.007827	0.007827
4	0.0086456	0.00835905	0.00803055	0.00773857	0.00743965	0.00747184	0.00750208	0.00760185	0.00760185
5	0.008024	0.00784518	0.00761802	0.00741163	0.00719132	0.00718529	0.00716174	0.0072174	0.0072174
7	0.00777954	0.0075868	0.00743466	0.00726955	0.00710966	0.00717731	0.00726665	0.00728989	0.00728989
10	0.00697384	0.00701303	0.00705259	0.00704171	0.0070555	0.00714878	0.00723997	0.00722954	0.00722954
15	0.00731508	0.00731858	0.00728587	0.00725674	0.0072209	0.00720153	0.00717821	0.0071639	0.0071639
    ];


    rawDataSwaptionSim = rawDataSwaption;
    
    for idxii=2: size(rawDataSwaption,1)
        for idxjj=2:size(rawDataSwaption,2)
            rawDataSwaptionSim(idxii,idxjj) = rawDataSwaption(idxii,idxjj) - maxIdx*shiftAmount + (idx2-1)*shiftAmount;
        end
    end
    
    rawDataSwaption = rawDataSwaptionSim;
    
%        rawDataSim = rawDataSwaption;

%     for idxhong1=2:size(rawDataSwaption,1)
%         for idxhong2=2:size(rawDataSwaption,2)
%             rawDataSim(idxhong1,idxhong2) = rawDataSwaption(idxhong1,idxhong2) + volShiftAmount*(idx2-1);
%         end
%     end
% 
%     rawDataSwaption = rawDataSim;
    
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    KRWAllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a KRWBlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(KRWAllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = KRWBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= KRWBlack.NormalATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
   %% create a CK1FRisky Model with initial modelParams
%     vol_r.tenor = [1;2;3;4;5;7;10;15;20];
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;
%     meanR_r = 0.0;
    
    
    joint_swap_tenor = 1;
%     targetSize = 10;

    targetSize = 15;
    
    freq = 4;
    
    modelParams = containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'},...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
    
%     import irModels.*;
%     import targetFunctions.*;

 %% allFixings
    %% CD91
    tempDate = rangeAccrualParams.startDate;
    irIndexFixing3 = containers.Map({StrDate(tempDate,'str')},{0.025});

    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        irIndexFixing3(StrDate(tempDate,'str')) = 0.025;
    end

    %% fixing info -> let it be the modelParams !
    allFixings = containers.Map({'LIBOR3M'},{irIndexFixing3});
    modelParams('allFixings') = allFixings;
    
    KRWCK1FRisky = CK1FRisky(IRModelRisky(Model(KRWAllMktData, modelParams)));
    
    tic
%     out = KRWLGM2F.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
    % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
%     out = KRWCK1FRisky.CalibrateToATMDiagSwaption(KRWBlack,'global');
    out = KRWCK1FRisky.CalibrateToATMDiagSwaptionNormal(KRWBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        KRWVol_r(idx2,idx3) = out.vol_r.quote(idx3);
        KRWRe(idx2,idx3) = out.re(idx3);
    end
    KRWRMSE(idx2) = out.rmseTotal;
    
    toc
    %% CK1FRisky model building end
    %% KRW model building end

    %% USD model building start
    %% import and construct a zeroCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

%  %simulation testCase
%     for idx=1:size(rawData,1)
%         rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
%     end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','USDZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    USDAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];


% rawData = [0.002739726	0.999952523
% 0.25	0.9950684
% 0.5	0.988031579
% 0.75	0.979080331
% 1	0.968776974
% 1.25	0.959202682
% 1.5	0.948993277
% 1.75	0.939322512
% 2	0.929369492
% 2.25	0.920482183
% 2.5	0.911621558
% 2.75	0.902762032
% 3	0.893905238
% 3.25	0.885611153
% 3.5	0.877424651
% 3.75	0.869320968
% 4	0.861299205
% 4.25	0.853502656
% 4.5	0.845805585
% 4.75	0.838202041
% 5	0.830690747
% 5.25	0.823531148
% 5.5	0.816487433
% 5.75	0.809552209
% 6	0.802723731
% 6.25	0.796000289
% 6.5	0.789380204
% 6.75	0.782861832
% 7	0.776443559
% 7.25	0.770025284
% 7.5	0.763695404
% 7.75	0.757455789
% 8	0.751305075
% 8.25	0.74524192
% 8.5	0.73926501
% 8.75	0.73337305
% 9	0.727564772
% 9.25	0.721838929
% 9.5	0.716194297
% 9.75	0.710629672
% 10	0.705143874
% 10.25	0.699194275
% 10.5	0.693278485
% 10.75	0.687420297
% 11	0.681619095
% 11.25	0.675874273
% 11.5	0.670185229
% 11.75	0.664551369
% 12	0.658972106
% 12.25	0.654130635
% 12.5	0.649391356
% 12.75	0.644722276
% 13	0.640122375
% 13.25	0.635590647
% 13.5	0.631126108
% 13.75	0.626727791
% 14	0.622394747
% 14.25	0.618126045
% 14.5	0.613920769
% 14.75	0.609778022
% 15	0.605696922
% 15.25	0.60235689
% 15.5	0.599139226
% 15.75	0.595995535
% 16	0.592924834
% 16.25	0.589926161
% 16.5	0.586998583
% 16.75	0.58414119
% 17	0.581353097
% 17.25	0.578633444
% 17.5	0.575981393
% 17.75	0.573396131
% 18	0.570876865
% 18.25	0.568422827
% 18.5	0.566033269
% 18.75	0.563707467
% 19	0.561444716
% 19.25	0.559244332
% 19.5	0.557105652
% 19.75	0.555028035
% 20	0.553010856
% 20.25	0.550157221
% 20.5	0.547261794
% 20.75	0.544409494
% 21	0.541599805
% 21.25	0.538832219
% 21.5	0.536106239
% 21.75	0.533421375
% 22	0.530777148
% 22.25	0.528173085
% 22.5	0.525608723
% 22.75	0.523083608
% 23	0.520597293
% 23.25	0.518149339
% 23.5	0.515739316
% 23.75	0.513366801
% 24	0.511031378
% 24.25	0.508732641
% 24.5	0.506470189
% 24.75	0.50424363
% 25	0.502052578
% 25.25	0.498564056
% 25.5	0.494988089
% 25.75	0.491434812
% 26	0.487904104
% 26.25	0.484395847
% 26.5	0.48090992
% 26.75	0.477446204
% 27	0.474004582
% 27.25	0.470584935
% 27.5	0.467187146
% 27.75	0.463811098
% 28	0.460456674
% 28.25	0.457123757
% 28.5	0.453812233
% 28.75	0.450521986
% 29	0.447252902
% 29.25	0.444004865
% 29.5	0.440777762
% 29.75	0.43757148
% 30	0.434385905
% 30.25	0.431221981
% 30.5	0.428081088
% 30.75	0.424963073
% 31	0.421867769
%     ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

%  %simulation testCase
%     for idx=1:size(rawData,1)
%         rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
%     end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);
    USDAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
% GIROPremiumToNormalVol From LIB
% rawDataSwaption = [0	1	2	3	4	5	7	10	12	15	20
% 1	0.0041	0.0045	0.0052	0.0061	0.0066	0.0071	0.0077	0.0080	0.0081	0.0082
% 2	0.0050	0.0057	0.0064	0.0066	0.0068	0.0071	0.0074	0.0077	0.0077	0.0078
% 3	0.0064	0.0069	0.0068	0.0068	0.0068	0.0072	0.0076	0.0078	0.0079	0.0079
% 4	0.0076	0.0071	0.0071	0.0070	0.0071	0.0072	0.0077	0.0081	0.0081	0.0081
% 5	0.0069	0.0070	0.0070	0.0071	0.0071	0.0070	0.0076	0.0083	0.0084	0.0083
% 7	0.0073	0.0074	0.0075	0.0072	0.0070	0.0075	0.0087	0.0096	0.0096	0.0095
% 10	0.0066	0.0066	0.0071	0.0074	0.0077	0.0092	0.0101	0.0102	0.0102	0.0102
% 15	0.0103	0.0105	0.0107	0.0109	0.0111	0.0108	0.0103	0.0102	0.0102	0.0101
% 20	0.0088	0.0089	0.0090	0.0091	0.0091	0.0091	0.0090	0.0089	0.0089	0.0089
%     ];
% 
rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.01025373	0.00996465	0.00939453	0.0088449	0.00838736	0.00835816	0.00827411	0.00844853	0.00844853
2	0.00987033	0.00941532	0.00896799	0.00850851	0.00803853	0.00809306	0.00814872	0.00832187	0.00832187
3	0.00965448	0.00917228	0.00868278	0.00815841	0.00766413	0.00768619	0.00766987	0.007827	0.007827
4	0.0086456	0.00835905	0.00803055	0.00773857	0.00743965	0.00747184	0.00750208	0.00760185	0.00760185
5	0.008024	0.00784518	0.00761802	0.00741163	0.00719132	0.00718529	0.00716174	0.0072174	0.0072174
7	0.00777954	0.0075868	0.00743466	0.00726955	0.00710966	0.00717731	0.00726665	0.00728989	0.00728989
10	0.00697384	0.00701303	0.00705259	0.00704171	0.0070555	0.00714878	0.00723997	0.00722954	0.00722954
15	0.00731508	0.00731858	0.00728587	0.00725674	0.0072209	0.00720153	0.00717821	0.0071639	0.0071639
    ];


    rawDataSwaptionSim = rawDataSwaption;
    
    for idxii=2: size(rawDataSwaption,1)
        for idxjj=2:size(rawDataSwaption,2)
            rawDataSwaptionSim(idxii,idxjj) = rawDataSwaption(idxii,idxjj) - maxIdx*shiftAmount + (idx2-1)*shiftAmount;
        end
    end
    
    rawDataSwaption = rawDataSwaptionSim;


    mktData = containers.Map({'rawData','asOfDate','ccy','name'},...
                            {rawDataSwaption,valueDate,'USD','USDSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    USDAllMktData('swaptionVol') = swaptionVolMktData;
    
   %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    USDBlack = IRBlack(IRModel(Model(USDAllMktData,modelParams))); 
    
    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);
    
    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = USDBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= USDBlack.BlackATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = USDBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
    %% create a CK1FModel with initial modelParams
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;

%     matu = 15 + 1.0/365.0;
    joint_swap_tenor = 1;
    targetSize = 15;
    freq = 4;
    
    modelParamsUSD= containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'}, ...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
                        
     %% allFixings
    %% CD91
    tempDate = rangeAccrualParams.startDate;
    irIndexFixing3USD = containers.Map({StrDate(tempDate,'str')},{0.025});

    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        irIndexFixing3USD(StrDate(tempDate,'str')) = 0.025;
    end

    %% fixing info -> let it be the modelParams !
    allFixingsUSD = containers.Map({'LIBOR3M'},{irIndexFixing3USD});
    modelParamsUSD('allFixings') = allFixingsUSD;
    
    USDCK1F = CK1F(IRModel(Model(USDAllMktData, modelParamsUSD)));
    
    tic
    outUSD = USDCK1F.CalibrateToATMDiagSwaptionNormal(USDBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        USDVol_r(idx2,idx3) = outUSD.vol_r.quote(idx3);
        USDRe(idx2,idx3) = outUSD.re(idx3);
    end
    USDRMSE(idx2) = outUSD.rmseTotal;
    
    toc
    %% USD model building end

    %% USDKRW fx model buidling start
    rawData = [0.083333333	0.0
0.25	0.0
0.5	0.0
1	0.0
2	0.0
3	0.0
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','USDKRWATMTermVol'});
    impliedTermVolMktData = MktData(mktData);
%     impliedTermVole = FXVolSurface(impliedTermVolMktData);
%     impliedVolSurface.params('moneyNessType') = 'deltaFwdMoneyNess';
    USDKRWAllMktData =containers.Map('impliedTermVol',impliedTermVolMktData);
%     AllMktData('impliedVolSurface') = impliedTermVolMktData;

    %% create  an FXBlackModel to calculate market PlainVanilla Options
%     modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});
%     USDKRWBlack = FXBlack(FXModel(Model(USDKRWAllMktData,modelParams))); 
    %% USDKRW fx model building end

    %% MultiAssetModel building start
    multiAssetMktData = containers.Map({'KRW','USD','USDKRW'}, ...
                            {KRWAllMktData, USDAllMktData, USDKRWAllMktData});

    % KRW,KTB                    
%     correlationMatrix = ones(2,2);
%     correlationMatrix(1,2) = 0.3;
%     correlationMatrix(2,1) = correlationMatrix(1,2);

    % KRW, USD & USDKRW
    correlationMatrix = ones(3, 3);
    correlationMatrix(1,2) = 0.845;
%     correlationMatrix(1,2) = 0.6;
    
%     correlationMatrix(1,2) = 0.7;
    
    correlationMatrix(1,3) = 0.0;
    correlationMatrix(2,3) = 0.0;
    correlationMatrix(2,1) = correlationMatrix(1,2);
    correlationMatrix(3,1) = correlationMatrix(1,3);
    correlationMatrix(3,2) = correlationMatrix(2,3);
    
    multiAssetMktData('correlationMatrix') = correlationMatrix;
    
    modelNameMap = containers.Map({'KRW','USD'},{KRWCK1FRisky, USDCK1F});
    modelParams = containers.Map({'modelNameMap'}, {modelNameMap});
    % correlationNames = {'KRW','USD'};
    modelParams('stochasticModelNames') = {'KRW','USD'};
    modelParams('correlationMatrix') = correlationMatrix;
    modelParams('refModelName') = 'KRW';
    % modelParams('useQuasiRandomYN') = 'true';
    modelParams('useQuasiRandomYN') = 'false';
    modelParams('fxVolCalibrated') = true;
    
    KRWCK1FRiskyUSDCK1F = MultiAssetModelCK1FCK1F(multiAssetMktData, modelParams);
    %% MultiAssetModel building end
   
    %% NumMethod Init Start
    numOfRandom = 40000;
    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                            {'AMC', 7, numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);
    %% NumMethod Init End
    
    % weekly step calculator
    % pricer = Pricer(rangeAccrual,KRWLGM,numMethodAMC);
    % pricingOut = pricer.computePrice(valueDate);
    
    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    % pricer = DailyWeeklyRiskyPricer_CDKTBSpreadRA(rangeAccrual,KRWLGM1FRiskyUSDLGM1F,numMethodAMC,false);
%     pricer = DailyWeeklyRiskyPricer_CDUSDLibor3MSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, true);    
    pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRAInverseFloater(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, true);    
     
    pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH, 'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH,'PSANo');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSANo');
%     pricingOutClosed = pricer.computePriceClosedSimple(valueDateH);
    
    callablePrice(idx2) = pricingOutMC1.callable.npv;
    nonCallPrice(idx2) = pricingOutMC1.nonCall.npv;
%     closedPrice(idx2) = pricingOutClosed.pricerInfo.nonCall.npv;
    
%     for idx3 = 1:length(pricingOutMC1.nonCall.cashflow_npv_e)
% %     for idx3 = 1:length(pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e)
%         nonCallPrice_cashflowNPV(idx2, idx3) = pricingOutMC1.nonCall.cashflow_npv_e(idx3);
% %         closedPrice_cashflowNPV(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
% %         
% %         closedPrice_fwdCD(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdCD(idx3);
% %         closedPrice_fwdKTB(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdKTB(idx3);
% %         closedPrice_fwdSpread(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSpread(idx3);
% %         closedPrice_fwdMu(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdMu(idx3);
% %         closedPrice_fwdSigma(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSigma(idx3);
% %                 
%     end
end


aaa =1.0;


In [ ]:

testCD_CDKTB3MSpreadInverseFloater_CK1F_CK1F_20220620_irsDelta.m

clc
clear
valueDate = '2022-06-20';


ktbshift = -0.01;

maxIdx = 0;
zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;

callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% CD-USDLibor3M Spread Range Accrual Pricing start

%RA info
rangeAccrualParams.payCcy = 'KRW';
rangeAccrualParams.startDate = H_Date('2019-08-23');

rangeAccrualParams.maturity = 15;

valueDateH = H_Date(valueDate);

nominal = 10000;
% coupon = 0.033;
coupon = 0.052;

% couponFreq = 1;
couponFreq = 0.25;


% Long: KRW
% Short: USD
tenorL = 0.25;
tenorS = 0.25;

% lowerS = 0.0;

lowerS = -0.001;
upperS = 999;

% 
% lower = -1;
% upper = 0.05;

tenor = 0.25;

lower = -999;
upper = 999;

%resetDate startDate endDate paymentDate
couponScheduleInt = [20190823	20190823	20191123	20191123
20191123	20191123	20200223	20200223
20200223	20200223	20200523	20200523
20200523	20200523	20200823	20200823
20200823	20200823	20201123	20201123
20201123	20201123	20210223	20210223
20210223	20210223	20210523	20210523
20210523	20210523	20210823	20210823
20210823	20210823	20211123	20211123
20211123	20211123	20220223	20220223
20220223	20220223	20220523	20220523
20220523	20220523	20220823	20220823
20220823	20220823	20221123	20221123
20221123	20221123	20230223	20230223
20230223	20230223	20230523	20230523
20230523	20230523	20230823	20230823
20230823	20230823	20231123	20231123
20231123	20231123	20240223	20240223
20240223	20240223	20240523	20240523
20240523	20240523	20240823	20240823
20240823	20240823	20241123	20241123
20241123	20241123	20250223	20250223
20250223	20250223	20250523	20250523
20250523	20250523	20250823	20250823
20250823	20250823	20251123	20251123
20251123	20251123	20260223	20260223
20260223	20260223	20260523	20260523
20260523	20260523	20260823	20260823
20260823	20260823	20261123	20261123
20261123	20261123	20270223	20270223
20270223	20270223	20270523	20270523
20270523	20270523	20270823	20270823
20270823	20270823	20271123	20271123
20271123	20271123	20280223	20280223
20280223	20280223	20280523	20280523
20280523	20280523	20280823	20280823
20280823	20280823	20281123	20281123
20281123	20281123	20290223	20290223
20290223	20290223	20290523	20290523
20290523	20290523	20290823	20290823
20290823	20290823	20291123	20291123
20291123	20291123	20300223	20300223
20300223	20300223	20300523	20300523
20300523	20300523	20300823	20300823
20300823	20300823	20301123	20301123
20301123	20301123	20310223	20310223
20310223	20310223	20310523	20310523
20310523	20310523	20310823	20310823
20310823	20310823	20311123	20311123
20311123	20311123	20320223	20320223
20320223	20320223	20320523	20320523
20320523	20320523	20320823	20320823
20320823	20320823	20321123	20321123
20321123	20321123	20330223	20330223
20330223	20330223	20330523	20330523
20330523	20330523	20330823	20330823
20330823	20330823	20331123	20331123
20331123	20331123	20340223	20340223
20340223	20340223	20340523	20340523
20340523	20340523	20340823	20340823
];

endDate = H_Date(couponScheduleInt(size(couponScheduleInt,1),4));

%% CK1F model param matu setting start
% matu = 15 + 1.0/365.0;
matu = DateDiff(H_Date(couponScheduleInt(end,4)),H_Date(valueDate))/365.0;
% matu = matu -10.0;

%% end

 rangeDays = zeros(60,1);
%     rangeDays(1) = 1;

for i=1:length(rangeDays)
    if DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) >= 0
        rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) + 1;
    end
end


%resetDate startDate endDate paymentDate
callScheduleInt = [20190823	20190823	20190830	20190830
20191116	20191116	20191123	20191123
20200216	20200216	20200223	20200223
20200516	20200516	20200523	20200523
20200816	20200816	20200823	20200823
20201116	20201116	20201123	20201123
20210216	20210216	20210223	20210223
20210516	20210516	20210523	20210523
20210816	20210816	20210823	20210823
20211116	20211116	20211123	20211123
20220216	20220216	20220223	20220223
20220516	20220516	20220523	20220523
20220816	20220816	20220823	20220823
20221116	20221116	20221123	20221123
20230216	20230216	20230223	20230223
20230516	20230516	20230523	20230523
20230816	20230816	20230823	20230823
20231116	20231116	20231123	20231123
20240216	20240216	20240223	20240223
20240516	20240516	20240523	20240523
20240816	20240816	20240823	20240823
20241116	20241116	20241123	20241123
20250216	20250216	20250223	20250223
20250516	20250516	20250523	20250523
20250816	20250816	20250823	20250823
20251116	20251116	20251123	20251123
20260216	20260216	20260223	20260223
20260516	20260516	20260523	20260523
20260816	20260816	20260823	20260823
20261116	20261116	20261123	20261123
20270216	20270216	20270223	20270223
20270516	20270516	20270523	20270523
20270816	20270816	20270823	20270823
20271116	20271116	20271123	20271123
20280216	20280216	20280223	20280223
20280516	20280516	20280523	20280523
20280816	20280816	20280823	20280823
20281116	20281116	20281123	20281123
20290216	20290216	20290223	20290223
20290516	20290516	20290523	20290523
20290816	20290816	20290823	20290823
20291116	20291116	20291123	20291123
20300216	20300216	20300223	20300223
20300516	20300516	20300523	20300523
20300816	20300816	20300823	20300823
20301116	20301116	20301123	20301123
20310216	20310216	20310223	20310223
20310516	20310516	20310523	20310523
20310816	20310816	20310823	20310823
20311116	20311116	20311123	20311123
20320216	20320216	20320223	20320223
20320516	20320516	20320523	20320523
20320816	20320816	20320823	20320823
20321116	20321116	20321123	20321123
20330216	20330216	20330223	20330223
20330516	20330516	20330523	20330523
20330816	20330816	20330823	20330823
20331116	20331116	20331123	20331123
20340216	20340216	20340223	20340223
20340516	20340516	20340523	20340523
];

callYN = ones(60,1);
callYN(1) = 0;

manualScheduleYN = 1;

rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                'tenorS','lower','upper','rangeDays','callYN', ...
                                'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                        {nominal,coupon,couponFreq,tenorL,tenorS,lower,upper,...
                        rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});
% 
% tenorL = 0.25;
% tenorS = 0.25;
% 
% tenor = 0.25;
% 
% lowerS = 0.0;
% upperS = 999;
% 
% 
% lower = 0.0;
% upper = 0.05;

rangeAccrualInfo('lowerS') =lowerS;
rangeAccrualInfo('upperS') =upperS;
rangeAccrualInfo('tenor') = 0.25;


rangeAccrualInfo('couponLowerBound') = -0.05;


rangeAccrualParams.params = rangeAccrualInfo;

% rangeAccrual = SpreadRangeAccrual(rangeAccrualParams);
% rangeAccrual = CDCMSSpreadDualRangeAccrual(rangeAccrualParams);
rangeAccrual = CDCMSSpreadDualRangeAccrualInverseFloater(rangeAccrualParams);

% maxIdx =2.5;

maxIdx = 20;

rmseSimul = zeros(100,1);
for idx2=1:51
    %% KRW Model building

    %% import and construct a zeroCurve
    rawData = [0.002739726	0.999952523
0.25	0.9950684
0.5	0.988031579
0.75	0.979080331
1	0.968776974
1.25	0.959202682
1.5	0.948993277
1.75	0.939322512
2	0.929369492
2.25	0.920482183
2.5	0.911621558
2.75	0.902762032
3	0.893905238
3.25	0.885611153
3.5	0.877424651
3.75	0.869320968
4	0.861299205
4.25	0.853502656
4.5	0.845805585
4.75	0.838202041
5	0.830690747
5.25	0.823531148
5.5	0.816487433
5.75	0.809552209
6	0.802723731
6.25	0.796000289
6.5	0.789380204
6.75	0.782861832
7	0.776443559
7.25	0.770025284
7.5	0.763695404
7.75	0.757455789
8	0.751305075
8.25	0.74524192
8.5	0.73926501
8.75	0.73337305
9	0.727564772
9.25	0.721838929
9.5	0.716194297
9.75	0.710629672
10	0.705143874
10.25	0.699194275
10.5	0.693278485
10.75	0.687420297
11	0.681619095
11.25	0.675874273
11.5	0.670185229
11.75	0.664551369
12	0.658972106
12.25	0.654130635
12.5	0.649391356
12.75	0.644722276
13	0.640122375
13.25	0.635590647
13.5	0.631126108
13.75	0.626727791
14	0.622394747
14.25	0.618126045
14.5	0.613920769
14.75	0.609778022
15	0.605696922
15.25	0.60235689
15.5	0.599139226
15.75	0.595995535
16	0.592924834
16.25	0.589926161
16.5	0.586998583
16.75	0.58414119
17	0.581353097
17.25	0.578633444
17.5	0.575981393
17.75	0.573396131
18	0.570876865
18.25	0.568422827
18.5	0.566033269
18.75	0.563707467
19	0.561444716
19.25	0.559244332
19.5	0.557105652
19.75	0.555028035
20	0.553010856
20.25	0.550157221
20.5	0.547261794
20.75	0.544409494
21	0.541599805
21.25	0.538832219
21.5	0.536106239
21.75	0.533421375
22	0.530777148
22.25	0.528173085
22.5	0.525608723
22.75	0.523083608
23	0.520597293
23.25	0.518149339
23.5	0.515739316
23.75	0.513366801
24	0.511031378
24.25	0.508732641
24.5	0.506470189
24.75	0.50424363
25	0.502052578
25.25	0.498564056
25.5	0.494988089
25.75	0.491434812
26	0.487904104
26.25	0.484395847
26.5	0.48090992
26.75	0.477446204
27	0.474004582
27.25	0.470584935
27.5	0.467187146
27.75	0.463811098
28	0.460456674
28.25	0.457123757
28.5	0.453812233
28.75	0.450521986
29	0.447252902
29.25	0.444004865
29.5	0.440777762
29.75	0.43757148
30	0.434385905
30.25	0.431221981
30.5	0.428081088
30.75	0.424963073
31	0.421867769
    ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

    
%     %simulation testCase
%     for idx=1:size(rawData,1)
%         rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
%     end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    KRWAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];

% rawData = [0.002739726	0.999952523
% 0.25	0.9950684
% 0.5	0.988031579
% 0.75	0.979080331
% 1	0.968776974
% 1.25	0.959202682
% 1.5	0.948993277
% 1.75	0.939322512
% 2	0.929369492
% 2.25	0.920482183
% 2.5	0.911621558
% 2.75	0.902762032
% 3	0.893905238
% 3.25	0.885611153
% 3.5	0.877424651
% 3.75	0.869320968
% 4	0.861299205
% 4.25	0.853502656
% 4.5	0.845805585
% 4.75	0.838202041
% 5	0.830690747
% 5.25	0.823531148
% 5.5	0.816487433
% 5.75	0.809552209
% 6	0.802723731
% 6.25	0.796000289
% 6.5	0.789380204
% 6.75	0.782861832
% 7	0.776443559
% 7.25	0.770025284
% 7.5	0.763695404
% 7.75	0.757455789
% 8	0.751305075
% 8.25	0.74524192
% 8.5	0.73926501
% 8.75	0.73337305
% 9	0.727564772
% 9.25	0.721838929
% 9.5	0.716194297
% 9.75	0.710629672
% 10	0.705143874
% 10.25	0.699194275
% 10.5	0.693278485
% 10.75	0.687420297
% 11	0.681619095
% 11.25	0.675874273
% 11.5	0.670185229
% 11.75	0.664551369
% 12	0.658972106
% 12.25	0.654130635
% 12.5	0.649391356
% 12.75	0.644722276
% 13	0.640122375
% 13.25	0.635590647
% 13.5	0.631126108
% 13.75	0.626727791
% 14	0.622394747
% 14.25	0.618126045
% 14.5	0.613920769
% 14.75	0.609778022
% 15	0.605696922
% 15.25	0.60235689
% 15.5	0.599139226
% 15.75	0.595995535
% 16	0.592924834
% 16.25	0.589926161
% 16.5	0.586998583
% 16.75	0.58414119
% 17	0.581353097
% 17.25	0.578633444
% 17.5	0.575981393
% 17.75	0.573396131
% 18	0.570876865
% 18.25	0.568422827
% 18.5	0.566033269
% 18.75	0.563707467
% 19	0.561444716
% 19.25	0.559244332
% 19.5	0.557105652
% 19.75	0.555028035
% 20	0.553010856
% 20.25	0.550157221
% 20.5	0.547261794
% 20.75	0.544409494
% 21	0.541599805
% 21.25	0.538832219
% 21.5	0.536106239
% 21.75	0.533421375
% 22	0.530777148
% 22.25	0.528173085
% 22.5	0.525608723
% 22.75	0.523083608
% 23	0.520597293
% 23.25	0.518149339
% 23.5	0.515739316
% 23.75	0.513366801
% 24	0.511031378
% 24.25	0.508732641
% 24.5	0.506470189
% 24.75	0.50424363
% 25	0.502052578
% 25.25	0.498564056
% 25.5	0.494988089
% 25.75	0.491434812
% 26	0.487904104
% 26.25	0.484395847
% 26.5	0.48090992
% 26.75	0.477446204
% 27	0.474004582
% 27.25	0.470584935
% 27.5	0.467187146
% 27.75	0.463811098
% 28	0.460456674
% 28.25	0.457123757
% 28.5	0.453812233
% 28.75	0.450521986
% 29	0.447252902
% 29.25	0.444004865
% 29.5	0.440777762
% 29.75	0.43757148
% 30	0.434385905
% 30.25	0.431221981
% 30.5	0.428081088
% 30.75	0.424963073
% 31	0.421867769
%     ];


for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end


% ktbshift
% 
%  %simulation testCase
%     for idx=1:size(rawData,1)
%         rawData(idx,2) = rawData(idx,2) + ktbshift;
%     end


 %simulation testCase
    for idx=1:size(rawData,1)
        rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
    end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);

    KRWAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurfaceNormal



rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.01025373	0.00996465	0.00939453	0.0088449	0.00838736	0.00835816	0.00827411	0.00844853	0.00844853
2	0.00987033	0.00941532	0.00896799	0.00850851	0.00803853	0.00809306	0.00814872	0.00832187	0.00832187
3	0.00965448	0.00917228	0.00868278	0.00815841	0.00766413	0.00768619	0.00766987	0.007827	0.007827
4	0.0086456	0.00835905	0.00803055	0.00773857	0.00743965	0.00747184	0.00750208	0.00760185	0.00760185
5	0.008024	0.00784518	0.00761802	0.00741163	0.00719132	0.00718529	0.00716174	0.0072174	0.0072174
7	0.00777954	0.0075868	0.00743466	0.00726955	0.00710966	0.00717731	0.00726665	0.00728989	0.00728989
10	0.00697384	0.00701303	0.00705259	0.00704171	0.0070555	0.00714878	0.00723997	0.00722954	0.00722954
15	0.00731508	0.00731858	0.00728587	0.00725674	0.0072209	0.00720153	0.00717821	0.0071639	0.0071639
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    KRWAllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a KRWBlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(KRWAllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = KRWBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= KRWBlack.NormalATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
   %% create a CK1FRisky Model with initial modelParams
%     vol_r.tenor = [1;2;3;4;5;7;10;15;20];
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;
%     meanR_r = 0.0;
    
    
    joint_swap_tenor = 1;
%     targetSize = 10;

    targetSize = 15;
    
    freq = 4;
    
    modelParams = containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'},...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
    
%     import irModels.*;
%     import targetFunctions.*;

 %% allFixings
    %% CD91
    tempDate = rangeAccrualParams.startDate;
    irIndexFixing3 = containers.Map({StrDate(tempDate,'str')},{0.025});

    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        irIndexFixing3(StrDate(tempDate,'str')) = 0.025;
    end

    %% fixing info -> let it be the modelParams !
    allFixings = containers.Map({'LIBOR3M'},{irIndexFixing3});
    modelParams('allFixings') = allFixings;
    
    KRWCK1FRisky = CK1FRisky(IRModelRisky(Model(KRWAllMktData, modelParams)));
    
    tic
%     out = KRWLGM2F.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
    % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
%     out = KRWCK1FRisky.CalibrateToATMDiagSwaption(KRWBlack,'global');
    out = KRWCK1FRisky.CalibrateToATMDiagSwaptionNormal(KRWBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        KRWVol_r(idx2,idx3) = out.vol_r.quote(idx3);
        KRWRe(idx2,idx3) = out.re(idx3);
    end
    KRWRMSE(idx2) = out.rmseTotal;
    
    toc
    %% CK1FRisky model building end
    %% KRW model building end

    %% USD model building start
    %% import and construct a zeroCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

 %simulation testCase
    for idx=1:size(rawData,1)
        rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
    end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','USDZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    USDAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];


% rawData = [0.002739726	0.999952523
% 0.25	0.9950684
% 0.5	0.988031579
% 0.75	0.979080331
% 1	0.968776974
% 1.25	0.959202682
% 1.5	0.948993277
% 1.75	0.939322512
% 2	0.929369492
% 2.25	0.920482183
% 2.5	0.911621558
% 2.75	0.902762032
% 3	0.893905238
% 3.25	0.885611153
% 3.5	0.877424651
% 3.75	0.869320968
% 4	0.861299205
% 4.25	0.853502656
% 4.5	0.845805585
% 4.75	0.838202041
% 5	0.830690747
% 5.25	0.823531148
% 5.5	0.816487433
% 5.75	0.809552209
% 6	0.802723731
% 6.25	0.796000289
% 6.5	0.789380204
% 6.75	0.782861832
% 7	0.776443559
% 7.25	0.770025284
% 7.5	0.763695404
% 7.75	0.757455789
% 8	0.751305075
% 8.25	0.74524192
% 8.5	0.73926501
% 8.75	0.73337305
% 9	0.727564772
% 9.25	0.721838929
% 9.5	0.716194297
% 9.75	0.710629672
% 10	0.705143874
% 10.25	0.699194275
% 10.5	0.693278485
% 10.75	0.687420297
% 11	0.681619095
% 11.25	0.675874273
% 11.5	0.670185229
% 11.75	0.664551369
% 12	0.658972106
% 12.25	0.654130635
% 12.5	0.649391356
% 12.75	0.644722276
% 13	0.640122375
% 13.25	0.635590647
% 13.5	0.631126108
% 13.75	0.626727791
% 14	0.622394747
% 14.25	0.618126045
% 14.5	0.613920769
% 14.75	0.609778022
% 15	0.605696922
% 15.25	0.60235689
% 15.5	0.599139226
% 15.75	0.595995535
% 16	0.592924834
% 16.25	0.589926161
% 16.5	0.586998583
% 16.75	0.58414119
% 17	0.581353097
% 17.25	0.578633444
% 17.5	0.575981393
% 17.75	0.573396131
% 18	0.570876865
% 18.25	0.568422827
% 18.5	0.566033269
% 18.75	0.563707467
% 19	0.561444716
% 19.25	0.559244332
% 19.5	0.557105652
% 19.75	0.555028035
% 20	0.553010856
% 20.25	0.550157221
% 20.5	0.547261794
% 20.75	0.544409494
% 21	0.541599805
% 21.25	0.538832219
% 21.5	0.536106239
% 21.75	0.533421375
% 22	0.530777148
% 22.25	0.528173085
% 22.5	0.525608723
% 22.75	0.523083608
% 23	0.520597293
% 23.25	0.518149339
% 23.5	0.515739316
% 23.75	0.513366801
% 24	0.511031378
% 24.25	0.508732641
% 24.5	0.506470189
% 24.75	0.50424363
% 25	0.502052578
% 25.25	0.498564056
% 25.5	0.494988089
% 25.75	0.491434812
% 26	0.487904104
% 26.25	0.484395847
% 26.5	0.48090992
% 26.75	0.477446204
% 27	0.474004582
% 27.25	0.470584935
% 27.5	0.467187146
% 27.75	0.463811098
% 28	0.460456674
% 28.25	0.457123757
% 28.5	0.453812233
% 28.75	0.450521986
% 29	0.447252902
% 29.25	0.444004865
% 29.5	0.440777762
% 29.75	0.43757148
% 30	0.434385905
% 30.25	0.431221981
% 30.5	0.428081088
% 30.75	0.424963073
% 31	0.421867769
%     ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

 %simulation testCase
    for idx=1:size(rawData,1)
        rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
    end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);
    USDAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
% GIROPremiumToNormalVol From LIB
% rawDataSwaption = [0	1	2	3	4	5	7	10	12	15	20
% 1	0.0041	0.0045	0.0052	0.0061	0.0066	0.0071	0.0077	0.0080	0.0081	0.0082
% 2	0.0050	0.0057	0.0064	0.0066	0.0068	0.0071	0.0074	0.0077	0.0077	0.0078
% 3	0.0064	0.0069	0.0068	0.0068	0.0068	0.0072	0.0076	0.0078	0.0079	0.0079
% 4	0.0076	0.0071	0.0071	0.0070	0.0071	0.0072	0.0077	0.0081	0.0081	0.0081
% 5	0.0069	0.0070	0.0070	0.0071	0.0071	0.0070	0.0076	0.0083	0.0084	0.0083
% 7	0.0073	0.0074	0.0075	0.0072	0.0070	0.0075	0.0087	0.0096	0.0096	0.0095
% 10	0.0066	0.0066	0.0071	0.0074	0.0077	0.0092	0.0101	0.0102	0.0102	0.0102
% 15	0.0103	0.0105	0.0107	0.0109	0.0111	0.0108	0.0103	0.0102	0.0102	0.0101
% 20	0.0088	0.0089	0.0090	0.0091	0.0091	0.0091	0.0090	0.0089	0.0089	0.0089
%     ];
% 
rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.01025373	0.00996465	0.00939453	0.0088449	0.00838736	0.00835816	0.00827411	0.00844853	0.00844853
2	0.00987033	0.00941532	0.00896799	0.00850851	0.00803853	0.00809306	0.00814872	0.00832187	0.00832187
3	0.00965448	0.00917228	0.00868278	0.00815841	0.00766413	0.00768619	0.00766987	0.007827	0.007827
4	0.0086456	0.00835905	0.00803055	0.00773857	0.00743965	0.00747184	0.00750208	0.00760185	0.00760185
5	0.008024	0.00784518	0.00761802	0.00741163	0.00719132	0.00718529	0.00716174	0.0072174	0.0072174
7	0.00777954	0.0075868	0.00743466	0.00726955	0.00710966	0.00717731	0.00726665	0.00728989	0.00728989
10	0.00697384	0.00701303	0.00705259	0.00704171	0.0070555	0.00714878	0.00723997	0.00722954	0.00722954
15	0.00731508	0.00731858	0.00728587	0.00725674	0.0072209	0.00720153	0.00717821	0.0071639	0.0071639
    ];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'},...
                            {rawDataSwaption,valueDate,'USD','USDSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    USDAllMktData('swaptionVol') = swaptionVolMktData;
    
   %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    USDBlack = IRBlack(IRModel(Model(USDAllMktData,modelParams))); 
    
    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);
    
    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = USDBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= USDBlack.BlackATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = USDBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
    %% create a CK1FModel with initial modelParams
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;

%     matu = 15 + 1.0/365.0;
    joint_swap_tenor = 1;
    targetSize = 15;
    freq = 4;
    
    modelParamsUSD= containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'}, ...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
                        
     %% allFixings
    %% CD91
    tempDate = rangeAccrualParams.startDate;
    irIndexFixing3USD = containers.Map({StrDate(tempDate,'str')},{0.025});

    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        irIndexFixing3USD(StrDate(tempDate,'str')) = 0.025;
    end

    %% fixing info -> let it be the modelParams !
    allFixingsUSD = containers.Map({'LIBOR3M'},{irIndexFixing3USD});
    modelParamsUSD('allFixings') = allFixingsUSD;
    
    USDCK1F = CK1F(IRModel(Model(USDAllMktData, modelParamsUSD)));
    
    tic
    outUSD = USDCK1F.CalibrateToATMDiagSwaptionNormal(USDBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        USDVol_r(idx2,idx3) = outUSD.vol_r.quote(idx3);
        USDRe(idx2,idx3) = outUSD.re(idx3);
    end
    USDRMSE(idx2) = outUSD.rmseTotal;
    
    toc
    %% USD model building end

    %% USDKRW fx model buidling start
    rawData = [0.083333333	0.0
0.25	0.0
0.5	0.0
1	0.0
2	0.0
3	0.0
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','USDKRWATMTermVol'});
    impliedTermVolMktData = MktData(mktData);
%     impliedTermVole = FXVolSurface(impliedTermVolMktData);
%     impliedVolSurface.params('moneyNessType') = 'deltaFwdMoneyNess';
    USDKRWAllMktData =containers.Map('impliedTermVol',impliedTermVolMktData);
%     AllMktData('impliedVolSurface') = impliedTermVolMktData;

    %% create  an FXBlackModel to calculate market PlainVanilla Options
%     modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});
%     USDKRWBlack = FXBlack(FXModel(Model(USDKRWAllMktData,modelParams))); 
    %% USDKRW fx model building end

    %% MultiAssetModel building start
    multiAssetMktData = containers.Map({'KRW','USD','USDKRW'}, ...
                            {KRWAllMktData, USDAllMktData, USDKRWAllMktData});

    % KRW,KTB                    
%     correlationMatrix = ones(2,2);
%     correlationMatrix(1,2) = 0.3;
%     correlationMatrix(2,1) = correlationMatrix(1,2);

    % KRW, USD & USDKRW
    correlationMatrix = ones(3, 3);
    correlationMatrix(1,2) = 0.845;
%     correlationMatrix(1,2) = 0.6;
    
%     correlationMatrix(1,2) = 0.7;
    
    correlationMatrix(1,3) = 0.0;
    correlationMatrix(2,3) = 0.0;
    correlationMatrix(2,1) = correlationMatrix(1,2);
    correlationMatrix(3,1) = correlationMatrix(1,3);
    correlationMatrix(3,2) = correlationMatrix(2,3);
    
    multiAssetMktData('correlationMatrix') = correlationMatrix;
    
    modelNameMap = containers.Map({'KRW','USD'},{KRWCK1FRisky, USDCK1F});
    modelParams = containers.Map({'modelNameMap'}, {modelNameMap});
    % correlationNames = {'KRW','USD'};
    modelParams('stochasticModelNames') = {'KRW','USD'};
    modelParams('correlationMatrix') = correlationMatrix;
    modelParams('refModelName') = 'KRW';
    % modelParams('useQuasiRandomYN') = 'true';
    modelParams('useQuasiRandomYN') = 'false';
    modelParams('fxVolCalibrated') = true;
    
    KRWCK1FRiskyUSDCK1F = MultiAssetModelCK1FCK1F(multiAssetMktData, modelParams);
    %% MultiAssetModel building end
   
    %% NumMethod Init Start
    numOfRandom = 40000;
    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                            {'AMC', 7, numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);
    %% NumMethod Init End
    
    % weekly step calculator
    % pricer = Pricer(rangeAccrual,KRWLGM,numMethodAMC);
    % pricingOut = pricer.computePrice(valueDate);
    
    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    % pricer = DailyWeeklyRiskyPricer_CDKTBSpreadRA(rangeAccrual,KRWLGM1FRiskyUSDLGM1F,numMethodAMC,false);
%     pricer = DailyWeeklyRiskyPricer_CDUSDLibor3MSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, true);    
    pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRAInverseFloater(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, true);    
     
    pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH, 'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH,'PSANo');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSANo');
%     pricingOutClosed = pricer.computePriceClosedSimple(valueDateH);
    
    callablePrice(idx2) = pricingOutMC1.callable.npv;
    nonCallPrice(idx2) = pricingOutMC1.nonCall.npv;
%     closedPrice(idx2) = pricingOutClosed.pricerInfo.nonCall.npv;
    
%     for idx3 = 1:length(pricingOutMC1.nonCall.cashflow_npv_e)
% %     for idx3 = 1:length(pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e)
%         nonCallPrice_cashflowNPV(idx2, idx3) = pricingOutMC1.nonCall.cashflow_npv_e(idx3);
% %         closedPrice_cashflowNPV(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
% %         
% %         closedPrice_fwdCD(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdCD(idx3);
% %         closedPrice_fwdKTB(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdKTB(idx3);
% %         closedPrice_fwdSpread(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSpread(idx3);
% %         closedPrice_fwdMu(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdMu(idx3);
% %         closedPrice_fwdSigma(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSigma(idx3);
% %                 
%     end
end

aaa =1.0;


In [ ]:
testCD_CDKTB3MSpreadInverseFloater_CK1F_CK1F_20220620.m

clc
clear
valueDate = '2022-06-20';

maxIdx = 0;
zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;

callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% CD-USDLibor3M Spread Range Accrual Pricing start

%RA info
rangeAccrualParams.payCcy = 'KRW';
rangeAccrualParams.startDate = H_Date('2019-08-23');

rangeAccrualParams.maturity = 15;

valueDateH = H_Date(valueDate);

nominal = 10000;
% coupon = 0.033;
coupon = 0.052;

% couponFreq = 1;
couponFreq = 0.25;


% Long: KRW
% Short: USD
tenorL = 0.25;
tenorS = 0.25;

% lowerS = 0.0;

lowerS = -0.001;
upperS = 999;

% 
% lower = -1;
% upper = 0.05;

tenor = 0.25;

lower = -999;
upper = 999;

%resetDate startDate endDate paymentDate
couponScheduleInt = [20190823	20190823	20191123	20191123
20191123	20191123	20200223	20200223
20200223	20200223	20200523	20200523
20200523	20200523	20200823	20200823
20200823	20200823	20201123	20201123
20201123	20201123	20210223	20210223
20210223	20210223	20210523	20210523
20210523	20210523	20210823	20210823
20210823	20210823	20211123	20211123
20211123	20211123	20220223	20220223
20220223	20220223	20220523	20220523
20220523	20220523	20220823	20220823
20220823	20220823	20221123	20221123
20221123	20221123	20230223	20230223
20230223	20230223	20230523	20230523
20230523	20230523	20230823	20230823
20230823	20230823	20231123	20231123
20231123	20231123	20240223	20240223
20240223	20240223	20240523	20240523
20240523	20240523	20240823	20240823
20240823	20240823	20241123	20241123
20241123	20241123	20250223	20250223
20250223	20250223	20250523	20250523
20250523	20250523	20250823	20250823
20250823	20250823	20251123	20251123
20251123	20251123	20260223	20260223
20260223	20260223	20260523	20260523
20260523	20260523	20260823	20260823
20260823	20260823	20261123	20261123
20261123	20261123	20270223	20270223
20270223	20270223	20270523	20270523
20270523	20270523	20270823	20270823
20270823	20270823	20271123	20271123
20271123	20271123	20280223	20280223
20280223	20280223	20280523	20280523
20280523	20280523	20280823	20280823
20280823	20280823	20281123	20281123
20281123	20281123	20290223	20290223
20290223	20290223	20290523	20290523
20290523	20290523	20290823	20290823
20290823	20290823	20291123	20291123
20291123	20291123	20300223	20300223
20300223	20300223	20300523	20300523
20300523	20300523	20300823	20300823
20300823	20300823	20301123	20301123
20301123	20301123	20310223	20310223
20310223	20310223	20310523	20310523
20310523	20310523	20310823	20310823
20310823	20310823	20311123	20311123
20311123	20311123	20320223	20320223
20320223	20320223	20320523	20320523
20320523	20320523	20320823	20320823
20320823	20320823	20321123	20321123
20321123	20321123	20330223	20330223
20330223	20330223	20330523	20330523
20330523	20330523	20330823	20330823
20330823	20330823	20331123	20331123
20331123	20331123	20340223	20340223
20340223	20340223	20340523	20340523
20340523	20340523	20340823	20340823
];

endDate = H_Date(couponScheduleInt(size(couponScheduleInt,1),4));

%% CK1F model param matu setting start
% matu = 15 + 1.0/365.0;
matu = DateDiff(H_Date(couponScheduleInt(end,4)),H_Date(valueDate))/365.0;
% matu = matu -10.0;

%% end

 rangeDays = zeros(60,1);
%     rangeDays(1) = 1;

for i=1:length(rangeDays)
    if DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) >= 0
        rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) + 1;
    end
end


%resetDate startDate endDate paymentDate
callScheduleInt = [20190823	20190823	20190830	20190830
20191116	20191116	20191123	20191123
20200216	20200216	20200223	20200223
20200516	20200516	20200523	20200523
20200816	20200816	20200823	20200823
20201116	20201116	20201123	20201123
20210216	20210216	20210223	20210223
20210516	20210516	20210523	20210523
20210816	20210816	20210823	20210823
20211116	20211116	20211123	20211123
20220216	20220216	20220223	20220223
20220516	20220516	20220523	20220523
20220816	20220816	20220823	20220823
20221116	20221116	20221123	20221123
20230216	20230216	20230223	20230223
20230516	20230516	20230523	20230523
20230816	20230816	20230823	20230823
20231116	20231116	20231123	20231123
20240216	20240216	20240223	20240223
20240516	20240516	20240523	20240523
20240816	20240816	20240823	20240823
20241116	20241116	20241123	20241123
20250216	20250216	20250223	20250223
20250516	20250516	20250523	20250523
20250816	20250816	20250823	20250823
20251116	20251116	20251123	20251123
20260216	20260216	20260223	20260223
20260516	20260516	20260523	20260523
20260816	20260816	20260823	20260823
20261116	20261116	20261123	20261123
20270216	20270216	20270223	20270223
20270516	20270516	20270523	20270523
20270816	20270816	20270823	20270823
20271116	20271116	20271123	20271123
20280216	20280216	20280223	20280223
20280516	20280516	20280523	20280523
20280816	20280816	20280823	20280823
20281116	20281116	20281123	20281123
20290216	20290216	20290223	20290223
20290516	20290516	20290523	20290523
20290816	20290816	20290823	20290823
20291116	20291116	20291123	20291123
20300216	20300216	20300223	20300223
20300516	20300516	20300523	20300523
20300816	20300816	20300823	20300823
20301116	20301116	20301123	20301123
20310216	20310216	20310223	20310223
20310516	20310516	20310523	20310523
20310816	20310816	20310823	20310823
20311116	20311116	20311123	20311123
20320216	20320216	20320223	20320223
20320516	20320516	20320523	20320523
20320816	20320816	20320823	20320823
20321116	20321116	20321123	20321123
20330216	20330216	20330223	20330223
20330516	20330516	20330523	20330523
20330816	20330816	20330823	20330823
20331116	20331116	20331123	20331123
20340216	20340216	20340223	20340223
20340516	20340516	20340523	20340523
];

callYN = ones(60,1);
callYN(1) = 0;

manualScheduleYN = 1;

rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                'tenorS','lower','upper','rangeDays','callYN', ...
                                'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                        {nominal,coupon,couponFreq,tenorL,tenorS,lower,upper,...
                        rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});
% 
% tenorL = 0.25;
% tenorS = 0.25;
% 
% tenor = 0.25;
% 
% lowerS = 0.0;
% upperS = 999;
% 
% 
% lower = 0.0;
% upper = 0.05;

rangeAccrualInfo('lowerS') =lowerS;
rangeAccrualInfo('upperS') =upperS;
rangeAccrualInfo('tenor') = 0.25;


rangeAccrualInfo('couponLowerBound') = -0.05;


rangeAccrualParams.params = rangeAccrualInfo;

% rangeAccrual = SpreadRangeAccrual(rangeAccrualParams);
% rangeAccrual = CDCMSSpreadDualRangeAccrual(rangeAccrualParams);
rangeAccrual = CDCMSSpreadDualRangeAccrualInverseFloater(rangeAccrualParams);

% maxIdx =2.5;

maxIdx = 10;

rmseSimul = zeros(100,1);
for idx2=1:21
    %% KRW Model building

    %% import and construct a zeroCurve
    rawData = [0.002739726	0.999952523
0.25	0.9950684
0.5	0.988031579
0.75	0.979080331
1	0.968776974
1.25	0.959202682
1.5	0.948993277
1.75	0.939322512
2	0.929369492
2.25	0.920482183
2.5	0.911621558
2.75	0.902762032
3	0.893905238
3.25	0.885611153
3.5	0.877424651
3.75	0.869320968
4	0.861299205
4.25	0.853502656
4.5	0.845805585
4.75	0.838202041
5	0.830690747
5.25	0.823531148
5.5	0.816487433
5.75	0.809552209
6	0.802723731
6.25	0.796000289
6.5	0.789380204
6.75	0.782861832
7	0.776443559
7.25	0.770025284
7.5	0.763695404
7.75	0.757455789
8	0.751305075
8.25	0.74524192
8.5	0.73926501
8.75	0.73337305
9	0.727564772
9.25	0.721838929
9.5	0.716194297
9.75	0.710629672
10	0.705143874
10.25	0.699194275
10.5	0.693278485
10.75	0.687420297
11	0.681619095
11.25	0.675874273
11.5	0.670185229
11.75	0.664551369
12	0.658972106
12.25	0.654130635
12.5	0.649391356
12.75	0.644722276
13	0.640122375
13.25	0.635590647
13.5	0.631126108
13.75	0.626727791
14	0.622394747
14.25	0.618126045
14.5	0.613920769
14.75	0.609778022
15	0.605696922
15.25	0.60235689
15.5	0.599139226
15.75	0.595995535
16	0.592924834
16.25	0.589926161
16.5	0.586998583
16.75	0.58414119
17	0.581353097
17.25	0.578633444
17.5	0.575981393
17.75	0.573396131
18	0.570876865
18.25	0.568422827
18.5	0.566033269
18.75	0.563707467
19	0.561444716
19.25	0.559244332
19.5	0.557105652
19.75	0.555028035
20	0.553010856
20.25	0.550157221
20.5	0.547261794
20.75	0.544409494
21	0.541599805
21.25	0.538832219
21.5	0.536106239
21.75	0.533421375
22	0.530777148
22.25	0.528173085
22.5	0.525608723
22.75	0.523083608
23	0.520597293
23.25	0.518149339
23.5	0.515739316
23.75	0.513366801
24	0.511031378
24.25	0.508732641
24.5	0.506470189
24.75	0.50424363
25	0.502052578
25.25	0.498564056
25.5	0.494988089
25.75	0.491434812
26	0.487904104
26.25	0.484395847
26.5	0.48090992
26.75	0.477446204
27	0.474004582
27.25	0.470584935
27.5	0.467187146
27.75	0.463811098
28	0.460456674
28.25	0.457123757
28.5	0.453812233
28.75	0.450521986
29	0.447252902
29.25	0.444004865
29.5	0.440777762
29.75	0.43757148
30	0.434385905
30.25	0.431221981
30.5	0.428081088
30.75	0.424963073
31	0.421867769
    ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

    
    %simulation testCase
    for idx=1:size(rawData,1)
        rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    KRWAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];

% rawData = [0.002739726	0.999952523
% 0.25	0.9950684
% 0.5	0.988031579
% 0.75	0.979080331
% 1	0.968776974
% 1.25	0.959202682
% 1.5	0.948993277
% 1.75	0.939322512
% 2	0.929369492
% 2.25	0.920482183
% 2.5	0.911621558
% 2.75	0.902762032
% 3	0.893905238
% 3.25	0.885611153
% 3.5	0.877424651
% 3.75	0.869320968
% 4	0.861299205
% 4.25	0.853502656
% 4.5	0.845805585
% 4.75	0.838202041
% 5	0.830690747
% 5.25	0.823531148
% 5.5	0.816487433
% 5.75	0.809552209
% 6	0.802723731
% 6.25	0.796000289
% 6.5	0.789380204
% 6.75	0.782861832
% 7	0.776443559
% 7.25	0.770025284
% 7.5	0.763695404
% 7.75	0.757455789
% 8	0.751305075
% 8.25	0.74524192
% 8.5	0.73926501
% 8.75	0.73337305
% 9	0.727564772
% 9.25	0.721838929
% 9.5	0.716194297
% 9.75	0.710629672
% 10	0.705143874
% 10.25	0.699194275
% 10.5	0.693278485
% 10.75	0.687420297
% 11	0.681619095
% 11.25	0.675874273
% 11.5	0.670185229
% 11.75	0.664551369
% 12	0.658972106
% 12.25	0.654130635
% 12.5	0.649391356
% 12.75	0.644722276
% 13	0.640122375
% 13.25	0.635590647
% 13.5	0.631126108
% 13.75	0.626727791
% 14	0.622394747
% 14.25	0.618126045
% 14.5	0.613920769
% 14.75	0.609778022
% 15	0.605696922
% 15.25	0.60235689
% 15.5	0.599139226
% 15.75	0.595995535
% 16	0.592924834
% 16.25	0.589926161
% 16.5	0.586998583
% 16.75	0.58414119
% 17	0.581353097
% 17.25	0.578633444
% 17.5	0.575981393
% 17.75	0.573396131
% 18	0.570876865
% 18.25	0.568422827
% 18.5	0.566033269
% 18.75	0.563707467
% 19	0.561444716
% 19.25	0.559244332
% 19.5	0.557105652
% 19.75	0.555028035
% 20	0.553010856
% 20.25	0.550157221
% 20.5	0.547261794
% 20.75	0.544409494
% 21	0.541599805
% 21.25	0.538832219
% 21.5	0.536106239
% 21.75	0.533421375
% 22	0.530777148
% 22.25	0.528173085
% 22.5	0.525608723
% 22.75	0.523083608
% 23	0.520597293
% 23.25	0.518149339
% 23.5	0.515739316
% 23.75	0.513366801
% 24	0.511031378
% 24.25	0.508732641
% 24.5	0.506470189
% 24.75	0.50424363
% 25	0.502052578
% 25.25	0.498564056
% 25.5	0.494988089
% 25.75	0.491434812
% 26	0.487904104
% 26.25	0.484395847
% 26.5	0.48090992
% 26.75	0.477446204
% 27	0.474004582
% 27.25	0.470584935
% 27.5	0.467187146
% 27.75	0.463811098
% 28	0.460456674
% 28.25	0.457123757
% 28.5	0.453812233
% 28.75	0.450521986
% 29	0.447252902
% 29.25	0.444004865
% 29.5	0.440777762
% 29.75	0.43757148
% 30	0.434385905
% 30.25	0.431221981
% 30.5	0.428081088
% 30.75	0.424963073
% 31	0.421867769
%     ];


for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

%  %simulation testCase
%     for idx=1:size(rawData,1)
%         rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
%     end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);

    KRWAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurfaceNormal



rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.01025373	0.00996465	0.00939453	0.0088449	0.00838736	0.00835816	0.00827411	0.00844853	0.00844853
2	0.00987033	0.00941532	0.00896799	0.00850851	0.00803853	0.00809306	0.00814872	0.00832187	0.00832187
3	0.00965448	0.00917228	0.00868278	0.00815841	0.00766413	0.00768619	0.00766987	0.007827	0.007827
4	0.0086456	0.00835905	0.00803055	0.00773857	0.00743965	0.00747184	0.00750208	0.00760185	0.00760185
5	0.008024	0.00784518	0.00761802	0.00741163	0.00719132	0.00718529	0.00716174	0.0072174	0.0072174
7	0.00777954	0.0075868	0.00743466	0.00726955	0.00710966	0.00717731	0.00726665	0.00728989	0.00728989
10	0.00697384	0.00701303	0.00705259	0.00704171	0.0070555	0.00714878	0.00723997	0.00722954	0.00722954
15	0.00731508	0.00731858	0.00728587	0.00725674	0.0072209	0.00720153	0.00717821	0.0071639	0.0071639
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    KRWAllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a KRWBlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(KRWAllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = KRWBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= KRWBlack.NormalATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
   %% create a CK1FRisky Model with initial modelParams
%     vol_r.tenor = [1;2;3;4;5;7;10;15;20];
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;
%     meanR_r = 0.0;
    
    
    joint_swap_tenor = 1;
%     targetSize = 10;

    targetSize = 15;
    
    freq = 4;
    
    modelParams = containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'},...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
    
%     import irModels.*;
%     import targetFunctions.*;

 %% allFixings
    %% CD91
    tempDate = rangeAccrualParams.startDate;
    irIndexFixing3 = containers.Map({StrDate(tempDate,'str')},{0.025});

    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        irIndexFixing3(StrDate(tempDate,'str')) = 0.025;
    end

    %% fixing info -> let it be the modelParams !
    allFixings = containers.Map({'LIBOR3M'},{irIndexFixing3});
    modelParams('allFixings') = allFixings;
    
    KRWCK1FRisky = CK1FRisky(IRModelRisky(Model(KRWAllMktData, modelParams)));
    
    tic
%     out = KRWLGM2F.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
    % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
%     out = KRWCK1FRisky.CalibrateToATMDiagSwaption(KRWBlack,'global');
    out = KRWCK1FRisky.CalibrateToATMDiagSwaptionNormal(KRWBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        KRWVol_r(idx2,idx3) = out.vol_r.quote(idx3);
        KRWRe(idx2,idx3) = out.re(idx3);
    end
    KRWRMSE(idx2) = out.rmseTotal;
    
    toc
    %% CK1FRisky model building end
    %% KRW model building end

    %% USD model building start
    %% import and construct a zeroCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

%  %simulation testCase
%     for idx=1:size(rawData,1)
%         rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
%     end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','USDZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    USDAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];


% rawData = [0.002739726	0.999952523
% 0.25	0.9950684
% 0.5	0.988031579
% 0.75	0.979080331
% 1	0.968776974
% 1.25	0.959202682
% 1.5	0.948993277
% 1.75	0.939322512
% 2	0.929369492
% 2.25	0.920482183
% 2.5	0.911621558
% 2.75	0.902762032
% 3	0.893905238
% 3.25	0.885611153
% 3.5	0.877424651
% 3.75	0.869320968
% 4	0.861299205
% 4.25	0.853502656
% 4.5	0.845805585
% 4.75	0.838202041
% 5	0.830690747
% 5.25	0.823531148
% 5.5	0.816487433
% 5.75	0.809552209
% 6	0.802723731
% 6.25	0.796000289
% 6.5	0.789380204
% 6.75	0.782861832
% 7	0.776443559
% 7.25	0.770025284
% 7.5	0.763695404
% 7.75	0.757455789
% 8	0.751305075
% 8.25	0.74524192
% 8.5	0.73926501
% 8.75	0.73337305
% 9	0.727564772
% 9.25	0.721838929
% 9.5	0.716194297
% 9.75	0.710629672
% 10	0.705143874
% 10.25	0.699194275
% 10.5	0.693278485
% 10.75	0.687420297
% 11	0.681619095
% 11.25	0.675874273
% 11.5	0.670185229
% 11.75	0.664551369
% 12	0.658972106
% 12.25	0.654130635
% 12.5	0.649391356
% 12.75	0.644722276
% 13	0.640122375
% 13.25	0.635590647
% 13.5	0.631126108
% 13.75	0.626727791
% 14	0.622394747
% 14.25	0.618126045
% 14.5	0.613920769
% 14.75	0.609778022
% 15	0.605696922
% 15.25	0.60235689
% 15.5	0.599139226
% 15.75	0.595995535
% 16	0.592924834
% 16.25	0.589926161
% 16.5	0.586998583
% 16.75	0.58414119
% 17	0.581353097
% 17.25	0.578633444
% 17.5	0.575981393
% 17.75	0.573396131
% 18	0.570876865
% 18.25	0.568422827
% 18.5	0.566033269
% 18.75	0.563707467
% 19	0.561444716
% 19.25	0.559244332
% 19.5	0.557105652
% 19.75	0.555028035
% 20	0.553010856
% 20.25	0.550157221
% 20.5	0.547261794
% 20.75	0.544409494
% 21	0.541599805
% 21.25	0.538832219
% 21.5	0.536106239
% 21.75	0.533421375
% 22	0.530777148
% 22.25	0.528173085
% 22.5	0.525608723
% 22.75	0.523083608
% 23	0.520597293
% 23.25	0.518149339
% 23.5	0.515739316
% 23.75	0.513366801
% 24	0.511031378
% 24.25	0.508732641
% 24.5	0.506470189
% 24.75	0.50424363
% 25	0.502052578
% 25.25	0.498564056
% 25.5	0.494988089
% 25.75	0.491434812
% 26	0.487904104
% 26.25	0.484395847
% 26.5	0.48090992
% 26.75	0.477446204
% 27	0.474004582
% 27.25	0.470584935
% 27.5	0.467187146
% 27.75	0.463811098
% 28	0.460456674
% 28.25	0.457123757
% 28.5	0.453812233
% 28.75	0.450521986
% 29	0.447252902
% 29.25	0.444004865
% 29.5	0.440777762
% 29.75	0.43757148
% 30	0.434385905
% 30.25	0.431221981
% 30.5	0.428081088
% 30.75	0.424963073
% 31	0.421867769
%     ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

%  %simulation testCase
%     for idx=1:size(rawData,1)
%         rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
%     end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);
    USDAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
% GIROPremiumToNormalVol From LIB
% rawDataSwaption = [0	1	2	3	4	5	7	10	12	15	20
% 1	0.0041	0.0045	0.0052	0.0061	0.0066	0.0071	0.0077	0.0080	0.0081	0.0082
% 2	0.0050	0.0057	0.0064	0.0066	0.0068	0.0071	0.0074	0.0077	0.0077	0.0078
% 3	0.0064	0.0069	0.0068	0.0068	0.0068	0.0072	0.0076	0.0078	0.0079	0.0079
% 4	0.0076	0.0071	0.0071	0.0070	0.0071	0.0072	0.0077	0.0081	0.0081	0.0081
% 5	0.0069	0.0070	0.0070	0.0071	0.0071	0.0070	0.0076	0.0083	0.0084	0.0083
% 7	0.0073	0.0074	0.0075	0.0072	0.0070	0.0075	0.0087	0.0096	0.0096	0.0095
% 10	0.0066	0.0066	0.0071	0.0074	0.0077	0.0092	0.0101	0.0102	0.0102	0.0102
% 15	0.0103	0.0105	0.0107	0.0109	0.0111	0.0108	0.0103	0.0102	0.0102	0.0101
% 20	0.0088	0.0089	0.0090	0.0091	0.0091	0.0091	0.0090	0.0089	0.0089	0.0089
%     ];
% 
rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.01025373	0.00996465	0.00939453	0.0088449	0.00838736	0.00835816	0.00827411	0.00844853	0.00844853
2	0.00987033	0.00941532	0.00896799	0.00850851	0.00803853	0.00809306	0.00814872	0.00832187	0.00832187
3	0.00965448	0.00917228	0.00868278	0.00815841	0.00766413	0.00768619	0.00766987	0.007827	0.007827
4	0.0086456	0.00835905	0.00803055	0.00773857	0.00743965	0.00747184	0.00750208	0.00760185	0.00760185
5	0.008024	0.00784518	0.00761802	0.00741163	0.00719132	0.00718529	0.00716174	0.0072174	0.0072174
7	0.00777954	0.0075868	0.00743466	0.00726955	0.00710966	0.00717731	0.00726665	0.00728989	0.00728989
10	0.00697384	0.00701303	0.00705259	0.00704171	0.0070555	0.00714878	0.00723997	0.00722954	0.00722954
15	0.00731508	0.00731858	0.00728587	0.00725674	0.0072209	0.00720153	0.00717821	0.0071639	0.0071639
    ];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'},...
                            {rawDataSwaption,valueDate,'USD','USDSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    USDAllMktData('swaptionVol') = swaptionVolMktData;
    
   %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    USDBlack = IRBlack(IRModel(Model(USDAllMktData,modelParams))); 
    
    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);
    
    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = USDBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= USDBlack.BlackATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = USDBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
    %% create a CK1FModel with initial modelParams
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;

%     matu = 15 + 1.0/365.0;
    joint_swap_tenor = 1;
    targetSize = 15;
    freq = 4;
    
    modelParamsUSD= containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'}, ...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
                        
     %% allFixings
    %% CD91
    tempDate = rangeAccrualParams.startDate;
    irIndexFixing3USD = containers.Map({StrDate(tempDate,'str')},{0.025});

    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        irIndexFixing3USD(StrDate(tempDate,'str')) = 0.025;
    end

    %% fixing info -> let it be the modelParams !
    allFixingsUSD = containers.Map({'LIBOR3M'},{irIndexFixing3USD});
    modelParamsUSD('allFixings') = allFixingsUSD;
    
    USDCK1F = CK1F(IRModel(Model(USDAllMktData, modelParamsUSD)));
    
    tic
    outUSD = USDCK1F.CalibrateToATMDiagSwaptionNormal(USDBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        USDVol_r(idx2,idx3) = outUSD.vol_r.quote(idx3);
        USDRe(idx2,idx3) = outUSD.re(idx3);
    end
    USDRMSE(idx2) = outUSD.rmseTotal;
    
    toc
    %% USD model building end

    %% USDKRW fx model buidling start
    rawData = [0.083333333	0.0
0.25	0.0
0.5	0.0
1	0.0
2	0.0
3	0.0
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','USDKRWATMTermVol'});
    impliedTermVolMktData = MktData(mktData);
%     impliedTermVole = FXVolSurface(impliedTermVolMktData);
%     impliedVolSurface.params('moneyNessType') = 'deltaFwdMoneyNess';
    USDKRWAllMktData =containers.Map('impliedTermVol',impliedTermVolMktData);
%     AllMktData('impliedVolSurface') = impliedTermVolMktData;

    %% create  an FXBlackModel to calculate market PlainVanilla Options
%     modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});
%     USDKRWBlack = FXBlack(FXModel(Model(USDKRWAllMktData,modelParams))); 
    %% USDKRW fx model building end

    %% MultiAssetModel building start
    multiAssetMktData = containers.Map({'KRW','USD','USDKRW'}, ...
                            {KRWAllMktData, USDAllMktData, USDKRWAllMktData});

    % KRW,KTB                    
%     correlationMatrix = ones(2,2);
%     correlationMatrix(1,2) = 0.3;
%     correlationMatrix(2,1) = correlationMatrix(1,2);

    % KRW, USD & USDKRW
    correlationMatrix = ones(3, 3);
    correlationMatrix(1,2) = 0.845;
%     correlationMatrix(1,2) = 0.6;
    
%     correlationMatrix(1,2) = 0.7;
    
    correlationMatrix(1,3) = 0.0;
    correlationMatrix(2,3) = 0.0;
    correlationMatrix(2,1) = correlationMatrix(1,2);
    correlationMatrix(3,1) = correlationMatrix(1,3);
    correlationMatrix(3,2) = correlationMatrix(2,3);
    
    multiAssetMktData('correlationMatrix') = correlationMatrix;
    
    modelNameMap = containers.Map({'KRW','USD'},{KRWCK1FRisky, USDCK1F});
    modelParams = containers.Map({'modelNameMap'}, {modelNameMap});
    % correlationNames = {'KRW','USD'};
    modelParams('stochasticModelNames') = {'KRW','USD'};
    modelParams('correlationMatrix') = correlationMatrix;
    modelParams('refModelName') = 'KRW';
    % modelParams('useQuasiRandomYN') = 'true';
    modelParams('useQuasiRandomYN') = 'false';
    modelParams('fxVolCalibrated') = true;
    
    KRWCK1FRiskyUSDCK1F = MultiAssetModelCK1FCK1F(multiAssetMktData, modelParams);
    %% MultiAssetModel building end
   
    %% NumMethod Init Start
    numOfRandom = 40000;
    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                            {'AMC', 7, numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);
    %% NumMethod Init End
    
    % weekly step calculator
    % pricer = Pricer(rangeAccrual,KRWLGM,numMethodAMC);
    % pricingOut = pricer.computePrice(valueDate);
    
    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    % pricer = DailyWeeklyRiskyPricer_CDKTBSpreadRA(rangeAccrual,KRWLGM1FRiskyUSDLGM1F,numMethodAMC,false);
%     pricer = DailyWeeklyRiskyPricer_CDUSDLibor3MSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, true);    
    pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRAInverseFloater(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, true);    
     
    pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH, 'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH,'PSANo');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSANo');
%     pricingOutClosed = pricer.computePriceClosedSimple(valueDateH);
    
    callablePrice(idx2) = pricingOutMC1.callable.npv;
    nonCallPrice(idx2) = pricingOutMC1.nonCall.npv;
%     closedPrice(idx2) = pricingOutClosed.pricerInfo.nonCall.npv;
    
%     for idx3 = 1:length(pricingOutMC1.nonCall.cashflow_npv_e)
% %     for idx3 = 1:length(pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e)
%         nonCallPrice_cashflowNPV(idx2, idx3) = pricingOutMC1.nonCall.cashflow_npv_e(idx3);
% %         closedPrice_cashflowNPV(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
% %         
% %         closedPrice_fwdCD(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdCD(idx3);
% %         closedPrice_fwdKTB(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdKTB(idx3);
% %         closedPrice_fwdSpread(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSpread(idx3);
% %         closedPrice_fwdMu(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdMu(idx3);
% %         closedPrice_fwdSigma(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSigma(idx3);
% %                 
%     end
end

aaa =1.0;


In [ ]:
testCD_CDKTB3MSpread_CK1F_CK1F_20230516_ycDelta_api2.m


clc
clear
valueDate = '2023-05-16';

maxIdx = 0;
zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;

callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% CD-USDLibor3M Spread Range Accrual Pricing start
%%
% global dictionary init
valueDateG = H_Date(valueDate);
globalDict = containers.Map();

%% static data
% default calendar
calendarNames = {'KRNT','KRBK'};
jointCalendar = GetCalendarsFromOctagon(valueDateG,calendarNames,'30Y');
globalDict('jointCalendar') = jointCalendar;
% static Setting for globalDict
%yldCurveSetting
yiledCurveSettings = containers.Map();
%KRW_IRS Setting
yieldCurveParams = {};
yieldCurveParams.floatingLegTenor = '3M';
yieldCurveParams.floatingLegDayCounter = 'Act365';
yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';

yieldCurveParams.fixingDays = 1;
yieldCurveParams.fixingCalendar = globalDict('jointCalendar');
yieldCurveParams.fixedLegTenor = '3M';
yieldCurveParams.fixedLegDayCounter = 'Act365';
yieldCurveParams.fixedLegConvention = 'ModifiedFollowing';
yieldCurveParams.targetTimeBucket = [0.0027397;0.25;0.5;0.75;1;2;3;4;5;7;10;12;15;20;25;30];
yiledCurveSettings('KRW_IRS') = yieldCurveParams;

%KRW_KTB
yieldCurveParamsDisc = {};
yieldCurveParamsDisc.floatingLegTenor = '3M';
yieldCurveParamsDisc.floatingLegDayCounter = 'Act365';
yieldCurveParamsDisc.floatingLegConvention = 'Following';
yieldCurveParamsDisc.fixingDays = 1;
yieldCurveParamsDisc.fixingCalendar = globalDict('jointCalendar');

yieldCurveParamsDisc.fixedLegTenor = '6M';
yieldCurveParamsDisc.fixedLegDayCounter = '30360';
yieldCurveParamsDisc.fixedLegConvention = 'Unadjusted';
yieldCurveParamsDisc.targetTimeBucket = [0.0027397;0.25;0.5;0.75;1;2;3;5;10;20;30];

yiledCurveSettings('KRW_KTB') = yieldCurveParamsDisc;

%USD_IRS
yieldCurveParams = {};
yieldCurveParams.floatingLegTenor = '3M';
yieldCurveParams.floatingLegDayCounter = 'Act360';
yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';
yieldCurveParams.fixingDays = 2;
yieldCurveParams.fixingCalendar = globalDict('jointCalendar');

yieldCurveParams.fixedLegTenor = '6M';
yieldCurveParams.fixedLegDayCounter = '30360';
yieldCurveParams.fixedLegConvention = 'Following';
yieldCurveParams.targetTimeBucket = [0.002739726;0.083333333;0.166666667;0.25;0.5;1;2;3;4;5;7;10;12;15;20;25;30];

yiledCurveSettings('USD_IRS') = yieldCurveParams;

globalDict('yieldCurveSettings') = yiledCurveSettings;

%modelSettings
modelSettings = containers.Map();

%CK1F
ck1FModelParams = containers.Map();
vol_r = struct;
vol_r.quote = [0.01];
vol_r.tenor = [1.0];
ck1FModelParams('vol_r') = vol_r;
ck1FModelParams('meanR_r') = 0.03;
ck1FModelParams('joint_swap_tenor') = 1.0;
ck1FModelParams('modelName') = 'CK1F';
ck1FModelParams('targetSize') = 15;
ck1FModelParams('freq') = 4;
ck1FModelParams('matu') = 15.0;

modelSettings('CK1F') = ck1FModelParams;
%LGM2F

%BS
globalDict('modelSettings') = modelSettings;

%modelNameMap

modelNameMap =  containers.Map();
modelNameMap('KRW') = 'CK1F';

globalDict('modelNameMap') = modelNameMap;

%assetModelMap
assetModelMap =  containers.Map();
assetModelMap('KRW') = struct;
globalDict('assetModelMap')=assetModelMap;

bootStrapFlag = true;
% bootStrapFlag = false;
yldCurveName = 'KRW_IRS';
ycData_KRW_IRS = GetYieldCurveFromOctagonAPI(valueDateG,yldCurveName);
zcData_KRW_IRS = BootStrapZeroFromYC(ycData_KRW_IRS,valueDateG,yldCurveName,globalDict);
rawData_KRW_IRS_zero = zcData_KRW_IRS.zcCurve.bootStrapOut.forwardCurve.params('rawData');

swaptionVolName = '032111_SWAPTN';
swaptionNormalVol_KRW_IRS = GetSwaptionVolSurfaceFromOctagon(valueDateG,swaptionVolName);
rawDataSwaption_1_15 = swaptionNormalVol_KRW_IRS.rawData;

krw_ktb_correl = GetYieldCurveCorrelFromOctagonAPI(valueDateG,'032111','032112');
raw_correl = krw_ktb_correl;

yldCurveName = 'KRW_KTB';
ycData_KRW_KTB = GetYieldCurveFromOctagonAPI(valueDateG,yldCurveName);
zcData_KRW_KTB = BootStrapZeroFromYC(ycData_KRW_KTB,valueDateG,yldCurveName,globalDict);
rawData_KRW_KTB_zero = zcData_KRW_KTB.zcCurve.bootStrapOut.forwardCurve.params('rawData');
%%
%RA info
rangeAccrualParams.payCcy = 'KRW';
rangeAccrualParams.startDate = H_Date('2022-02-09');
rangeAccrualParams.maturity = 10;

% valueDateH = H_Date('2017-04-06');
valueDateH = H_Date(valueDate);

nominal = 10000;
coupon = 0.041;
couponFreq = 1;

% Long: KRW
% Short: USD
tenorL = 0.25;
tenorS = 0.25;

lowerS = 0.0;
upperS = 999;

% 
% lower = -1;
% upper = 0.05;

tenor = 0.25;

lower = -999;
upper = 999;

%resetDate startDate endDate paymentDate
couponScheduleInt = [20220209	20220209	20220809	20220809
20220809	20220809	20230209	20230209
20230209	20230209	20230809	20230809
20230809	20230809	20240209	20240209
20240209	20240209	20240809	20240809
20240809	20240809	20250209	20250209
20250209	20250209	20250809	20250809
20250809	20250809	20260209	20260209
20260209	20260209	20260809	20260809
20260809	20260809	20270209	20270209
20270209	20270209	20270809	20270809
20270809	20270809	20280209	20280209
20280209	20280209	20280809	20280809
20280809	20280809	20290209	20290209
20290209	20290209	20290809	20290809
20290809	20290809	20300209	20300209
20300209	20300209	20300809	20300809
20300809	20300809	20310209	20310209
20310209	20310209	20310809	20310809
20310809	20310809	20320209	20320209
];

endDate = H_Date(couponScheduleInt(size(couponScheduleInt,1),4));

%% CK1F model param matu setting start
% matu = 15 + 1.0/365.0;
matu = DateDiff(H_Date(couponScheduleInt(end,4)),H_Date(valueDate))/365.0;
% matu = matu -10.0;

%% end

 rangeDays = zeros(20,1);
%     rangeDays(1) = 1;

for i=1:length(rangeDays)
    if DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) >= 0
        rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) + 1;
    end
end


%resetDate startDate endDate paymentDate
callScheduleInt = [20220209	20220209	20220809	20220809
20220809	20220809	20230209	20230209
20230209	20230209	20230809	20230809
20230809	20230809	20240209	20240209
20240209	20240209	20240809	20240809
20240809	20240809	20250209	20250209
20250209	20250209	20250809	20250809
20250809	20250809	20260209	20260209
20260209	20260209	20260809	20260809
20260809	20260809	20270209	20270209
20270209	20270209	20270809	20270809
20270809	20270809	20280209	20280209
20280209	20280209	20280809	20280809
20280809	20280809	20290209	20290209
20290209	20290209	20290809	20290809
20290809	20290809	20300209	20300209
20300209	20300209	20300809	20300809
20300809	20300809	20310209	20310209
20310209	20310209	20310809	20310809
20310809	20310809	20320209	20320209
];

callYN = ones(20,1);
callYN(1) = 0;

manualScheduleYN = 1;

rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                'tenorS','lower','upper','rangeDays','callYN', ...
                                'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                        {nominal,coupon,couponFreq,tenorL,tenorS,lower,upper,...
                        rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});

rangeAccrualInfo('lowerS') =lowerS;
rangeAccrualInfo('upperS') =upperS;
rangeAccrualInfo('tenor') = 0.25;

rangeAccrualParams.params = rangeAccrualInfo;
rangeAccrual = CDCMSSpreadDualRangeAccrual(rangeAccrualParams);

maxIdx = 0;

rmseSimul = zeros(100,1);
simulationType = 'PointByPointDelta';

if strcmp(simulationType,'PointByPointDelta')
    numberOfTenor = 17;
elseif strcmp(simulationType,'ParallelDelta')
    numberOfTenor = 1;
else
    numberOfTenor = 1;
end

idxMax = numberOfTenor + 1;
deltaShiftAmount = 0.0001;
for idx2=1:idxMax
    yldCurveName = 'KRW_IRS';
    ycData_KRW_IRS_Sim = ycData_KRW_IRS;
    
    if strcmp(simulationType,'PointByPointDelta')
        if idx2==1
            aaa = 1;
        else
                tempIdx = idx2-1;
                ycData_KRW_IRS_Sim.dataRate{tempIdx} = ycData_KRW_IRS.dataRate{tempIdx} + 1.0*deltaShiftAmount;
        end
        
        zcData_KRW_IRS_Sim = BootStrapZeroFromYC(ycData_KRW_IRS_Sim,valueDateG,yldCurveName,globalDict);
        rawData_KRW_IRS_Sim_zero = zcData_KRW_IRS_Sim.zcCurve.bootStrapOut.forwardCurve.params('rawData');
        
    end
    
    %% KRW Model building

    %% import and construct a zeroCurve
    rawData = rawData_KRW_IRS_Sim_zero;
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    KRWAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = rawData_KRW_KTB_zero;
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);

    KRWAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurfaceNormal
    rawDataSwaption = rawDataSwaption_1_15;

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    KRWAllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a KRWBlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(KRWAllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = KRWBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= KRWBlack.NormalATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
   %% create a CK1FRisky Model with initial modelParams
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;
    joint_swap_tenor = 1;
    targetSize = 15;
    freq = 4;
    
    modelParams = containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'},...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
    
    KRWCK1FRisky = CK1FRisky(IRModelRisky(Model(KRWAllMktData, modelParams)));
    
    tic
    out = KRWCK1FRisky.CalibrateToATMDiagSwaptionNormal(KRWBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        KRWVol_r(idx2,idx3) = out.vol_r.quote(idx3);
        KRWRe(idx2,idx3) = out.re(idx3);
    end
    KRWRMSE(idx2) = out.rmseTotal;
    
    toc
    %% CK1FRisky model building end
    %% KRW model building end

    %% USD model building start
    %% import and construct a zeroCurve
    rawData = rawData_KRW_KTB_zero;

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','USDZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    USDAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = rawData_KRW_KTB_zero;

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);
    USDAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
    rawDataSwaption = rawDataSwaption_1_15;
    
    mktData = containers.Map({'rawData','asOfDate','ccy','name'},...
                            {rawDataSwaption,valueDate,'USD','USDSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    USDAllMktData('swaptionVol') = swaptionVolMktData;
    
   %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    USDBlack = IRBlack(IRModel(Model(USDAllMktData,modelParams))); 
    
    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);
    
    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = USDBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= USDBlack.BlackATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = USDBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
    %% create a CK1FModel with initial modelParams
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;

%     matu = 15 + 1.0/365.0;
    joint_swap_tenor = 1;
    targetSize = 15;
    freq = 4;
    
    modelParamsUSD= containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'}, ...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
    USDCK1F = CK1F(IRModel(Model(USDAllMktData, modelParamsUSD)));
    
    tic
    outUSD = USDCK1F.CalibrateToATMDiagSwaptionNormal(USDBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        USDVol_r(idx2,idx3) = outUSD.vol_r.quote(idx3);
        USDRe(idx2,idx3) = outUSD.re(idx3);
    end
    USDRMSE(idx2) = outUSD.rmseTotal;
    
    toc
    %% USD model building end

    %% USDKRW fx model buidling start
    rawData = [0.083333333	0.0
0.25	0.0
0.5	0.0
1	0.0
2	0.0
3	0.0
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','USDKRWATMTermVol'});
    impliedTermVolMktData = MktData(mktData);
%     impliedTermVole = FXVolSurface(impliedTermVolMktData);
%     impliedVolSurface.params('moneyNessType') = 'deltaFwdMoneyNess';
    USDKRWAllMktData =containers.Map('impliedTermVol',impliedTermVolMktData);
%     AllMktData('impliedVolSurface') = impliedTermVolMktData;

    %% create  an FXBlackModel to calculate market PlainVanilla Options
%     modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});
%     USDKRWBlack = FXBlack(FXModel(Model(USDKRWAllMktData,modelParams))); 
    %% USDKRW fx model building end

    %% MultiAssetModel building start
    multiAssetMktData = containers.Map({'KRW','USD','USDKRW'}, ...
                            {KRWAllMktData, USDAllMktData, USDKRWAllMktData});

    % KRW,KTB                    
%     correlationMatrix = ones(2,2);
%     correlationMatrix(1,2) = 0.3;
%     correlationMatrix(2,1) = correlationMatrix(1,2);

    % KRW, USD & USDKRW
    correlationMatrix = ones(3, 3);
    correlationMatrix(1,2) = raw_correl;
    
%     correlationMatrix(1,2) = 0.85;
    
%     correlationMatrix(1,2) = 0.85 - 0.05*(idx2-1);
    
    correlationMatrix(1,3) = 0.0;
    correlationMatrix(2,3) = 0.0;
    correlationMatrix(2,1) = correlationMatrix(1,2);
    correlationMatrix(3,1) = correlationMatrix(1,3);
    correlationMatrix(3,2) = correlationMatrix(2,3);
    
    multiAssetMktData('correlationMatrix') = correlationMatrix;
    
    modelNameMap = containers.Map({'KRW','USD'},{KRWCK1FRisky, USDCK1F});
    modelParams = containers.Map({'modelNameMap'}, {modelNameMap});
    % correlationNames = {'KRW','USD'};
    modelParams('stochasticModelNames') = {'KRW','USD'};
    modelParams('correlationMatrix') = correlationMatrix;
    modelParams('refModelName') = 'KRW';
    % modelParams('useQuasiRandomYN') = 'true';
    modelParams('useQuasiRandomYN') = 'false';
    modelParams('fxVolCalibrated') = true;
    
    KRWCK1FRiskyUSDCK1F = MultiAssetModelCK1FCK1F(multiAssetMktData, modelParams);
    %% MultiAssetModel building end
   
    %% NumMethod Init Start
    numOfRandom = 40000;
    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                            {'AMC', 7, numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);
    %% NumMethod Init End
    
    % weekly step calculator
    % pricer = Pricer(rangeAccrual,KRWLGM,numMethodAMC);
    % pricingOut = pricer.computePrice(valueDate);
    
    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    % pricer = DailyWeeklyRiskyPricer_CDKTBSpreadRA(rangeAccrual,KRWLGM1FRiskyUSDLGM1F,numMethodAMC,false);
%     pricer = DailyWeeklyRiskyPricer_CDUSDLibor3MSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
    pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, true);
    
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, true);    
     
    pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH, 'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH,'PSANo');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSANo');
%     pricingOutClosed = pricer.computePriceClosedSimple(valueDateH);
    
    callablePrice(idx2) = pricingOutMC1.callable.npv;
    nonCallPrice(idx2) = pricingOutMC1.nonCall.npv;
%     closedPrice(idx2) = pricingOutClosed.pricerInfo.nonCall.npv;
    
    for idx3 = 1:length(pricingOutMC1.nonCall.cashflow_npv_e)
%     for idx3 = 1:length(pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e)
        nonCallPrice_cashflowNPV(idx2, idx3) = pricingOutMC1.nonCall.cashflow_npv_e(idx3);
%         closedPrice_cashflowNPV(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
%         
%         closedPrice_fwdCD(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdCD(idx3);
%         closedPrice_fwdKTB(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdKTB(idx3);
%         closedPrice_fwdSpread(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSpread(idx3);
%         closedPrice_fwdMu(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdMu(idx3);
%         closedPrice_fwdSigma(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSigma(idx3);
%                 
    end
end

aaa =1.0;


In [ ]:
testCD_CDKTB3MSpread_CK1F_CK1F_20230516_ycDelta_api_theta.m

clc
clear

maxIdx = 0;
zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;

callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);



%% CD-USDLibor3M Spread Range Accrual Pricing start
%%
% global dictionary init
globalDict = containers.Map();

idxMax = 10;

valueDate = '2023-05-08';

valueDateG = H_Date(valueDate);
tempValueDate = H_Date(valueDate);

for idx2=1:idxMax
tempValueDate = AddDate(H_Date(valueDate),idx2-1,'day');
valueDateG = tempValueDate;

    
%% static data
% default calendar
calendarNames = {'KRNT','KRBK'};
jointCalendar = GetCalendarsFromOctagon(valueDateG,calendarNames,'30Y');
globalDict('jointCalendar') = jointCalendar;
% static Setting for globalDict
%yldCurveSetting
yiledCurveSettings = containers.Map();
%KRW_IRS Setting
yieldCurveParams = {};
yieldCurveParams.floatingLegTenor = '3M';
yieldCurveParams.floatingLegDayCounter = 'Act365';
yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';

yieldCurveParams.fixingDays = 1;
yieldCurveParams.fixingCalendar = globalDict('jointCalendar');
yieldCurveParams.fixedLegTenor = '3M';
yieldCurveParams.fixedLegDayCounter = 'Act365';
yieldCurveParams.fixedLegConvention = 'ModifiedFollowing';
yieldCurveParams.targetTimeBucket = [0.0027397;0.25;0.5;0.75;1;2;3;4;5;7;10;12;15;20;25;30];
yiledCurveSettings('KRW_IRS') = yieldCurveParams;

%KRW_KTB
yieldCurveParamsDisc = {};
yieldCurveParamsDisc.floatingLegTenor = '3M';
yieldCurveParamsDisc.floatingLegDayCounter = 'Act365';
yieldCurveParamsDisc.floatingLegConvention = 'Following';
yieldCurveParamsDisc.fixingDays = 1;
yieldCurveParamsDisc.fixingCalendar = globalDict('jointCalendar');

yieldCurveParamsDisc.fixedLegTenor = '6M';
yieldCurveParamsDisc.fixedLegDayCounter = '30360';
yieldCurveParamsDisc.fixedLegConvention = 'Unadjusted';
yieldCurveParamsDisc.targetTimeBucket = [0.0027397;0.25;0.5;0.75;1;2;3;5;10;20;30];

yiledCurveSettings('KRW_KTB') = yieldCurveParamsDisc;

%USD_IRS
yieldCurveParams = {};
yieldCurveParams.floatingLegTenor = '3M';
yieldCurveParams.floatingLegDayCounter = 'Act360';
yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';
yieldCurveParams.fixingDays = 2;
yieldCurveParams.fixingCalendar = globalDict('jointCalendar');

yieldCurveParams.fixedLegTenor = '6M';
yieldCurveParams.fixedLegDayCounter = '30360';
yieldCurveParams.fixedLegConvention = 'Following';
yieldCurveParams.targetTimeBucket = [0.002739726;0.083333333;0.166666667;0.25;0.5;1;2;3;4;5;7;10;12;15;20;25;30];

yiledCurveSettings('USD_IRS') = yieldCurveParams;

globalDict('yieldCurveSettings') = yiledCurveSettings;

%modelSettings
modelSettings = containers.Map();

%CK1F
ck1FModelParams = containers.Map();
vol_r = struct;
vol_r.quote = [0.01];
vol_r.tenor = [1.0];
ck1FModelParams('vol_r') = vol_r;
ck1FModelParams('meanR_r') = 0.03;
ck1FModelParams('joint_swap_tenor') = 1.0;
ck1FModelParams('modelName') = 'CK1F';
ck1FModelParams('targetSize') = 15;
ck1FModelParams('freq') = 4;
ck1FModelParams('matu') = 15.0;

modelSettings('CK1F') = ck1FModelParams;
%LGM2F

%BS
globalDict('modelSettings') = modelSettings;

%modelNameMap

modelNameMap =  containers.Map();
modelNameMap('KRW') = 'CK1F';

globalDict('modelNameMap') = modelNameMap;

%assetModelMap
assetModelMap =  containers.Map();
assetModelMap('KRW') = struct;
globalDict('assetModelMap')=assetModelMap;

bootStrapFlag = true;
% bootStrapFlag = false;
yldCurveName = 'KRW_IRS';
ycData_KRW_IRS = GetYieldCurveFromOctagonAPI(valueDateG,yldCurveName);
zcData_KRW_IRS = BootStrapZeroFromYC(ycData_KRW_IRS,valueDateG,yldCurveName,globalDict);
rawData_KRW_IRS_zero = zcData_KRW_IRS.zcCurve.bootStrapOut.forwardCurve.params('rawData');

swaptionVolName = '032111_SWAPTN';
swaptionNormalVol_KRW_IRS = GetSwaptionVolSurfaceFromOctagon(valueDateG,swaptionVolName);
rawDataSwaption_1_15 = swaptionNormalVol_KRW_IRS.rawData;

krw_ktb_correl = GetYieldCurveCorrelFromOctagonAPI(valueDateG,'032111','032112');
raw_correl = krw_ktb_correl;

yldCurveName = 'KRW_KTB';
ycData_KRW_KTB = GetYieldCurveFromOctagonAPI(valueDateG,yldCurveName);
zcData_KRW_KTB = BootStrapZeroFromYC(ycData_KRW_KTB,valueDateG,yldCurveName,globalDict);
rawData_KRW_KTB_zero = zcData_KRW_KTB.zcCurve.bootStrapOut.forwardCurve.params('rawData');
%%
%RA info
rangeAccrualParams.payCcy = 'KRW';
rangeAccrualParams.startDate = H_Date('2022-02-09');
rangeAccrualParams.maturity = 10;

% valueDateH = H_Date('2017-04-06');
valueDateH = valueDateG;

nominal = 10000;
coupon = 0.041;
couponFreq = 1;

% Long: KRW
% Short: USD
tenorL = 0.25;
tenorS = 0.25;

lowerS = 0.0;
upperS = 999;

% 
% lower = -1;
% upper = 0.05;

tenor = 0.25;

lower = -999;
upper = 999;

%resetDate startDate endDate paymentDate
couponScheduleInt = [20220209	20220209	20220809	20220809
20220809	20220809	20230209	20230209
20230209	20230209	20230809	20230809
20230809	20230809	20240209	20240209
20240209	20240209	20240809	20240809
20240809	20240809	20250209	20250209
20250209	20250209	20250809	20250809
20250809	20250809	20260209	20260209
20260209	20260209	20260809	20260809
20260809	20260809	20270209	20270209
20270209	20270209	20270809	20270809
20270809	20270809	20280209	20280209
20280209	20280209	20280809	20280809
20280809	20280809	20290209	20290209
20290209	20290209	20290809	20290809
20290809	20290809	20300209	20300209
20300209	20300209	20300809	20300809
20300809	20300809	20310209	20310209
20310209	20310209	20310809	20310809
20310809	20310809	20320209	20320209
];

endDate = H_Date(couponScheduleInt(size(couponScheduleInt,1),4));

%% CK1F model param matu setting start
% matu = 15 + 1.0/365.0;
matu = DateDiff(H_Date(couponScheduleInt(end,4)),valueDateH)/365.0;
% matu = matu -10.0;

%% end

 rangeDays = zeros(20,1);
%     rangeDays(1) = 1;

for i=1:length(rangeDays)
    if DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) >= 0
        rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) + 1;
    end
end


%resetDate startDate endDate paymentDate
callScheduleInt = [20220209	20220209	20220809	20220809
20220809	20220809	20230209	20230209
20230209	20230209	20230809	20230809
20230809	20230809	20240209	20240209
20240209	20240209	20240809	20240809
20240809	20240809	20250209	20250209
20250209	20250209	20250809	20250809
20250809	20250809	20260209	20260209
20260209	20260209	20260809	20260809
20260809	20260809	20270209	20270209
20270209	20270209	20270809	20270809
20270809	20270809	20280209	20280209
20280209	20280209	20280809	20280809
20280809	20280809	20290209	20290209
20290209	20290209	20290809	20290809
20290809	20290809	20300209	20300209
20300209	20300209	20300809	20300809
20300809	20300809	20310209	20310209
20310209	20310209	20310809	20310809
20310809	20310809	20320209	20320209
];

callYN = ones(20,1);
callYN(1) = 0;

manualScheduleYN = 1;

rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                'tenorS','lower','upper','rangeDays','callYN', ...
                                'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                        {nominal,coupon,couponFreq,tenorL,tenorS,lower,upper,...
                        rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});

rangeAccrualInfo('lowerS') =lowerS;
rangeAccrualInfo('upperS') =upperS;
rangeAccrualInfo('tenor') = 0.25;

rangeAccrualParams.params = rangeAccrualInfo;
rangeAccrual = CDCMSSpreadDualRangeAccrual(rangeAccrualParams);



    
    rawData = rawData_KRW_IRS_zero;
    
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    KRWAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = rawData_KRW_KTB_zero;
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);

    KRWAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurfaceNormal
    rawDataSwaption = rawDataSwaption_1_15;

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    KRWAllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a KRWBlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(KRWAllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = KRWBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= KRWBlack.NormalATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
   %% create a CK1FRisky Model with initial modelParams
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;
    joint_swap_tenor = 1;
    targetSize = 15;
    freq = 4;
    
    modelParams = containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'},...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
    
    KRWCK1FRisky = CK1FRisky(IRModelRisky(Model(KRWAllMktData, modelParams)));
    
    tic
    out = KRWCK1FRisky.CalibrateToATMDiagSwaptionNormal(KRWBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        KRWVol_r(idx2,idx3) = out.vol_r.quote(idx3);
        KRWRe(idx2,idx3) = out.re(idx3);
    end
    KRWRMSE(idx2) = out.rmseTotal;
    
    toc
    %% CK1FRisky model building end
    %% KRW model building end

    %% USD model building start
    %% import and construct a zeroCurve
    rawData = rawData_KRW_KTB_zero;

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','USDZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    USDAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = rawData_KRW_KTB_zero;

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);
    USDAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
    rawDataSwaption = rawDataSwaption_1_15;
    
    mktData = containers.Map({'rawData','asOfDate','ccy','name'},...
                            {rawDataSwaption,valueDate,'USD','USDSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    USDAllMktData('swaptionVol') = swaptionVolMktData;
    
   %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    USDBlack = IRBlack(IRModel(Model(USDAllMktData,modelParams))); 
    
    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);
    
    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = USDBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= USDBlack.BlackATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = USDBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
    %% create a CK1FModel with initial modelParams
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;

%     matu = 15 + 1.0/365.0;
    joint_swap_tenor = 1;
    targetSize = 15;
    freq = 4;
    
    modelParamsUSD= containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'}, ...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
    USDCK1F = CK1F(IRModel(Model(USDAllMktData, modelParamsUSD)));
    
    tic
    outUSD = USDCK1F.CalibrateToATMDiagSwaptionNormal(USDBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        USDVol_r(idx2,idx3) = outUSD.vol_r.quote(idx3);
        USDRe(idx2,idx3) = outUSD.re(idx3);
    end
    USDRMSE(idx2) = outUSD.rmseTotal;
    
    toc
    %% USD model building end

    %% USDKRW fx model buidling start
    rawData = [0.083333333	0.0
0.25	0.0
0.5	0.0
1	0.0
2	0.0
3	0.0
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','USDKRWATMTermVol'});
    impliedTermVolMktData = MktData(mktData);
%     impliedTermVole = FXVolSurface(impliedTermVolMktData);
%     impliedVolSurface.params('moneyNessType') = 'deltaFwdMoneyNess';
    USDKRWAllMktData =containers.Map('impliedTermVol',impliedTermVolMktData);
%     AllMktData('impliedVolSurface') = impliedTermVolMktData;

    %% create  an FXBlackModel to calculate market PlainVanilla Options
%     modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});
%     USDKRWBlack = FXBlack(FXModel(Model(USDKRWAllMktData,modelParams))); 
    %% USDKRW fx model building end

    %% MultiAssetModel building start
    multiAssetMktData = containers.Map({'KRW','USD','USDKRW'}, ...
                            {KRWAllMktData, USDAllMktData, USDKRWAllMktData});

    % KRW,KTB                    
%     correlationMatrix = ones(2,2);
%     correlationMatrix(1,2) = 0.3;
%     correlationMatrix(2,1) = correlationMatrix(1,2);

    % KRW, USD & USDKRW
    correlationMatrix = ones(3, 3);
    correlationMatrix(1,2) = raw_correl;
    
%     correlationMatrix(1,2) = 0.85;
    
%     correlationMatrix(1,2) = 0.85 - 0.05*(idx2-1);
    
    correlationMatrix(1,3) = 0.0;
    correlationMatrix(2,3) = 0.0;
    correlationMatrix(2,1) = correlationMatrix(1,2);
    correlationMatrix(3,1) = correlationMatrix(1,3);
    correlationMatrix(3,2) = correlationMatrix(2,3);
    
    multiAssetMktData('correlationMatrix') = correlationMatrix;
    
    modelNameMap = containers.Map({'KRW','USD'},{KRWCK1FRisky, USDCK1F});
    modelParams = containers.Map({'modelNameMap'}, {modelNameMap});
    % correlationNames = {'KRW','USD'};
    modelParams('stochasticModelNames') = {'KRW','USD'};
    modelParams('correlationMatrix') = correlationMatrix;
    modelParams('refModelName') = 'KRW';
    % modelParams('useQuasiRandomYN') = 'true';
    modelParams('useQuasiRandomYN') = 'false';
    modelParams('fxVolCalibrated') = true;
    
    KRWCK1FRiskyUSDCK1F = MultiAssetModelCK1FCK1F(multiAssetMktData, modelParams);
    %% MultiAssetModel building end
   
    %% NumMethod Init Start
    numOfRandom = 40000;
    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                            {'AMC', 7, numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);
    %% NumMethod Init End
    
    % weekly step calculator
    % pricer = Pricer(rangeAccrual,KRWLGM,numMethodAMC);
    % pricingOut = pricer.computePrice(valueDate);
    
    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    % pricer = DailyWeeklyRiskyPricer_CDKTBSpreadRA(rangeAccrual,KRWLGM1FRiskyUSDLGM1F,numMethodAMC,false);
%     pricer = DailyWeeklyRiskyPricer_CDUSDLibor3MSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
    pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, true);
    
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, true);    
     
    pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH, 'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH,'PSANo');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSANo');
%     pricingOutClosed = pricer.computePriceClosedSimple(valueDateH);
    
    callablePrice(idx2) = pricingOutMC1.callable.npv;
    nonCallPrice(idx2) = pricingOutMC1.nonCall.npv;
%     closedPrice(idx2) = pricingOutClosed.pricerInfo.nonCall.npv;
    
    for idx3 = 1:length(pricingOutMC1.nonCall.cashflow_npv_e)
%     for idx3 = 1:length(pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e)
        nonCallPrice_cashflowNPV(idx2, idx3) = pricingOutMC1.nonCall.cashflow_npv_e(idx3);
%         closedPrice_cashflowNPV(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
%         
%         closedPrice_fwdCD(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdCD(idx3);
%         closedPrice_fwdKTB(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdKTB(idx3);
%         closedPrice_fwdSpread(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSpread(idx3);
%         closedPrice_fwdMu(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdMu(idx3);
%         closedPrice_fwdSigma(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSigma(idx3);
%                 
    end
end

aaa =1.0;


In [ ]:
testCD_CDKTB3MSpread_CK1F_CK1F_20230516_ycDelta_api.m

clc
clear
valueDate = '2023-05-16';

maxIdx = 0;
zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;

callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% CD-USDLibor3M Spread Range Accrual Pricing start
%%
% global dictionary init
valueDateG = H_Date(valueDate);
globalDict = containers.Map();

%% static data
% default calendar
calendarNames = {'KRNT','KRBK'};
jointCalendar = GetCalendarsFromOctagon(valueDateG,calendarNames,'30Y');

% KRBK.name = 'KRBK';
% KRBK.holidaysList = {'2015-01-01'
% '2015-02-18'
% '2015-02-19'
% '2015-02-20'
% '2015-05-01'
% '2015-05-05'
% '2015-05-25'
% '2015-06-06'
% '2015-08-15'
% '2015-09-26'
% '2015-09-27'
% '2015-09-28'
% '2015-09-29'
% '2015-10-03'
% '2015-10-09'
% '2015-12-25'
% '2016-01-01'
% '2016-02-08'
% '2016-02-09'
% '2016-02-10'
% '2016-03-01'
% '2016-04-13'
% '2016-05-01'
% '2016-05-05'
% '2016-06-06'
% '2016-08-15'
% '2016-09-14'
% '2016-09-15'
% '2016-09-16'
% '2016-10-03'
% '2016-10-09'
% '2017-01-27'
% '2017-01-28'
% '2017-01-29'
% '2017-01-30'
% '2017-03-01'
% '2017-05-01'
% '2017-05-03'
% '2017-05-05'
% '2017-05-09'
% '2017-06-06'
% '2017-08-15'
% '2017-10-02'
% '2017-10-03'
% '2017-10-04'
% '2017-10-05'
% '2017-10-06'
% '2017-10-09'
% '2017-12-25'
% '2018-01-01'
% '2018-02-15'
% '2018-02-16'
% '2018-03-01'
% '2018-05-01'
% '2018-05-05'
% '2018-05-07'
% '2018-05-22'
% '2018-06-06'
% '2018-06-13'
% '2018-08-15'
% '2018-09-24'
% '2018-09-25'
% '2018-09-26'
% '2018-10-03'
% '2018-10-09'
% '2018-12-25'
% '2019-01-01'
% '2019-02-04'
% '2019-02-05'
% '2019-02-06'
% '2019-03-01'
% '2019-05-01'
% '2019-05-05'
% '2019-05-06'
% '2019-05-12'
% '2019-06-06'
% '2019-08-15'
% '2019-09-12'
% '2019-09-13'
% '2019-09-14'
% '2019-10-03'
% '2019-10-09'
% '2019-12-25'
% '2020-01-01'
% '2020-01-24'
% '2020-01-27'
% '2020-04-15'
% '2020-04-30'
% '2020-05-01'
% '2020-05-05'
% '2020-08-17'
% '2020-09-30'
% '2020-10-01'
% '2020-10-02'
% '2020-10-09'
% '2020-12-25'
% '2021-01-01'
% '2021-02-11'
% '2021-02-12'
% '2021-03-01'
% '2021-05-05'
% '2021-05-19'
% '2021-08-16'
% '2021-09-20'
% '2021-09-21'
% '2021-09-22'
% '2021-10-03'
% '2021-10-04'
% '2021-10-09'
% '2021-10-11'
% '2022-01-31'
% '2022-02-01'
% '2022-02-02'
% '2022-03-01'
% '2022-03-09'
% '2022-05-05'
% '2022-06-01'
% '2022-06-06'
% '2022-08-15'
% '2022-09-09'
% '2022-09-12'
% '2022-10-03'
% '2022-10-09'
% '2022-10-10'
% '2023-01-23'
% '2023-01-24'
% '2023-03-01'
% '2023-05-01'
% '2023-05-05'
% '2023-06-06'
% '2023-08-15'
% '2023-09-28'
% '2023-09-29'
% '2023-10-03'
% '2023-10-09'
% '2023-12-25'
% '2024-01-01'
% '2024-02-09'
% '2024-02-12'
% '2024-03-01'
% '2024-04-10'
% '2024-05-01'
% '2024-05-06'
% '2024-05-15'
% '2024-06-06'
% '2024-08-15'
% '2024-09-16'
% '2024-09-17'
% '2024-09-18'
% '2024-10-03'
% '2024-10-09'
% '2024-12-25'
% '2025-01-01'
% '2025-01-28'
% '2025-01-29'
% '2025-01-30'
% '2025-03-01'
% '2025-03-03'
% '2025-05-01'
% '2025-05-05'
% '2025-05-06'
% '2025-06-06'
% '2025-08-15'
% '2025-10-03'
% '2025-10-06'
% '2025-10-07'
% '2025-10-08'
% '2025-10-09'
% '2025-12-25'
% '2026-01-01'
% '2026-02-16'
% '2026-02-17'
% '2026-02-18'
% '2026-03-01'
% '2026-03-02'
% '2026-05-01'
% '2026-05-05'
% '2026-06-03'
% '2026-08-17'
% '2026-09-24'
% '2026-09-25'
% '2026-10-03'
% '2026-10-05'
% '2026-10-09'
% '2026-12-25'
% '2027-01-01'
% '2027-02-08'
% '2027-02-09'
% '2027-03-01'
% '2027-05-05'
% '2027-05-13'
% '2027-08-16'
% '2027-09-14'
% '2027-09-15'
% '2027-09-16'
% '2027-10-03'
% '2027-10-04'
% '2027-10-09'
% '2027-10-11'
% '2028-01-01'
% '2028-01-26'
% '2028-01-27'
% '2028-01-28'
% '2028-03-01'
% '2028-04-12'
% '2028-05-01'
% '2028-05-02'
% '2028-05-05'
% '2028-06-06'
% '2028-08-15'
% '2028-10-02'
% '2028-10-03'
% '2028-10-04'
% '2028-10-05'
% '2028-10-09'
% '2028-12-25'
% '2029-01-01'
% '2029-02-12'
% '2029-02-13'
% '2029-02-14'
% '2029-03-01'
% '2029-05-01'
% '2029-05-05'
% '2029-05-07'
% '2029-06-06'
% '2029-08-15'
% '2029-09-21'
% '2029-09-24'
% '2029-10-03'
% '2029-10-09'
% '2029-12-25'
% '2030-01-01'
% '2030-02-04'
% '2030-02-05'
% '2030-03-01'
% '2030-05-01'
% '2030-05-06'
% '2030-05-09'
% '2030-06-06'
% '2030-08-15'
% '2030-09-11'
% '2030-09-12'
% '2030-09-13'
% '2030-10-03'
% '2030-10-09'
% '2030-12-25'
% '2031-01-01'
% '2031-01-22'
% '2031-01-23'
% '2031-01-24'
% '2031-03-03'
% '2031-05-01'
% '2031-05-05'
% '2031-05-28'
% '2031-06-06'
% '2031-08-15'
% '2031-09-30'
% '2031-10-01'
% '2031-10-02'
% '2031-10-03'
% '2031-10-09'
% '2031-12-25'
% '2032-01-01'
% '2032-02-10'
% '2032-02-11'
% '2032-02-12'
% '2032-03-01'
% '2032-05-05'
% '2032-08-16'
% '2032-09-20'
% '2032-09-21'
% '2032-10-04'
% '2032-10-11'
% '2033-01-31'
% '2033-02-01'
% '2033-02-02'
% '2033-03-01'
% '2033-05-05'
% '2033-05-06'
% '2033-06-06'
% '2033-08-15'
% '2033-09-07'
% '2033-09-08'
% '2033-09-09'
% '2033-10-03'
% '2033-10-10'
% '2034-02-20'
% '2034-02-21'
% '2034-03-01'
% '2034-05-01'
% '2034-05-05'
% '2034-05-25'
% '2034-06-06'
% '2034-08-15'
% '2034-09-26'
% '2034-09-27'
% '2034-09-28'
% '2034-10-03'
% '2034-10-09'
% '2034-12-25'
% '2035-01-01'
% '2035-02-07'
% '2035-02-08'
% '2035-02-09'
% '2035-03-01'
% '2035-05-01'
% '2035-05-07'
% '2035-05-15'
% '2035-06-06'
% '2035-08-15'
% '2035-09-17'
% '2035-09-18'
% '2035-10-03'
% '2035-10-09'
% '2035-12-25'
% '2036-01-01'
% '2036-01-28'
% '2036-01-29'
% '2036-03-03'
% '2036-05-05'
% '2036-06-06'
% '2036-08-15'
% '2036-10-03'
% '2036-10-06'
% '2036-10-07'
% '2036-10-09'
% '2036-12-25'
% '2037-01-01'
% '2037-02-16'
% '2037-02-17'
% '2037-03-02'
% '2037-05-05'
% '2037-05-22'
% '2037-08-17'
% '2037-09-23'
% '2037-09-24'
% '2037-09-25'
% '2037-10-05'
% '2037-10-09'
% '2037-12-25'
% '2038-01-01'
% '2038-02-03'
% '2038-02-04'
% '2038-02-05'
% '2038-03-01'
% '2038-05-05'
% '2038-05-11'
% '2038-08-16'
% '2038-09-13'
% '2038-09-14'
% '2038-09-15'
% '2038-10-04'
% '2038-10-11'
% '2039-01-24'
% '2039-01-25'
% '2039-01-26'
% '2039-03-01'
% '2039-05-05'
% '2039-06-06'
% '2039-08-15'
% '2039-10-03'
% '2039-10-04'
% '2039-10-05'
% '2039-10-10'
% '2040-01-01'
% '2040-02-11'
% '2040-02-12'
% '2040-02-13'
% '2040-02-14'
% '2040-03-01'
% '2040-05-01'
% '2040-05-05'
% '2040-05-07'
% '2040-05-18'
% '2040-06-06'
% '2040-08-15'
% '2040-09-19'
% '2040-09-20'
% '2040-09-21'
% '2040-10-03'
% '2040-10-09'
% '2040-12-25'
% '2041-01-01'
% '2041-01-31'
% '2041-02-01'
% '2041-03-01'
% '2041-05-01'
% '2041-05-06'
% '2041-05-07'
% '2041-06-06'
% '2041-08-15'
% '2041-09-09'
% '2041-09-10'
% '2041-09-11'
% '2041-10-03'
% '2041-10-09'
% '2041-12-25'
%     };
% 
% KRBKCalendar = Calendar();
% KRBKCalendar.name = KRBK.name;
% 
% for i=1:length(KRBK.holidaysList)
%     KRBKCalendar.AddHoliday(KRBK.holidaysList{i})
% end
% 
% % jointCalendar = JointCalendar({KRNTCalendar,KRBKCalendar});
% jointCalendar = KRBKCalendar;

globalDict('jointCalendar') = jointCalendar;

% static Setting for globalDict
%yldCurveSetting

yiledCurveSettings = containers.Map();

%KRW_IRS Setting

yieldCurveParams = {};
yieldCurveParams.floatingLegTenor = '3M';
yieldCurveParams.floatingLegDayCounter = 'Act365';
yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';

yieldCurveParams.fixingDays = 1;
yieldCurveParams.fixingCalendar = globalDict('jointCalendar');
yieldCurveParams.fixedLegTenor = '3M';
yieldCurveParams.fixedLegDayCounter = 'Act365';
yieldCurveParams.fixedLegConvention = 'ModifiedFollowing';
yieldCurveParams.targetTimeBucket = [0.0027397;0.25;0.5;0.75;1;2;3;4;5;7;10;12;15;20;25;30];
yiledCurveSettings('KRW_IRS') = yieldCurveParams;

%KRW_KTB
yieldCurveParamsDisc = {};
yieldCurveParamsDisc.floatingLegTenor = '3M';
yieldCurveParamsDisc.floatingLegDayCounter = 'Act365';
yieldCurveParamsDisc.floatingLegConvention = 'Following';
yieldCurveParamsDisc.fixingDays = 1;
yieldCurveParamsDisc.fixingCalendar = globalDict('jointCalendar');

yieldCurveParamsDisc.fixedLegTenor = '6M';
yieldCurveParamsDisc.fixedLegDayCounter = '30360';
yieldCurveParamsDisc.fixedLegConvention = 'Unadjusted';
yieldCurveParamsDisc.targetTimeBucket = [0.0027397;0.25;0.5;0.75;1;2;3;5;10;20;30];

yiledCurveSettings('KRW_KTB') = yieldCurveParamsDisc;

%USD_IRS
yieldCurveParams = {};
yieldCurveParams.floatingLegTenor = '3M';
yieldCurveParams.floatingLegDayCounter = 'Act360';
yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';
yieldCurveParams.fixingDays = 2;
yieldCurveParams.fixingCalendar = globalDict('jointCalendar');

yieldCurveParams.fixedLegTenor = '6M';
yieldCurveParams.fixedLegDayCounter = '30360';
yieldCurveParams.fixedLegConvention = 'Following';
yieldCurveParams.targetTimeBucket = [0.002739726;0.083333333;0.166666667;0.25;0.5;1;2;3;4;5;7;10;12;15;20;25;30];

yiledCurveSettings('USD_IRS') = yieldCurveParams;

globalDict('yieldCurveSettings') = yiledCurveSettings;

%modelSettings
modelSettings = containers.Map();

%CK1F
ck1FModelParams = containers.Map();
vol_r = struct;
vol_r.quote = [0.01];
vol_r.tenor = [1.0];
ck1FModelParams('vol_r') = vol_r;
ck1FModelParams('meanR_r') = 0.03;
ck1FModelParams('joint_swap_tenor') = 1.0;
ck1FModelParams('modelName') = 'CK1F';
ck1FModelParams('targetSize') = 15;
ck1FModelParams('freq') = 4;
ck1FModelParams('matu') = 15.0;

modelSettings('CK1F') = ck1FModelParams;
%LGM2F

%BS
globalDict('modelSettings') = modelSettings;

%modelNameMap

modelNameMap =  containers.Map();
modelNameMap('KRW') = 'CK1F';


globalDict('modelNameMap') = modelNameMap;
targetTenor = [0.002739726
0.25
0.5
0.75
1
1.25
1.5
1.75
2
2.25
2.5
2.75
3
3.25
3.5
3.75
4
4.25
4.5
4.75
5
5.25
5.5
5.75
6
6.25
6.5
6.75
7
7.25
7.5
7.75
8
8.25
8.5
8.75
9
9.25
9.5
9.75
10
10.25
10.5
10.75
11
11.25
11.5
11.75
12
12.25
12.5
12.75
13
13.25
13.5
13.75
14
14.25
14.5
14.75
15
15.25
15.5
15.75
16
16.25
16.5
16.75
17
17.25
17.5
17.75
18
18.25
18.5
18.75
19
19.25
19.5
19.75
20
20.25
20.5
20.75
21
21.25
21.5
21.75
22
22.25
22.5
22.75
23
23.25
23.5
23.75
24
24.25
24.5
24.75
25
25.25
25.5
25.75
26
26.25
26.5
26.75
27
27.25
27.5
27.75
28
28.25
28.5
28.75
29
29.25
29.5
29.75
30
30.25
30.5
30.75
31
];

%assetModelMap
assetModelMap =  containers.Map();
assetModelMap('KRW') = struct;
globalDict('assetModelMap')=assetModelMap;

bootStrapFlag = true;
% bootStrapFlag = false;
yldCurveName = 'KRW_IRS';
ycData_KRW_IRS = GetYieldCurveFromOctagonAPI(valueDateG,yldCurveName);
zcData_KRW_IRS = BootStrapZeroFromYC(ycData_KRW_IRS,valueDateG,yldCurveName,globalDict);
rawData_KRW_IRS_zero = zcData_KRW_IRS.zcCurve.bootStrapOut.forwardCurve.params('rawData');


swaptionVolName = '032111_SWAPTN';
swaptionNormalVol_KRW_IRS = GetSwaptionVolSurfaceFromOctagon(valueDateG,swaptionVolName);
rawDataSwaption_1_15 = swaptionNormalVol_KRW_IRS.rawData;

krw_ktb_correl = GetYieldCurveCorrelFromOctagonAPI(valueDateG,'032111','032112');
raw_correl = krw_ktb_correl;

% aaa = 0.0;
% 
% 
% rawDataSwaption_input = [0	1	2	3	4	5	7	10	12	15
% 1	0.00726605	0.00740456	0.00733467	0.00725197	0.00724005	0.00728073	0.0073448	0.00747042	0.00747042
% 2	0.00721442	0.00715563	0.00710281	0.00700997	0.00693234	0.00690678	0.00685873	0.0069827	0.0069827
% 3	0.00745313	0.00726106	0.007014	0.00679279	0.00657002	0.00652308	0.00643375	0.0065201	0.0065201
% 4	0.00591876	0.00594663	0.0059777	0.0060218	0.0060595	0.00603983	0.00599371	0.00604989	0.00604989
% 5	0.00566879	0.00562841	0.0056001	0.00556791	0.00553558	0.00554117	0.0055434	0.00556717	0.00556717
% 7	0.00534335	0.00535691	0.00536734	0.00537591	0.00538597	0.00540221	0.00541413	0.00542501	0.00542501
% 10	0.00485869	0.00494013	0.00505524	0.00516879	0.00527637	0.00527205	0.0052421	0.0052345	0.0052345
% 15	0.00502939	0.00511104	0.0052	0.00530223	0.00539931	0.00532063	0.00519822	0.00518659	0.00518659
%     ];

% zeroRates = zeros(size(rawData_KRW_IRS_zero,1),size(rawData_KRW_IRS_zero,1)+1);
% 
% zeroRates(:,1) = rawData_KRW_IRS_zero(:,2);
% for idxhong=1:length(targetTenor)
%     ycData_KRW_IRS_dum = ycData_KRW_IRS;
%     ycData_KRW_IRS_dum.dataRate{idxhong} = ycData_KRW_IRS.dataRate{idxhong} + 0.0001;
%     zcData_KRW_IRS_dum = BootStrapZeroFromYC(ycData_KRW_IRS_dum,valueDateG,yldCurveName,globalDict);
%     rawData_KRW_IRS_zero_dum = zcData_KRW_IRS_dum.zcCurve.bootStrapOut.forwardCurve.params('rawData');
%     zeroRates(:,idxhong+1) = rawData_KRW_IRS_zero_dum(:,2);
% 
% end
% for idxhong = 1:length(targetTenor)
%     zeroRates(idxhong) = zcData_KRW_IRS.zcCurve.bootStrapOut.forwardCurve.Zero(targetTenor(idxhong));
% end

aaa = 1.0;


yldCurveName = 'KRW_KTB';
ycData_KRW_KTB = GetYieldCurveFromOctagonAPI(valueDateG,yldCurveName);
zcData_KRW_KTB = BootStrapZeroFromYC(ycData_KRW_KTB,valueDateG,yldCurveName,globalDict);
rawData_KRW_KTB_zero = zcData_KRW_KTB.zcCurve.bootStrapOut.forwardCurve.params('rawData');
% ycData = GetYieldCurveFromOctagon(valueDateG,yldCurveName,globalDict,bootStrapFlag);
% rawData_KRW_IRS_zero = ycData.zcCurve.bootStrapOut.forwardCurve.params('rawData');

% yldCurveName = 'KRW_KTB';
% ycData2 = GetYieldCurveFromOctagon(valueDateG,yldCurveName,globalDict,bootStrapFlag);
% rawData_KRW_KTB_zero = ycData2.zcCurve.bootStrapOut.forwardCurve.params('rawData');

aaa = 1.0;
%%

%RA info
rangeAccrualParams.payCcy = 'KRW';
rangeAccrualParams.startDate = H_Date('2022-02-09');
rangeAccrualParams.maturity = 10;

% valueDateH = H_Date('2017-04-06');
valueDateH = H_Date(valueDate);

nominal = 10000;
coupon = 0.041;
couponFreq = 1;

% Long: KRW
% Short: USD
tenorL = 0.25;
tenorS = 0.25;

lowerS = 0.0;
upperS = 999;

% 
% lower = -1;
% upper = 0.05;

tenor = 0.25;

lower = -999;
upper = 999;

%resetDate startDate endDate paymentDate
couponScheduleInt = [20220209	20220209	20220809	20220809
20220809	20220809	20230209	20230209
20230209	20230209	20230809	20230809
20230809	20230809	20240209	20240209
20240209	20240209	20240809	20240809
20240809	20240809	20250209	20250209
20250209	20250209	20250809	20250809
20250809	20250809	20260209	20260209
20260209	20260209	20260809	20260809
20260809	20260809	20270209	20270209
20270209	20270209	20270809	20270809
20270809	20270809	20280209	20280209
20280209	20280209	20280809	20280809
20280809	20280809	20290209	20290209
20290209	20290209	20290809	20290809
20290809	20290809	20300209	20300209
20300209	20300209	20300809	20300809
20300809	20300809	20310209	20310209
20310209	20310209	20310809	20310809
20310809	20310809	20320209	20320209
];

endDate = H_Date(couponScheduleInt(size(couponScheduleInt,1),4));

%% CK1F model param matu setting start
% matu = 15 + 1.0/365.0;
matu = DateDiff(H_Date(couponScheduleInt(end,4)),H_Date(valueDate))/365.0;
% matu = matu -10.0;

%% end

 rangeDays = zeros(20,1);
%     rangeDays(1) = 1;

for i=1:length(rangeDays)
    if DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) >= 0
        rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) + 1;
    end
end


%resetDate startDate endDate paymentDate
callScheduleInt = [20220209	20220209	20220809	20220809
20220809	20220809	20230209	20230209
20230209	20230209	20230809	20230809
20230809	20230809	20240209	20240209
20240209	20240209	20240809	20240809
20240809	20240809	20250209	20250209
20250209	20250209	20250809	20250809
20250809	20250809	20260209	20260209
20260209	20260209	20260809	20260809
20260809	20260809	20270209	20270209
20270209	20270209	20270809	20270809
20270809	20270809	20280209	20280209
20280209	20280209	20280809	20280809
20280809	20280809	20290209	20290209
20290209	20290209	20290809	20290809
20290809	20290809	20300209	20300209
20300209	20300209	20300809	20300809
20300809	20300809	20310209	20310209
20310209	20310209	20310809	20310809
20310809	20310809	20320209	20320209
];

callYN = ones(20,1);
callYN(1) = 0;

manualScheduleYN = 1;

rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                'tenorS','lower','upper','rangeDays','callYN', ...
                                'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                        {nominal,coupon,couponFreq,tenorL,tenorS,lower,upper,...
                        rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});
% 
% tenorL = 0.25;
% tenorS = 0.25;
% 
% tenor = 0.25;
% 
% lowerS = 0.0;
% upperS = 999;
% 
% 
% lower = 0.0;
% upper = 0.05;

rangeAccrualInfo('lowerS') =lowerS;
rangeAccrualInfo('upperS') =upperS;
rangeAccrualInfo('tenor') = 0.25;

rangeAccrualParams.params = rangeAccrualInfo;

% rangeAccrual = SpreadRangeAccrual(rangeAccrualParams);
rangeAccrual = CDCMSSpreadDualRangeAccrual(rangeAccrualParams);

% maxIdx =2.5;

maxIdx = 0;

rmseSimul = zeros(100,1);



simulationType = 'PointByPointDelta';

if strcmp(simulationType,'PointByPointDelta')
    numberOfTenor = 17;
elseif strcmp(simulationType,'ParallelDelta')
    numberOfTenor = 1;
else
    numberOfTenor = 1;
end

% idxMax = 2*numberOfTenor + 1;
idxMax = numberOfTenor + 1;
deltaShiftAmount = 0.0001;
for idx2=1:idxMax
    yldCurveName = 'KRW_IRS';
    ycData_KRW_IRS_Sim = ycData_KRW_IRS;
    
    
    if strcmp(simulationType,'PointByPointDelta')
        if idx2==1
            aaa = 1;
        else
                tempIdx = idx2-1;
                ycData_KRW_IRS_Sim.dataRate{tempIdx} = ycData_KRW_IRS.dataRate{tempIdx} + 1.0*deltaShiftAmount;
        end
        
        zcData_KRW_IRS_Sim = BootStrapZeroFromYC(ycData_KRW_IRS_Sim,valueDateG,yldCurveName,globalDict);
        rawData_KRW_IRS_Sim_zero = zcData_KRW_IRS_Sim.zcCurve.bootStrapOut.forwardCurve.params('rawData');
        
    end
    
    %% KRW Model building

    %% import and construct a zeroCurve
        rawData = rawData_KRW_IRS_Sim_zero;

%     rawData = [0.01095890	0.01288977
% 0.26027397	0.01490597
% 0.51232877	0.01600635
% 0.76164384	0.01732110
% 1.00821918	0.01862444
% 1.51232877	0.02033641
% 2.01095890	0.02148639
% 3.01917808	0.02248311
% 4.01643836	0.02295767
% 5.01369863	0.02315906
% 7.01369863	0.02312159
% 10.01917808	0.02316791
% 12.01643836	0.02333248
% 15.02465753	0.02272163
% 20.03013699	0.02073587
% 25.11232877	0.01953241
%     ];

% for idx=1:size(rawData,1)
%     rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
% end

%     
%     %simulation testCase
%     for idx=1:size(rawData,1)
%         rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
%     end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    KRWAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = rawData_KRW_KTB_zero;
    
%     rawData = [0.0109589041096	0.0128897723998
% 0.2602739726027	0.0122615805749
% 0.5123287671233	0.0126436586237
% 0.7616438356164	0.0132888549689
% 1.0082191780822	0.0149651333289
% 2.0109589041096	0.0194062486354
% 3.0191780821918	0.0220738728647
% 5.0136986301370	0.0246213621680
% 10.0191780821917	0.0268179784314
% 20.0301369863013	0.0270848466204
% 30.0301369863013	0.0262832530607
%     ];

% for idx=1:size(rawData,1)
%     rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
% end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);

    KRWAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurfaceNormal


% 
% rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
% 1	0.00726605	0.00740456	0.00733467	0.00725197	0.00724005	0.00728073	0.0073448	0.00747042	0.00747042
% 2	0.00721442	0.00715563	0.00710281	0.00700997	0.00693234	0.00690678	0.00685873	0.0069827	0.0069827
% 3	0.00745313	0.00726106	0.007014	0.00679279	0.00657002	0.00652308	0.00643375	0.0065201	0.0065201
% 4	0.00591876	0.00594663	0.0059777	0.0060218	0.0060595	0.00603983	0.00599371	0.00604989	0.00604989
% 5	0.00566879	0.00562841	0.0056001	0.00556791	0.00553558	0.00554117	0.0055434	0.00556717	0.00556717
% 7	0.00534335	0.00535691	0.00536734	0.00537591	0.00538597	0.00540221	0.00541413	0.00542501	0.00542501
% 10	0.00485869	0.00494013	0.00505524	0.00516879	0.00527637	0.00527205	0.0052421	0.0052345	0.0052345
% 15	0.00502939	0.00511104	0.0052	0.00530223	0.00539931	0.00532063	0.00519822	0.00518659	0.00518659
%     ];


rawDataSwaption = rawDataSwaption_1_15;

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    KRWAllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a KRWBlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(KRWAllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = KRWBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= KRWBlack.NormalATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
   %% create a CK1FRisky Model with initial modelParams
%     vol_r.tenor = [1;2;3;4;5;7;10;15;20];
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;
%     meanR_r = 0.0;
    
    
    joint_swap_tenor = 1;
%     targetSize = 10;

    targetSize = 15;
    
    freq = 4;
    
    modelParams = containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'},...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
    
%     import irModels.*;
%     import targetFunctions.*;
    
    KRWCK1FRisky = CK1FRisky(IRModelRisky(Model(KRWAllMktData, modelParams)));
    
    tic
%     out = KRWLGM2F.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
    % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
%     out = KRWCK1FRisky.CalibrateToATMDiagSwaption(KRWBlack,'global');
    out = KRWCK1FRisky.CalibrateToATMDiagSwaptionNormal(KRWBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        KRWVol_r(idx2,idx3) = out.vol_r.quote(idx3);
        KRWRe(idx2,idx3) = out.re(idx3);
    end
    KRWRMSE(idx2) = out.rmseTotal;
    
    toc
    %% CK1FRisky model building end
    %% KRW model building end

    %% USD model building start
    %% import and construct a zeroCurve
    rawData = rawData_KRW_KTB_zero;
%     rawData = [0.0109589041096	0.0128897723998
% 0.2602739726027	0.0122615805749
% 0.5123287671233	0.0126436586237
% 0.7616438356164	0.0132888549689
% 1.0082191780822	0.0149651333289
% 2.0109589041096	0.0194062486354
% 3.0191780821918	0.0220738728647
% 5.0136986301370	0.0246213621680
% 10.0191780821917	0.0268179784314
% 20.0301369863013	0.0270848466204
% 30.0301369863013	0.0262832530607
%     ];

% for idx=1:size(rawData,1)
%     rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
% end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','USDZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    USDAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = rawData_KRW_KTB_zero;
%     rawData = [0.0109589041096	0.0128897723998
% 0.2602739726027	0.0122615805749
% 0.5123287671233	0.0126436586237
% 0.7616438356164	0.0132888549689
% 1.0082191780822	0.0149651333289
% 2.0109589041096	0.0194062486354
% 3.0191780821918	0.0220738728647
% 5.0136986301370	0.0246213621680
% 10.0191780821917	0.0268179784314
% 20.0301369863013	0.0270848466204
% 30.0301369863013	0.0262832530607
%     ];

% for idx=1:size(rawData,1)
%     rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
% end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);
    USDAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
% GIROPremiumToNormalVol From LIB
% rawDataSwaption = [0	1	2	3	4	5	7	10	12	15	20
% 1	0.0041	0.0045	0.0052	0.0061	0.0066	0.0071	0.0077	0.0080	0.0081	0.0082
% 2	0.0050	0.0057	0.0064	0.0066	0.0068	0.0071	0.0074	0.0077	0.0077	0.0078
% 3	0.0064	0.0069	0.0068	0.0068	0.0068	0.0072	0.0076	0.0078	0.0079	0.0079
% 4	0.0076	0.0071	0.0071	0.0070	0.0071	0.0072	0.0077	0.0081	0.0081	0.0081
% 5	0.0069	0.0070	0.0070	0.0071	0.0071	0.0070	0.0076	0.0083	0.0084	0.0083
% 7	0.0073	0.0074	0.0075	0.0072	0.0070	0.0075	0.0087	0.0096	0.0096	0.0095
% 10	0.0066	0.0066	0.0071	0.0074	0.0077	0.0092	0.0101	0.0102	0.0102	0.0102
% 15	0.0103	0.0105	0.0107	0.0109	0.0111	0.0108	0.0103	0.0102	0.0102	0.0101
% 20	0.0088	0.0089	0.0090	0.0091	0.0091	0.0091	0.0090	0.0089	0.0089	0.0089
%     ];
% 
rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.00726605	0.00740456	0.00733467	0.00725197	0.00724005	0.00728073	0.0073448	0.00747042	0.00747042
2	0.00721442	0.00715563	0.00710281	0.00700997	0.00693234	0.00690678	0.00685873	0.0069827	0.0069827
3	0.00745313	0.00726106	0.007014	0.00679279	0.00657002	0.00652308	0.00643375	0.0065201	0.0065201
4	0.00591876	0.00594663	0.0059777	0.0060218	0.0060595	0.00603983	0.00599371	0.00604989	0.00604989
5	0.00566879	0.00562841	0.0056001	0.00556791	0.00553558	0.00554117	0.0055434	0.00556717	0.00556717
7	0.00534335	0.00535691	0.00536734	0.00537591	0.00538597	0.00540221	0.00541413	0.00542501	0.00542501
10	0.00485869	0.00494013	0.00505524	0.00516879	0.00527637	0.00527205	0.0052421	0.0052345	0.0052345
15	0.00502939	0.00511104	0.0052	0.00530223	0.00539931	0.00532063	0.00519822	0.00518659	0.00518659
    ];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'},...
                            {rawDataSwaption,valueDate,'USD','USDSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    USDAllMktData('swaptionVol') = swaptionVolMktData;
    
   %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    USDBlack = IRBlack(IRModel(Model(USDAllMktData,modelParams))); 
    
    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);
    
    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = USDBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= USDBlack.BlackATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = USDBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
    %% create a CK1FModel with initial modelParams
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;

%     matu = 15 + 1.0/365.0;
    joint_swap_tenor = 1;
    targetSize = 15;
    freq = 4;
    
    modelParamsUSD= containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'}, ...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
    USDCK1F = CK1F(IRModel(Model(USDAllMktData, modelParamsUSD)));
    
    tic
    outUSD = USDCK1F.CalibrateToATMDiagSwaptionNormal(USDBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        USDVol_r(idx2,idx3) = outUSD.vol_r.quote(idx3);
        USDRe(idx2,idx3) = outUSD.re(idx3);
    end
    USDRMSE(idx2) = outUSD.rmseTotal;
    
    toc
    %% USD model building end

    %% USDKRW fx model buidling start
    rawData = [0.083333333	0.0
0.25	0.0
0.5	0.0
1	0.0
2	0.0
3	0.0
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','USDKRWATMTermVol'});
    impliedTermVolMktData = MktData(mktData);
%     impliedTermVole = FXVolSurface(impliedTermVolMktData);
%     impliedVolSurface.params('moneyNessType') = 'deltaFwdMoneyNess';
    USDKRWAllMktData =containers.Map('impliedTermVol',impliedTermVolMktData);
%     AllMktData('impliedVolSurface') = impliedTermVolMktData;

    %% create  an FXBlackModel to calculate market PlainVanilla Options
%     modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});
%     USDKRWBlack = FXBlack(FXModel(Model(USDKRWAllMktData,modelParams))); 
    %% USDKRW fx model building end

    %% MultiAssetModel building start
    multiAssetMktData = containers.Map({'KRW','USD','USDKRW'}, ...
                            {KRWAllMktData, USDAllMktData, USDKRWAllMktData});

    % KRW,KTB                    
%     correlationMatrix = ones(2,2);
%     correlationMatrix(1,2) = 0.3;
%     correlationMatrix(2,1) = correlationMatrix(1,2);

    % KRW, USD & USDKRW
    correlationMatrix = ones(3, 3);
    correlationMatrix(1,2) = raw_correl;
    
%     correlationMatrix(1,2) = 0.85;
    
%     correlationMatrix(1,2) = 0.85 - 0.05*(idx2-1);
    
    correlationMatrix(1,3) = 0.0;
    correlationMatrix(2,3) = 0.0;
    correlationMatrix(2,1) = correlationMatrix(1,2);
    correlationMatrix(3,1) = correlationMatrix(1,3);
    correlationMatrix(3,2) = correlationMatrix(2,3);
    
    multiAssetMktData('correlationMatrix') = correlationMatrix;
    
    modelNameMap = containers.Map({'KRW','USD'},{KRWCK1FRisky, USDCK1F});
    modelParams = containers.Map({'modelNameMap'}, {modelNameMap});
    % correlationNames = {'KRW','USD'};
    modelParams('stochasticModelNames') = {'KRW','USD'};
    modelParams('correlationMatrix') = correlationMatrix;
    modelParams('refModelName') = 'KRW';
    % modelParams('useQuasiRandomYN') = 'true';
    modelParams('useQuasiRandomYN') = 'false';
    modelParams('fxVolCalibrated') = true;
    
    KRWCK1FRiskyUSDCK1F = MultiAssetModelCK1FCK1F(multiAssetMktData, modelParams);
    %% MultiAssetModel building end
   
    %% NumMethod Init Start
    numOfRandom = 40000;
    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                            {'AMC', 7, numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);
    %% NumMethod Init End
    
    % weekly step calculator
    % pricer = Pricer(rangeAccrual,KRWLGM,numMethodAMC);
    % pricingOut = pricer.computePrice(valueDate);
    
    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    % pricer = DailyWeeklyRiskyPricer_CDKTBSpreadRA(rangeAccrual,KRWLGM1FRiskyUSDLGM1F,numMethodAMC,false);
%     pricer = DailyWeeklyRiskyPricer_CDUSDLibor3MSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
    pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, true);
    
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, true);    
     
    pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH, 'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH,'PSANo');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSANo');
%     pricingOutClosed = pricer.computePriceClosedSimple(valueDateH);
    
    callablePrice(idx2) = pricingOutMC1.callable.npv;
    nonCallPrice(idx2) = pricingOutMC1.nonCall.npv;
%     closedPrice(idx2) = pricingOutClosed.pricerInfo.nonCall.npv;
    
    for idx3 = 1:length(pricingOutMC1.nonCall.cashflow_npv_e)
%     for idx3 = 1:length(pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e)
        nonCallPrice_cashflowNPV(idx2, idx3) = pricingOutMC1.nonCall.cashflow_npv_e(idx3);
%         closedPrice_cashflowNPV(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
%         
%         closedPrice_fwdCD(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdCD(idx3);
%         closedPrice_fwdKTB(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdKTB(idx3);
%         closedPrice_fwdSpread(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSpread(idx3);
%         closedPrice_fwdMu(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdMu(idx3);
%         closedPrice_fwdSigma(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSigma(idx3);
%                 
    end
end

aaa =1.0;


In [ ]:
testCD_CDKTB3MSpread_CK1F_CK1F_20220810_ycDelta_api_zero.m


clc
clear
valueDate = '2023-05-16';

maxIdx = 0;
zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;

callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% CD-USDLibor3M Spread Range Accrual Pricing start
%%
% global dictionary init
valueDateG = H_Date(valueDate);
globalDict = containers.Map();

%% static data
% default calendar
% calendarNames = {'KRNT','KRBK'};
% jointCalendar = GetCalendarsFromOctagon(valueDateG,calendarNames,'30Y');

KRBK.name = 'KRBK';
KRBK.holidaysList = {'2015-01-01'
'2015-02-18'
'2015-02-19'
'2015-02-20'
'2015-05-01'
'2015-05-05'
'2015-05-25'
'2015-06-06'
'2015-08-15'
'2015-09-26'
'2015-09-27'
'2015-09-28'
'2015-09-29'
'2015-10-03'
'2015-10-09'
'2015-12-25'
'2016-01-01'
'2016-02-08'
'2016-02-09'
'2016-02-10'
'2016-03-01'
'2016-04-13'
'2016-05-01'
'2016-05-05'
'2016-06-06'
'2016-08-15'
'2016-09-14'
'2016-09-15'
'2016-09-16'
'2016-10-03'
'2016-10-09'
'2017-01-27'
'2017-01-28'
'2017-01-29'
'2017-01-30'
'2017-03-01'
'2017-05-01'
'2017-05-03'
'2017-05-05'
'2017-05-09'
'2017-06-06'
'2017-08-15'
'2017-10-02'
'2017-10-03'
'2017-10-04'
'2017-10-05'
'2017-10-06'
'2017-10-09'
'2017-12-25'
'2018-01-01'
'2018-02-15'
'2018-02-16'
'2018-03-01'
'2018-05-01'
'2018-05-05'
'2018-05-07'
'2018-05-22'
'2018-06-06'
'2018-06-13'
'2018-08-15'
'2018-09-24'
'2018-09-25'
'2018-09-26'
'2018-10-03'
'2018-10-09'
'2018-12-25'
'2019-01-01'
'2019-02-04'
'2019-02-05'
'2019-02-06'
'2019-03-01'
'2019-05-01'
'2019-05-05'
'2019-05-06'
'2019-05-12'
'2019-06-06'
'2019-08-15'
'2019-09-12'
'2019-09-13'
'2019-09-14'
'2019-10-03'
'2019-10-09'
'2019-12-25'
'2020-01-01'
'2020-01-24'
'2020-01-27'
'2020-04-15'
'2020-04-30'
'2020-05-01'
'2020-05-05'
'2020-08-17'
'2020-09-30'
'2020-10-01'
'2020-10-02'
'2020-10-09'
'2020-12-25'
'2021-01-01'
'2021-02-11'
'2021-02-12'
'2021-03-01'
'2021-05-05'
'2021-05-19'
'2021-08-16'
'2021-09-20'
'2021-09-21'
'2021-09-22'
'2021-10-03'
'2021-10-04'
'2021-10-09'
'2021-10-11'
'2022-01-31'
'2022-02-01'
'2022-02-02'
'2022-03-01'
'2022-03-09'
'2022-05-05'
'2022-06-01'
'2022-06-06'
'2022-08-15'
'2022-09-09'
'2022-09-12'
'2022-10-03'
'2022-10-09'
'2022-10-10'
'2023-01-23'
'2023-01-24'
'2023-03-01'
'2023-05-01'
'2023-05-05'
'2023-06-06'
'2023-08-15'
'2023-09-28'
'2023-09-29'
'2023-10-03'
'2023-10-09'
'2023-12-25'
'2024-01-01'
'2024-02-09'
'2024-02-12'
'2024-03-01'
'2024-04-10'
'2024-05-01'
'2024-05-06'
'2024-05-15'
'2024-06-06'
'2024-08-15'
'2024-09-16'
'2024-09-17'
'2024-09-18'
'2024-10-03'
'2024-10-09'
'2024-12-25'
'2025-01-01'
'2025-01-28'
'2025-01-29'
'2025-01-30'
'2025-03-01'
'2025-03-03'
'2025-05-01'
'2025-05-05'
'2025-05-06'
'2025-06-06'
'2025-08-15'
'2025-10-03'
'2025-10-06'
'2025-10-07'
'2025-10-08'
'2025-10-09'
'2025-12-25'
'2026-01-01'
'2026-02-16'
'2026-02-17'
'2026-02-18'
'2026-03-01'
'2026-03-02'
'2026-05-01'
'2026-05-05'
'2026-06-03'
'2026-08-17'
'2026-09-24'
'2026-09-25'
'2026-10-03'
'2026-10-05'
'2026-10-09'
'2026-12-25'
'2027-01-01'
'2027-02-08'
'2027-02-09'
'2027-03-01'
'2027-05-05'
'2027-05-13'
'2027-08-16'
'2027-09-14'
'2027-09-15'
'2027-09-16'
'2027-10-03'
'2027-10-04'
'2027-10-09'
'2027-10-11'
'2028-01-01'
'2028-01-26'
'2028-01-27'
'2028-01-28'
'2028-03-01'
'2028-04-12'
'2028-05-01'
'2028-05-02'
'2028-05-05'
'2028-06-06'
'2028-08-15'
'2028-10-02'
'2028-10-03'
'2028-10-04'
'2028-10-05'
'2028-10-09'
'2028-12-25'
'2029-01-01'
'2029-02-12'
'2029-02-13'
'2029-02-14'
'2029-03-01'
'2029-05-01'
'2029-05-05'
'2029-05-07'
'2029-06-06'
'2029-08-15'
'2029-09-21'
'2029-09-24'
'2029-10-03'
'2029-10-09'
'2029-12-25'
'2030-01-01'
'2030-02-04'
'2030-02-05'
'2030-03-01'
'2030-05-01'
'2030-05-06'
'2030-05-09'
'2030-06-06'
'2030-08-15'
'2030-09-11'
'2030-09-12'
'2030-09-13'
'2030-10-03'
'2030-10-09'
'2030-12-25'
'2031-01-01'
'2031-01-22'
'2031-01-23'
'2031-01-24'
'2031-03-03'
'2031-05-01'
'2031-05-05'
'2031-05-28'
'2031-06-06'
'2031-08-15'
'2031-09-30'
'2031-10-01'
'2031-10-02'
'2031-10-03'
'2031-10-09'
'2031-12-25'
'2032-01-01'
'2032-02-10'
'2032-02-11'
'2032-02-12'
'2032-03-01'
'2032-05-05'
'2032-08-16'
'2032-09-20'
'2032-09-21'
'2032-10-04'
'2032-10-11'
'2033-01-31'
'2033-02-01'
'2033-02-02'
'2033-03-01'
'2033-05-05'
'2033-05-06'
'2033-06-06'
'2033-08-15'
'2033-09-07'
'2033-09-08'
'2033-09-09'
'2033-10-03'
'2033-10-10'
'2034-02-20'
'2034-02-21'
'2034-03-01'
'2034-05-01'
'2034-05-05'
'2034-05-25'
'2034-06-06'
'2034-08-15'
'2034-09-26'
'2034-09-27'
'2034-09-28'
'2034-10-03'
'2034-10-09'
'2034-12-25'
'2035-01-01'
'2035-02-07'
'2035-02-08'
'2035-02-09'
'2035-03-01'
'2035-05-01'
'2035-05-07'
'2035-05-15'
'2035-06-06'
'2035-08-15'
'2035-09-17'
'2035-09-18'
'2035-10-03'
'2035-10-09'
'2035-12-25'
'2036-01-01'
'2036-01-28'
'2036-01-29'
'2036-03-03'
'2036-05-05'
'2036-06-06'
'2036-08-15'
'2036-10-03'
'2036-10-06'
'2036-10-07'
'2036-10-09'
'2036-12-25'
'2037-01-01'
'2037-02-16'
'2037-02-17'
'2037-03-02'
'2037-05-05'
'2037-05-22'
'2037-08-17'
'2037-09-23'
'2037-09-24'
'2037-09-25'
'2037-10-05'
'2037-10-09'
'2037-12-25'
'2038-01-01'
'2038-02-03'
'2038-02-04'
'2038-02-05'
'2038-03-01'
'2038-05-05'
'2038-05-11'
'2038-08-16'
'2038-09-13'
'2038-09-14'
'2038-09-15'
'2038-10-04'
'2038-10-11'
'2039-01-24'
'2039-01-25'
'2039-01-26'
'2039-03-01'
'2039-05-05'
'2039-06-06'
'2039-08-15'
'2039-10-03'
'2039-10-04'
'2039-10-05'
'2039-10-10'
'2040-01-01'
'2040-02-11'
'2040-02-12'
'2040-02-13'
'2040-02-14'
'2040-03-01'
'2040-05-01'
'2040-05-05'
'2040-05-07'
'2040-05-18'
'2040-06-06'
'2040-08-15'
'2040-09-19'
'2040-09-20'
'2040-09-21'
'2040-10-03'
'2040-10-09'
'2040-12-25'
'2041-01-01'
'2041-01-31'
'2041-02-01'
'2041-03-01'
'2041-05-01'
'2041-05-06'
'2041-05-07'
'2041-06-06'
'2041-08-15'
'2041-09-09'
'2041-09-10'
'2041-09-11'
'2041-10-03'
'2041-10-09'
'2041-12-25'
    };

KRBKCalendar = Calendar();
KRBKCalendar.name = KRBK.name;

for i=1:length(KRBK.holidaysList)
    KRBKCalendar.AddHoliday(KRBK.holidaysList{i})
end

% jointCalendar = JointCalendar({KRNTCalendar,KRBKCalendar});
jointCalendar = KRBKCalendar;

globalDict('jointCalendar') = jointCalendar;

% static Setting for globalDict
%yldCurveSetting

yiledCurveSettings = containers.Map();

%KRW_IRS Setting

yieldCurveParams = {};
yieldCurveParams.floatingLegTenor = '3M';
yieldCurveParams.floatingLegDayCounter = 'Act365';
yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';

yieldCurveParams.fixingDays = 1;
yieldCurveParams.fixingCalendar = globalDict('jointCalendar');
yieldCurveParams.fixedLegTenor = '3M';
yieldCurveParams.fixedLegDayCounter = 'Act365';
yieldCurveParams.fixedLegConvention = 'ModifiedFollowing';
yieldCurveParams.targetTimeBucket = [0.0027397;0.25;0.5;0.75;1;2;3;4;5;7;10;12;15;20;25;30];
yiledCurveSettings('KRW_IRS') = yieldCurveParams;

%KRW_KTB
yieldCurveParamsDisc = {};
yieldCurveParamsDisc.floatingLegTenor = '3M';
yieldCurveParamsDisc.floatingLegDayCounter = 'Act365';
yieldCurveParamsDisc.floatingLegConvention = 'Following';
yieldCurveParamsDisc.fixingDays = 1;
yieldCurveParamsDisc.fixingCalendar = globalDict('jointCalendar');

yieldCurveParamsDisc.fixedLegTenor = '6M';
yieldCurveParamsDisc.fixedLegDayCounter = '30360';
yieldCurveParamsDisc.fixedLegConvention = 'Unadjusted';
yieldCurveParamsDisc.targetTimeBucket = [0.0027397;0.25;0.5;0.75;1;2;3;5;10;20;30];

yiledCurveSettings('KRW_KTB') = yieldCurveParamsDisc;

%USD_IRS
yieldCurveParams = {};
yieldCurveParams.floatingLegTenor = '3M';
yieldCurveParams.floatingLegDayCounter = 'Act360';
yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';
yieldCurveParams.fixingDays = 2;
yieldCurveParams.fixingCalendar = globalDict('jointCalendar');

yieldCurveParams.fixedLegTenor = '6M';
yieldCurveParams.fixedLegDayCounter = '30360';
yieldCurveParams.fixedLegConvention = 'Following';
yieldCurveParams.targetTimeBucket = [0.002739726;0.083333333;0.166666667;0.25;0.5;1;2;3;4;5;7;10;12;15;20;25;30];

yiledCurveSettings('USD_IRS') = yieldCurveParams;

globalDict('yieldCurveSettings') = yiledCurveSettings;

%modelSettings
modelSettings = containers.Map();

%CK1F
ck1FModelParams = containers.Map();
vol_r = struct;
vol_r.quote = [0.01];
vol_r.tenor = [1.0];
ck1FModelParams('vol_r') = vol_r;
ck1FModelParams('meanR_r') = 0.03;
ck1FModelParams('joint_swap_tenor') = 1.0;
ck1FModelParams('modelName') = 'CK1F';
ck1FModelParams('targetSize') = 15;
ck1FModelParams('freq') = 4;
ck1FModelParams('matu') = 15.0;

modelSettings('CK1F') = ck1FModelParams;
%LGM2F

%BS
globalDict('modelSettings') = modelSettings;

%modelNameMap

modelNameMap =  containers.Map();
modelNameMap('KRW') = 'CK1F';


globalDict('modelNameMap') = modelNameMap;
targetTenor = [0.002739726
0.25
0.5
0.75
1
1.25
1.5
1.75
2
2.25
2.5
2.75
3
3.25
3.5
3.75
4
4.25
4.5
4.75
5
5.25
5.5
5.75
6
6.25
6.5
6.75
7
7.25
7.5
7.75
8
8.25
8.5
8.75
9
9.25
9.5
9.75
10
10.25
10.5
10.75
11
11.25
11.5
11.75
12
12.25
12.5
12.75
13
13.25
13.5
13.75
14
14.25
14.5
14.75
15
15.25
15.5
15.75
16
16.25
16.5
16.75
17
17.25
17.5
17.75
18
18.25
18.5
18.75
19
19.25
19.5
19.75
20
20.25
20.5
20.75
21
21.25
21.5
21.75
22
22.25
22.5
22.75
23
23.25
23.5
23.75
24
24.25
24.5
24.75
25
25.25
25.5
25.75
26
26.25
26.5
26.75
27
27.25
27.5
27.75
28
28.25
28.5
28.75
29
29.25
29.5
29.75
30
30.25
30.5
30.75
31
];

%assetModelMap
assetModelMap =  containers.Map();
assetModelMap('KRW') = struct;
globalDict('assetModelMap')=assetModelMap;

bootStrapFlag = true;
% bootStrapFlag = false;
yldCurveName = 'KRW_IRS';
ycData_KRW_IRS = GetYieldCurveFromOctagonAPI(valueDateG,yldCurveName);
zcData_KRW_IRS = BootStrapZeroFromYC(ycData_KRW_IRS,valueDateG,yldCurveName,globalDict);
rawData_KRW_IRS_zero = zcData_KRW_IRS.zcCurve.bootStrapOut.forwardCurve.params('rawData');

zeroRates = zeros(size(rawData_KRW_IRS_zero,1),size(rawData_KRW_IRS_zero,1)+1);

zeroRates(:,1) = rawData_KRW_IRS_zero(:,2);
for idxhong=1:length(targetTenor)
    ycData_KRW_IRS_dum = ycData_KRW_IRS;
    ycData_KRW_IRS_dum.dataRate{idxhong} = ycData_KRW_IRS.dataRate{idxhong} + 0.0001;
    zcData_KRW_IRS_dum = BootStrapZeroFromYC(ycData_KRW_IRS_dum,valueDateG,yldCurveName,globalDict);
    rawData_KRW_IRS_zero_dum = zcData_KRW_IRS_dum.zcCurve.bootStrapOut.forwardCurve.params('rawData');
    zeroRates(:,idxhong+1) = rawData_KRW_IRS_zero_dum(:,2);

end
% for idxhong = 1:length(targetTenor)
%     zeroRates(idxhong) = zcData_KRW_IRS.zcCurve.bootStrapOut.forwardCurve.Zero(targetTenor(idxhong));
% end

aaa = 1.0;


yldCurveName = 'KRW_KTB';
ycData_KRW_KTB = GetYieldCurveFromOctagonAPI(valueDateG,yldCurveName);
zcData_KRW_KTB = BootStrapZeroFromYC(ycData_KRW_KTB,valueDateG,yldCurveName,globalDict);
rawData_KRW_KTB_zero = zcData_KRW_KTB.zcCurve.bootStrapOut.forwardCurve.params('rawData');
% ycData = GetYieldCurveFromOctagon(valueDateG,yldCurveName,globalDict,bootStrapFlag);
% rawData_KRW_IRS_zero = ycData.zcCurve.bootStrapOut.forwardCurve.params('rawData');

% yldCurveName = 'KRW_KTB';
% ycData2 = GetYieldCurveFromOctagon(valueDateG,yldCurveName,globalDict,bootStrapFlag);
% rawData_KRW_KTB_zero = ycData2.zcCurve.bootStrapOut.forwardCurve.params('rawData');

aaa = 1.0;
%%

%RA info
rangeAccrualParams.payCcy = 'KRW';
rangeAccrualParams.startDate = H_Date('2022-02-09');
rangeAccrualParams.maturity = 10;

% valueDateH = H_Date('2017-04-06');
valueDateH = H_Date(valueDate);

nominal = 10000;
coupon = 0.041;
couponFreq = 1;

% Long: KRW
% Short: USD
tenorL = 0.25;
tenorS = 0.25;

lowerS = 0.0;
upperS = 999;

% 
% lower = -1;
% upper = 0.05;

tenor = 0.25;

lower = -999;
upper = 999;

%resetDate startDate endDate paymentDate
couponScheduleInt = [20220209	20220209	20220809	20220809
20220809	20220809	20230209	20230209
20230209	20230209	20230809	20230809
20230809	20230809	20240209	20240209
20240209	20240209	20240809	20240809
20240809	20240809	20250209	20250209
20250209	20250209	20250809	20250809
20250809	20250809	20260209	20260209
20260209	20260209	20260809	20260809
20260809	20260809	20270209	20270209
20270209	20270209	20270809	20270809
20270809	20270809	20280209	20280209
20280209	20280209	20280809	20280809
20280809	20280809	20290209	20290209
20290209	20290209	20290809	20290809
20290809	20290809	20300209	20300209
20300209	20300209	20300809	20300809
20300809	20300809	20310209	20310209
20310209	20310209	20310809	20310809
20310809	20310809	20320209	20320209
];

endDate = H_Date(couponScheduleInt(size(couponScheduleInt,1),4));

%% CK1F model param matu setting start
% matu = 15 + 1.0/365.0;
matu = DateDiff(H_Date(couponScheduleInt(end,4)),H_Date(valueDate))/365.0;
% matu = matu -10.0;

%% end

 rangeDays = zeros(20,1);
%     rangeDays(1) = 1;

for i=1:length(rangeDays)
    if DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) >= 0
        rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) + 1;
    end
end


%resetDate startDate endDate paymentDate
callScheduleInt = [20220209	20220209	20220809	20220809
20220809	20220809	20230209	20230209
20230209	20230209	20230809	20230809
20230809	20230809	20240209	20240209
20240209	20240209	20240809	20240809
20240809	20240809	20250209	20250209
20250209	20250209	20250809	20250809
20250809	20250809	20260209	20260209
20260209	20260209	20260809	20260809
20260809	20260809	20270209	20270209
20270209	20270209	20270809	20270809
20270809	20270809	20280209	20280209
20280209	20280209	20280809	20280809
20280809	20280809	20290209	20290209
20290209	20290209	20290809	20290809
20290809	20290809	20300209	20300209
20300209	20300209	20300809	20300809
20300809	20300809	20310209	20310209
20310209	20310209	20310809	20310809
20310809	20310809	20320209	20320209
];

callYN = ones(20,1);
callYN(1) = 0;

manualScheduleYN = 1;

rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                'tenorS','lower','upper','rangeDays','callYN', ...
                                'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                        {nominal,coupon,couponFreq,tenorL,tenorS,lower,upper,...
                        rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});
% 
% tenorL = 0.25;
% tenorS = 0.25;
% 
% tenor = 0.25;
% 
% lowerS = 0.0;
% upperS = 999;
% 
% 
% lower = 0.0;
% upper = 0.05;

rangeAccrualInfo('lowerS') =lowerS;
rangeAccrualInfo('upperS') =upperS;
rangeAccrualInfo('tenor') = 0.25;

rangeAccrualParams.params = rangeAccrualInfo;

% rangeAccrual = SpreadRangeAccrual(rangeAccrualParams);
rangeAccrual = CDCMSSpreadDualRangeAccrual(rangeAccrualParams);

% maxIdx =2.5;

maxIdx = 0;

rmseSimul = zeros(100,1);



simulationType = 'PointByPointDelta';

if strcmp(simulationType,'PointByPointDelta')
    numberOfTenor = 17;
elseif strcmp(simulationType,'ParallelDelta')
    numberOfTenor = 1;
else
    numberOfTenor = 1;
end

% idxMax = 2*numberOfTenor + 1;
idxMax = numberOfTenor + 1;
deltaShiftAmount = 0.0001;
for idx2=1:idxMax
    yldCurveName = 'KRW_IRS';
    ycData_KRW_IRS_Sim = ycData_KRW_IRS;
    
    
    if strcmp(simulationType,'PointByPointDelta')
        if idx2==1
            aaa = 1;
        else
                tempIdx = idx2-1;
                ycData_KRW_IRS_Sim.dataRate{tempIdx} = ycData_KRW_IRS.dataRate{tempIdx} + 1.0*deltaShiftAmount;
        end
        
        zcData_KRW_IRS_Sim = BootStrapZeroFromYC(ycData_KRW_IRS_Sim,valueDateG,yldCurveName,globalDict);
        rawData_KRW_IRS_Sim_zero = zcData_KRW_IRS_Sim.zcCurve.bootStrapOut.forwardCurve.params('rawData');
        
    end
    
    %% KRW Model building

    %% import and construct a zeroCurve
        rawData = rawData_KRW_IRS_Sim_zero;

%     rawData = [0.01095890	0.01288977
% 0.26027397	0.01490597
% 0.51232877	0.01600635
% 0.76164384	0.01732110
% 1.00821918	0.01862444
% 1.51232877	0.02033641
% 2.01095890	0.02148639
% 3.01917808	0.02248311
% 4.01643836	0.02295767
% 5.01369863	0.02315906
% 7.01369863	0.02312159
% 10.01917808	0.02316791
% 12.01643836	0.02333248
% 15.02465753	0.02272163
% 20.03013699	0.02073587
% 25.11232877	0.01953241
%     ];

% for idx=1:size(rawData,1)
%     rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
% end

%     
%     %simulation testCase
%     for idx=1:size(rawData,1)
%         rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
%     end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    KRWAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = rawData_KRW_KTB_zero;
    
%     rawData = [0.0109589041096	0.0128897723998
% 0.2602739726027	0.0122615805749
% 0.5123287671233	0.0126436586237
% 0.7616438356164	0.0132888549689
% 1.0082191780822	0.0149651333289
% 2.0109589041096	0.0194062486354
% 3.0191780821918	0.0220738728647
% 5.0136986301370	0.0246213621680
% 10.0191780821917	0.0268179784314
% 20.0301369863013	0.0270848466204
% 30.0301369863013	0.0262832530607
%     ];

% for idx=1:size(rawData,1)
%     rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
% end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);

    KRWAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurfaceNormal



rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.00726605	0.00740456	0.00733467	0.00725197	0.00724005	0.00728073	0.0073448	0.00747042	0.00747042
2	0.00721442	0.00715563	0.00710281	0.00700997	0.00693234	0.00690678	0.00685873	0.0069827	0.0069827
3	0.00745313	0.00726106	0.007014	0.00679279	0.00657002	0.00652308	0.00643375	0.0065201	0.0065201
4	0.00591876	0.00594663	0.0059777	0.0060218	0.0060595	0.00603983	0.00599371	0.00604989	0.00604989
5	0.00566879	0.00562841	0.0056001	0.00556791	0.00553558	0.00554117	0.0055434	0.00556717	0.00556717
7	0.00534335	0.00535691	0.00536734	0.00537591	0.00538597	0.00540221	0.00541413	0.00542501	0.00542501
10	0.00485869	0.00494013	0.00505524	0.00516879	0.00527637	0.00527205	0.0052421	0.0052345	0.0052345
15	0.00502939	0.00511104	0.0052	0.00530223	0.00539931	0.00532063	0.00519822	0.00518659	0.00518659
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    KRWAllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a KRWBlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(KRWAllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = KRWBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= KRWBlack.NormalATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
   %% create a CK1FRisky Model with initial modelParams
%     vol_r.tenor = [1;2;3;4;5;7;10;15;20];
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;
%     meanR_r = 0.0;
    
    
    joint_swap_tenor = 1;
%     targetSize = 10;

    targetSize = 15;
    
    freq = 4;
    
    modelParams = containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'},...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
    
%     import irModels.*;
%     import targetFunctions.*;
    
    KRWCK1FRisky = CK1FRisky(IRModelRisky(Model(KRWAllMktData, modelParams)));
    
    tic
%     out = KRWLGM2F.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
    % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
%     out = KRWCK1FRisky.CalibrateToATMDiagSwaption(KRWBlack,'global');
    out = KRWCK1FRisky.CalibrateToATMDiagSwaptionNormal(KRWBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        KRWVol_r(idx2,idx3) = out.vol_r.quote(idx3);
        KRWRe(idx2,idx3) = out.re(idx3);
    end
    KRWRMSE(idx2) = out.rmseTotal;
    
    toc
    %% CK1FRisky model building end
    %% KRW model building end

    %% USD model building start
    %% import and construct a zeroCurve
    rawData = rawData_KRW_KTB_zero;
%     rawData = [0.0109589041096	0.0128897723998
% 0.2602739726027	0.0122615805749
% 0.5123287671233	0.0126436586237
% 0.7616438356164	0.0132888549689
% 1.0082191780822	0.0149651333289
% 2.0109589041096	0.0194062486354
% 3.0191780821918	0.0220738728647
% 5.0136986301370	0.0246213621680
% 10.0191780821917	0.0268179784314
% 20.0301369863013	0.0270848466204
% 30.0301369863013	0.0262832530607
%     ];

% for idx=1:size(rawData,1)
%     rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
% end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','USDZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    USDAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = rawData_KRW_KTB_zero;
%     rawData = [0.0109589041096	0.0128897723998
% 0.2602739726027	0.0122615805749
% 0.5123287671233	0.0126436586237
% 0.7616438356164	0.0132888549689
% 1.0082191780822	0.0149651333289
% 2.0109589041096	0.0194062486354
% 3.0191780821918	0.0220738728647
% 5.0136986301370	0.0246213621680
% 10.0191780821917	0.0268179784314
% 20.0301369863013	0.0270848466204
% 30.0301369863013	0.0262832530607
%     ];

% for idx=1:size(rawData,1)
%     rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
% end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);
    USDAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
% GIROPremiumToNormalVol From LIB
% rawDataSwaption = [0	1	2	3	4	5	7	10	12	15	20
% 1	0.0041	0.0045	0.0052	0.0061	0.0066	0.0071	0.0077	0.0080	0.0081	0.0082
% 2	0.0050	0.0057	0.0064	0.0066	0.0068	0.0071	0.0074	0.0077	0.0077	0.0078
% 3	0.0064	0.0069	0.0068	0.0068	0.0068	0.0072	0.0076	0.0078	0.0079	0.0079
% 4	0.0076	0.0071	0.0071	0.0070	0.0071	0.0072	0.0077	0.0081	0.0081	0.0081
% 5	0.0069	0.0070	0.0070	0.0071	0.0071	0.0070	0.0076	0.0083	0.0084	0.0083
% 7	0.0073	0.0074	0.0075	0.0072	0.0070	0.0075	0.0087	0.0096	0.0096	0.0095
% 10	0.0066	0.0066	0.0071	0.0074	0.0077	0.0092	0.0101	0.0102	0.0102	0.0102
% 15	0.0103	0.0105	0.0107	0.0109	0.0111	0.0108	0.0103	0.0102	0.0102	0.0101
% 20	0.0088	0.0089	0.0090	0.0091	0.0091	0.0091	0.0090	0.0089	0.0089	0.0089
%     ];
% 
rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.00726605	0.00740456	0.00733467	0.00725197	0.00724005	0.00728073	0.0073448	0.00747042	0.00747042
2	0.00721442	0.00715563	0.00710281	0.00700997	0.00693234	0.00690678	0.00685873	0.0069827	0.0069827
3	0.00745313	0.00726106	0.007014	0.00679279	0.00657002	0.00652308	0.00643375	0.0065201	0.0065201
4	0.00591876	0.00594663	0.0059777	0.0060218	0.0060595	0.00603983	0.00599371	0.00604989	0.00604989
5	0.00566879	0.00562841	0.0056001	0.00556791	0.00553558	0.00554117	0.0055434	0.00556717	0.00556717
7	0.00534335	0.00535691	0.00536734	0.00537591	0.00538597	0.00540221	0.00541413	0.00542501	0.00542501
10	0.00485869	0.00494013	0.00505524	0.00516879	0.00527637	0.00527205	0.0052421	0.0052345	0.0052345
15	0.00502939	0.00511104	0.0052	0.00530223	0.00539931	0.00532063	0.00519822	0.00518659	0.00518659
    ];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'},...
                            {rawDataSwaption,valueDate,'USD','USDSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    USDAllMktData('swaptionVol') = swaptionVolMktData;
    
   %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    USDBlack = IRBlack(IRModel(Model(USDAllMktData,modelParams))); 
    
    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);
    
    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = USDBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= USDBlack.BlackATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = USDBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
    %% create a CK1FModel with initial modelParams
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;

%     matu = 15 + 1.0/365.0;
    joint_swap_tenor = 1;
    targetSize = 15;
    freq = 4;
    
    modelParamsUSD= containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'}, ...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
    USDCK1F = CK1F(IRModel(Model(USDAllMktData, modelParamsUSD)));
    
    tic
    outUSD = USDCK1F.CalibrateToATMDiagSwaptionNormal(USDBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        USDVol_r(idx2,idx3) = outUSD.vol_r.quote(idx3);
        USDRe(idx2,idx3) = outUSD.re(idx3);
    end
    USDRMSE(idx2) = outUSD.rmseTotal;
    
    toc
    %% USD model building end

    %% USDKRW fx model buidling start
    rawData = [0.083333333	0.0
0.25	0.0
0.5	0.0
1	0.0
2	0.0
3	0.0
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','USDKRWATMTermVol'});
    impliedTermVolMktData = MktData(mktData);
%     impliedTermVole = FXVolSurface(impliedTermVolMktData);
%     impliedVolSurface.params('moneyNessType') = 'deltaFwdMoneyNess';
    USDKRWAllMktData =containers.Map('impliedTermVol',impliedTermVolMktData);
%     AllMktData('impliedVolSurface') = impliedTermVolMktData;

    %% create  an FXBlackModel to calculate market PlainVanilla Options
%     modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});
%     USDKRWBlack = FXBlack(FXModel(Model(USDKRWAllMktData,modelParams))); 
    %% USDKRW fx model building end

    %% MultiAssetModel building start
    multiAssetMktData = containers.Map({'KRW','USD','USDKRW'}, ...
                            {KRWAllMktData, USDAllMktData, USDKRWAllMktData});

    % KRW,KTB                    
%     correlationMatrix = ones(2,2);
%     correlationMatrix(1,2) = 0.3;
%     correlationMatrix(2,1) = correlationMatrix(1,2);

    % KRW, USD & USDKRW
    correlationMatrix = ones(3, 3);
    correlationMatrix(1,2) = 0.85;
%     correlationMatrix(1,2) = 0.85 - 0.05*(idx2-1);
    
    correlationMatrix(1,3) = 0.0;
    correlationMatrix(2,3) = 0.0;
    correlationMatrix(2,1) = correlationMatrix(1,2);
    correlationMatrix(3,1) = correlationMatrix(1,3);
    correlationMatrix(3,2) = correlationMatrix(2,3);
    
    multiAssetMktData('correlationMatrix') = correlationMatrix;
    
    modelNameMap = containers.Map({'KRW','USD'},{KRWCK1FRisky, USDCK1F});
    modelParams = containers.Map({'modelNameMap'}, {modelNameMap});
    % correlationNames = {'KRW','USD'};
    modelParams('stochasticModelNames') = {'KRW','USD'};
    modelParams('correlationMatrix') = correlationMatrix;
    modelParams('refModelName') = 'KRW';
    % modelParams('useQuasiRandomYN') = 'true';
    modelParams('useQuasiRandomYN') = 'false';
    modelParams('fxVolCalibrated') = true;
    
    KRWCK1FRiskyUSDCK1F = MultiAssetModelCK1FCK1F(multiAssetMktData, modelParams);
    %% MultiAssetModel building end
   
    %% NumMethod Init Start
    numOfRandom = 40000;
    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                            {'AMC', 7, numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);
    %% NumMethod Init End
    
    % weekly step calculator
    % pricer = Pricer(rangeAccrual,KRWLGM,numMethodAMC);
    % pricingOut = pricer.computePrice(valueDate);
    
    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    % pricer = DailyWeeklyRiskyPricer_CDKTBSpreadRA(rangeAccrual,KRWLGM1FRiskyUSDLGM1F,numMethodAMC,false);
%     pricer = DailyWeeklyRiskyPricer_CDUSDLibor3MSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
    pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, true);
    
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, true);    
     
    pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH, 'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH,'PSANo');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSANo');
%     pricingOutClosed = pricer.computePriceClosedSimple(valueDateH);
    
    callablePrice(idx2) = pricingOutMC1.callable.npv;
    nonCallPrice(idx2) = pricingOutMC1.nonCall.npv;
%     closedPrice(idx2) = pricingOutClosed.pricerInfo.nonCall.npv;
    
    for idx3 = 1:length(pricingOutMC1.nonCall.cashflow_npv_e)
%     for idx3 = 1:length(pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e)
        nonCallPrice_cashflowNPV(idx2, idx3) = pricingOutMC1.nonCall.cashflow_npv_e(idx3);
%         closedPrice_cashflowNPV(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
%         
%         closedPrice_fwdCD(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdCD(idx3);
%         closedPrice_fwdKTB(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdKTB(idx3);
%         closedPrice_fwdSpread(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSpread(idx3);
%         closedPrice_fwdMu(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdMu(idx3);
%         closedPrice_fwdSigma(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSigma(idx3);
%                 
    end
end

aaa =1.0;


In [ ]:
testCD_CDKTB3MSpread_CK1F_CK1F_20220810_generic_ycDelta_api.m

clc
clear
valueDate = '2023-05-16';

maxIdx = 0;
zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;

callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% CD-USDLibor3M Spread Range Accrual Pricing start
%%
% global dictionary init
valueDateG = H_Date(valueDate);
globalDict = containers.Map();

%% static data
% default calendar
% calendarNames = {'KRNT','KRBK'};
% jointCalendar = GetCalendarsFromOctagon(valueDateG,calendarNames,'30Y');

KRBK.name = 'KRBK';
KRBK.holidaysList = {'2015-01-01'
'2015-02-18'
'2015-02-19'
'2015-02-20'
'2015-05-01'
'2015-05-05'
'2015-05-25'
'2015-06-06'
'2015-08-15'
'2015-09-26'
'2015-09-27'
'2015-09-28'
'2015-09-29'
'2015-10-03'
'2015-10-09'
'2015-12-25'
'2016-01-01'
'2016-02-08'
'2016-02-09'
'2016-02-10'
'2016-03-01'
'2016-04-13'
'2016-05-01'
'2016-05-05'
'2016-06-06'
'2016-08-15'
'2016-09-14'
'2016-09-15'
'2016-09-16'
'2016-10-03'
'2016-10-09'
'2017-01-27'
'2017-01-28'
'2017-01-29'
'2017-01-30'
'2017-03-01'
'2017-05-01'
'2017-05-03'
'2017-05-05'
'2017-05-09'
'2017-06-06'
'2017-08-15'
'2017-10-02'
'2017-10-03'
'2017-10-04'
'2017-10-05'
'2017-10-06'
'2017-10-09'
'2017-12-25'
'2018-01-01'
'2018-02-15'
'2018-02-16'
'2018-03-01'
'2018-05-01'
'2018-05-05'
'2018-05-07'
'2018-05-22'
'2018-06-06'
'2018-06-13'
'2018-08-15'
'2018-09-24'
'2018-09-25'
'2018-09-26'
'2018-10-03'
'2018-10-09'
'2018-12-25'
'2019-01-01'
'2019-02-04'
'2019-02-05'
'2019-02-06'
'2019-03-01'
'2019-05-01'
'2019-05-05'
'2019-05-06'
'2019-05-12'
'2019-06-06'
'2019-08-15'
'2019-09-12'
'2019-09-13'
'2019-09-14'
'2019-10-03'
'2019-10-09'
'2019-12-25'
'2020-01-01'
'2020-01-24'
'2020-01-27'
'2020-04-15'
'2020-04-30'
'2020-05-01'
'2020-05-05'
'2020-08-17'
'2020-09-30'
'2020-10-01'
'2020-10-02'
'2020-10-09'
'2020-12-25'
'2021-01-01'
'2021-02-11'
'2021-02-12'
'2021-03-01'
'2021-05-05'
'2021-05-19'
'2021-08-16'
'2021-09-20'
'2021-09-21'
'2021-09-22'
'2021-10-03'
'2021-10-04'
'2021-10-09'
'2021-10-11'
'2022-01-31'
'2022-02-01'
'2022-02-02'
'2022-03-01'
'2022-03-09'
'2022-05-05'
'2022-06-01'
'2022-06-06'
'2022-08-15'
'2022-09-09'
'2022-09-12'
'2022-10-03'
'2022-10-09'
'2022-10-10'
'2023-01-23'
'2023-01-24'
'2023-03-01'
'2023-05-01'
'2023-05-05'
'2023-06-06'
'2023-08-15'
'2023-09-28'
'2023-09-29'
'2023-10-03'
'2023-10-09'
'2023-12-25'
'2024-01-01'
'2024-02-09'
'2024-02-12'
'2024-03-01'
'2024-04-10'
'2024-05-01'
'2024-05-06'
'2024-05-15'
'2024-06-06'
'2024-08-15'
'2024-09-16'
'2024-09-17'
'2024-09-18'
'2024-10-03'
'2024-10-09'
'2024-12-25'
'2025-01-01'
'2025-01-28'
'2025-01-29'
'2025-01-30'
'2025-03-01'
'2025-03-03'
'2025-05-01'
'2025-05-05'
'2025-05-06'
'2025-06-06'
'2025-08-15'
'2025-10-03'
'2025-10-06'
'2025-10-07'
'2025-10-08'
'2025-10-09'
'2025-12-25'
'2026-01-01'
'2026-02-16'
'2026-02-17'
'2026-02-18'
'2026-03-01'
'2026-03-02'
'2026-05-01'
'2026-05-05'
'2026-06-03'
'2026-08-17'
'2026-09-24'
'2026-09-25'
'2026-10-03'
'2026-10-05'
'2026-10-09'
'2026-12-25'
'2027-01-01'
'2027-02-08'
'2027-02-09'
'2027-03-01'
'2027-05-05'
'2027-05-13'
'2027-08-16'
'2027-09-14'
'2027-09-15'
'2027-09-16'
'2027-10-03'
'2027-10-04'
'2027-10-09'
'2027-10-11'
'2028-01-01'
'2028-01-26'
'2028-01-27'
'2028-01-28'
'2028-03-01'
'2028-04-12'
'2028-05-01'
'2028-05-02'
'2028-05-05'
'2028-06-06'
'2028-08-15'
'2028-10-02'
'2028-10-03'
'2028-10-04'
'2028-10-05'
'2028-10-09'
'2028-12-25'
'2029-01-01'
'2029-02-12'
'2029-02-13'
'2029-02-14'
'2029-03-01'
'2029-05-01'
'2029-05-05'
'2029-05-07'
'2029-06-06'
'2029-08-15'
'2029-09-21'
'2029-09-24'
'2029-10-03'
'2029-10-09'
'2029-12-25'
'2030-01-01'
'2030-02-04'
'2030-02-05'
'2030-03-01'
'2030-05-01'
'2030-05-06'
'2030-05-09'
'2030-06-06'
'2030-08-15'
'2030-09-11'
'2030-09-12'
'2030-09-13'
'2030-10-03'
'2030-10-09'
'2030-12-25'
'2031-01-01'
'2031-01-22'
'2031-01-23'
'2031-01-24'
'2031-03-03'
'2031-05-01'
'2031-05-05'
'2031-05-28'
'2031-06-06'
'2031-08-15'
'2031-09-30'
'2031-10-01'
'2031-10-02'
'2031-10-03'
'2031-10-09'
'2031-12-25'
'2032-01-01'
'2032-02-10'
'2032-02-11'
'2032-02-12'
'2032-03-01'
'2032-05-05'
'2032-08-16'
'2032-09-20'
'2032-09-21'
'2032-10-04'
'2032-10-11'
'2033-01-31'
'2033-02-01'
'2033-02-02'
'2033-03-01'
'2033-05-05'
'2033-05-06'
'2033-06-06'
'2033-08-15'
'2033-09-07'
'2033-09-08'
'2033-09-09'
'2033-10-03'
'2033-10-10'
'2034-02-20'
'2034-02-21'
'2034-03-01'
'2034-05-01'
'2034-05-05'
'2034-05-25'
'2034-06-06'
'2034-08-15'
'2034-09-26'
'2034-09-27'
'2034-09-28'
'2034-10-03'
'2034-10-09'
'2034-12-25'
'2035-01-01'
'2035-02-07'
'2035-02-08'
'2035-02-09'
'2035-03-01'
'2035-05-01'
'2035-05-07'
'2035-05-15'
'2035-06-06'
'2035-08-15'
'2035-09-17'
'2035-09-18'
'2035-10-03'
'2035-10-09'
'2035-12-25'
'2036-01-01'
'2036-01-28'
'2036-01-29'
'2036-03-03'
'2036-05-05'
'2036-06-06'
'2036-08-15'
'2036-10-03'
'2036-10-06'
'2036-10-07'
'2036-10-09'
'2036-12-25'
'2037-01-01'
'2037-02-16'
'2037-02-17'
'2037-03-02'
'2037-05-05'
'2037-05-22'
'2037-08-17'
'2037-09-23'
'2037-09-24'
'2037-09-25'
'2037-10-05'
'2037-10-09'
'2037-12-25'
'2038-01-01'
'2038-02-03'
'2038-02-04'
'2038-02-05'
'2038-03-01'
'2038-05-05'
'2038-05-11'
'2038-08-16'
'2038-09-13'
'2038-09-14'
'2038-09-15'
'2038-10-04'
'2038-10-11'
'2039-01-24'
'2039-01-25'
'2039-01-26'
'2039-03-01'
'2039-05-05'
'2039-06-06'
'2039-08-15'
'2039-10-03'
'2039-10-04'
'2039-10-05'
'2039-10-10'
'2040-01-01'
'2040-02-11'
'2040-02-12'
'2040-02-13'
'2040-02-14'
'2040-03-01'
'2040-05-01'
'2040-05-05'
'2040-05-07'
'2040-05-18'
'2040-06-06'
'2040-08-15'
'2040-09-19'
'2040-09-20'
'2040-09-21'
'2040-10-03'
'2040-10-09'
'2040-12-25'
'2041-01-01'
'2041-01-31'
'2041-02-01'
'2041-03-01'
'2041-05-01'
'2041-05-06'
'2041-05-07'
'2041-06-06'
'2041-08-15'
'2041-09-09'
'2041-09-10'
'2041-09-11'
'2041-10-03'
'2041-10-09'
'2041-12-25'
    };

KRBKCalendar = Calendar();
KRBKCalendar.name = KRBK.name;

for i=1:length(KRBK.holidaysList)
    KRBKCalendar.AddHoliday(KRBK.holidaysList{i})
end

% jointCalendar = JointCalendar({KRNTCalendar,KRBKCalendar});
jointCalendar = KRBKCalendar;

globalDict('jointCalendar') = jointCalendar;

% static Setting for globalDict
%yldCurveSetting

yiledCurveSettings = containers.Map();

%KRW_IRS Setting

yieldCurveParams = {};
yieldCurveParams.floatingLegTenor = '3M';
yieldCurveParams.floatingLegDayCounter = 'Act365';
yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';

yieldCurveParams.fixingDays = 1;
yieldCurveParams.fixingCalendar = globalDict('jointCalendar');
yieldCurveParams.fixedLegTenor = '3M';
yieldCurveParams.fixedLegDayCounter = 'Act365';
yieldCurveParams.fixedLegConvention = 'ModifiedFollowing';
yieldCurveParams.targetTimeBucket = [0.0027397;0.25;0.5;0.75;1;2;3;4;5;7;10;12;15;20;25;30];
yiledCurveSettings('KRW_IRS') = yieldCurveParams;

%KRW_KTB
yieldCurveParamsDisc = {};
yieldCurveParamsDisc.floatingLegTenor = '3M';
yieldCurveParamsDisc.floatingLegDayCounter = 'Act365';
yieldCurveParamsDisc.floatingLegConvention = 'Following';
yieldCurveParamsDisc.fixingDays = 1;
yieldCurveParamsDisc.fixingCalendar = globalDict('jointCalendar');

yieldCurveParamsDisc.fixedLegTenor = '6M';
yieldCurveParamsDisc.fixedLegDayCounter = '30360';
yieldCurveParamsDisc.fixedLegConvention = 'Unadjusted';
yieldCurveParamsDisc.targetTimeBucket = [0.0027397;0.25;0.5;0.75;1;2;3;5;10;20;30];

yiledCurveSettings('KRW_KTB') = yieldCurveParamsDisc;

%USD_IRS
yieldCurveParams = {};
yieldCurveParams.floatingLegTenor = '3M';
yieldCurveParams.floatingLegDayCounter = 'Act360';
yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';
yieldCurveParams.fixingDays = 2;
yieldCurveParams.fixingCalendar = globalDict('jointCalendar');

yieldCurveParams.fixedLegTenor = '6M';
yieldCurveParams.fixedLegDayCounter = '30360';
yieldCurveParams.fixedLegConvention = 'Following';
yieldCurveParams.targetTimeBucket = [0.002739726;0.083333333;0.166666667;0.25;0.5;1;2;3;4;5;7;10;12;15;20;25;30];

yiledCurveSettings('USD_IRS') = yieldCurveParams;

globalDict('yieldCurveSettings') = yiledCurveSettings;

%modelSettings
modelSettings = containers.Map();

%CK1F
ck1FModelParams = containers.Map();
vol_r = struct;
vol_r.quote = [0.01];
vol_r.tenor = [1.0];
ck1FModelParams('vol_r') = vol_r;
ck1FModelParams('meanR_r') = 0.03;
ck1FModelParams('joint_swap_tenor') = 1.0;
ck1FModelParams('modelName') = 'CK1F';
ck1FModelParams('targetSize') = 15;
ck1FModelParams('freq') = 4;
ck1FModelParams('matu') = 15.0;

modelSettings('CK1F') = ck1FModelParams;
%LGM2F

%BS
globalDict('modelSettings') = modelSettings;

%modelNameMap

modelNameMap =  containers.Map();
modelNameMap('KRW') = 'CK1F';


globalDict('modelNameMap') = modelNameMap;
targetTenor = [0.002739726
0.25
0.5
0.75
1
1.25
1.5
1.75
2
2.25
2.5
2.75
3
3.25
3.5
3.75
4
4.25
4.5
4.75
5
5.25
5.5
5.75
6
6.25
6.5
6.75
7
7.25
7.5
7.75
8
8.25
8.5
8.75
9
9.25
9.5
9.75
10
10.25
10.5
10.75
11
11.25
11.5
11.75
12
12.25
12.5
12.75
13
13.25
13.5
13.75
14
14.25
14.5
14.75
15
15.25
15.5
15.75
16
16.25
16.5
16.75
17
17.25
17.5
17.75
18
18.25
18.5
18.75
19
19.25
19.5
19.75
20
20.25
20.5
20.75
21
21.25
21.5
21.75
22
22.25
22.5
22.75
23
23.25
23.5
23.75
24
24.25
24.5
24.75
25
25.25
25.5
25.75
26
26.25
26.5
26.75
27
27.25
27.5
27.75
28
28.25
28.5
28.75
29
29.25
29.5
29.75
30
30.25
30.5
30.75
31
];

%assetModelMap
assetModelMap =  containers.Map();
assetModelMap('KRW') = struct;
globalDict('assetModelMap')=assetModelMap;

bootStrapFlag = true;
% bootStrapFlag = false;
yldCurveName = 'KRW_IRS';
ycData_KRW_IRS = GetYieldCurveFromOctagonAPI(valueDateG,yldCurveName);
zcData_KRW_IRS = BootStrapZeroFromYC(ycData_KRW_IRS,valueDateG,yldCurveName,globalDict);
rawData_KRW_IRS_zero = zcData_KRW_IRS.zcCurve.bootStrapOut.forwardCurve.params('rawData');

zeroRates = zeros(size(rawData_KRW_IRS_zero,1),size(rawData_KRW_IRS_zero,1)+1);

zeroRates(:,1) = rawData_KRW_IRS_zero(:,2);
for idxhong=1:length(targetTenor)
    ycData_KRW_IRS_dum = ycData_KRW_IRS;
    ycData_KRW_IRS_dum.dataRate{idxhong} = ycData_KRW_IRS.dataRate{idxhong} + 0.0001;
    zcData_KRW_IRS_dum = BootStrapZeroFromYC(ycData_KRW_IRS_dum,valueDateG,yldCurveName,globalDict);
    rawData_KRW_IRS_zero_dum = zcData_KRW_IRS_dum.zcCurve.bootStrapOut.forwardCurve.params('rawData');
    zeroRates(:,idxhong+1) = rawData_KRW_IRS_zero_dum(:,2);

end
% for idxhong = 1:length(targetTenor)
%     zeroRates(idxhong) = zcData_KRW_IRS.zcCurve.bootStrapOut.forwardCurve.Zero(targetTenor(idxhong));
% end

aaa = 1.0;


yldCurveName = 'KRW_KTB';
ycData_KRW_KTB = GetYieldCurveFromOctagonAPI(valueDateG,yldCurveName);
zcData_KRW_KTB = BootStrapZeroFromYC(ycData_KRW_KTB,valueDateG,yldCurveName,globalDict);
rawData_KRW_KTB_zero = zcData_KRW_KTB.zcCurve.bootStrapOut.forwardCurve.params('rawData');
% ycData = GetYieldCurveFromOctagon(valueDateG,yldCurveName,globalDict,bootStrapFlag);
% rawData_KRW_IRS_zero = ycData.zcCurve.bootStrapOut.forwardCurve.params('rawData');

% yldCurveName = 'KRW_KTB';
% ycData2 = GetYieldCurveFromOctagon(valueDateG,yldCurveName,globalDict,bootStrapFlag);
% rawData_KRW_KTB_zero = ycData2.zcCurve.bootStrapOut.forwardCurve.params('rawData');

aaa = 1.0;
%%

%RA info
rangeAccrualParams.payCcy = 'KRW';
rangeAccrualParams.startDate = H_Date('2022-02-09');
rangeAccrualParams.maturity = 10;

% valueDateH = H_Date('2017-04-06');
valueDateH = H_Date(valueDate);

nominal = 10000;
coupon = 0.041;
couponFreq = 1;

% Long: KRW
% Short: USD
tenorL = 0.25;
tenorS = 0.25;

lowerS = 0.0;
upperS = 999;

% 
% lower = -1;
% upper = 0.05;

tenor = 0.25;

lower = -999;
upper = 999;

%resetDate startDate endDate paymentDate
couponScheduleInt = [20220209	20220209	20220809	20220809
20220809	20220809	20230209	20230209
20230209	20230209	20230809	20230809
20230809	20230809	20240209	20240209
20240209	20240209	20240809	20240809
20240809	20240809	20250209	20250209
20250209	20250209	20250809	20250809
20250809	20250809	20260209	20260209
20260209	20260209	20260809	20260809
20260809	20260809	20270209	20270209
20270209	20270209	20270809	20270809
20270809	20270809	20280209	20280209
20280209	20280209	20280809	20280809
20280809	20280809	20290209	20290209
20290209	20290209	20290809	20290809
20290809	20290809	20300209	20300209
20300209	20300209	20300809	20300809
20300809	20300809	20310209	20310209
20310209	20310209	20310809	20310809
20310809	20310809	20320209	20320209
];

endDate = H_Date(couponScheduleInt(size(couponScheduleInt,1),4));

%% CK1F model param matu setting start
% matu = 15 + 1.0/365.0;
matu = DateDiff(H_Date(couponScheduleInt(end,4)),H_Date(valueDate))/365.0;
% matu = matu -10.0;

%% end

 rangeDays = zeros(20,1);
%     rangeDays(1) = 1;

for i=1:length(rangeDays)
    if DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) >= 0
        rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) + 1;
    end
end


%resetDate startDate endDate paymentDate
callScheduleInt = [20220209	20220209	20220809	20220809
20220809	20220809	20230209	20230209
20230209	20230209	20230809	20230809
20230809	20230809	20240209	20240209
20240209	20240209	20240809	20240809
20240809	20240809	20250209	20250209
20250209	20250209	20250809	20250809
20250809	20250809	20260209	20260209
20260209	20260209	20260809	20260809
20260809	20260809	20270209	20270209
20270209	20270209	20270809	20270809
20270809	20270809	20280209	20280209
20280209	20280209	20280809	20280809
20280809	20280809	20290209	20290209
20290209	20290209	20290809	20290809
20290809	20290809	20300209	20300209
20300209	20300209	20300809	20300809
20300809	20300809	20310209	20310209
20310209	20310209	20310809	20310809
20310809	20310809	20320209	20320209
];

callYN = ones(20,1);
callYN(1) = 0;

manualScheduleYN = 1;

rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                'tenorS','lower','upper','rangeDays','callYN', ...
                                'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                        {nominal,coupon,couponFreq,tenorL,tenorS,lower,upper,...
                        rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});
% 
% tenorL = 0.25;
% tenorS = 0.25;
% 
% tenor = 0.25;
% 
% lowerS = 0.0;
% upperS = 999;
% 
% 
% lower = 0.0;
% upper = 0.05;

rangeAccrualInfo('lowerS') =lowerS;
rangeAccrualInfo('upperS') =upperS;
rangeAccrualInfo('tenor') = 0.25;

rangeAccrualParams.params = rangeAccrualInfo;

% rangeAccrual = SpreadRangeAccrual(rangeAccrualParams);
rangeAccrual = CDCMSSpreadDualRangeAccrual(rangeAccrualParams);

% maxIdx =2.5;

maxIdx = 0;

rmseSimul = zeros(100,1);



simulationType = 'PointByPointDelta';

if strcmp(simulationType,'PointByPointDelta')
    numberOfTenor = 17;
elseif strcmp(simulationType,'ParallelDelta')
    numberOfTenor = 1;
else
    numberOfTenor = 1;
end

% idxMax = 2*numberOfTenor + 1;
idxMax = numberOfTenor + 1;
deltaShiftAmount = 0.0001;
for idx2=1:idxMax
    yldCurveName = 'KRW_IRS';
    ycData_KRW_IRS_Sim = ycData_KRW_IRS;
    
    
    if strcmp(simulationType,'PointByPointDelta')
        if idx2==1
            aaa = 1;
        else
                tempIdx = idx2-1;
                ycData_KRW_IRS_Sim.dataRate{tempIdx} = ycData_KRW_IRS.dataRate{tempIdx} + 1.0*deltaShiftAmount;
        end
        
        zcData_KRW_IRS_Sim = BootStrapZeroFromYC(ycData_KRW_IRS_Sim,valueDateG,yldCurveName,globalDict);
        rawData_KRW_IRS_Sim_zero = zcData_KRW_IRS_Sim.zcCurve.bootStrapOut.forwardCurve.params('rawData');
        
    end
    
    %% KRW Model building

    %% import and construct a zeroCurve
        rawData = rawData_KRW_IRS_Sim_zero;

%     rawData = [0.01095890	0.01288977
% 0.26027397	0.01490597
% 0.51232877	0.01600635
% 0.76164384	0.01732110
% 1.00821918	0.01862444
% 1.51232877	0.02033641
% 2.01095890	0.02148639
% 3.01917808	0.02248311
% 4.01643836	0.02295767
% 5.01369863	0.02315906
% 7.01369863	0.02312159
% 10.01917808	0.02316791
% 12.01643836	0.02333248
% 15.02465753	0.02272163
% 20.03013699	0.02073587
% 25.11232877	0.01953241
%     ];

% for idx=1:size(rawData,1)
%     rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
% end

%     
%     %simulation testCase
%     for idx=1:size(rawData,1)
%         rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
%     end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    KRWAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = rawData_KRW_KTB_zero;
    
%     rawData = [0.0109589041096	0.0128897723998
% 0.2602739726027	0.0122615805749
% 0.5123287671233	0.0126436586237
% 0.7616438356164	0.0132888549689
% 1.0082191780822	0.0149651333289
% 2.0109589041096	0.0194062486354
% 3.0191780821918	0.0220738728647
% 5.0136986301370	0.0246213621680
% 10.0191780821917	0.0268179784314
% 20.0301369863013	0.0270848466204
% 30.0301369863013	0.0262832530607
%     ];

% for idx=1:size(rawData,1)
%     rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
% end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);

    KRWAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurfaceNormal



rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.00726605	0.00740456	0.00733467	0.00725197	0.00724005	0.00728073	0.0073448	0.00747042	0.00747042
2	0.00721442	0.00715563	0.00710281	0.00700997	0.00693234	0.00690678	0.00685873	0.0069827	0.0069827
3	0.00745313	0.00726106	0.007014	0.00679279	0.00657002	0.00652308	0.00643375	0.0065201	0.0065201
4	0.00591876	0.00594663	0.0059777	0.0060218	0.0060595	0.00603983	0.00599371	0.00604989	0.00604989
5	0.00566879	0.00562841	0.0056001	0.00556791	0.00553558	0.00554117	0.0055434	0.00556717	0.00556717
7	0.00534335	0.00535691	0.00536734	0.00537591	0.00538597	0.00540221	0.00541413	0.00542501	0.00542501
10	0.00485869	0.00494013	0.00505524	0.00516879	0.00527637	0.00527205	0.0052421	0.0052345	0.0052345
15	0.00502939	0.00511104	0.0052	0.00530223	0.00539931	0.00532063	0.00519822	0.00518659	0.00518659
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    KRWAllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a KRWBlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(KRWAllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = KRWBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= KRWBlack.NormalATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
   %% create a CK1FRisky Model with initial modelParams
%     vol_r.tenor = [1;2;3;4;5;7;10;15;20];
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;
%     meanR_r = 0.0;
    
    
    joint_swap_tenor = 1;
%     targetSize = 10;

    targetSize = 15;
    
    freq = 4;
    
    modelParams = containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'},...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
    
%     import irModels.*;
%     import targetFunctions.*;
    
    KRWCK1FRisky = CK1FRisky(IRModelRisky(Model(KRWAllMktData, modelParams)));
    
    tic
%     out = KRWLGM2F.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
    % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
%     out = KRWCK1FRisky.CalibrateToATMDiagSwaption(KRWBlack,'global');
    out = KRWCK1FRisky.CalibrateToATMDiagSwaptionNormal(KRWBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        KRWVol_r(idx2,idx3) = out.vol_r.quote(idx3);
        KRWRe(idx2,idx3) = out.re(idx3);
    end
    KRWRMSE(idx2) = out.rmseTotal;
    
    toc
    %% CK1FRisky model building end
    %% KRW model building end

    %% USD model building start
    %% import and construct a zeroCurve
    rawData = rawData_KRW_KTB_zero;
%     rawData = [0.0109589041096	0.0128897723998
% 0.2602739726027	0.0122615805749
% 0.5123287671233	0.0126436586237
% 0.7616438356164	0.0132888549689
% 1.0082191780822	0.0149651333289
% 2.0109589041096	0.0194062486354
% 3.0191780821918	0.0220738728647
% 5.0136986301370	0.0246213621680
% 10.0191780821917	0.0268179784314
% 20.0301369863013	0.0270848466204
% 30.0301369863013	0.0262832530607
%     ];

% for idx=1:size(rawData,1)
%     rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
% end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','USDZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    USDAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = rawData_KRW_KTB_zero;
%     rawData = [0.0109589041096	0.0128897723998
% 0.2602739726027	0.0122615805749
% 0.5123287671233	0.0126436586237
% 0.7616438356164	0.0132888549689
% 1.0082191780822	0.0149651333289
% 2.0109589041096	0.0194062486354
% 3.0191780821918	0.0220738728647
% 5.0136986301370	0.0246213621680
% 10.0191780821917	0.0268179784314
% 20.0301369863013	0.0270848466204
% 30.0301369863013	0.0262832530607
%     ];

% for idx=1:size(rawData,1)
%     rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
% end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);
    USDAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
% GIROPremiumToNormalVol From LIB
% rawDataSwaption = [0	1	2	3	4	5	7	10	12	15	20
% 1	0.0041	0.0045	0.0052	0.0061	0.0066	0.0071	0.0077	0.0080	0.0081	0.0082
% 2	0.0050	0.0057	0.0064	0.0066	0.0068	0.0071	0.0074	0.0077	0.0077	0.0078
% 3	0.0064	0.0069	0.0068	0.0068	0.0068	0.0072	0.0076	0.0078	0.0079	0.0079
% 4	0.0076	0.0071	0.0071	0.0070	0.0071	0.0072	0.0077	0.0081	0.0081	0.0081
% 5	0.0069	0.0070	0.0070	0.0071	0.0071	0.0070	0.0076	0.0083	0.0084	0.0083
% 7	0.0073	0.0074	0.0075	0.0072	0.0070	0.0075	0.0087	0.0096	0.0096	0.0095
% 10	0.0066	0.0066	0.0071	0.0074	0.0077	0.0092	0.0101	0.0102	0.0102	0.0102
% 15	0.0103	0.0105	0.0107	0.0109	0.0111	0.0108	0.0103	0.0102	0.0102	0.0101
% 20	0.0088	0.0089	0.0090	0.0091	0.0091	0.0091	0.0090	0.0089	0.0089	0.0089
%     ];
% 
rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.00726605	0.00740456	0.00733467	0.00725197	0.00724005	0.00728073	0.0073448	0.00747042	0.00747042
2	0.00721442	0.00715563	0.00710281	0.00700997	0.00693234	0.00690678	0.00685873	0.0069827	0.0069827
3	0.00745313	0.00726106	0.007014	0.00679279	0.00657002	0.00652308	0.00643375	0.0065201	0.0065201
4	0.00591876	0.00594663	0.0059777	0.0060218	0.0060595	0.00603983	0.00599371	0.00604989	0.00604989
5	0.00566879	0.00562841	0.0056001	0.00556791	0.00553558	0.00554117	0.0055434	0.00556717	0.00556717
7	0.00534335	0.00535691	0.00536734	0.00537591	0.00538597	0.00540221	0.00541413	0.00542501	0.00542501
10	0.00485869	0.00494013	0.00505524	0.00516879	0.00527637	0.00527205	0.0052421	0.0052345	0.0052345
15	0.00502939	0.00511104	0.0052	0.00530223	0.00539931	0.00532063	0.00519822	0.00518659	0.00518659
    ];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'},...
                            {rawDataSwaption,valueDate,'USD','USDSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    USDAllMktData('swaptionVol') = swaptionVolMktData;
    
   %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    USDBlack = IRBlack(IRModel(Model(USDAllMktData,modelParams))); 
    
    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);
    
    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = USDBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= USDBlack.BlackATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = USDBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
    %% create a CK1FModel with initial modelParams
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;

%     matu = 15 + 1.0/365.0;
    joint_swap_tenor = 1;
    targetSize = 15;
    freq = 4;
    
    modelParamsUSD= containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'}, ...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
    USDCK1F = CK1F(IRModel(Model(USDAllMktData, modelParamsUSD)));
    
    tic
    outUSD = USDCK1F.CalibrateToATMDiagSwaptionNormal(USDBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        USDVol_r(idx2,idx3) = outUSD.vol_r.quote(idx3);
        USDRe(idx2,idx3) = outUSD.re(idx3);
    end
    USDRMSE(idx2) = outUSD.rmseTotal;
    
    toc
    %% USD model building end

    %% USDKRW fx model buidling start
    rawData = [0.083333333	0.0
0.25	0.0
0.5	0.0
1	0.0
2	0.0
3	0.0
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','USDKRWATMTermVol'});
    impliedTermVolMktData = MktData(mktData);
%     impliedTermVole = FXVolSurface(impliedTermVolMktData);
%     impliedVolSurface.params('moneyNessType') = 'deltaFwdMoneyNess';
    USDKRWAllMktData =containers.Map('impliedTermVol',impliedTermVolMktData);
%     AllMktData('impliedVolSurface') = impliedTermVolMktData;

    %% create  an FXBlackModel to calculate market PlainVanilla Options
%     modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});
%     USDKRWBlack = FXBlack(FXModel(Model(USDKRWAllMktData,modelParams))); 
    %% USDKRW fx model building end

    %% MultiAssetModel building start
    multiAssetMktData = containers.Map({'KRW','USD','USDKRW'}, ...
                            {KRWAllMktData, USDAllMktData, USDKRWAllMktData});

    % KRW,KTB                    
%     correlationMatrix = ones(2,2);
%     correlationMatrix(1,2) = 0.3;
%     correlationMatrix(2,1) = correlationMatrix(1,2);

    % KRW, USD & USDKRW
    correlationMatrix = ones(3, 3);
    correlationMatrix(1,2) = 0.85;
%     correlationMatrix(1,2) = 0.85 - 0.05*(idx2-1);
    
    correlationMatrix(1,3) = 0.0;
    correlationMatrix(2,3) = 0.0;
    correlationMatrix(2,1) = correlationMatrix(1,2);
    correlationMatrix(3,1) = correlationMatrix(1,3);
    correlationMatrix(3,2) = correlationMatrix(2,3);
    
    multiAssetMktData('correlationMatrix') = correlationMatrix;
    
    modelNameMap = containers.Map({'KRW','USD'},{KRWCK1FRisky, USDCK1F});
    modelParams = containers.Map({'modelNameMap'}, {modelNameMap});
    % correlationNames = {'KRW','USD'};
    modelParams('stochasticModelNames') = {'KRW','USD'};
    modelParams('correlationMatrix') = correlationMatrix;
    modelParams('refModelName') = 'KRW';
    % modelParams('useQuasiRandomYN') = 'true';
    modelParams('useQuasiRandomYN') = 'false';
    modelParams('fxVolCalibrated') = true;
    
    KRWCK1FRiskyUSDCK1F = MultiAssetModelCK1FCK1F(multiAssetMktData, modelParams);
    %% MultiAssetModel building end
   
    %% NumMethod Init Start
    numOfRandom = 40000;
    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                            {'AMC', 7, numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);
    %% NumMethod Init End
    
    % weekly step calculator
    % pricer = Pricer(rangeAccrual,KRWLGM,numMethodAMC);
    % pricingOut = pricer.computePrice(valueDate);
    
    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    % pricer = DailyWeeklyRiskyPricer_CDKTBSpreadRA(rangeAccrual,KRWLGM1FRiskyUSDLGM1F,numMethodAMC,false);
%     pricer = DailyWeeklyRiskyPricer_CDUSDLibor3MSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
    pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, true);
    
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, true);    
     
    pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH, 'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH,'PSANo');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSANo');
%     pricingOutClosed = pricer.computePriceClosedSimple(valueDateH);
    
    callablePrice(idx2) = pricingOutMC1.callable.npv;
    nonCallPrice(idx2) = pricingOutMC1.nonCall.npv;
%     closedPrice(idx2) = pricingOutClosed.pricerInfo.nonCall.npv;
    
    for idx3 = 1:length(pricingOutMC1.nonCall.cashflow_npv_e)
%     for idx3 = 1:length(pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e)
        nonCallPrice_cashflowNPV(idx2, idx3) = pricingOutMC1.nonCall.cashflow_npv_e(idx3);
%         closedPrice_cashflowNPV(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
%         
%         closedPrice_fwdCD(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdCD(idx3);
%         closedPrice_fwdKTB(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdKTB(idx3);
%         closedPrice_fwdSpread(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSpread(idx3);
%         closedPrice_fwdMu(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdMu(idx3);
%         closedPrice_fwdSigma(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSigma(idx3);
%                 
    end
end

aaa =1.0;


In [ ]:

testCD_CDKTB3MSpread_CK1F_CK1F_20220620_vega.m

clc
clear
valueDate = '2022-06-20';

maxIdx = 0;
zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;
volShiftAmount = 0.001;

callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% CD-USDLibor3M Spread Range Accrual Pricing start

%RA info
rangeAccrualParams.payCcy = 'KRW';
rangeAccrualParams.startDate = H_Date('2020-02-03');
rangeAccrualParams.maturity = 15;

valueDateH = H_Date(valueDate);

nominal = 10000;
coupon = 0.033;
couponFreq = 1;

% Long: KRW
% Short: USD
tenorL = 0.25;
tenorS = 0.25;

lowerS = 0.0;
upperS = 999;

% 
% lower = -1;
% upper = 0.05;

tenor = 0.25;

lower = -999;
upper = 999;

%resetDate startDate endDate paymentDate
couponScheduleInt = [20200203	20200203	20210203	20210203
20210203	20210203	20220203	20220203
20220203	20220203	20230203	20230203
20230203	20230203	20240205	20240205
20240205	20240205	20250203	20250203
20250203	20250203	20260203	20260203
20260203	20260203	20270203	20270203
20270203	20270203	20280203	20280203
20280203	20280203	20290205	20290205
20290205	20290205	20300206	20300206
20300206	20300206	20310203	20310203
20310203	20310203	20320203	20320203
20320203	20320203	20330203	20330203
20330203	20330203	20340203	20340203
20340203	20340203	20350205	20350205
];

endDate = H_Date(couponScheduleInt(size(couponScheduleInt,1),4));

%% CK1F model param matu setting start
% matu = 15 + 1.0/365.0;
matu = DateDiff(H_Date(couponScheduleInt(end,4)),H_Date(valueDate))/365.0;
% matu = matu -10.0;

%% end

 rangeDays = zeros(15,1);
%     rangeDays(1) = 1;

for i=1:length(rangeDays)
    if DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) >= 0
        rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) + 1;
    end
end


%resetDate startDate endDate paymentDate
callScheduleInt = [20200203	20200203	20200210	20200210
20210127	20210127	20210203	20210203
20220124	20220124	20220203	20220203
20230127	20230127	20230203	20230203
20240129	20240129	20240205	20240205
20250122	20250122	20250203	20250203
20260127	20260127	20260203	20260203
20270127	20270127	20270203	20270203
20280124	20280124	20280203	20280203
20290129	20290129	20290205	20290205
20300128	20300128	20300206	20300206
20310127	20310127	20310203	20310203
20320127	20320127	20320203	20320203
20330124	20330124	20330203	20330203
20340127	20340127	20340203	20340203
];

callYN = ones(15,1);
callYN(1) = 0;

manualScheduleYN = 1;

rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                'tenorS','lower','upper','rangeDays','callYN', ...
                                'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                        {nominal,coupon,couponFreq,tenorL,tenorS,lower,upper,...
                        rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});
% 
% tenorL = 0.25;
% tenorS = 0.25;
% 
% tenor = 0.25;
% 
% lowerS = 0.0;
% upperS = 999;
% 
% 
% lower = 0.0;
% upper = 0.05;

rangeAccrualInfo('lowerS') =lowerS;
rangeAccrualInfo('upperS') =upperS;
rangeAccrualInfo('tenor') = 0.25;

rangeAccrualParams.params = rangeAccrualInfo;

% rangeAccrual = SpreadRangeAccrual(rangeAccrualParams);
rangeAccrual = CDCMSSpreadDualRangeAccrual(rangeAccrualParams);

% maxIdx =2.5;

maxIdx = 1;

rmseSimul = zeros(100,1);
for idx2=1:3
    %% KRW Model building

    %% import and construct a zeroCurve
    rawData = [0.002739726	0.999952523
0.25	0.9950684
0.5	0.988031579
0.75	0.979080331
1	0.968776974
1.25	0.959202682
1.5	0.948993277
1.75	0.939322512
2	0.929369492
2.25	0.920482183
2.5	0.911621558
2.75	0.902762032
3	0.893905238
3.25	0.885611153
3.5	0.877424651
3.75	0.869320968
4	0.861299205
4.25	0.853502656
4.5	0.845805585
4.75	0.838202041
5	0.830690747
5.25	0.823531148
5.5	0.816487433
5.75	0.809552209
6	0.802723731
6.25	0.796000289
6.5	0.789380204
6.75	0.782861832
7	0.776443559
7.25	0.770025284
7.5	0.763695404
7.75	0.757455789
8	0.751305075
8.25	0.74524192
8.5	0.73926501
8.75	0.73337305
9	0.727564772
9.25	0.721838929
9.5	0.716194297
9.75	0.710629672
10	0.705143874
10.25	0.699194275
10.5	0.693278485
10.75	0.687420297
11	0.681619095
11.25	0.675874273
11.5	0.670185229
11.75	0.664551369
12	0.658972106
12.25	0.654130635
12.5	0.649391356
12.75	0.644722276
13	0.640122375
13.25	0.635590647
13.5	0.631126108
13.75	0.626727791
14	0.622394747
14.25	0.618126045
14.5	0.613920769
14.75	0.609778022
15	0.605696922
15.25	0.60235689
15.5	0.599139226
15.75	0.595995535
16	0.592924834
16.25	0.589926161
16.5	0.586998583
16.75	0.58414119
17	0.581353097
17.25	0.578633444
17.5	0.575981393
17.75	0.573396131
18	0.570876865
18.25	0.568422827
18.5	0.566033269
18.75	0.563707467
19	0.561444716
19.25	0.559244332
19.5	0.557105652
19.75	0.555028035
20	0.553010856
20.25	0.550157221
20.5	0.547261794
20.75	0.544409494
21	0.541599805
21.25	0.538832219
21.5	0.536106239
21.75	0.533421375
22	0.530777148
22.25	0.528173085
22.5	0.525608723
22.75	0.523083608
23	0.520597293
23.25	0.518149339
23.5	0.515739316
23.75	0.513366801
24	0.511031378
24.25	0.508732641
24.5	0.506470189
24.75	0.50424363
25	0.502052578
25.25	0.498564056
25.5	0.494988089
25.75	0.491434812
26	0.487904104
26.25	0.484395847
26.5	0.48090992
26.75	0.477446204
27	0.474004582
27.25	0.470584935
27.5	0.467187146
27.75	0.463811098
28	0.460456674
28.25	0.457123757
28.5	0.453812233
28.75	0.450521986
29	0.447252902
29.25	0.444004865
29.5	0.440777762
29.75	0.43757148
30	0.434385905
30.25	0.431221981
30.5	0.428081088
30.75	0.424963073
31	0.421867769
    ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

    
%     %simulation testCase
%     for idx=1:size(rawData,1)
%         rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
%     end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    KRWAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];

% rawData = [0.002739726	0.999952523
% 0.25	0.9950684
% 0.5	0.988031579
% 0.75	0.979080331
% 1	0.968776974
% 1.25	0.959202682
% 1.5	0.948993277
% 1.75	0.939322512
% 2	0.929369492
% 2.25	0.920482183
% 2.5	0.911621558
% 2.75	0.902762032
% 3	0.893905238
% 3.25	0.885611153
% 3.5	0.877424651
% 3.75	0.869320968
% 4	0.861299205
% 4.25	0.853502656
% 4.5	0.845805585
% 4.75	0.838202041
% 5	0.830690747
% 5.25	0.823531148
% 5.5	0.816487433
% 5.75	0.809552209
% 6	0.802723731
% 6.25	0.796000289
% 6.5	0.789380204
% 6.75	0.782861832
% 7	0.776443559
% 7.25	0.770025284
% 7.5	0.763695404
% 7.75	0.757455789
% 8	0.751305075
% 8.25	0.74524192
% 8.5	0.73926501
% 8.75	0.73337305
% 9	0.727564772
% 9.25	0.721838929
% 9.5	0.716194297
% 9.75	0.710629672
% 10	0.705143874
% 10.25	0.699194275
% 10.5	0.693278485
% 10.75	0.687420297
% 11	0.681619095
% 11.25	0.675874273
% 11.5	0.670185229
% 11.75	0.664551369
% 12	0.658972106
% 12.25	0.654130635
% 12.5	0.649391356
% 12.75	0.644722276
% 13	0.640122375
% 13.25	0.635590647
% 13.5	0.631126108
% 13.75	0.626727791
% 14	0.622394747
% 14.25	0.618126045
% 14.5	0.613920769
% 14.75	0.609778022
% 15	0.605696922
% 15.25	0.60235689
% 15.5	0.599139226
% 15.75	0.595995535
% 16	0.592924834
% 16.25	0.589926161
% 16.5	0.586998583
% 16.75	0.58414119
% 17	0.581353097
% 17.25	0.578633444
% 17.5	0.575981393
% 17.75	0.573396131
% 18	0.570876865
% 18.25	0.568422827
% 18.5	0.566033269
% 18.75	0.563707467
% 19	0.561444716
% 19.25	0.559244332
% 19.5	0.557105652
% 19.75	0.555028035
% 20	0.553010856
% 20.25	0.550157221
% 20.5	0.547261794
% 20.75	0.544409494
% 21	0.541599805
% 21.25	0.538832219
% 21.5	0.536106239
% 21.75	0.533421375
% 22	0.530777148
% 22.25	0.528173085
% 22.5	0.525608723
% 22.75	0.523083608
% 23	0.520597293
% 23.25	0.518149339
% 23.5	0.515739316
% 23.75	0.513366801
% 24	0.511031378
% 24.25	0.508732641
% 24.5	0.506470189
% 24.75	0.50424363
% 25	0.502052578
% 25.25	0.498564056
% 25.5	0.494988089
% 25.75	0.491434812
% 26	0.487904104
% 26.25	0.484395847
% 26.5	0.48090992
% 26.75	0.477446204
% 27	0.474004582
% 27.25	0.470584935
% 27.5	0.467187146
% 27.75	0.463811098
% 28	0.460456674
% 28.25	0.457123757
% 28.5	0.453812233
% 28.75	0.450521986
% 29	0.447252902
% 29.25	0.444004865
% 29.5	0.440777762
% 29.75	0.43757148
% 30	0.434385905
% 30.25	0.431221981
% 30.5	0.428081088
% 30.75	0.424963073
% 31	0.421867769
%     ];


for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

%  %simulation testCase
%     for idx=1:size(rawData,1)
%         rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
%     end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);

    KRWAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurfaceNormal



rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.01025373	0.00996465	0.00939453	0.0088449	0.00838736	0.00835816	0.00827411	0.00844853	0.00844853
2	0.00987033	0.00941532	0.00896799	0.00850851	0.00803853	0.00809306	0.00814872	0.00832187	0.00832187
3	0.00965448	0.00917228	0.00868278	0.00815841	0.00766413	0.00768619	0.00766987	0.007827	0.007827
4	0.0086456	0.00835905	0.00803055	0.00773857	0.00743965	0.00747184	0.00750208	0.00760185	0.00760185
5	0.008024	0.00784518	0.00761802	0.00741163	0.00719132	0.00718529	0.00716174	0.0072174	0.0072174
7	0.00777954	0.0075868	0.00743466	0.00726955	0.00710966	0.00717731	0.00726665	0.00728989	0.00728989
10	0.00697384	0.00701303	0.00705259	0.00704171	0.0070555	0.00714878	0.00723997	0.00722954	0.00722954
15	0.00731508	0.00731858	0.00728587	0.00725674	0.0072209	0.00720153	0.00717821	0.0071639	0.0071639
    ];

    rawDataSim = rawDataSwaption;

    for idxhong1=2:size(rawDataSwaption,1)
        for idxhong2=2:size(rawDataSwaption,2)
            rawDataSim(idxhong1,idxhong2) = rawDataSwaption(idxhong1,idxhong2) + volShiftAmount*(idx2-1);
        end
    end

    rawDataSwaption = rawDataSim;
    
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    KRWAllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a KRWBlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(KRWAllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = KRWBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= KRWBlack.NormalATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
   %% create a CK1FRisky Model with initial modelParams
%     vol_r.tenor = [1;2;3;4;5;7;10;15;20];
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;
%     meanR_r = 0.0;
    
    
    joint_swap_tenor = 1;
%     targetSize = 10;

    targetSize = 15;
    
    freq = 4;
    
    modelParams = containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'},...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
    
%     import irModels.*;
%     import targetFunctions.*;
    
    KRWCK1FRisky = CK1FRisky(IRModelRisky(Model(KRWAllMktData, modelParams)));
    
    tic
%     out = KRWLGM2F.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
    % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
%     out = KRWCK1FRisky.CalibrateToATMDiagSwaption(KRWBlack,'global');
    out = KRWCK1FRisky.CalibrateToATMDiagSwaptionNormal(KRWBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        KRWVol_r(idx2,idx3) = out.vol_r.quote(idx3);
        KRWRe(idx2,idx3) = out.re(idx3);
    end
    KRWRMSE(idx2) = out.rmseTotal;
    
    toc
    %% CK1FRisky model building end
    %% KRW model building end

    %% USD model building start
    %% import and construct a zeroCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

%  %simulation testCase
%     for idx=1:size(rawData,1)
%         rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
%     end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','USDZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    USDAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];


% rawData = [0.002739726	0.999952523
% 0.25	0.9950684
% 0.5	0.988031579
% 0.75	0.979080331
% 1	0.968776974
% 1.25	0.959202682
% 1.5	0.948993277
% 1.75	0.939322512
% 2	0.929369492
% 2.25	0.920482183
% 2.5	0.911621558
% 2.75	0.902762032
% 3	0.893905238
% 3.25	0.885611153
% 3.5	0.877424651
% 3.75	0.869320968
% 4	0.861299205
% 4.25	0.853502656
% 4.5	0.845805585
% 4.75	0.838202041
% 5	0.830690747
% 5.25	0.823531148
% 5.5	0.816487433
% 5.75	0.809552209
% 6	0.802723731
% 6.25	0.796000289
% 6.5	0.789380204
% 6.75	0.782861832
% 7	0.776443559
% 7.25	0.770025284
% 7.5	0.763695404
% 7.75	0.757455789
% 8	0.751305075
% 8.25	0.74524192
% 8.5	0.73926501
% 8.75	0.73337305
% 9	0.727564772
% 9.25	0.721838929
% 9.5	0.716194297
% 9.75	0.710629672
% 10	0.705143874
% 10.25	0.699194275
% 10.5	0.693278485
% 10.75	0.687420297
% 11	0.681619095
% 11.25	0.675874273
% 11.5	0.670185229
% 11.75	0.664551369
% 12	0.658972106
% 12.25	0.654130635
% 12.5	0.649391356
% 12.75	0.644722276
% 13	0.640122375
% 13.25	0.635590647
% 13.5	0.631126108
% 13.75	0.626727791
% 14	0.622394747
% 14.25	0.618126045
% 14.5	0.613920769
% 14.75	0.609778022
% 15	0.605696922
% 15.25	0.60235689
% 15.5	0.599139226
% 15.75	0.595995535
% 16	0.592924834
% 16.25	0.589926161
% 16.5	0.586998583
% 16.75	0.58414119
% 17	0.581353097
% 17.25	0.578633444
% 17.5	0.575981393
% 17.75	0.573396131
% 18	0.570876865
% 18.25	0.568422827
% 18.5	0.566033269
% 18.75	0.563707467
% 19	0.561444716
% 19.25	0.559244332
% 19.5	0.557105652
% 19.75	0.555028035
% 20	0.553010856
% 20.25	0.550157221
% 20.5	0.547261794
% 20.75	0.544409494
% 21	0.541599805
% 21.25	0.538832219
% 21.5	0.536106239
% 21.75	0.533421375
% 22	0.530777148
% 22.25	0.528173085
% 22.5	0.525608723
% 22.75	0.523083608
% 23	0.520597293
% 23.25	0.518149339
% 23.5	0.515739316
% 23.75	0.513366801
% 24	0.511031378
% 24.25	0.508732641
% 24.5	0.506470189
% 24.75	0.50424363
% 25	0.502052578
% 25.25	0.498564056
% 25.5	0.494988089
% 25.75	0.491434812
% 26	0.487904104
% 26.25	0.484395847
% 26.5	0.48090992
% 26.75	0.477446204
% 27	0.474004582
% 27.25	0.470584935
% 27.5	0.467187146
% 27.75	0.463811098
% 28	0.460456674
% 28.25	0.457123757
% 28.5	0.453812233
% 28.75	0.450521986
% 29	0.447252902
% 29.25	0.444004865
% 29.5	0.440777762
% 29.75	0.43757148
% 30	0.434385905
% 30.25	0.431221981
% 30.5	0.428081088
% 30.75	0.424963073
% 31	0.421867769
%     ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

%  %simulation testCase
%     for idx=1:size(rawData,1)
%         rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
%     end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);
    USDAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
% GIROPremiumToNormalVol From LIB
% rawDataSwaption = [0	1	2	3	4	5	7	10	12	15	20
% 1	0.0041	0.0045	0.0052	0.0061	0.0066	0.0071	0.0077	0.0080	0.0081	0.0082
% 2	0.0050	0.0057	0.0064	0.0066	0.0068	0.0071	0.0074	0.0077	0.0077	0.0078
% 3	0.0064	0.0069	0.0068	0.0068	0.0068	0.0072	0.0076	0.0078	0.0079	0.0079
% 4	0.0076	0.0071	0.0071	0.0070	0.0071	0.0072	0.0077	0.0081	0.0081	0.0081
% 5	0.0069	0.0070	0.0070	0.0071	0.0071	0.0070	0.0076	0.0083	0.0084	0.0083
% 7	0.0073	0.0074	0.0075	0.0072	0.0070	0.0075	0.0087	0.0096	0.0096	0.0095
% 10	0.0066	0.0066	0.0071	0.0074	0.0077	0.0092	0.0101	0.0102	0.0102	0.0102
% 15	0.0103	0.0105	0.0107	0.0109	0.0111	0.0108	0.0103	0.0102	0.0102	0.0101
% 20	0.0088	0.0089	0.0090	0.0091	0.0091	0.0091	0.0090	0.0089	0.0089	0.0089
%     ];
% 
rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.01025373	0.00996465	0.00939453	0.0088449	0.00838736	0.00835816	0.00827411	0.00844853	0.00844853
2	0.00987033	0.00941532	0.00896799	0.00850851	0.00803853	0.00809306	0.00814872	0.00832187	0.00832187
3	0.00965448	0.00917228	0.00868278	0.00815841	0.00766413	0.00768619	0.00766987	0.007827	0.007827
4	0.0086456	0.00835905	0.00803055	0.00773857	0.00743965	0.00747184	0.00750208	0.00760185	0.00760185
5	0.008024	0.00784518	0.00761802	0.00741163	0.00719132	0.00718529	0.00716174	0.0072174	0.0072174
7	0.00777954	0.0075868	0.00743466	0.00726955	0.00710966	0.00717731	0.00726665	0.00728989	0.00728989
10	0.00697384	0.00701303	0.00705259	0.00704171	0.0070555	0.00714878	0.00723997	0.00722954	0.00722954
15	0.00731508	0.00731858	0.00728587	0.00725674	0.0072209	0.00720153	0.00717821	0.0071639	0.0071639
    ];


    rawDataSim = rawDataSwaption;

    for idxhong1=2:size(rawDataSwaption,1)
        for idxhong2=2:size(rawDataSwaption,2)
            rawDataSim(idxhong1,idxhong2) = rawDataSwaption(idxhong1,idxhong2) + volShiftAmount*(idx2-1);
        end
    end

    rawDataSwaption = rawDataSim;
    
    mktData = containers.Map({'rawData','asOfDate','ccy','name'},...
                            {rawDataSwaption,valueDate,'USD','USDSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    USDAllMktData('swaptionVol') = swaptionVolMktData;
    
   %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    USDBlack = IRBlack(IRModel(Model(USDAllMktData,modelParams))); 
    
    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);
    
    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = USDBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= USDBlack.BlackATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = USDBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
    %% create a CK1FModel with initial modelParams
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;

%     matu = 15 + 1.0/365.0;
    joint_swap_tenor = 1;
    targetSize = 15;
    freq = 4;
    
    modelParamsUSD= containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'}, ...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
    USDCK1F = CK1F(IRModel(Model(USDAllMktData, modelParamsUSD)));
    
    tic
    outUSD = USDCK1F.CalibrateToATMDiagSwaptionNormal(USDBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        USDVol_r(idx2,idx3) = outUSD.vol_r.quote(idx3);
        USDRe(idx2,idx3) = outUSD.re(idx3);
    end
    USDRMSE(idx2) = outUSD.rmseTotal;
    
    toc
    %% USD model building end

    %% USDKRW fx model buidling start
    rawData = [0.083333333	0.0
0.25	0.0
0.5	0.0
1	0.0
2	0.0
3	0.0
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','USDKRWATMTermVol'});
    impliedTermVolMktData = MktData(mktData);
%     impliedTermVole = FXVolSurface(impliedTermVolMktData);
%     impliedVolSurface.params('moneyNessType') = 'deltaFwdMoneyNess';
    USDKRWAllMktData =containers.Map('impliedTermVol',impliedTermVolMktData);
%     AllMktData('impliedVolSurface') = impliedTermVolMktData;

    %% create  an FXBlackModel to calculate market PlainVanilla Options
%     modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});
%     USDKRWBlack = FXBlack(FXModel(Model(USDKRWAllMktData,modelParams))); 
    %% USDKRW fx model building end

    %% MultiAssetModel building start
    multiAssetMktData = containers.Map({'KRW','USD','USDKRW'}, ...
                            {KRWAllMktData, USDAllMktData, USDKRWAllMktData});

    % KRW,KTB                    
%     correlationMatrix = ones(2,2);
%     correlationMatrix(1,2) = 0.3;
%     correlationMatrix(2,1) = correlationMatrix(1,2);

    % KRW, USD & USDKRW
    correlationMatrix = ones(3, 3);
    correlationMatrix(1,2) = 0.845;
%     correlationMatrix(1,2) = 0.6;
    
%     correlationMatrix(1,2) = 0.7;
    
    correlationMatrix(1,3) = 0.0;
    correlationMatrix(2,3) = 0.0;
    correlationMatrix(2,1) = correlationMatrix(1,2);
    correlationMatrix(3,1) = correlationMatrix(1,3);
    correlationMatrix(3,2) = correlationMatrix(2,3);
    
    multiAssetMktData('correlationMatrix') = correlationMatrix;
    
    modelNameMap = containers.Map({'KRW','USD'},{KRWCK1FRisky, USDCK1F});
    modelParams = containers.Map({'modelNameMap'}, {modelNameMap});
    % correlationNames = {'KRW','USD'};
    modelParams('stochasticModelNames') = {'KRW','USD'};
    modelParams('correlationMatrix') = correlationMatrix;
    modelParams('refModelName') = 'KRW';
    % modelParams('useQuasiRandomYN') = 'true';
    modelParams('useQuasiRandomYN') = 'false';
    modelParams('fxVolCalibrated') = true;
    
    KRWCK1FRiskyUSDCK1F = MultiAssetModelCK1FCK1F(multiAssetMktData, modelParams);
    %% MultiAssetModel building end
   
    %% NumMethod Init Start
    numOfRandom = 40000;
    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                            {'AMC', 7, numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);
    %% NumMethod Init End
    
    % weekly step calculator
    % pricer = Pricer(rangeAccrual,KRWLGM,numMethodAMC);
    % pricingOut = pricer.computePrice(valueDate);
    
    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    % pricer = DailyWeeklyRiskyPricer_CDKTBSpreadRA(rangeAccrual,KRWLGM1FRiskyUSDLGM1F,numMethodAMC,false);
%     pricer = DailyWeeklyRiskyPricer_CDUSDLibor3MSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
    pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, true);    
     
    pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH, 'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH,'PSANo');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSANo');
%     pricingOutClosed = pricer.computePriceClosedSimple(valueDateH);
    
    callablePrice(idx2) = pricingOutMC1.callable.npv;
    nonCallPrice(idx2) = pricingOutMC1.nonCall.npv;
%     closedPrice(idx2) = pricingOutClosed.pricerInfo.nonCall.npv;
    
%     for idx3 = 1:length(pricingOutMC1.nonCall.cashflow_npv_e)
% %     for idx3 = 1:length(pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e)
%         nonCallPrice_cashflowNPV(idx2, idx3) = pricingOutMC1.nonCall.cashflow_npv_e(idx3);
% %         closedPrice_cashflowNPV(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
% %         
% %         closedPrice_fwdCD(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdCD(idx3);
% %         closedPrice_fwdKTB(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdKTB(idx3);
% %         closedPrice_fwdSpread(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSpread(idx3);
% %         closedPrice_fwdMu(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdMu(idx3);
% %         closedPrice_fwdSigma(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSigma(idx3);
% %                 
%     end
end

aaa =1.0;


In [ ]:
testCD_CDKTB3MSpread_CK1F_CK1F_20220620_irs_pointbypoint_delta.m

clc
clear
valueDate = '2022-06-20';

maxIdx = 0;
zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;

callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% CD-USDLibor3M Spread Range Accrual Pricing start

%RA info
rangeAccrualParams.payCcy = 'KRW';
rangeAccrualParams.startDate = H_Date('2020-02-03');
rangeAccrualParams.maturity = 15;

valueDateH = H_Date(valueDate);

nominal = 10000;
coupon = 0.033;
couponFreq = 1;

% Long: KRW
% Short: USD
tenorL = 0.25;
tenorS = 0.25;

lowerS = 0.0;
upperS = 999;

% 
% lower = -1;
% upper = 0.05;

tenor = 0.25;

lower = -999;
upper = 999;

%resetDate startDate endDate paymentDate
couponScheduleInt = [20200203	20200203	20210203	20210203
20210203	20210203	20220203	20220203
20220203	20220203	20230203	20230203
20230203	20230203	20240205	20240205
20240205	20240205	20250203	20250203
20250203	20250203	20260203	20260203
20260203	20260203	20270203	20270203
20270203	20270203	20280203	20280203
20280203	20280203	20290205	20290205
20290205	20290205	20300206	20300206
20300206	20300206	20310203	20310203
20310203	20310203	20320203	20320203
20320203	20320203	20330203	20330203
20330203	20330203	20340203	20340203
20340203	20340203	20350205	20350205
];

endDate = H_Date(couponScheduleInt(size(couponScheduleInt,1),4));

%% CK1F model param matu setting start
% matu = 15 + 1.0/365.0;
matu = DateDiff(H_Date(couponScheduleInt(end,4)),H_Date(valueDate))/365.0;
% matu = matu -10.0;

%% end

 rangeDays = zeros(15,1);
%     rangeDays(1) = 1;

for i=1:length(rangeDays)
    if DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) >= 0
        rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) + 1;
    end
end


%resetDate startDate endDate paymentDate
callScheduleInt = [20200203	20200203	20200210	20200210
20210127	20210127	20210203	20210203
20220124	20220124	20220203	20220203
20230127	20230127	20230203	20230203
20240129	20240129	20240205	20240205
20250122	20250122	20250203	20250203
20260127	20260127	20260203	20260203
20270127	20270127	20270203	20270203
20280124	20280124	20280203	20280203
20290129	20290129	20290205	20290205
20300128	20300128	20300206	20300206
20310127	20310127	20310203	20310203
20320127	20320127	20320203	20320203
20330124	20330124	20330203	20330203
20340127	20340127	20340203	20340203
];

callYN = ones(15,1);
callYN(1) = 0;

manualScheduleYN = 1;

rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                'tenorS','lower','upper','rangeDays','callYN', ...
                                'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                        {nominal,coupon,couponFreq,tenorL,tenorS,lower,upper,...
                        rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});
% 
% tenorL = 0.25;
% tenorS = 0.25;
% 
% tenor = 0.25;
% 
% lowerS = 0.0;
% upperS = 999;
% 
% 
% lower = 0.0;
% upper = 0.05;

rangeAccrualInfo('lowerS') =lowerS;
rangeAccrualInfo('upperS') =upperS;
rangeAccrualInfo('tenor') = 0.25;

rangeAccrualParams.params = rangeAccrualInfo;

% rangeAccrual = SpreadRangeAccrual(rangeAccrualParams);
rangeAccrual = CDCMSSpreadDualRangeAccrual(rangeAccrualParams);

% maxIdx =2.5;

maxIdx = 1;

rmseSimul = zeros(100,1);
for idx2=1:3
    %% KRW Model building

    %% import and construct a zeroCurve
    rawData = [0.002739726	0.999952523
0.25	0.9950684
0.5	0.988031579
0.75	0.979080331
1	0.968776974
1.25	0.959202682
1.5	0.948993277
1.75	0.939322512
2	0.929369492
2.25	0.920482183
2.5	0.911621558
2.75	0.902762032
3	0.893905238
3.25	0.885611153
3.5	0.877424651
3.75	0.869320968
4	0.861299205
4.25	0.853502656
4.5	0.845805585
4.75	0.838202041
5	0.830690747
5.25	0.823531148
5.5	0.816487433
5.75	0.809552209
6	0.802723731
6.25	0.796000289
6.5	0.789380204
6.75	0.782861832
7	0.776443559
7.25	0.770025284
7.5	0.763695404
7.75	0.757455789
8	0.751305075
8.25	0.74524192
8.5	0.73926501
8.75	0.73337305
9	0.727564772
9.25	0.721838929
9.5	0.716194297
9.75	0.710629672
10	0.705143874
10.25	0.699194275
10.5	0.693278485
10.75	0.687420297
11	0.681619095
11.25	0.675874273
11.5	0.670185229
11.75	0.664551369
12	0.658972106
12.25	0.654130635
12.5	0.649391356
12.75	0.644722276
13	0.640122375
13.25	0.635590647
13.5	0.631126108
13.75	0.626727791
14	0.622394747
14.25	0.618126045
14.5	0.613920769
14.75	0.609778022
15	0.605696922
15.25	0.60235689
15.5	0.599139226
15.75	0.595995535
16	0.592924834
16.25	0.589926161
16.5	0.586998583
16.75	0.58414119
17	0.581353097
17.25	0.578633444
17.5	0.575981393
17.75	0.573396131
18	0.570876865
18.25	0.568422827
18.5	0.566033269
18.75	0.563707467
19	0.561444716
19.25	0.559244332
19.5	0.557105652
19.75	0.555028035
20	0.553010856
20.25	0.550157221
20.5	0.547261794
20.75	0.544409494
21	0.541599805
21.25	0.538832219
21.5	0.536106239
21.75	0.533421375
22	0.530777148
22.25	0.528173085
22.5	0.525608723
22.75	0.523083608
23	0.520597293
23.25	0.518149339
23.5	0.515739316
23.75	0.513366801
24	0.511031378
24.25	0.508732641
24.5	0.506470189
24.75	0.50424363
25	0.502052578
25.25	0.498564056
25.5	0.494988089
25.75	0.491434812
26	0.487904104
26.25	0.484395847
26.5	0.48090992
26.75	0.477446204
27	0.474004582
27.25	0.470584935
27.5	0.467187146
27.75	0.463811098
28	0.460456674
28.25	0.457123757
28.5	0.453812233
28.75	0.450521986
29	0.447252902
29.25	0.444004865
29.5	0.440777762
29.75	0.43757148
30	0.434385905
30.25	0.431221981
30.5	0.428081088
30.75	0.424963073
31	0.421867769
    ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

    
%     %simulation testCase
%     for idx=1:size(rawData,1)
%         rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
%     end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    KRWAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];

% rawData = [0.002739726	0.999952523
% 0.25	0.9950684
% 0.5	0.988031579
% 0.75	0.979080331
% 1	0.968776974
% 1.25	0.959202682
% 1.5	0.948993277
% 1.75	0.939322512
% 2	0.929369492
% 2.25	0.920482183
% 2.5	0.911621558
% 2.75	0.902762032
% 3	0.893905238
% 3.25	0.885611153
% 3.5	0.877424651
% 3.75	0.869320968
% 4	0.861299205
% 4.25	0.853502656
% 4.5	0.845805585
% 4.75	0.838202041
% 5	0.830690747
% 5.25	0.823531148
% 5.5	0.816487433
% 5.75	0.809552209
% 6	0.802723731
% 6.25	0.796000289
% 6.5	0.789380204
% 6.75	0.782861832
% 7	0.776443559
% 7.25	0.770025284
% 7.5	0.763695404
% 7.75	0.757455789
% 8	0.751305075
% 8.25	0.74524192
% 8.5	0.73926501
% 8.75	0.73337305
% 9	0.727564772
% 9.25	0.721838929
% 9.5	0.716194297
% 9.75	0.710629672
% 10	0.705143874
% 10.25	0.699194275
% 10.5	0.693278485
% 10.75	0.687420297
% 11	0.681619095
% 11.25	0.675874273
% 11.5	0.670185229
% 11.75	0.664551369
% 12	0.658972106
% 12.25	0.654130635
% 12.5	0.649391356
% 12.75	0.644722276
% 13	0.640122375
% 13.25	0.635590647
% 13.5	0.631126108
% 13.75	0.626727791
% 14	0.622394747
% 14.25	0.618126045
% 14.5	0.613920769
% 14.75	0.609778022
% 15	0.605696922
% 15.25	0.60235689
% 15.5	0.599139226
% 15.75	0.595995535
% 16	0.592924834
% 16.25	0.589926161
% 16.5	0.586998583
% 16.75	0.58414119
% 17	0.581353097
% 17.25	0.578633444
% 17.5	0.575981393
% 17.75	0.573396131
% 18	0.570876865
% 18.25	0.568422827
% 18.5	0.566033269
% 18.75	0.563707467
% 19	0.561444716
% 19.25	0.559244332
% 19.5	0.557105652
% 19.75	0.555028035
% 20	0.553010856
% 20.25	0.550157221
% 20.5	0.547261794
% 20.75	0.544409494
% 21	0.541599805
% 21.25	0.538832219
% 21.5	0.536106239
% 21.75	0.533421375
% 22	0.530777148
% 22.25	0.528173085
% 22.5	0.525608723
% 22.75	0.523083608
% 23	0.520597293
% 23.25	0.518149339
% 23.5	0.515739316
% 23.75	0.513366801
% 24	0.511031378
% 24.25	0.508732641
% 24.5	0.506470189
% 24.75	0.50424363
% 25	0.502052578
% 25.25	0.498564056
% 25.5	0.494988089
% 25.75	0.491434812
% 26	0.487904104
% 26.25	0.484395847
% 26.5	0.48090992
% 26.75	0.477446204
% 27	0.474004582
% 27.25	0.470584935
% 27.5	0.467187146
% 27.75	0.463811098
% 28	0.460456674
% 28.25	0.457123757
% 28.5	0.453812233
% 28.75	0.450521986
% 29	0.447252902
% 29.25	0.444004865
% 29.5	0.440777762
% 29.75	0.43757148
% 30	0.434385905
% 30.25	0.431221981
% 30.5	0.428081088
% 30.75	0.424963073
% 31	0.421867769
%     ];


for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

 %simulation testCase
    for idx=1:size(rawData,1)
        rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
    end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);

    KRWAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurfaceNormal



rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.01025373	0.00996465	0.00939453	0.0088449	0.00838736	0.00835816	0.00827411	0.00844853	0.00844853
2	0.00987033	0.00941532	0.00896799	0.00850851	0.00803853	0.00809306	0.00814872	0.00832187	0.00832187
3	0.00965448	0.00917228	0.00868278	0.00815841	0.00766413	0.00768619	0.00766987	0.007827	0.007827
4	0.0086456	0.00835905	0.00803055	0.00773857	0.00743965	0.00747184	0.00750208	0.00760185	0.00760185
5	0.008024	0.00784518	0.00761802	0.00741163	0.00719132	0.00718529	0.00716174	0.0072174	0.0072174
7	0.00777954	0.0075868	0.00743466	0.00726955	0.00710966	0.00717731	0.00726665	0.00728989	0.00728989
10	0.00697384	0.00701303	0.00705259	0.00704171	0.0070555	0.00714878	0.00723997	0.00722954	0.00722954
15	0.00731508	0.00731858	0.00728587	0.00725674	0.0072209	0.00720153	0.00717821	0.0071639	0.0071639
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    KRWAllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a KRWBlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(KRWAllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = KRWBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= KRWBlack.NormalATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
   %% create a CK1FRisky Model with initial modelParams
%     vol_r.tenor = [1;2;3;4;5;7;10;15;20];
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;
%     meanR_r = 0.0;
    
    
    joint_swap_tenor = 1;
%     targetSize = 10;

    targetSize = 15;
    
    freq = 4;
    
    modelParams = containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'},...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
    
%     import irModels.*;
%     import targetFunctions.*;
    
    KRWCK1FRisky = CK1FRisky(IRModelRisky(Model(KRWAllMktData, modelParams)));
    
    tic
%     out = KRWLGM2F.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
    % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
%     out = KRWCK1FRisky.CalibrateToATMDiagSwaption(KRWBlack,'global');
    out = KRWCK1FRisky.CalibrateToATMDiagSwaptionNormal(KRWBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        KRWVol_r(idx2,idx3) = out.vol_r.quote(idx3);
        KRWRe(idx2,idx3) = out.re(idx3);
    end
    KRWRMSE(idx2) = out.rmseTotal;
    
    toc
    %% CK1FRisky model building end
    %% KRW model building end

    %% USD model building start
    %% import and construct a zeroCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

%  %simulation testCase
%     for idx=1:size(rawData,1)
%         rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
%     end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','USDZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    USDAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];


% rawData = [0.002739726	0.999952523
% 0.25	0.9950684
% 0.5	0.988031579
% 0.75	0.979080331
% 1	0.968776974
% 1.25	0.959202682
% 1.5	0.948993277
% 1.75	0.939322512
% 2	0.929369492
% 2.25	0.920482183
% 2.5	0.911621558
% 2.75	0.902762032
% 3	0.893905238
% 3.25	0.885611153
% 3.5	0.877424651
% 3.75	0.869320968
% 4	0.861299205
% 4.25	0.853502656
% 4.5	0.845805585
% 4.75	0.838202041
% 5	0.830690747
% 5.25	0.823531148
% 5.5	0.816487433
% 5.75	0.809552209
% 6	0.802723731
% 6.25	0.796000289
% 6.5	0.789380204
% 6.75	0.782861832
% 7	0.776443559
% 7.25	0.770025284
% 7.5	0.763695404
% 7.75	0.757455789
% 8	0.751305075
% 8.25	0.74524192
% 8.5	0.73926501
% 8.75	0.73337305
% 9	0.727564772
% 9.25	0.721838929
% 9.5	0.716194297
% 9.75	0.710629672
% 10	0.705143874
% 10.25	0.699194275
% 10.5	0.693278485
% 10.75	0.687420297
% 11	0.681619095
% 11.25	0.675874273
% 11.5	0.670185229
% 11.75	0.664551369
% 12	0.658972106
% 12.25	0.654130635
% 12.5	0.649391356
% 12.75	0.644722276
% 13	0.640122375
% 13.25	0.635590647
% 13.5	0.631126108
% 13.75	0.626727791
% 14	0.622394747
% 14.25	0.618126045
% 14.5	0.613920769
% 14.75	0.609778022
% 15	0.605696922
% 15.25	0.60235689
% 15.5	0.599139226
% 15.75	0.595995535
% 16	0.592924834
% 16.25	0.589926161
% 16.5	0.586998583
% 16.75	0.58414119
% 17	0.581353097
% 17.25	0.578633444
% 17.5	0.575981393
% 17.75	0.573396131
% 18	0.570876865
% 18.25	0.568422827
% 18.5	0.566033269
% 18.75	0.563707467
% 19	0.561444716
% 19.25	0.559244332
% 19.5	0.557105652
% 19.75	0.555028035
% 20	0.553010856
% 20.25	0.550157221
% 20.5	0.547261794
% 20.75	0.544409494
% 21	0.541599805
% 21.25	0.538832219
% 21.5	0.536106239
% 21.75	0.533421375
% 22	0.530777148
% 22.25	0.528173085
% 22.5	0.525608723
% 22.75	0.523083608
% 23	0.520597293
% 23.25	0.518149339
% 23.5	0.515739316
% 23.75	0.513366801
% 24	0.511031378
% 24.25	0.508732641
% 24.5	0.506470189
% 24.75	0.50424363
% 25	0.502052578
% 25.25	0.498564056
% 25.5	0.494988089
% 25.75	0.491434812
% 26	0.487904104
% 26.25	0.484395847
% 26.5	0.48090992
% 26.75	0.477446204
% 27	0.474004582
% 27.25	0.470584935
% 27.5	0.467187146
% 27.75	0.463811098
% 28	0.460456674
% 28.25	0.457123757
% 28.5	0.453812233
% 28.75	0.450521986
% 29	0.447252902
% 29.25	0.444004865
% 29.5	0.440777762
% 29.75	0.43757148
% 30	0.434385905
% 30.25	0.431221981
% 30.5	0.428081088
% 30.75	0.424963073
% 31	0.421867769
%     ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

 %simulation testCase
    for idx=1:size(rawData,1)
        rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
    end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);
    USDAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
% GIROPremiumToNormalVol From LIB
% rawDataSwaption = [0	1	2	3	4	5	7	10	12	15	20
% 1	0.0041	0.0045	0.0052	0.0061	0.0066	0.0071	0.0077	0.0080	0.0081	0.0082
% 2	0.0050	0.0057	0.0064	0.0066	0.0068	0.0071	0.0074	0.0077	0.0077	0.0078
% 3	0.0064	0.0069	0.0068	0.0068	0.0068	0.0072	0.0076	0.0078	0.0079	0.0079
% 4	0.0076	0.0071	0.0071	0.0070	0.0071	0.0072	0.0077	0.0081	0.0081	0.0081
% 5	0.0069	0.0070	0.0070	0.0071	0.0071	0.0070	0.0076	0.0083	0.0084	0.0083
% 7	0.0073	0.0074	0.0075	0.0072	0.0070	0.0075	0.0087	0.0096	0.0096	0.0095
% 10	0.0066	0.0066	0.0071	0.0074	0.0077	0.0092	0.0101	0.0102	0.0102	0.0102
% 15	0.0103	0.0105	0.0107	0.0109	0.0111	0.0108	0.0103	0.0102	0.0102	0.0101
% 20	0.0088	0.0089	0.0090	0.0091	0.0091	0.0091	0.0090	0.0089	0.0089	0.0089
%     ];
% 
rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.01025373	0.00996465	0.00939453	0.0088449	0.00838736	0.00835816	0.00827411	0.00844853	0.00844853
2	0.00987033	0.00941532	0.00896799	0.00850851	0.00803853	0.00809306	0.00814872	0.00832187	0.00832187
3	0.00965448	0.00917228	0.00868278	0.00815841	0.00766413	0.00768619	0.00766987	0.007827	0.007827
4	0.0086456	0.00835905	0.00803055	0.00773857	0.00743965	0.00747184	0.00750208	0.00760185	0.00760185
5	0.008024	0.00784518	0.00761802	0.00741163	0.00719132	0.00718529	0.00716174	0.0072174	0.0072174
7	0.00777954	0.0075868	0.00743466	0.00726955	0.00710966	0.00717731	0.00726665	0.00728989	0.00728989
10	0.00697384	0.00701303	0.00705259	0.00704171	0.0070555	0.00714878	0.00723997	0.00722954	0.00722954
15	0.00731508	0.00731858	0.00728587	0.00725674	0.0072209	0.00720153	0.00717821	0.0071639	0.0071639
    ];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'},...
                            {rawDataSwaption,valueDate,'USD','USDSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    USDAllMktData('swaptionVol') = swaptionVolMktData;
    
   %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    USDBlack = IRBlack(IRModel(Model(USDAllMktData,modelParams))); 
    
    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);
    
    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = USDBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= USDBlack.BlackATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = USDBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
    %% create a CK1FModel with initial modelParams
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;

%     matu = 15 + 1.0/365.0;
    joint_swap_tenor = 1;
    targetSize = 15;
    freq = 4;
    
    modelParamsUSD= containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'}, ...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
    USDCK1F = CK1F(IRModel(Model(USDAllMktData, modelParamsUSD)));
    
    tic
    outUSD = USDCK1F.CalibrateToATMDiagSwaptionNormal(USDBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        USDVol_r(idx2,idx3) = outUSD.vol_r.quote(idx3);
        USDRe(idx2,idx3) = outUSD.re(idx3);
    end
    USDRMSE(idx2) = outUSD.rmseTotal;
    
    toc
    %% USD model building end

    %% USDKRW fx model buidling start
    rawData = [0.083333333	0.0
0.25	0.0
0.5	0.0
1	0.0
2	0.0
3	0.0
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','USDKRWATMTermVol'});
    impliedTermVolMktData = MktData(mktData);
%     impliedTermVole = FXVolSurface(impliedTermVolMktData);
%     impliedVolSurface.params('moneyNessType') = 'deltaFwdMoneyNess';
    USDKRWAllMktData =containers.Map('impliedTermVol',impliedTermVolMktData);
%     AllMktData('impliedVolSurface') = impliedTermVolMktData;

    %% create  an FXBlackModel to calculate market PlainVanilla Options
%     modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});
%     USDKRWBlack = FXBlack(FXModel(Model(USDKRWAllMktData,modelParams))); 
    %% USDKRW fx model building end

    %% MultiAssetModel building start
    multiAssetMktData = containers.Map({'KRW','USD','USDKRW'}, ...
                            {KRWAllMktData, USDAllMktData, USDKRWAllMktData});

    % KRW,KTB                    
%     correlationMatrix = ones(2,2);
%     correlationMatrix(1,2) = 0.3;
%     correlationMatrix(2,1) = correlationMatrix(1,2);

    % KRW, USD & USDKRW
    correlationMatrix = ones(3, 3);
    correlationMatrix(1,2) = 0.845;
%     correlationMatrix(1,2) = 0.6;
    
%     correlationMatrix(1,2) = 0.7;
    
    correlationMatrix(1,3) = 0.0;
    correlationMatrix(2,3) = 0.0;
    correlationMatrix(2,1) = correlationMatrix(1,2);
    correlationMatrix(3,1) = correlationMatrix(1,3);
    correlationMatrix(3,2) = correlationMatrix(2,3);
    
    multiAssetMktData('correlationMatrix') = correlationMatrix;
    
    modelNameMap = containers.Map({'KRW','USD'},{KRWCK1FRisky, USDCK1F});
    modelParams = containers.Map({'modelNameMap'}, {modelNameMap});
    % correlationNames = {'KRW','USD'};
    modelParams('stochasticModelNames') = {'KRW','USD'};
    modelParams('correlationMatrix') = correlationMatrix;
    modelParams('refModelName') = 'KRW';
    % modelParams('useQuasiRandomYN') = 'true';
    modelParams('useQuasiRandomYN') = 'false';
    modelParams('fxVolCalibrated') = true;
    
    KRWCK1FRiskyUSDCK1F = MultiAssetModelCK1FCK1F(multiAssetMktData, modelParams);
    %% MultiAssetModel building end
   
    %% NumMethod Init Start
    numOfRandom = 40000;
    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                            {'AMC', 7, numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);
    %% NumMethod Init End
    
    % weekly step calculator
    % pricer = Pricer(rangeAccrual,KRWLGM,numMethodAMC);
    % pricingOut = pricer.computePrice(valueDate);
    
    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    % pricer = DailyWeeklyRiskyPricer_CDKTBSpreadRA(rangeAccrual,KRWLGM1FRiskyUSDLGM1F,numMethodAMC,false);
%     pricer = DailyWeeklyRiskyPricer_CDUSDLibor3MSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
    pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, true);    
     
    pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH, 'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH,'PSANo');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSANo');
%     pricingOutClosed = pricer.computePriceClosedSimple(valueDateH);
    
    callablePrice(idx2) = pricingOutMC1.callable.npv;
    nonCallPrice(idx2) = pricingOutMC1.nonCall.npv;
%     closedPrice(idx2) = pricingOutClosed.pricerInfo.nonCall.npv;
    
%     for idx3 = 1:length(pricingOutMC1.nonCall.cashflow_npv_e)
% %     for idx3 = 1:length(pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e)
%         nonCallPrice_cashflowNPV(idx2, idx3) = pricingOutMC1.nonCall.cashflow_npv_e(idx3);
% %         closedPrice_cashflowNPV(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
% %         
% %         closedPrice_fwdCD(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdCD(idx3);
% %         closedPrice_fwdKTB(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdKTB(idx3);
% %         closedPrice_fwdSpread(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSpread(idx3);
% %         closedPrice_fwdMu(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdMu(idx3);
% %         closedPrice_fwdSigma(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSigma(idx3);
% %                 
%     end
end

aaa =1.0;


In [ ]:

testCD_CDKTB3MSpread_CK1F_CK1F_20220620_irs_delta.m

clc
clear
valueDate = '2022-06-20';

maxIdx = 0;
zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;

callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% CD-USDLibor3M Spread Range Accrual Pricing start

%RA info
rangeAccrualParams.payCcy = 'KRW';
rangeAccrualParams.startDate = H_Date('2020-02-03');
rangeAccrualParams.maturity = 15;

valueDateH = H_Date(valueDate);

nominal = 10000;
coupon = 0.033;
couponFreq = 1;

% Long: KRW
% Short: USD
tenorL = 0.25;
tenorS = 0.25;

lowerS = 0.0;
upperS = 999;

% 
% lower = -1;
% upper = 0.05;

tenor = 0.25;

lower = -999;
upper = 999;

%resetDate startDate endDate paymentDate
couponScheduleInt = [20200203	20200203	20210203	20210203
20210203	20210203	20220203	20220203
20220203	20220203	20230203	20230203
20230203	20230203	20240205	20240205
20240205	20240205	20250203	20250203
20250203	20250203	20260203	20260203
20260203	20260203	20270203	20270203
20270203	20270203	20280203	20280203
20280203	20280203	20290205	20290205
20290205	20290205	20300206	20300206
20300206	20300206	20310203	20310203
20310203	20310203	20320203	20320203
20320203	20320203	20330203	20330203
20330203	20330203	20340203	20340203
20340203	20340203	20350205	20350205
];

endDate = H_Date(couponScheduleInt(size(couponScheduleInt,1),4));

%% CK1F model param matu setting start
% matu = 15 + 1.0/365.0;
matu = DateDiff(H_Date(couponScheduleInt(end,4)),H_Date(valueDate))/365.0;
% matu = matu -10.0;

%% end

 rangeDays = zeros(15,1);
%     rangeDays(1) = 1;

for i=1:length(rangeDays)
    if DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) >= 0
        rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) + 1;
    end
end


%resetDate startDate endDate paymentDate
callScheduleInt = [20200203	20200203	20200210	20200210
20210127	20210127	20210203	20210203
20220124	20220124	20220203	20220203
20230127	20230127	20230203	20230203
20240129	20240129	20240205	20240205
20250122	20250122	20250203	20250203
20260127	20260127	20260203	20260203
20270127	20270127	20270203	20270203
20280124	20280124	20280203	20280203
20290129	20290129	20290205	20290205
20300128	20300128	20300206	20300206
20310127	20310127	20310203	20310203
20320127	20320127	20320203	20320203
20330124	20330124	20330203	20330203
20340127	20340127	20340203	20340203
];

callYN = ones(15,1);
callYN(1) = 0;

manualScheduleYN = 1;

rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                'tenorS','lower','upper','rangeDays','callYN', ...
                                'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                        {nominal,coupon,couponFreq,tenorL,tenorS,lower,upper,...
                        rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});
% 
% tenorL = 0.25;
% tenorS = 0.25;
% 
% tenor = 0.25;
% 
% lowerS = 0.0;
% upperS = 999;
% 
% 
% lower = 0.0;
% upper = 0.05;

rangeAccrualInfo('lowerS') =lowerS;
rangeAccrualInfo('upperS') =upperS;
rangeAccrualInfo('tenor') = 0.25;

rangeAccrualParams.params = rangeAccrualInfo;

% rangeAccrual = SpreadRangeAccrual(rangeAccrualParams);
rangeAccrual = CDCMSSpreadDualRangeAccrual(rangeAccrualParams);

% maxIdx =2.5;

maxIdx = 1;

rmseSimul = zeros(100,1);
for idx2=1:3
    %% KRW Model building

    %% import and construct a zeroCurve
    rawData = [0.002739726	0.999952523
0.25	0.9950684
0.5	0.988031579
0.75	0.979080331
1	0.968776974
1.25	0.959202682
1.5	0.948993277
1.75	0.939322512
2	0.929369492
2.25	0.920482183
2.5	0.911621558
2.75	0.902762032
3	0.893905238
3.25	0.885611153
3.5	0.877424651
3.75	0.869320968
4	0.861299205
4.25	0.853502656
4.5	0.845805585
4.75	0.838202041
5	0.830690747
5.25	0.823531148
5.5	0.816487433
5.75	0.809552209
6	0.802723731
6.25	0.796000289
6.5	0.789380204
6.75	0.782861832
7	0.776443559
7.25	0.770025284
7.5	0.763695404
7.75	0.757455789
8	0.751305075
8.25	0.74524192
8.5	0.73926501
8.75	0.73337305
9	0.727564772
9.25	0.721838929
9.5	0.716194297
9.75	0.710629672
10	0.705143874
10.25	0.699194275
10.5	0.693278485
10.75	0.687420297
11	0.681619095
11.25	0.675874273
11.5	0.670185229
11.75	0.664551369
12	0.658972106
12.25	0.654130635
12.5	0.649391356
12.75	0.644722276
13	0.640122375
13.25	0.635590647
13.5	0.631126108
13.75	0.626727791
14	0.622394747
14.25	0.618126045
14.5	0.613920769
14.75	0.609778022
15	0.605696922
15.25	0.60235689
15.5	0.599139226
15.75	0.595995535
16	0.592924834
16.25	0.589926161
16.5	0.586998583
16.75	0.58414119
17	0.581353097
17.25	0.578633444
17.5	0.575981393
17.75	0.573396131
18	0.570876865
18.25	0.568422827
18.5	0.566033269
18.75	0.563707467
19	0.561444716
19.25	0.559244332
19.5	0.557105652
19.75	0.555028035
20	0.553010856
20.25	0.550157221
20.5	0.547261794
20.75	0.544409494
21	0.541599805
21.25	0.538832219
21.5	0.536106239
21.75	0.533421375
22	0.530777148
22.25	0.528173085
22.5	0.525608723
22.75	0.523083608
23	0.520597293
23.25	0.518149339
23.5	0.515739316
23.75	0.513366801
24	0.511031378
24.25	0.508732641
24.5	0.506470189
24.75	0.50424363
25	0.502052578
25.25	0.498564056
25.5	0.494988089
25.75	0.491434812
26	0.487904104
26.25	0.484395847
26.5	0.48090992
26.75	0.477446204
27	0.474004582
27.25	0.470584935
27.5	0.467187146
27.75	0.463811098
28	0.460456674
28.25	0.457123757
28.5	0.453812233
28.75	0.450521986
29	0.447252902
29.25	0.444004865
29.5	0.440777762
29.75	0.43757148
30	0.434385905
30.25	0.431221981
30.5	0.428081088
30.75	0.424963073
31	0.421867769
    ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

    
    %simulation testCase
    for idx=1:size(rawData,1)
        rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    KRWAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];

% rawData = [0.002739726	0.999952523
% 0.25	0.9950684
% 0.5	0.988031579
% 0.75	0.979080331
% 1	0.968776974
% 1.25	0.959202682
% 1.5	0.948993277
% 1.75	0.939322512
% 2	0.929369492
% 2.25	0.920482183
% 2.5	0.911621558
% 2.75	0.902762032
% 3	0.893905238
% 3.25	0.885611153
% 3.5	0.877424651
% 3.75	0.869320968
% 4	0.861299205
% 4.25	0.853502656
% 4.5	0.845805585
% 4.75	0.838202041
% 5	0.830690747
% 5.25	0.823531148
% 5.5	0.816487433
% 5.75	0.809552209
% 6	0.802723731
% 6.25	0.796000289
% 6.5	0.789380204
% 6.75	0.782861832
% 7	0.776443559
% 7.25	0.770025284
% 7.5	0.763695404
% 7.75	0.757455789
% 8	0.751305075
% 8.25	0.74524192
% 8.5	0.73926501
% 8.75	0.73337305
% 9	0.727564772
% 9.25	0.721838929
% 9.5	0.716194297
% 9.75	0.710629672
% 10	0.705143874
% 10.25	0.699194275
% 10.5	0.693278485
% 10.75	0.687420297
% 11	0.681619095
% 11.25	0.675874273
% 11.5	0.670185229
% 11.75	0.664551369
% 12	0.658972106
% 12.25	0.654130635
% 12.5	0.649391356
% 12.75	0.644722276
% 13	0.640122375
% 13.25	0.635590647
% 13.5	0.631126108
% 13.75	0.626727791
% 14	0.622394747
% 14.25	0.618126045
% 14.5	0.613920769
% 14.75	0.609778022
% 15	0.605696922
% 15.25	0.60235689
% 15.5	0.599139226
% 15.75	0.595995535
% 16	0.592924834
% 16.25	0.589926161
% 16.5	0.586998583
% 16.75	0.58414119
% 17	0.581353097
% 17.25	0.578633444
% 17.5	0.575981393
% 17.75	0.573396131
% 18	0.570876865
% 18.25	0.568422827
% 18.5	0.566033269
% 18.75	0.563707467
% 19	0.561444716
% 19.25	0.559244332
% 19.5	0.557105652
% 19.75	0.555028035
% 20	0.553010856
% 20.25	0.550157221
% 20.5	0.547261794
% 20.75	0.544409494
% 21	0.541599805
% 21.25	0.538832219
% 21.5	0.536106239
% 21.75	0.533421375
% 22	0.530777148
% 22.25	0.528173085
% 22.5	0.525608723
% 22.75	0.523083608
% 23	0.520597293
% 23.25	0.518149339
% 23.5	0.515739316
% 23.75	0.513366801
% 24	0.511031378
% 24.25	0.508732641
% 24.5	0.506470189
% 24.75	0.50424363
% 25	0.502052578
% 25.25	0.498564056
% 25.5	0.494988089
% 25.75	0.491434812
% 26	0.487904104
% 26.25	0.484395847
% 26.5	0.48090992
% 26.75	0.477446204
% 27	0.474004582
% 27.25	0.470584935
% 27.5	0.467187146
% 27.75	0.463811098
% 28	0.460456674
% 28.25	0.457123757
% 28.5	0.453812233
% 28.75	0.450521986
% 29	0.447252902
% 29.25	0.444004865
% 29.5	0.440777762
% 29.75	0.43757148
% 30	0.434385905
% 30.25	0.431221981
% 30.5	0.428081088
% 30.75	0.424963073
% 31	0.421867769
%     ];


for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

%  %simulation testCase
%     for idx=1:size(rawData,1)
%         rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
%     end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);

    KRWAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurfaceNormal



rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.01025373	0.00996465	0.00939453	0.0088449	0.00838736	0.00835816	0.00827411	0.00844853	0.00844853
2	0.00987033	0.00941532	0.00896799	0.00850851	0.00803853	0.00809306	0.00814872	0.00832187	0.00832187
3	0.00965448	0.00917228	0.00868278	0.00815841	0.00766413	0.00768619	0.00766987	0.007827	0.007827
4	0.0086456	0.00835905	0.00803055	0.00773857	0.00743965	0.00747184	0.00750208	0.00760185	0.00760185
5	0.008024	0.00784518	0.00761802	0.00741163	0.00719132	0.00718529	0.00716174	0.0072174	0.0072174
7	0.00777954	0.0075868	0.00743466	0.00726955	0.00710966	0.00717731	0.00726665	0.00728989	0.00728989
10	0.00697384	0.00701303	0.00705259	0.00704171	0.0070555	0.00714878	0.00723997	0.00722954	0.00722954
15	0.00731508	0.00731858	0.00728587	0.00725674	0.0072209	0.00720153	0.00717821	0.0071639	0.0071639
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    KRWAllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a KRWBlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(KRWAllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = KRWBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= KRWBlack.NormalATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
   %% create a CK1FRisky Model with initial modelParams
%     vol_r.tenor = [1;2;3;4;5;7;10;15;20];
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;
%     meanR_r = 0.0;
    
    
    joint_swap_tenor = 1;
%     targetSize = 10;

    targetSize = 15;
    
    freq = 4;
    
    modelParams = containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'},...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
    
%     import irModels.*;
%     import targetFunctions.*;
    
    KRWCK1FRisky = CK1FRisky(IRModelRisky(Model(KRWAllMktData, modelParams)));
    
    tic
%     out = KRWLGM2F.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
    % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
%     out = KRWCK1FRisky.CalibrateToATMDiagSwaption(KRWBlack,'global');
    out = KRWCK1FRisky.CalibrateToATMDiagSwaptionNormal(KRWBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        KRWVol_r(idx2,idx3) = out.vol_r.quote(idx3);
        KRWRe(idx2,idx3) = out.re(idx3);
    end
    KRWRMSE(idx2) = out.rmseTotal;
    
    toc
    %% CK1FRisky model building end
    %% KRW model building end

    %% USD model building start
    %% import and construct a zeroCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

%  %simulation testCase
%     for idx=1:size(rawData,1)
%         rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
%     end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','USDZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    USDAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];


% rawData = [0.002739726	0.999952523
% 0.25	0.9950684
% 0.5	0.988031579
% 0.75	0.979080331
% 1	0.968776974
% 1.25	0.959202682
% 1.5	0.948993277
% 1.75	0.939322512
% 2	0.929369492
% 2.25	0.920482183
% 2.5	0.911621558
% 2.75	0.902762032
% 3	0.893905238
% 3.25	0.885611153
% 3.5	0.877424651
% 3.75	0.869320968
% 4	0.861299205
% 4.25	0.853502656
% 4.5	0.845805585
% 4.75	0.838202041
% 5	0.830690747
% 5.25	0.823531148
% 5.5	0.816487433
% 5.75	0.809552209
% 6	0.802723731
% 6.25	0.796000289
% 6.5	0.789380204
% 6.75	0.782861832
% 7	0.776443559
% 7.25	0.770025284
% 7.5	0.763695404
% 7.75	0.757455789
% 8	0.751305075
% 8.25	0.74524192
% 8.5	0.73926501
% 8.75	0.73337305
% 9	0.727564772
% 9.25	0.721838929
% 9.5	0.716194297
% 9.75	0.710629672
% 10	0.705143874
% 10.25	0.699194275
% 10.5	0.693278485
% 10.75	0.687420297
% 11	0.681619095
% 11.25	0.675874273
% 11.5	0.670185229
% 11.75	0.664551369
% 12	0.658972106
% 12.25	0.654130635
% 12.5	0.649391356
% 12.75	0.644722276
% 13	0.640122375
% 13.25	0.635590647
% 13.5	0.631126108
% 13.75	0.626727791
% 14	0.622394747
% 14.25	0.618126045
% 14.5	0.613920769
% 14.75	0.609778022
% 15	0.605696922
% 15.25	0.60235689
% 15.5	0.599139226
% 15.75	0.595995535
% 16	0.592924834
% 16.25	0.589926161
% 16.5	0.586998583
% 16.75	0.58414119
% 17	0.581353097
% 17.25	0.578633444
% 17.5	0.575981393
% 17.75	0.573396131
% 18	0.570876865
% 18.25	0.568422827
% 18.5	0.566033269
% 18.75	0.563707467
% 19	0.561444716
% 19.25	0.559244332
% 19.5	0.557105652
% 19.75	0.555028035
% 20	0.553010856
% 20.25	0.550157221
% 20.5	0.547261794
% 20.75	0.544409494
% 21	0.541599805
% 21.25	0.538832219
% 21.5	0.536106239
% 21.75	0.533421375
% 22	0.530777148
% 22.25	0.528173085
% 22.5	0.525608723
% 22.75	0.523083608
% 23	0.520597293
% 23.25	0.518149339
% 23.5	0.515739316
% 23.75	0.513366801
% 24	0.511031378
% 24.25	0.508732641
% 24.5	0.506470189
% 24.75	0.50424363
% 25	0.502052578
% 25.25	0.498564056
% 25.5	0.494988089
% 25.75	0.491434812
% 26	0.487904104
% 26.25	0.484395847
% 26.5	0.48090992
% 26.75	0.477446204
% 27	0.474004582
% 27.25	0.470584935
% 27.5	0.467187146
% 27.75	0.463811098
% 28	0.460456674
% 28.25	0.457123757
% 28.5	0.453812233
% 28.75	0.450521986
% 29	0.447252902
% 29.25	0.444004865
% 29.5	0.440777762
% 29.75	0.43757148
% 30	0.434385905
% 30.25	0.431221981
% 30.5	0.428081088
% 30.75	0.424963073
% 31	0.421867769
%     ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

%  %simulation testCase
%     for idx=1:size(rawData,1)
%         rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
%     end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);
    USDAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
% GIROPremiumToNormalVol From LIB
% rawDataSwaption = [0	1	2	3	4	5	7	10	12	15	20
% 1	0.0041	0.0045	0.0052	0.0061	0.0066	0.0071	0.0077	0.0080	0.0081	0.0082
% 2	0.0050	0.0057	0.0064	0.0066	0.0068	0.0071	0.0074	0.0077	0.0077	0.0078
% 3	0.0064	0.0069	0.0068	0.0068	0.0068	0.0072	0.0076	0.0078	0.0079	0.0079
% 4	0.0076	0.0071	0.0071	0.0070	0.0071	0.0072	0.0077	0.0081	0.0081	0.0081
% 5	0.0069	0.0070	0.0070	0.0071	0.0071	0.0070	0.0076	0.0083	0.0084	0.0083
% 7	0.0073	0.0074	0.0075	0.0072	0.0070	0.0075	0.0087	0.0096	0.0096	0.0095
% 10	0.0066	0.0066	0.0071	0.0074	0.0077	0.0092	0.0101	0.0102	0.0102	0.0102
% 15	0.0103	0.0105	0.0107	0.0109	0.0111	0.0108	0.0103	0.0102	0.0102	0.0101
% 20	0.0088	0.0089	0.0090	0.0091	0.0091	0.0091	0.0090	0.0089	0.0089	0.0089
%     ];
% 
rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.01025373	0.00996465	0.00939453	0.0088449	0.00838736	0.00835816	0.00827411	0.00844853	0.00844853
2	0.00987033	0.00941532	0.00896799	0.00850851	0.00803853	0.00809306	0.00814872	0.00832187	0.00832187
3	0.00965448	0.00917228	0.00868278	0.00815841	0.00766413	0.00768619	0.00766987	0.007827	0.007827
4	0.0086456	0.00835905	0.00803055	0.00773857	0.00743965	0.00747184	0.00750208	0.00760185	0.00760185
5	0.008024	0.00784518	0.00761802	0.00741163	0.00719132	0.00718529	0.00716174	0.0072174	0.0072174
7	0.00777954	0.0075868	0.00743466	0.00726955	0.00710966	0.00717731	0.00726665	0.00728989	0.00728989
10	0.00697384	0.00701303	0.00705259	0.00704171	0.0070555	0.00714878	0.00723997	0.00722954	0.00722954
15	0.00731508	0.00731858	0.00728587	0.00725674	0.0072209	0.00720153	0.00717821	0.0071639	0.0071639
    ];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'},...
                            {rawDataSwaption,valueDate,'USD','USDSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    USDAllMktData('swaptionVol') = swaptionVolMktData;
    
   %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    USDBlack = IRBlack(IRModel(Model(USDAllMktData,modelParams))); 
    
    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);
    
    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = USDBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= USDBlack.BlackATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = USDBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
    %% create a CK1FModel with initial modelParams
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;

%     matu = 15 + 1.0/365.0;
    joint_swap_tenor = 1;
    targetSize = 15;
    freq = 4;
    
    modelParamsUSD= containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'}, ...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
    USDCK1F = CK1F(IRModel(Model(USDAllMktData, modelParamsUSD)));
    
    tic
    outUSD = USDCK1F.CalibrateToATMDiagSwaptionNormal(USDBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        USDVol_r(idx2,idx3) = outUSD.vol_r.quote(idx3);
        USDRe(idx2,idx3) = outUSD.re(idx3);
    end
    USDRMSE(idx2) = outUSD.rmseTotal;
    
    toc
    %% USD model building end

    %% USDKRW fx model buidling start
    rawData = [0.083333333	0.0
0.25	0.0
0.5	0.0
1	0.0
2	0.0
3	0.0
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','USDKRWATMTermVol'});
    impliedTermVolMktData = MktData(mktData);
%     impliedTermVole = FXVolSurface(impliedTermVolMktData);
%     impliedVolSurface.params('moneyNessType') = 'deltaFwdMoneyNess';
    USDKRWAllMktData =containers.Map('impliedTermVol',impliedTermVolMktData);
%     AllMktData('impliedVolSurface') = impliedTermVolMktData;

    %% create  an FXBlackModel to calculate market PlainVanilla Options
%     modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});
%     USDKRWBlack = FXBlack(FXModel(Model(USDKRWAllMktData,modelParams))); 
    %% USDKRW fx model building end

    %% MultiAssetModel building start
    multiAssetMktData = containers.Map({'KRW','USD','USDKRW'}, ...
                            {KRWAllMktData, USDAllMktData, USDKRWAllMktData});

    % KRW,KTB                    
%     correlationMatrix = ones(2,2);
%     correlationMatrix(1,2) = 0.3;
%     correlationMatrix(2,1) = correlationMatrix(1,2);

    % KRW, USD & USDKRW
    correlationMatrix = ones(3, 3);
    correlationMatrix(1,2) = 0.845;
%     correlationMatrix(1,2) = 0.6;
    
%     correlationMatrix(1,2) = 0.7;
    
    correlationMatrix(1,3) = 0.0;
    correlationMatrix(2,3) = 0.0;
    correlationMatrix(2,1) = correlationMatrix(1,2);
    correlationMatrix(3,1) = correlationMatrix(1,3);
    correlationMatrix(3,2) = correlationMatrix(2,3);
    
    multiAssetMktData('correlationMatrix') = correlationMatrix;
    
    modelNameMap = containers.Map({'KRW','USD'},{KRWCK1FRisky, USDCK1F});
    modelParams = containers.Map({'modelNameMap'}, {modelNameMap});
    % correlationNames = {'KRW','USD'};
    modelParams('stochasticModelNames') = {'KRW','USD'};
    modelParams('correlationMatrix') = correlationMatrix;
    modelParams('refModelName') = 'KRW';
    % modelParams('useQuasiRandomYN') = 'true';
    modelParams('useQuasiRandomYN') = 'false';
    modelParams('fxVolCalibrated') = true;
    
    KRWCK1FRiskyUSDCK1F = MultiAssetModelCK1FCK1F(multiAssetMktData, modelParams);
    %% MultiAssetModel building end
   
    %% NumMethod Init Start
    numOfRandom = 40000;
    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                            {'AMC', 7, numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);
    %% NumMethod Init End
    
    % weekly step calculator
    % pricer = Pricer(rangeAccrual,KRWLGM,numMethodAMC);
    % pricingOut = pricer.computePrice(valueDate);
    
    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    % pricer = DailyWeeklyRiskyPricer_CDKTBSpreadRA(rangeAccrual,KRWLGM1FRiskyUSDLGM1F,numMethodAMC,false);
%     pricer = DailyWeeklyRiskyPricer_CDUSDLibor3MSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
    pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, true);    
     
    pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH, 'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH,'PSANo');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSANo');
%     pricingOutClosed = pricer.computePriceClosedSimple(valueDateH);
    
    callablePrice(idx2) = pricingOutMC1.callable.npv;
    nonCallPrice(idx2) = pricingOutMC1.nonCall.npv;
%     closedPrice(idx2) = pricingOutClosed.pricerInfo.nonCall.npv;
    
%     for idx3 = 1:length(pricingOutMC1.nonCall.cashflow_npv_e)
% %     for idx3 = 1:length(pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e)
%         nonCallPrice_cashflowNPV(idx2, idx3) = pricingOutMC1.nonCall.cashflow_npv_e(idx3);
% %         closedPrice_cashflowNPV(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
% %         
% %         closedPrice_fwdCD(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdCD(idx3);
% %         closedPrice_fwdKTB(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdKTB(idx3);
% %         closedPrice_fwdSpread(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSpread(idx3);
% %         closedPrice_fwdMu(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdMu(idx3);
% %         closedPrice_fwdSigma(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSigma(idx3);
% %                 
%     end
end

aaa =1.0;


In [ ]:
testCD_CDKTB3MSpread_CK1F_CK1F_20220620_20220926_IRSKTBShift.m

clc
clear
valueDate = '2022-06-20';

maxIdx = 0;
zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;

callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% CD-USDLibor3M Spread Range Accrual Pricing start

%RA info
rangeAccrualParams.payCcy = 'KRW';
rangeAccrualParams.startDate = H_Date('2020-02-03');
rangeAccrualParams.maturity = 15;

valueDateH = H_Date(valueDate);

nominal = 10000;
coupon = 0.033;
couponFreq = 1;

% Long: KRW
% Short: USD
tenorL = 0.25;
tenorS = 0.25;

lowerS = 0.0;
upperS = 999;

% 
% lower = -1;
% upper = 0.05;

tenor = 0.25;

lower = -999;
upper = 999;

%resetDate startDate endDate paymentDate
couponScheduleInt = [20200203	20200203	20210203	20210203
20210203	20210203	20220203	20220203
20220203	20220203	20230203	20230203
20230203	20230203	20240205	20240205
20240205	20240205	20250203	20250203
20250203	20250203	20260203	20260203
20260203	20260203	20270203	20270203
20270203	20270203	20280203	20280203
20280203	20280203	20290205	20290205
20290205	20290205	20300206	20300206
20300206	20300206	20310203	20310203
20310203	20310203	20320203	20320203
20320203	20320203	20330203	20330203
20330203	20330203	20340203	20340203
20340203	20340203	20350205	20350205
];

endDate = H_Date(couponScheduleInt(size(couponScheduleInt,1),4));

%% CK1F model param matu setting start
% matu = 15 + 1.0/365.0;
matu = DateDiff(H_Date(couponScheduleInt(end,4)),H_Date(valueDate))/365.0;
% matu = matu -10.0;

%% end

 rangeDays = zeros(15,1);
%     rangeDays(1) = 1;

for i=1:length(rangeDays)
    if DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) >= 0
        rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) + 1;
    end
end


%resetDate startDate endDate paymentDate
callScheduleInt = [20200203	20200203	20200210	20200210
20210127	20210127	20210203	20210203
20220124	20220124	20220203	20220203
20230127	20230127	20230203	20230203
20240129	20240129	20240205	20240205
20250122	20250122	20250203	20250203
20260127	20260127	20260203	20260203
20270127	20270127	20270203	20270203
20280124	20280124	20280203	20280203
20290129	20290129	20290205	20290205
20300128	20300128	20300206	20300206
20310127	20310127	20310203	20310203
20320127	20320127	20320203	20320203
20330124	20330124	20330203	20330203
20340127	20340127	20340203	20340203
];

callYN = ones(15,1);
callYN(1) = 0;

manualScheduleYN = 1;

rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                'tenorS','lower','upper','rangeDays','callYN', ...
                                'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                        {nominal,coupon,couponFreq,tenorL,tenorS,lower,upper,...
                        rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});
% 
% tenorL = 0.25;
% tenorS = 0.25;
% 
% tenor = 0.25;
% 
% lowerS = 0.0;
% upperS = 999;
% 
% 
% lower = 0.0;
% upper = 0.05;

rangeAccrualInfo('lowerS') =lowerS;
rangeAccrualInfo('upperS') =upperS;
rangeAccrualInfo('tenor') = 0.25;

rangeAccrualParams.params = rangeAccrualInfo;

% rangeAccrual = SpreadRangeAccrual(rangeAccrualParams);
rangeAccrual = CDCMSSpreadDualRangeAccrual(rangeAccrualParams);

% maxIdx =2.5;

maxIdx = 5;

rmseSimul = zeros(100,1);

for idx2=1:11
    %% KRW Model building

    %% import and construct a zeroCurve
    rawData = [0.002739726	0.999952523
0.25	0.9950684
0.5	0.988031579
0.75	0.979080331
1	0.968776974
1.25	0.959202682
1.5	0.948993277
1.75	0.939322512
2	0.929369492
2.25	0.920482183
2.5	0.911621558
2.75	0.902762032
3	0.893905238
3.25	0.885611153
3.5	0.877424651
3.75	0.869320968
4	0.861299205
4.25	0.853502656
4.5	0.845805585
4.75	0.838202041
5	0.830690747
5.25	0.823531148
5.5	0.816487433
5.75	0.809552209
6	0.802723731
6.25	0.796000289
6.5	0.789380204
6.75	0.782861832
7	0.776443559
7.25	0.770025284
7.5	0.763695404
7.75	0.757455789
8	0.751305075
8.25	0.74524192
8.5	0.73926501
8.75	0.73337305
9	0.727564772
9.25	0.721838929
9.5	0.716194297
9.75	0.710629672
10	0.705143874
10.25	0.699194275
10.5	0.693278485
10.75	0.687420297
11	0.681619095
11.25	0.675874273
11.5	0.670185229
11.75	0.664551369
12	0.658972106
12.25	0.654130635
12.5	0.649391356
12.75	0.644722276
13	0.640122375
13.25	0.635590647
13.5	0.631126108
13.75	0.626727791
14	0.622394747
14.25	0.618126045
14.5	0.613920769
14.75	0.609778022
15	0.605696922
15.25	0.60235689
15.5	0.599139226
15.75	0.595995535
16	0.592924834
16.25	0.589926161
16.5	0.586998583
16.75	0.58414119
17	0.581353097
17.25	0.578633444
17.5	0.575981393
17.75	0.573396131
18	0.570876865
18.25	0.568422827
18.5	0.566033269
18.75	0.563707467
19	0.561444716
19.25	0.559244332
19.5	0.557105652
19.75	0.555028035
20	0.553010856
20.25	0.550157221
20.5	0.547261794
20.75	0.544409494
21	0.541599805
21.25	0.538832219
21.5	0.536106239
21.75	0.533421375
22	0.530777148
22.25	0.528173085
22.5	0.525608723
22.75	0.523083608
23	0.520597293
23.25	0.518149339
23.5	0.515739316
23.75	0.513366801
24	0.511031378
24.25	0.508732641
24.5	0.506470189
24.75	0.50424363
25	0.502052578
25.25	0.498564056
25.5	0.494988089
25.75	0.491434812
26	0.487904104
26.25	0.484395847
26.5	0.48090992
26.75	0.477446204
27	0.474004582
27.25	0.470584935
27.5	0.467187146
27.75	0.463811098
28	0.460456674
28.25	0.457123757
28.5	0.453812233
28.75	0.450521986
29	0.447252902
29.25	0.444004865
29.5	0.440777762
29.75	0.43757148
30	0.434385905
30.25	0.431221981
30.5	0.428081088
30.75	0.424963073
31	0.421867769
    ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

    
    %simulation testCase
    for idx=1:size(rawData,1)
        rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    KRWAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];

% rawData = [0.002739726	0.999952523
% 0.25	0.9950684
% 0.5	0.988031579
% 0.75	0.979080331
% 1	0.968776974
% 1.25	0.959202682
% 1.5	0.948993277
% 1.75	0.939322512
% 2	0.929369492
% 2.25	0.920482183
% 2.5	0.911621558
% 2.75	0.902762032
% 3	0.893905238
% 3.25	0.885611153
% 3.5	0.877424651
% 3.75	0.869320968
% 4	0.861299205
% 4.25	0.853502656
% 4.5	0.845805585
% 4.75	0.838202041
% 5	0.830690747
% 5.25	0.823531148
% 5.5	0.816487433
% 5.75	0.809552209
% 6	0.802723731
% 6.25	0.796000289
% 6.5	0.789380204
% 6.75	0.782861832
% 7	0.776443559
% 7.25	0.770025284
% 7.5	0.763695404
% 7.75	0.757455789
% 8	0.751305075
% 8.25	0.74524192
% 8.5	0.73926501
% 8.75	0.73337305
% 9	0.727564772
% 9.25	0.721838929
% 9.5	0.716194297
% 9.75	0.710629672
% 10	0.705143874
% 10.25	0.699194275
% 10.5	0.693278485
% 10.75	0.687420297
% 11	0.681619095
% 11.25	0.675874273
% 11.5	0.670185229
% 11.75	0.664551369
% 12	0.658972106
% 12.25	0.654130635
% 12.5	0.649391356
% 12.75	0.644722276
% 13	0.640122375
% 13.25	0.635590647
% 13.5	0.631126108
% 13.75	0.626727791
% 14	0.622394747
% 14.25	0.618126045
% 14.5	0.613920769
% 14.75	0.609778022
% 15	0.605696922
% 15.25	0.60235689
% 15.5	0.599139226
% 15.75	0.595995535
% 16	0.592924834
% 16.25	0.589926161
% 16.5	0.586998583
% 16.75	0.58414119
% 17	0.581353097
% 17.25	0.578633444
% 17.5	0.575981393
% 17.75	0.573396131
% 18	0.570876865
% 18.25	0.568422827
% 18.5	0.566033269
% 18.75	0.563707467
% 19	0.561444716
% 19.25	0.559244332
% 19.5	0.557105652
% 19.75	0.555028035
% 20	0.553010856
% 20.25	0.550157221
% 20.5	0.547261794
% 20.75	0.544409494
% 21	0.541599805
% 21.25	0.538832219
% 21.5	0.536106239
% 21.75	0.533421375
% 22	0.530777148
% 22.25	0.528173085
% 22.5	0.525608723
% 22.75	0.523083608
% 23	0.520597293
% 23.25	0.518149339
% 23.5	0.515739316
% 23.75	0.513366801
% 24	0.511031378
% 24.25	0.508732641
% 24.5	0.506470189
% 24.75	0.50424363
% 25	0.502052578
% 25.25	0.498564056
% 25.5	0.494988089
% 25.75	0.491434812
% 26	0.487904104
% 26.25	0.484395847
% 26.5	0.48090992
% 26.75	0.477446204
% 27	0.474004582
% 27.25	0.470584935
% 27.5	0.467187146
% 27.75	0.463811098
% 28	0.460456674
% 28.25	0.457123757
% 28.5	0.453812233
% 28.75	0.450521986
% 29	0.447252902
% 29.25	0.444004865
% 29.5	0.440777762
% 29.75	0.43757148
% 30	0.434385905
% 30.25	0.431221981
% 30.5	0.428081088
% 30.75	0.424963073
% 31	0.421867769
%     ];


for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

 %simulation testCase
    for idx=1:size(rawData,1)
        rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
    end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);

    KRWAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurfaceNormal



rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.01025373	0.00996465	0.00939453	0.0088449	0.00838736	0.00835816	0.00827411	0.00844853	0.00844853
2	0.00987033	0.00941532	0.00896799	0.00850851	0.00803853	0.00809306	0.00814872	0.00832187	0.00832187
3	0.00965448	0.00917228	0.00868278	0.00815841	0.00766413	0.00768619	0.00766987	0.007827	0.007827
4	0.0086456	0.00835905	0.00803055	0.00773857	0.00743965	0.00747184	0.00750208	0.00760185	0.00760185
5	0.008024	0.00784518	0.00761802	0.00741163	0.00719132	0.00718529	0.00716174	0.0072174	0.0072174
7	0.00777954	0.0075868	0.00743466	0.00726955	0.00710966	0.00717731	0.00726665	0.00728989	0.00728989
10	0.00697384	0.00701303	0.00705259	0.00704171	0.0070555	0.00714878	0.00723997	0.00722954	0.00722954
15	0.00731508	0.00731858	0.00728587	0.00725674	0.0072209	0.00720153	0.00717821	0.0071639	0.0071639
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    KRWAllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a KRWBlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(KRWAllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = KRWBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= KRWBlack.NormalATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
   %% create a CK1FRisky Model with initial modelParams
%     vol_r.tenor = [1;2;3;4;5;7;10;15;20];
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;
%     meanR_r = 0.0;
    
    
    joint_swap_tenor = 1;
%     targetSize = 10;

    targetSize = 15;
    
    freq = 4;
    
    modelParams = containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'},...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
    
%     import irModels.*;
%     import targetFunctions.*;
    
    KRWCK1FRisky = CK1FRisky(IRModelRisky(Model(KRWAllMktData, modelParams)));
    
    tic
%     out = KRWLGM2F.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
    % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
%     out = KRWCK1FRisky.CalibrateToATMDiagSwaption(KRWBlack,'global');
    out = KRWCK1FRisky.CalibrateToATMDiagSwaptionNormal(KRWBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        KRWVol_r(idx2,idx3) = out.vol_r.quote(idx3);
        KRWRe(idx2,idx3) = out.re(idx3);
    end
    KRWRMSE(idx2) = out.rmseTotal;
    
    toc
    %% CK1FRisky model building end
    %% KRW model building end

    %% USD model building start
    %% import and construct a zeroCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

 %simulation testCase
    for idx=1:size(rawData,1)
        rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
    end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','USDZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    USDAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];


% rawData = [0.002739726	0.999952523
% 0.25	0.9950684
% 0.5	0.988031579
% 0.75	0.979080331
% 1	0.968776974
% 1.25	0.959202682
% 1.5	0.948993277
% 1.75	0.939322512
% 2	0.929369492
% 2.25	0.920482183
% 2.5	0.911621558
% 2.75	0.902762032
% 3	0.893905238
% 3.25	0.885611153
% 3.5	0.877424651
% 3.75	0.869320968
% 4	0.861299205
% 4.25	0.853502656
% 4.5	0.845805585
% 4.75	0.838202041
% 5	0.830690747
% 5.25	0.823531148
% 5.5	0.816487433
% 5.75	0.809552209
% 6	0.802723731
% 6.25	0.796000289
% 6.5	0.789380204
% 6.75	0.782861832
% 7	0.776443559
% 7.25	0.770025284
% 7.5	0.763695404
% 7.75	0.757455789
% 8	0.751305075
% 8.25	0.74524192
% 8.5	0.73926501
% 8.75	0.73337305
% 9	0.727564772
% 9.25	0.721838929
% 9.5	0.716194297
% 9.75	0.710629672
% 10	0.705143874
% 10.25	0.699194275
% 10.5	0.693278485
% 10.75	0.687420297
% 11	0.681619095
% 11.25	0.675874273
% 11.5	0.670185229
% 11.75	0.664551369
% 12	0.658972106
% 12.25	0.654130635
% 12.5	0.649391356
% 12.75	0.644722276
% 13	0.640122375
% 13.25	0.635590647
% 13.5	0.631126108
% 13.75	0.626727791
% 14	0.622394747
% 14.25	0.618126045
% 14.5	0.613920769
% 14.75	0.609778022
% 15	0.605696922
% 15.25	0.60235689
% 15.5	0.599139226
% 15.75	0.595995535
% 16	0.592924834
% 16.25	0.589926161
% 16.5	0.586998583
% 16.75	0.58414119
% 17	0.581353097
% 17.25	0.578633444
% 17.5	0.575981393
% 17.75	0.573396131
% 18	0.570876865
% 18.25	0.568422827
% 18.5	0.566033269
% 18.75	0.563707467
% 19	0.561444716
% 19.25	0.559244332
% 19.5	0.557105652
% 19.75	0.555028035
% 20	0.553010856
% 20.25	0.550157221
% 20.5	0.547261794
% 20.75	0.544409494
% 21	0.541599805
% 21.25	0.538832219
% 21.5	0.536106239
% 21.75	0.533421375
% 22	0.530777148
% 22.25	0.528173085
% 22.5	0.525608723
% 22.75	0.523083608
% 23	0.520597293
% 23.25	0.518149339
% 23.5	0.515739316
% 23.75	0.513366801
% 24	0.511031378
% 24.25	0.508732641
% 24.5	0.506470189
% 24.75	0.50424363
% 25	0.502052578
% 25.25	0.498564056
% 25.5	0.494988089
% 25.75	0.491434812
% 26	0.487904104
% 26.25	0.484395847
% 26.5	0.48090992
% 26.75	0.477446204
% 27	0.474004582
% 27.25	0.470584935
% 27.5	0.467187146
% 27.75	0.463811098
% 28	0.460456674
% 28.25	0.457123757
% 28.5	0.453812233
% 28.75	0.450521986
% 29	0.447252902
% 29.25	0.444004865
% 29.5	0.440777762
% 29.75	0.43757148
% 30	0.434385905
% 30.25	0.431221981
% 30.5	0.428081088
% 30.75	0.424963073
% 31	0.421867769
%     ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

 %simulation testCase
    for idx=1:size(rawData,1)
        rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
    end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);
    USDAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
% GIROPremiumToNormalVol From LIB
% rawDataSwaption = [0	1	2	3	4	5	7	10	12	15	20
% 1	0.0041	0.0045	0.0052	0.0061	0.0066	0.0071	0.0077	0.0080	0.0081	0.0082
% 2	0.0050	0.0057	0.0064	0.0066	0.0068	0.0071	0.0074	0.0077	0.0077	0.0078
% 3	0.0064	0.0069	0.0068	0.0068	0.0068	0.0072	0.0076	0.0078	0.0079	0.0079
% 4	0.0076	0.0071	0.0071	0.0070	0.0071	0.0072	0.0077	0.0081	0.0081	0.0081
% 5	0.0069	0.0070	0.0070	0.0071	0.0071	0.0070	0.0076	0.0083	0.0084	0.0083
% 7	0.0073	0.0074	0.0075	0.0072	0.0070	0.0075	0.0087	0.0096	0.0096	0.0095
% 10	0.0066	0.0066	0.0071	0.0074	0.0077	0.0092	0.0101	0.0102	0.0102	0.0102
% 15	0.0103	0.0105	0.0107	0.0109	0.0111	0.0108	0.0103	0.0102	0.0102	0.0101
% 20	0.0088	0.0089	0.0090	0.0091	0.0091	0.0091	0.0090	0.0089	0.0089	0.0089
%     ];
% 
rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.01025373	0.00996465	0.00939453	0.0088449	0.00838736	0.00835816	0.00827411	0.00844853	0.00844853
2	0.00987033	0.00941532	0.00896799	0.00850851	0.00803853	0.00809306	0.00814872	0.00832187	0.00832187
3	0.00965448	0.00917228	0.00868278	0.00815841	0.00766413	0.00768619	0.00766987	0.007827	0.007827
4	0.0086456	0.00835905	0.00803055	0.00773857	0.00743965	0.00747184	0.00750208	0.00760185	0.00760185
5	0.008024	0.00784518	0.00761802	0.00741163	0.00719132	0.00718529	0.00716174	0.0072174	0.0072174
7	0.00777954	0.0075868	0.00743466	0.00726955	0.00710966	0.00717731	0.00726665	0.00728989	0.00728989
10	0.00697384	0.00701303	0.00705259	0.00704171	0.0070555	0.00714878	0.00723997	0.00722954	0.00722954
15	0.00731508	0.00731858	0.00728587	0.00725674	0.0072209	0.00720153	0.00717821	0.0071639	0.0071639
    ];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'},...
                            {rawDataSwaption,valueDate,'USD','USDSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    USDAllMktData('swaptionVol') = swaptionVolMktData;
    
   %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    USDBlack = IRBlack(IRModel(Model(USDAllMktData,modelParams))); 
    
    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);
    
    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = USDBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= USDBlack.BlackATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = USDBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
    %% create a CK1FModel with initial modelParams
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;

%     matu = 15 + 1.0/365.0;
    joint_swap_tenor = 1;
    targetSize = 15;
    freq = 4;
    
    modelParamsUSD= containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'}, ...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
    USDCK1F = CK1F(IRModel(Model(USDAllMktData, modelParamsUSD)));
    
    tic
    outUSD = USDCK1F.CalibrateToATMDiagSwaptionNormal(USDBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        USDVol_r(idx2,idx3) = outUSD.vol_r.quote(idx3);
        USDRe(idx2,idx3) = outUSD.re(idx3);
    end
    USDRMSE(idx2) = outUSD.rmseTotal;
    
    toc
    %% USD model building end

    %% USDKRW fx model buidling start
    rawData = [0.083333333	0.0
0.25	0.0
0.5	0.0
1	0.0
2	0.0
3	0.0
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','USDKRWATMTermVol'});
    impliedTermVolMktData = MktData(mktData);
%     impliedTermVole = FXVolSurface(impliedTermVolMktData);
%     impliedVolSurface.params('moneyNessType') = 'deltaFwdMoneyNess';
    USDKRWAllMktData =containers.Map('impliedTermVol',impliedTermVolMktData);
%     AllMktData('impliedVolSurface') = impliedTermVolMktData;

    %% create  an FXBlackModel to calculate market PlainVanilla Options
%     modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});
%     USDKRWBlack = FXBlack(FXModel(Model(USDKRWAllMktData,modelParams))); 
    %% USDKRW fx model building end

    %% MultiAssetModel building start
    multiAssetMktData = containers.Map({'KRW','USD','USDKRW'}, ...
                            {KRWAllMktData, USDAllMktData, USDKRWAllMktData});

    % KRW,KTB                    
%     correlationMatrix = ones(2,2);
%     correlationMatrix(1,2) = 0.3;
%     correlationMatrix(2,1) = correlationMatrix(1,2);

    % KRW, USD & USDKRW
    correlationMatrix = ones(3, 3);
    correlationMatrix(1,2) = 0.845;
%     correlationMatrix(1,2) = 0.6;
    
%     correlationMatrix(1,2) = 0.7;
    
    correlationMatrix(1,3) = 0.0;
    correlationMatrix(2,3) = 0.0;
    correlationMatrix(2,1) = correlationMatrix(1,2);
    correlationMatrix(3,1) = correlationMatrix(1,3);
    correlationMatrix(3,2) = correlationMatrix(2,3);
    
    multiAssetMktData('correlationMatrix') = correlationMatrix;
    
    modelNameMap = containers.Map({'KRW','USD'},{KRWCK1FRisky, USDCK1F});
    modelParams = containers.Map({'modelNameMap'}, {modelNameMap});
    % correlationNames = {'KRW','USD'};
    modelParams('stochasticModelNames') = {'KRW','USD'};
    modelParams('correlationMatrix') = correlationMatrix;
    modelParams('refModelName') = 'KRW';
    % modelParams('useQuasiRandomYN') = 'true';
    modelParams('useQuasiRandomYN') = 'false';
    modelParams('fxVolCalibrated') = true;
    
    KRWCK1FRiskyUSDCK1F = MultiAssetModelCK1FCK1F(multiAssetMktData, modelParams);
    %% MultiAssetModel building end
   
    %% NumMethod Init Start
    numOfRandom = 40000;
    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                            {'AMC', 7, numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);
    %% NumMethod Init End
    
    % weekly step calculator
    % pricer = Pricer(rangeAccrual,KRWLGM,numMethodAMC);
    % pricingOut = pricer.computePrice(valueDate);
    
    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    % pricer = DailyWeeklyRiskyPricer_CDKTBSpreadRA(rangeAccrual,KRWLGM1FRiskyUSDLGM1F,numMethodAMC,false);
%     pricer = DailyWeeklyRiskyPricer_CDUSDLibor3MSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
    pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, true);    
     
    pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH, 'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH,'PSANo');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSANo');
%     pricingOutClosed = pricer.computePriceClosedSimple(valueDateH);
    
    callablePrice(idx2) = pricingOutMC1.callable.npv;
    nonCallPrice(idx2) = pricingOutMC1.nonCall.npv;
%     closedPrice(idx2) = pricingOutClosed.pricerInfo.nonCall.npv;
    
%     for idx3 = 1:length(pricingOutMC1.nonCall.cashflow_npv_e)
% %     for idx3 = 1:length(pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e)
%         nonCallPrice_cashflowNPV(idx2, idx3) = pricingOutMC1.nonCall.cashflow_npv_e(idx3);
% %         closedPrice_cashflowNPV(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
% %         
% %         closedPrice_fwdCD(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdCD(idx3);
% %         closedPrice_fwdKTB(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdKTB(idx3);
% %         closedPrice_fwdSpread(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSpread(idx3);
% %         closedPrice_fwdMu(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdMu(idx3);
% %         closedPrice_fwdSigma(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSigma(idx3);
% %                 
%     end
end

aaa =1.0;



In [ ]:

testCD_CDKTB3MSpread_CK1F_CK1F_20220620_20220926.m

clc
clear
valueDate = '2022-06-20';

maxIdx = 0;
zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;

callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% CD-USDLibor3M Spread Range Accrual Pricing start

%RA info
rangeAccrualParams.payCcy = 'KRW';
rangeAccrualParams.startDate = H_Date('2020-02-03');
rangeAccrualParams.maturity = 15;

valueDateH = H_Date(valueDate);

nominal = 10000;
coupon = 0.033;
couponFreq = 1;

% Long: KRW
% Short: USD
tenorL = 0.25;
tenorS = 0.25;

lowerS = 0.0;
upperS = 999;

% 
% lower = -1;
% upper = 0.05;

tenor = 0.25;

lower = -999;
upper = 999;

%resetDate startDate endDate paymentDate
couponScheduleInt = [20200203	20200203	20210203	20210203
20210203	20210203	20220203	20220203
20220203	20220203	20230203	20230203
20230203	20230203	20240205	20240205
20240205	20240205	20250203	20250203
20250203	20250203	20260203	20260203
20260203	20260203	20270203	20270203
20270203	20270203	20280203	20280203
20280203	20280203	20290205	20290205
20290205	20290205	20300206	20300206
20300206	20300206	20310203	20310203
20310203	20310203	20320203	20320203
20320203	20320203	20330203	20330203
20330203	20330203	20340203	20340203
20340203	20340203	20350205	20350205
];

endDate = H_Date(couponScheduleInt(size(couponScheduleInt,1),4));

%% CK1F model param matu setting start
% matu = 15 + 1.0/365.0;
matu = DateDiff(H_Date(couponScheduleInt(end,4)),H_Date(valueDate))/365.0;
% matu = matu -10.0;

%% end

 rangeDays = zeros(15,1);
%     rangeDays(1) = 1;

for i=1:length(rangeDays)
    if DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) >= 0
        rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) + 1;
    end
end


%resetDate startDate endDate paymentDate
callScheduleInt = [20200203	20200203	20200210	20200210
20210127	20210127	20210203	20210203
20220124	20220124	20220203	20220203
20230127	20230127	20230203	20230203
20240129	20240129	20240205	20240205
20250122	20250122	20250203	20250203
20260127	20260127	20260203	20260203
20270127	20270127	20270203	20270203
20280124	20280124	20280203	20280203
20290129	20290129	20290205	20290205
20300128	20300128	20300206	20300206
20310127	20310127	20310203	20310203
20320127	20320127	20320203	20320203
20330124	20330124	20330203	20330203
20340127	20340127	20340203	20340203
];

callYN = ones(15,1);
callYN(1) = 0;

manualScheduleYN = 1;

rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                'tenorS','lower','upper','rangeDays','callYN', ...
                                'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                        {nominal,coupon,couponFreq,tenorL,tenorS,lower,upper,...
                        rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});
% 
% tenorL = 0.25;
% tenorS = 0.25;
% 
% tenor = 0.25;
% 
% lowerS = 0.0;
% upperS = 999;
% 
% 
% lower = 0.0;
% upper = 0.05;

rangeAccrualInfo('lowerS') =lowerS;
rangeAccrualInfo('upperS') =upperS;
rangeAccrualInfo('tenor') = 0.25;

rangeAccrualParams.params = rangeAccrualInfo;

% rangeAccrual = SpreadRangeAccrual(rangeAccrualParams);
rangeAccrual = CDCMSSpreadDualRangeAccrual(rangeAccrualParams);

% maxIdx =2.5;

maxIdx = 5;

rmseSimul = zeros(100,1);

for idx2=1:11
    %% KRW Model building

    %% import and construct a zeroCurve
    rawData = [0.002739726	0.999952523
0.25	0.9950684
0.5	0.988031579
0.75	0.979080331
1	0.968776974
1.25	0.959202682
1.5	0.948993277
1.75	0.939322512
2	0.929369492
2.25	0.920482183
2.5	0.911621558
2.75	0.902762032
3	0.893905238
3.25	0.885611153
3.5	0.877424651
3.75	0.869320968
4	0.861299205
4.25	0.853502656
4.5	0.845805585
4.75	0.838202041
5	0.830690747
5.25	0.823531148
5.5	0.816487433
5.75	0.809552209
6	0.802723731
6.25	0.796000289
6.5	0.789380204
6.75	0.782861832
7	0.776443559
7.25	0.770025284
7.5	0.763695404
7.75	0.757455789
8	0.751305075
8.25	0.74524192
8.5	0.73926501
8.75	0.73337305
9	0.727564772
9.25	0.721838929
9.5	0.716194297
9.75	0.710629672
10	0.705143874
10.25	0.699194275
10.5	0.693278485
10.75	0.687420297
11	0.681619095
11.25	0.675874273
11.5	0.670185229
11.75	0.664551369
12	0.658972106
12.25	0.654130635
12.5	0.649391356
12.75	0.644722276
13	0.640122375
13.25	0.635590647
13.5	0.631126108
13.75	0.626727791
14	0.622394747
14.25	0.618126045
14.5	0.613920769
14.75	0.609778022
15	0.605696922
15.25	0.60235689
15.5	0.599139226
15.75	0.595995535
16	0.592924834
16.25	0.589926161
16.5	0.586998583
16.75	0.58414119
17	0.581353097
17.25	0.578633444
17.5	0.575981393
17.75	0.573396131
18	0.570876865
18.25	0.568422827
18.5	0.566033269
18.75	0.563707467
19	0.561444716
19.25	0.559244332
19.5	0.557105652
19.75	0.555028035
20	0.553010856
20.25	0.550157221
20.5	0.547261794
20.75	0.544409494
21	0.541599805
21.25	0.538832219
21.5	0.536106239
21.75	0.533421375
22	0.530777148
22.25	0.528173085
22.5	0.525608723
22.75	0.523083608
23	0.520597293
23.25	0.518149339
23.5	0.515739316
23.75	0.513366801
24	0.511031378
24.25	0.508732641
24.5	0.506470189
24.75	0.50424363
25	0.502052578
25.25	0.498564056
25.5	0.494988089
25.75	0.491434812
26	0.487904104
26.25	0.484395847
26.5	0.48090992
26.75	0.477446204
27	0.474004582
27.25	0.470584935
27.5	0.467187146
27.75	0.463811098
28	0.460456674
28.25	0.457123757
28.5	0.453812233
28.75	0.450521986
29	0.447252902
29.25	0.444004865
29.5	0.440777762
29.75	0.43757148
30	0.434385905
30.25	0.431221981
30.5	0.428081088
30.75	0.424963073
31	0.421867769
    ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

    
    %simulation testCase
    for idx=1:size(rawData,1)
        rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    KRWAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];

% rawData = [0.002739726	0.999952523
% 0.25	0.9950684
% 0.5	0.988031579
% 0.75	0.979080331
% 1	0.968776974
% 1.25	0.959202682
% 1.5	0.948993277
% 1.75	0.939322512
% 2	0.929369492
% 2.25	0.920482183
% 2.5	0.911621558
% 2.75	0.902762032
% 3	0.893905238
% 3.25	0.885611153
% 3.5	0.877424651
% 3.75	0.869320968
% 4	0.861299205
% 4.25	0.853502656
% 4.5	0.845805585
% 4.75	0.838202041
% 5	0.830690747
% 5.25	0.823531148
% 5.5	0.816487433
% 5.75	0.809552209
% 6	0.802723731
% 6.25	0.796000289
% 6.5	0.789380204
% 6.75	0.782861832
% 7	0.776443559
% 7.25	0.770025284
% 7.5	0.763695404
% 7.75	0.757455789
% 8	0.751305075
% 8.25	0.74524192
% 8.5	0.73926501
% 8.75	0.73337305
% 9	0.727564772
% 9.25	0.721838929
% 9.5	0.716194297
% 9.75	0.710629672
% 10	0.705143874
% 10.25	0.699194275
% 10.5	0.693278485
% 10.75	0.687420297
% 11	0.681619095
% 11.25	0.675874273
% 11.5	0.670185229
% 11.75	0.664551369
% 12	0.658972106
% 12.25	0.654130635
% 12.5	0.649391356
% 12.75	0.644722276
% 13	0.640122375
% 13.25	0.635590647
% 13.5	0.631126108
% 13.75	0.626727791
% 14	0.622394747
% 14.25	0.618126045
% 14.5	0.613920769
% 14.75	0.609778022
% 15	0.605696922
% 15.25	0.60235689
% 15.5	0.599139226
% 15.75	0.595995535
% 16	0.592924834
% 16.25	0.589926161
% 16.5	0.586998583
% 16.75	0.58414119
% 17	0.581353097
% 17.25	0.578633444
% 17.5	0.575981393
% 17.75	0.573396131
% 18	0.570876865
% 18.25	0.568422827
% 18.5	0.566033269
% 18.75	0.563707467
% 19	0.561444716
% 19.25	0.559244332
% 19.5	0.557105652
% 19.75	0.555028035
% 20	0.553010856
% 20.25	0.550157221
% 20.5	0.547261794
% 20.75	0.544409494
% 21	0.541599805
% 21.25	0.538832219
% 21.5	0.536106239
% 21.75	0.533421375
% 22	0.530777148
% 22.25	0.528173085
% 22.5	0.525608723
% 22.75	0.523083608
% 23	0.520597293
% 23.25	0.518149339
% 23.5	0.515739316
% 23.75	0.513366801
% 24	0.511031378
% 24.25	0.508732641
% 24.5	0.506470189
% 24.75	0.50424363
% 25	0.502052578
% 25.25	0.498564056
% 25.5	0.494988089
% 25.75	0.491434812
% 26	0.487904104
% 26.25	0.484395847
% 26.5	0.48090992
% 26.75	0.477446204
% 27	0.474004582
% 27.25	0.470584935
% 27.5	0.467187146
% 27.75	0.463811098
% 28	0.460456674
% 28.25	0.457123757
% 28.5	0.453812233
% 28.75	0.450521986
% 29	0.447252902
% 29.25	0.444004865
% 29.5	0.440777762
% 29.75	0.43757148
% 30	0.434385905
% 30.25	0.431221981
% 30.5	0.428081088
% 30.75	0.424963073
% 31	0.421867769
%     ];


for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

%  %simulation testCase
%     for idx=1:size(rawData,1)
%         rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
%     end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);

    KRWAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurfaceNormal



rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.01025373	0.00996465	0.00939453	0.0088449	0.00838736	0.00835816	0.00827411	0.00844853	0.00844853
2	0.00987033	0.00941532	0.00896799	0.00850851	0.00803853	0.00809306	0.00814872	0.00832187	0.00832187
3	0.00965448	0.00917228	0.00868278	0.00815841	0.00766413	0.00768619	0.00766987	0.007827	0.007827
4	0.0086456	0.00835905	0.00803055	0.00773857	0.00743965	0.00747184	0.00750208	0.00760185	0.00760185
5	0.008024	0.00784518	0.00761802	0.00741163	0.00719132	0.00718529	0.00716174	0.0072174	0.0072174
7	0.00777954	0.0075868	0.00743466	0.00726955	0.00710966	0.00717731	0.00726665	0.00728989	0.00728989
10	0.00697384	0.00701303	0.00705259	0.00704171	0.0070555	0.00714878	0.00723997	0.00722954	0.00722954
15	0.00731508	0.00731858	0.00728587	0.00725674	0.0072209	0.00720153	0.00717821	0.0071639	0.0071639
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    KRWAllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a KRWBlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(KRWAllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = KRWBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= KRWBlack.NormalATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
   %% create a CK1FRisky Model with initial modelParams
%     vol_r.tenor = [1;2;3;4;5;7;10;15;20];
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;
%     meanR_r = 0.0;
    
    
    joint_swap_tenor = 1;
%     targetSize = 10;

    targetSize = 15;
    
    freq = 4;
    
    modelParams = containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'},...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
    
%     import irModels.*;
%     import targetFunctions.*;
    
    KRWCK1FRisky = CK1FRisky(IRModelRisky(Model(KRWAllMktData, modelParams)));
    
    tic
%     out = KRWLGM2F.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
    % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
%     out = KRWCK1FRisky.CalibrateToATMDiagSwaption(KRWBlack,'global');
    out = KRWCK1FRisky.CalibrateToATMDiagSwaptionNormal(KRWBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        KRWVol_r(idx2,idx3) = out.vol_r.quote(idx3);
        KRWRe(idx2,idx3) = out.re(idx3);
    end
    KRWRMSE(idx2) = out.rmseTotal;
    
    toc
    %% CK1FRisky model building end
    %% KRW model building end

    %% USD model building start
    %% import and construct a zeroCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

%  %simulation testCase
%     for idx=1:size(rawData,1)
%         rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
%     end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','USDZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    USDAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];


% rawData = [0.002739726	0.999952523
% 0.25	0.9950684
% 0.5	0.988031579
% 0.75	0.979080331
% 1	0.968776974
% 1.25	0.959202682
% 1.5	0.948993277
% 1.75	0.939322512
% 2	0.929369492
% 2.25	0.920482183
% 2.5	0.911621558
% 2.75	0.902762032
% 3	0.893905238
% 3.25	0.885611153
% 3.5	0.877424651
% 3.75	0.869320968
% 4	0.861299205
% 4.25	0.853502656
% 4.5	0.845805585
% 4.75	0.838202041
% 5	0.830690747
% 5.25	0.823531148
% 5.5	0.816487433
% 5.75	0.809552209
% 6	0.802723731
% 6.25	0.796000289
% 6.5	0.789380204
% 6.75	0.782861832
% 7	0.776443559
% 7.25	0.770025284
% 7.5	0.763695404
% 7.75	0.757455789
% 8	0.751305075
% 8.25	0.74524192
% 8.5	0.73926501
% 8.75	0.73337305
% 9	0.727564772
% 9.25	0.721838929
% 9.5	0.716194297
% 9.75	0.710629672
% 10	0.705143874
% 10.25	0.699194275
% 10.5	0.693278485
% 10.75	0.687420297
% 11	0.681619095
% 11.25	0.675874273
% 11.5	0.670185229
% 11.75	0.664551369
% 12	0.658972106
% 12.25	0.654130635
% 12.5	0.649391356
% 12.75	0.644722276
% 13	0.640122375
% 13.25	0.635590647
% 13.5	0.631126108
% 13.75	0.626727791
% 14	0.622394747
% 14.25	0.618126045
% 14.5	0.613920769
% 14.75	0.609778022
% 15	0.605696922
% 15.25	0.60235689
% 15.5	0.599139226
% 15.75	0.595995535
% 16	0.592924834
% 16.25	0.589926161
% 16.5	0.586998583
% 16.75	0.58414119
% 17	0.581353097
% 17.25	0.578633444
% 17.5	0.575981393
% 17.75	0.573396131
% 18	0.570876865
% 18.25	0.568422827
% 18.5	0.566033269
% 18.75	0.563707467
% 19	0.561444716
% 19.25	0.559244332
% 19.5	0.557105652
% 19.75	0.555028035
% 20	0.553010856
% 20.25	0.550157221
% 20.5	0.547261794
% 20.75	0.544409494
% 21	0.541599805
% 21.25	0.538832219
% 21.5	0.536106239
% 21.75	0.533421375
% 22	0.530777148
% 22.25	0.528173085
% 22.5	0.525608723
% 22.75	0.523083608
% 23	0.520597293
% 23.25	0.518149339
% 23.5	0.515739316
% 23.75	0.513366801
% 24	0.511031378
% 24.25	0.508732641
% 24.5	0.506470189
% 24.75	0.50424363
% 25	0.502052578
% 25.25	0.498564056
% 25.5	0.494988089
% 25.75	0.491434812
% 26	0.487904104
% 26.25	0.484395847
% 26.5	0.48090992
% 26.75	0.477446204
% 27	0.474004582
% 27.25	0.470584935
% 27.5	0.467187146
% 27.75	0.463811098
% 28	0.460456674
% 28.25	0.457123757
% 28.5	0.453812233
% 28.75	0.450521986
% 29	0.447252902
% 29.25	0.444004865
% 29.5	0.440777762
% 29.75	0.43757148
% 30	0.434385905
% 30.25	0.431221981
% 30.5	0.428081088
% 30.75	0.424963073
% 31	0.421867769
%     ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

%  %simulation testCase
%     for idx=1:size(rawData,1)
%         rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
%     end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);
    USDAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
% GIROPremiumToNormalVol From LIB
% rawDataSwaption = [0	1	2	3	4	5	7	10	12	15	20
% 1	0.0041	0.0045	0.0052	0.0061	0.0066	0.0071	0.0077	0.0080	0.0081	0.0082
% 2	0.0050	0.0057	0.0064	0.0066	0.0068	0.0071	0.0074	0.0077	0.0077	0.0078
% 3	0.0064	0.0069	0.0068	0.0068	0.0068	0.0072	0.0076	0.0078	0.0079	0.0079
% 4	0.0076	0.0071	0.0071	0.0070	0.0071	0.0072	0.0077	0.0081	0.0081	0.0081
% 5	0.0069	0.0070	0.0070	0.0071	0.0071	0.0070	0.0076	0.0083	0.0084	0.0083
% 7	0.0073	0.0074	0.0075	0.0072	0.0070	0.0075	0.0087	0.0096	0.0096	0.0095
% 10	0.0066	0.0066	0.0071	0.0074	0.0077	0.0092	0.0101	0.0102	0.0102	0.0102
% 15	0.0103	0.0105	0.0107	0.0109	0.0111	0.0108	0.0103	0.0102	0.0102	0.0101
% 20	0.0088	0.0089	0.0090	0.0091	0.0091	0.0091	0.0090	0.0089	0.0089	0.0089
%     ];
% 
rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.01025373	0.00996465	0.00939453	0.0088449	0.00838736	0.00835816	0.00827411	0.00844853	0.00844853
2	0.00987033	0.00941532	0.00896799	0.00850851	0.00803853	0.00809306	0.00814872	0.00832187	0.00832187
3	0.00965448	0.00917228	0.00868278	0.00815841	0.00766413	0.00768619	0.00766987	0.007827	0.007827
4	0.0086456	0.00835905	0.00803055	0.00773857	0.00743965	0.00747184	0.00750208	0.00760185	0.00760185
5	0.008024	0.00784518	0.00761802	0.00741163	0.00719132	0.00718529	0.00716174	0.0072174	0.0072174
7	0.00777954	0.0075868	0.00743466	0.00726955	0.00710966	0.00717731	0.00726665	0.00728989	0.00728989
10	0.00697384	0.00701303	0.00705259	0.00704171	0.0070555	0.00714878	0.00723997	0.00722954	0.00722954
15	0.00731508	0.00731858	0.00728587	0.00725674	0.0072209	0.00720153	0.00717821	0.0071639	0.0071639
    ];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'},...
                            {rawDataSwaption,valueDate,'USD','USDSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    USDAllMktData('swaptionVol') = swaptionVolMktData;
    
   %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    USDBlack = IRBlack(IRModel(Model(USDAllMktData,modelParams))); 
    
    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);
    
    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = USDBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= USDBlack.BlackATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = USDBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
    %% create a CK1FModel with initial modelParams
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;

%     matu = 15 + 1.0/365.0;
    joint_swap_tenor = 1;
    targetSize = 15;
    freq = 4;
    
    modelParamsUSD= containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'}, ...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
    USDCK1F = CK1F(IRModel(Model(USDAllMktData, modelParamsUSD)));
    
    tic
    outUSD = USDCK1F.CalibrateToATMDiagSwaptionNormal(USDBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        USDVol_r(idx2,idx3) = outUSD.vol_r.quote(idx3);
        USDRe(idx2,idx3) = outUSD.re(idx3);
    end
    USDRMSE(idx2) = outUSD.rmseTotal;
    
    toc
    %% USD model building end

    %% USDKRW fx model buidling start
    rawData = [0.083333333	0.0
0.25	0.0
0.5	0.0
1	0.0
2	0.0
3	0.0
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','USDKRWATMTermVol'});
    impliedTermVolMktData = MktData(mktData);
%     impliedTermVole = FXVolSurface(impliedTermVolMktData);
%     impliedVolSurface.params('moneyNessType') = 'deltaFwdMoneyNess';
    USDKRWAllMktData =containers.Map('impliedTermVol',impliedTermVolMktData);
%     AllMktData('impliedVolSurface') = impliedTermVolMktData;

    %% create  an FXBlackModel to calculate market PlainVanilla Options
%     modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});
%     USDKRWBlack = FXBlack(FXModel(Model(USDKRWAllMktData,modelParams))); 
    %% USDKRW fx model building end

    %% MultiAssetModel building start
    multiAssetMktData = containers.Map({'KRW','USD','USDKRW'}, ...
                            {KRWAllMktData, USDAllMktData, USDKRWAllMktData});

    % KRW,KTB                    
%     correlationMatrix = ones(2,2);
%     correlationMatrix(1,2) = 0.3;
%     correlationMatrix(2,1) = correlationMatrix(1,2);

    % KRW, USD & USDKRW
    correlationMatrix = ones(3, 3);
    correlationMatrix(1,2) = 0.845;
%     correlationMatrix(1,2) = 0.6;
    
%     correlationMatrix(1,2) = 0.7;
    
    correlationMatrix(1,3) = 0.0;
    correlationMatrix(2,3) = 0.0;
    correlationMatrix(2,1) = correlationMatrix(1,2);
    correlationMatrix(3,1) = correlationMatrix(1,3);
    correlationMatrix(3,2) = correlationMatrix(2,3);
    
    multiAssetMktData('correlationMatrix') = correlationMatrix;
    
    modelNameMap = containers.Map({'KRW','USD'},{KRWCK1FRisky, USDCK1F});
    modelParams = containers.Map({'modelNameMap'}, {modelNameMap});
    % correlationNames = {'KRW','USD'};
    modelParams('stochasticModelNames') = {'KRW','USD'};
    modelParams('correlationMatrix') = correlationMatrix;
    modelParams('refModelName') = 'KRW';
    % modelParams('useQuasiRandomYN') = 'true';
    modelParams('useQuasiRandomYN') = 'false';
    modelParams('fxVolCalibrated') = true;
    
    KRWCK1FRiskyUSDCK1F = MultiAssetModelCK1FCK1F(multiAssetMktData, modelParams);
    %% MultiAssetModel building end
   
    %% NumMethod Init Start
    numOfRandom = 40000;
    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                            {'AMC', 7, numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);
    %% NumMethod Init End
    
    % weekly step calculator
    % pricer = Pricer(rangeAccrual,KRWLGM,numMethodAMC);
    % pricingOut = pricer.computePrice(valueDate);
    
    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    % pricer = DailyWeeklyRiskyPricer_CDKTBSpreadRA(rangeAccrual,KRWLGM1FRiskyUSDLGM1F,numMethodAMC,false);
%     pricer = DailyWeeklyRiskyPricer_CDUSDLibor3MSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
    pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, true);    
     
    pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH, 'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH,'PSANo');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSANo');
%     pricingOutClosed = pricer.computePriceClosedSimple(valueDateH);
    
    callablePrice(idx2) = pricingOutMC1.callable.npv;
    nonCallPrice(idx2) = pricingOutMC1.nonCall.npv;
%     closedPrice(idx2) = pricingOutClosed.pricerInfo.nonCall.npv;
    
%     for idx3 = 1:length(pricingOutMC1.nonCall.cashflow_npv_e)
% %     for idx3 = 1:length(pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e)
%         nonCallPrice_cashflowNPV(idx2, idx3) = pricingOutMC1.nonCall.cashflow_npv_e(idx3);
% %         closedPrice_cashflowNPV(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
% %         
% %         closedPrice_fwdCD(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdCD(idx3);
% %         closedPrice_fwdKTB(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdKTB(idx3);
% %         closedPrice_fwdSpread(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSpread(idx3);
% %         closedPrice_fwdMu(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdMu(idx3);
% %         closedPrice_fwdSigma(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSigma(idx3);
% %                 
%     end
end

aaa =1.0;


In [ ]:
testCD_CDKTB3MSpread_CK1F_CK1F_20220620.m

clc
clear
valueDate = '2022-06-20';

maxIdx = 0;
zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;

callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% CD-USDLibor3M Spread Range Accrual Pricing start

%RA info
rangeAccrualParams.payCcy = 'KRW';
rangeAccrualParams.startDate = H_Date('2020-02-03');
rangeAccrualParams.maturity = 15;

valueDateH = H_Date(valueDate);

nominal = 10000;
coupon = 0.033;
couponFreq = 1;

% Long: KRW
% Short: USD
tenorL = 0.25;
tenorS = 0.25;

lowerS = 0.0;
upperS = 999;

% 
% lower = -1;
% upper = 0.05;

tenor = 0.25;

lower = -999;
upper = 999;

%resetDate startDate endDate paymentDate
couponScheduleInt = [20200203	20200203	20210203	20210203
20210203	20210203	20220203	20220203
20220203	20220203	20230203	20230203
20230203	20230203	20240205	20240205
20240205	20240205	20250203	20250203
20250203	20250203	20260203	20260203
20260203	20260203	20270203	20270203
20270203	20270203	20280203	20280203
20280203	20280203	20290205	20290205
20290205	20290205	20300206	20300206
20300206	20300206	20310203	20310203
20310203	20310203	20320203	20320203
20320203	20320203	20330203	20330203
20330203	20330203	20340203	20340203
20340203	20340203	20350205	20350205
];

endDate = H_Date(couponScheduleInt(size(couponScheduleInt,1),4));

%% CK1F model param matu setting start
% matu = 15 + 1.0/365.0;
matu = DateDiff(H_Date(couponScheduleInt(end,4)),H_Date(valueDate))/365.0;
% matu = matu -10.0;

%% end

 rangeDays = zeros(15,1);
%     rangeDays(1) = 1;

for i=1:length(rangeDays)
    if DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) >= 0
        rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) + 1;
    end
end


%resetDate startDate endDate paymentDate
callScheduleInt = [20200203	20200203	20200210	20200210
20210127	20210127	20210203	20210203
20220124	20220124	20220203	20220203
20230127	20230127	20230203	20230203
20240129	20240129	20240205	20240205
20250122	20250122	20250203	20250203
20260127	20260127	20260203	20260203
20270127	20270127	20270203	20270203
20280124	20280124	20280203	20280203
20290129	20290129	20290205	20290205
20300128	20300128	20300206	20300206
20310127	20310127	20310203	20310203
20320127	20320127	20320203	20320203
20330124	20330124	20330203	20330203
20340127	20340127	20340203	20340203
];

callYN = ones(15,1);
callYN(1) = 0;

manualScheduleYN = 1;

rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                'tenorS','lower','upper','rangeDays','callYN', ...
                                'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                        {nominal,coupon,couponFreq,tenorL,tenorS,lower,upper,...
                        rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});
% 
% tenorL = 0.25;
% tenorS = 0.25;
% 
% tenor = 0.25;
% 
% lowerS = 0.0;
% upperS = 999;
% 
% 
% lower = 0.0;
% upper = 0.05;

rangeAccrualInfo('lowerS') =lowerS;
rangeAccrualInfo('upperS') =upperS;
rangeAccrualInfo('tenor') = 0.25;

rangeAccrualParams.params = rangeAccrualInfo;

% rangeAccrual = SpreadRangeAccrual(rangeAccrualParams);
rangeAccrual = CDCMSSpreadDualRangeAccrual(rangeAccrualParams);

% maxIdx =2.5;

maxIdx = 5;

rmseSimul = zeros(100,1);
for idx2=1:20
    %% KRW Model building

    %% import and construct a zeroCurve
    rawData = [0.002739726	0.999952523
0.25	0.9950684
0.5	0.988031579
0.75	0.979080331
1	0.968776974
1.25	0.959202682
1.5	0.948993277
1.75	0.939322512
2	0.929369492
2.25	0.920482183
2.5	0.911621558
2.75	0.902762032
3	0.893905238
3.25	0.885611153
3.5	0.877424651
3.75	0.869320968
4	0.861299205
4.25	0.853502656
4.5	0.845805585
4.75	0.838202041
5	0.830690747
5.25	0.823531148
5.5	0.816487433
5.75	0.809552209
6	0.802723731
6.25	0.796000289
6.5	0.789380204
6.75	0.782861832
7	0.776443559
7.25	0.770025284
7.5	0.763695404
7.75	0.757455789
8	0.751305075
8.25	0.74524192
8.5	0.73926501
8.75	0.73337305
9	0.727564772
9.25	0.721838929
9.5	0.716194297
9.75	0.710629672
10	0.705143874
10.25	0.699194275
10.5	0.693278485
10.75	0.687420297
11	0.681619095
11.25	0.675874273
11.5	0.670185229
11.75	0.664551369
12	0.658972106
12.25	0.654130635
12.5	0.649391356
12.75	0.644722276
13	0.640122375
13.25	0.635590647
13.5	0.631126108
13.75	0.626727791
14	0.622394747
14.25	0.618126045
14.5	0.613920769
14.75	0.609778022
15	0.605696922
15.25	0.60235689
15.5	0.599139226
15.75	0.595995535
16	0.592924834
16.25	0.589926161
16.5	0.586998583
16.75	0.58414119
17	0.581353097
17.25	0.578633444
17.5	0.575981393
17.75	0.573396131
18	0.570876865
18.25	0.568422827
18.5	0.566033269
18.75	0.563707467
19	0.561444716
19.25	0.559244332
19.5	0.557105652
19.75	0.555028035
20	0.553010856
20.25	0.550157221
20.5	0.547261794
20.75	0.544409494
21	0.541599805
21.25	0.538832219
21.5	0.536106239
21.75	0.533421375
22	0.530777148
22.25	0.528173085
22.5	0.525608723
22.75	0.523083608
23	0.520597293
23.25	0.518149339
23.5	0.515739316
23.75	0.513366801
24	0.511031378
24.25	0.508732641
24.5	0.506470189
24.75	0.50424363
25	0.502052578
25.25	0.498564056
25.5	0.494988089
25.75	0.491434812
26	0.487904104
26.25	0.484395847
26.5	0.48090992
26.75	0.477446204
27	0.474004582
27.25	0.470584935
27.5	0.467187146
27.75	0.463811098
28	0.460456674
28.25	0.457123757
28.5	0.453812233
28.75	0.450521986
29	0.447252902
29.25	0.444004865
29.5	0.440777762
29.75	0.43757148
30	0.434385905
30.25	0.431221981
30.5	0.428081088
30.75	0.424963073
31	0.421867769
    ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

    
    %simulation testCase
    for idx=1:size(rawData,1)
        rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    KRWAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];

% rawData = [0.002739726	0.999952523
% 0.25	0.9950684
% 0.5	0.988031579
% 0.75	0.979080331
% 1	0.968776974
% 1.25	0.959202682
% 1.5	0.948993277
% 1.75	0.939322512
% 2	0.929369492
% 2.25	0.920482183
% 2.5	0.911621558
% 2.75	0.902762032
% 3	0.893905238
% 3.25	0.885611153
% 3.5	0.877424651
% 3.75	0.869320968
% 4	0.861299205
% 4.25	0.853502656
% 4.5	0.845805585
% 4.75	0.838202041
% 5	0.830690747
% 5.25	0.823531148
% 5.5	0.816487433
% 5.75	0.809552209
% 6	0.802723731
% 6.25	0.796000289
% 6.5	0.789380204
% 6.75	0.782861832
% 7	0.776443559
% 7.25	0.770025284
% 7.5	0.763695404
% 7.75	0.757455789
% 8	0.751305075
% 8.25	0.74524192
% 8.5	0.73926501
% 8.75	0.73337305
% 9	0.727564772
% 9.25	0.721838929
% 9.5	0.716194297
% 9.75	0.710629672
% 10	0.705143874
% 10.25	0.699194275
% 10.5	0.693278485
% 10.75	0.687420297
% 11	0.681619095
% 11.25	0.675874273
% 11.5	0.670185229
% 11.75	0.664551369
% 12	0.658972106
% 12.25	0.654130635
% 12.5	0.649391356
% 12.75	0.644722276
% 13	0.640122375
% 13.25	0.635590647
% 13.5	0.631126108
% 13.75	0.626727791
% 14	0.622394747
% 14.25	0.618126045
% 14.5	0.613920769
% 14.75	0.609778022
% 15	0.605696922
% 15.25	0.60235689
% 15.5	0.599139226
% 15.75	0.595995535
% 16	0.592924834
% 16.25	0.589926161
% 16.5	0.586998583
% 16.75	0.58414119
% 17	0.581353097
% 17.25	0.578633444
% 17.5	0.575981393
% 17.75	0.573396131
% 18	0.570876865
% 18.25	0.568422827
% 18.5	0.566033269
% 18.75	0.563707467
% 19	0.561444716
% 19.25	0.559244332
% 19.5	0.557105652
% 19.75	0.555028035
% 20	0.553010856
% 20.25	0.550157221
% 20.5	0.547261794
% 20.75	0.544409494
% 21	0.541599805
% 21.25	0.538832219
% 21.5	0.536106239
% 21.75	0.533421375
% 22	0.530777148
% 22.25	0.528173085
% 22.5	0.525608723
% 22.75	0.523083608
% 23	0.520597293
% 23.25	0.518149339
% 23.5	0.515739316
% 23.75	0.513366801
% 24	0.511031378
% 24.25	0.508732641
% 24.5	0.506470189
% 24.75	0.50424363
% 25	0.502052578
% 25.25	0.498564056
% 25.5	0.494988089
% 25.75	0.491434812
% 26	0.487904104
% 26.25	0.484395847
% 26.5	0.48090992
% 26.75	0.477446204
% 27	0.474004582
% 27.25	0.470584935
% 27.5	0.467187146
% 27.75	0.463811098
% 28	0.460456674
% 28.25	0.457123757
% 28.5	0.453812233
% 28.75	0.450521986
% 29	0.447252902
% 29.25	0.444004865
% 29.5	0.440777762
% 29.75	0.43757148
% 30	0.434385905
% 30.25	0.431221981
% 30.5	0.428081088
% 30.75	0.424963073
% 31	0.421867769
%     ];


for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

 %simulation testCase
    for idx=1:size(rawData,1)
        rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
    end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);

    KRWAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurfaceNormal



rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.01025373	0.00996465	0.00939453	0.0088449	0.00838736	0.00835816	0.00827411	0.00844853	0.00844853
2	0.00987033	0.00941532	0.00896799	0.00850851	0.00803853	0.00809306	0.00814872	0.00832187	0.00832187
3	0.00965448	0.00917228	0.00868278	0.00815841	0.00766413	0.00768619	0.00766987	0.007827	0.007827
4	0.0086456	0.00835905	0.00803055	0.00773857	0.00743965	0.00747184	0.00750208	0.00760185	0.00760185
5	0.008024	0.00784518	0.00761802	0.00741163	0.00719132	0.00718529	0.00716174	0.0072174	0.0072174
7	0.00777954	0.0075868	0.00743466	0.00726955	0.00710966	0.00717731	0.00726665	0.00728989	0.00728989
10	0.00697384	0.00701303	0.00705259	0.00704171	0.0070555	0.00714878	0.00723997	0.00722954	0.00722954
15	0.00731508	0.00731858	0.00728587	0.00725674	0.0072209	0.00720153	0.00717821	0.0071639	0.0071639
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    KRWAllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a KRWBlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(KRWAllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = KRWBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= KRWBlack.NormalATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
   %% create a CK1FRisky Model with initial modelParams
%     vol_r.tenor = [1;2;3;4;5;7;10;15;20];
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;
%     meanR_r = 0.0;
    
    
    joint_swap_tenor = 1;
%     targetSize = 10;

    targetSize = 15;
    
    freq = 4;
    
    modelParams = containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'},...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
    
%     import irModels.*;
%     import targetFunctions.*;
    
    KRWCK1FRisky = CK1FRisky(IRModelRisky(Model(KRWAllMktData, modelParams)));
    
    tic
%     out = KRWLGM2F.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
    % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
%     out = KRWCK1FRisky.CalibrateToATMDiagSwaption(KRWBlack,'global');
    out = KRWCK1FRisky.CalibrateToATMDiagSwaptionNormal(KRWBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        KRWVol_r(idx2,idx3) = out.vol_r.quote(idx3);
        KRWRe(idx2,idx3) = out.re(idx3);
    end
    KRWRMSE(idx2) = out.rmseTotal;
    
    toc
    %% CK1FRisky model building end
    %% KRW model building end

    %% USD model building start
    %% import and construct a zeroCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

 %simulation testCase
    for idx=1:size(rawData,1)
        rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
    end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','USDZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    USDAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = [0.002739726	0.999952523
0.25	0.995423959
0.5	0.989371104
0.75	0.981362974
1	0.972438345
1.5	0.953405996
2	0.931163144
2.5	0.913843378
3	0.896610278
3.5	0.879110832
4	0.861571008
4.5	0.844012396
5	0.826450556
5.5	0.811101115
6	0.796108365
6.5	0.781441658
7	0.767093167
7.5	0.75305527
8	0.739320542
8.5	0.72588175
9	0.712731846
9.5	0.699863968
10	0.687271425
10.5	0.674808593
11	0.662597018
11.5	0.650634092
12	0.638914318
12.5	0.627432328
13	0.616182883
13.5	0.605160863
14	0.594361272
14.5	0.583779231
15	0.573409974
15.5	0.563248848
16	0.553291308
16.5	0.543532916
17	0.533969336
17.5	0.524596335
18	0.515409777
18.5	0.506405624
19	0.497579928
19.5	0.488928837
20	0.480448585
20.5	0.472587271
21	0.464916925
21.5	0.457412839
22	0.45007097
22.5	0.442887384
23	0.435858254
23.5	0.428979854
24	0.42224856
24.5	0.415660842
25	0.409213269
25.5	0.402902496
26	0.396725271
26.5	0.390678427
27	0.384758881
27.5	0.378963631
28	0.373289754
28.5	0.367734404
29	0.362294811
29.5	0.356968276
30	0.351752171
    ];


% rawData = [0.002739726	0.999952523
% 0.25	0.9950684
% 0.5	0.988031579
% 0.75	0.979080331
% 1	0.968776974
% 1.25	0.959202682
% 1.5	0.948993277
% 1.75	0.939322512
% 2	0.929369492
% 2.25	0.920482183
% 2.5	0.911621558
% 2.75	0.902762032
% 3	0.893905238
% 3.25	0.885611153
% 3.5	0.877424651
% 3.75	0.869320968
% 4	0.861299205
% 4.25	0.853502656
% 4.5	0.845805585
% 4.75	0.838202041
% 5	0.830690747
% 5.25	0.823531148
% 5.5	0.816487433
% 5.75	0.809552209
% 6	0.802723731
% 6.25	0.796000289
% 6.5	0.789380204
% 6.75	0.782861832
% 7	0.776443559
% 7.25	0.770025284
% 7.5	0.763695404
% 7.75	0.757455789
% 8	0.751305075
% 8.25	0.74524192
% 8.5	0.73926501
% 8.75	0.73337305
% 9	0.727564772
% 9.25	0.721838929
% 9.5	0.716194297
% 9.75	0.710629672
% 10	0.705143874
% 10.25	0.699194275
% 10.5	0.693278485
% 10.75	0.687420297
% 11	0.681619095
% 11.25	0.675874273
% 11.5	0.670185229
% 11.75	0.664551369
% 12	0.658972106
% 12.25	0.654130635
% 12.5	0.649391356
% 12.75	0.644722276
% 13	0.640122375
% 13.25	0.635590647
% 13.5	0.631126108
% 13.75	0.626727791
% 14	0.622394747
% 14.25	0.618126045
% 14.5	0.613920769
% 14.75	0.609778022
% 15	0.605696922
% 15.25	0.60235689
% 15.5	0.599139226
% 15.75	0.595995535
% 16	0.592924834
% 16.25	0.589926161
% 16.5	0.586998583
% 16.75	0.58414119
% 17	0.581353097
% 17.25	0.578633444
% 17.5	0.575981393
% 17.75	0.573396131
% 18	0.570876865
% 18.25	0.568422827
% 18.5	0.566033269
% 18.75	0.563707467
% 19	0.561444716
% 19.25	0.559244332
% 19.5	0.557105652
% 19.75	0.555028035
% 20	0.553010856
% 20.25	0.550157221
% 20.5	0.547261794
% 20.75	0.544409494
% 21	0.541599805
% 21.25	0.538832219
% 21.5	0.536106239
% 21.75	0.533421375
% 22	0.530777148
% 22.25	0.528173085
% 22.5	0.525608723
% 22.75	0.523083608
% 23	0.520597293
% 23.25	0.518149339
% 23.5	0.515739316
% 23.75	0.513366801
% 24	0.511031378
% 24.25	0.508732641
% 24.5	0.506470189
% 24.75	0.50424363
% 25	0.502052578
% 25.25	0.498564056
% 25.5	0.494988089
% 25.75	0.491434812
% 26	0.487904104
% 26.25	0.484395847
% 26.5	0.48090992
% 26.75	0.477446204
% 27	0.474004582
% 27.25	0.470584935
% 27.5	0.467187146
% 27.75	0.463811098
% 28	0.460456674
% 28.25	0.457123757
% 28.5	0.453812233
% 28.75	0.450521986
% 29	0.447252902
% 29.25	0.444004865
% 29.5	0.440777762
% 29.75	0.43757148
% 30	0.434385905
% 30.25	0.431221981
% 30.5	0.428081088
% 30.75	0.424963073
% 31	0.421867769
%     ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

 %simulation testCase
    for idx=1:size(rawData,1)
        rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
    end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);
    USDAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
% GIROPremiumToNormalVol From LIB
% rawDataSwaption = [0	1	2	3	4	5	7	10	12	15	20
% 1	0.0041	0.0045	0.0052	0.0061	0.0066	0.0071	0.0077	0.0080	0.0081	0.0082
% 2	0.0050	0.0057	0.0064	0.0066	0.0068	0.0071	0.0074	0.0077	0.0077	0.0078
% 3	0.0064	0.0069	0.0068	0.0068	0.0068	0.0072	0.0076	0.0078	0.0079	0.0079
% 4	0.0076	0.0071	0.0071	0.0070	0.0071	0.0072	0.0077	0.0081	0.0081	0.0081
% 5	0.0069	0.0070	0.0070	0.0071	0.0071	0.0070	0.0076	0.0083	0.0084	0.0083
% 7	0.0073	0.0074	0.0075	0.0072	0.0070	0.0075	0.0087	0.0096	0.0096	0.0095
% 10	0.0066	0.0066	0.0071	0.0074	0.0077	0.0092	0.0101	0.0102	0.0102	0.0102
% 15	0.0103	0.0105	0.0107	0.0109	0.0111	0.0108	0.0103	0.0102	0.0102	0.0101
% 20	0.0088	0.0089	0.0090	0.0091	0.0091	0.0091	0.0090	0.0089	0.0089	0.0089
%     ];
% 
rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.01025373	0.00996465	0.00939453	0.0088449	0.00838736	0.00835816	0.00827411	0.00844853	0.00844853
2	0.00987033	0.00941532	0.00896799	0.00850851	0.00803853	0.00809306	0.00814872	0.00832187	0.00832187
3	0.00965448	0.00917228	0.00868278	0.00815841	0.00766413	0.00768619	0.00766987	0.007827	0.007827
4	0.0086456	0.00835905	0.00803055	0.00773857	0.00743965	0.00747184	0.00750208	0.00760185	0.00760185
5	0.008024	0.00784518	0.00761802	0.00741163	0.00719132	0.00718529	0.00716174	0.0072174	0.0072174
7	0.00777954	0.0075868	0.00743466	0.00726955	0.00710966	0.00717731	0.00726665	0.00728989	0.00728989
10	0.00697384	0.00701303	0.00705259	0.00704171	0.0070555	0.00714878	0.00723997	0.00722954	0.00722954
15	0.00731508	0.00731858	0.00728587	0.00725674	0.0072209	0.00720153	0.00717821	0.0071639	0.0071639
    ];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'},...
                            {rawDataSwaption,valueDate,'USD','USDSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    USDAllMktData('swaptionVol') = swaptionVolMktData;
    
   %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    USDBlack = IRBlack(IRModel(Model(USDAllMktData,modelParams))); 
    
    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);
    
    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = USDBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= USDBlack.BlackATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = USDBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
    %% create a CK1FModel with initial modelParams
    vol_r.tenor = [1;2;3;4;5;7;10;15];
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;

%     matu = 15 + 1.0/365.0;
    joint_swap_tenor = 1;
    targetSize = 15;
    freq = 4;
    
    modelParamsUSD= containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'}, ...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
    USDCK1F = CK1F(IRModel(Model(USDAllMktData, modelParamsUSD)));
    
    tic
    outUSD = USDCK1F.CalibrateToATMDiagSwaptionNormal(USDBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        USDVol_r(idx2,idx3) = outUSD.vol_r.quote(idx3);
        USDRe(idx2,idx3) = outUSD.re(idx3);
    end
    USDRMSE(idx2) = outUSD.rmseTotal;
    
    toc
    %% USD model building end

    %% USDKRW fx model buidling start
    rawData = [0.083333333	0.0
0.25	0.0
0.5	0.0
1	0.0
2	0.0
3	0.0
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','USDKRWATMTermVol'});
    impliedTermVolMktData = MktData(mktData);
%     impliedTermVole = FXVolSurface(impliedTermVolMktData);
%     impliedVolSurface.params('moneyNessType') = 'deltaFwdMoneyNess';
    USDKRWAllMktData =containers.Map('impliedTermVol',impliedTermVolMktData);
%     AllMktData('impliedVolSurface') = impliedTermVolMktData;

    %% create  an FXBlackModel to calculate market PlainVanilla Options
%     modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});
%     USDKRWBlack = FXBlack(FXModel(Model(USDKRWAllMktData,modelParams))); 
    %% USDKRW fx model building end

    %% MultiAssetModel building start
    multiAssetMktData = containers.Map({'KRW','USD','USDKRW'}, ...
                            {KRWAllMktData, USDAllMktData, USDKRWAllMktData});

    % KRW,KTB                    
%     correlationMatrix = ones(2,2);
%     correlationMatrix(1,2) = 0.3;
%     correlationMatrix(2,1) = correlationMatrix(1,2);

    % KRW, USD & USDKRW
    correlationMatrix = ones(3, 3);
    correlationMatrix(1,2) = 0.845;
%     correlationMatrix(1,2) = 0.6;
    
%     correlationMatrix(1,2) = 0.7;
    
    correlationMatrix(1,3) = 0.0;
    correlationMatrix(2,3) = 0.0;
    correlationMatrix(2,1) = correlationMatrix(1,2);
    correlationMatrix(3,1) = correlationMatrix(1,3);
    correlationMatrix(3,2) = correlationMatrix(2,3);
    
    multiAssetMktData('correlationMatrix') = correlationMatrix;
    
    modelNameMap = containers.Map({'KRW','USD'},{KRWCK1FRisky, USDCK1F});
    modelParams = containers.Map({'modelNameMap'}, {modelNameMap});
    % correlationNames = {'KRW','USD'};
    modelParams('stochasticModelNames') = {'KRW','USD'};
    modelParams('correlationMatrix') = correlationMatrix;
    modelParams('refModelName') = 'KRW';
    % modelParams('useQuasiRandomYN') = 'true';
    modelParams('useQuasiRandomYN') = 'false';
    modelParams('fxVolCalibrated') = true;
    
    KRWCK1FRiskyUSDCK1F = MultiAssetModelCK1FCK1F(multiAssetMktData, modelParams);
    %% MultiAssetModel building end
   
    %% NumMethod Init Start
    numOfRandom = 40000;
    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                            {'AMC', 7, numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);
    %% NumMethod Init End
    
    % weekly step calculator
    % pricer = Pricer(rangeAccrual,KRWLGM,numMethodAMC);
    % pricingOut = pricer.computePrice(valueDate);
    
    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    % pricer = DailyWeeklyRiskyPricer_CDKTBSpreadRA(rangeAccrual,KRWLGM1FRiskyUSDLGM1F,numMethodAMC,false);
%     pricer = DailyWeeklyRiskyPricer_CDUSDLibor3MSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
    pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, true);    
     
    pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH, 'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH,'PSANo');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSA');
%     pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSANo');
%     pricingOutClosed = pricer.computePriceClosedSimple(valueDateH);
    
    callablePrice(idx2) = pricingOutMC1.callable.npv;
    nonCallPrice(idx2) = pricingOutMC1.nonCall.npv;
%     closedPrice(idx2) = pricingOutClosed.pricerInfo.nonCall.npv;
    
%     for idx3 = 1:length(pricingOutMC1.nonCall.cashflow_npv_e)
% %     for idx3 = 1:length(pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e)
%         nonCallPrice_cashflowNPV(idx2, idx3) = pricingOutMC1.nonCall.cashflow_npv_e(idx3);
% %         closedPrice_cashflowNPV(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
% %         
% %         closedPrice_fwdCD(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdCD(idx3);
% %         closedPrice_fwdKTB(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdKTB(idx3);
% %         closedPrice_fwdSpread(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSpread(idx3);
% %         closedPrice_fwdMu(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdMu(idx3);
% %         closedPrice_fwdSigma(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSigma(idx3);
% %                 
%     end
end

aaa =1.0;


In [ ]:
testCD_CDKTB3MSpread_CK1F_CK1F_20200317_PricingColumn.m

clc
clear
valueDate = '2020-03-17';

%% CD-USDLibor3M Spread Range Accrual Pricing start

rangeAccrualParams.payCcy = 'KRW';
rangeAccrualParams.startDate = H_Date('2019-08-22');
rangeAccrualParams.endDate   = H_Date('2039-08-22');


valueDateH = H_Date(valueDate);

nominal = 10000;

%% coupon info start
couponInfo = struct;
couponInfo.coupon = 0.03;
couponInfo.eventType = 'PeriodStart';

couponInfo.couponType = 'CDAndCDKTBSpreadDualRA';
% Long: KRW
% Short: KTB

couponInfo.tenorL = 0.25;
couponInfo.tenorS = 0.25;
couponInfo.tenor = 0.25;

couponInfo.lowerS = 0.0;
couponInfo.upperS = 999;

couponInfo.lower = 0.0;
couponInfo.upper = 0.05;

couponInfo.EODFlag_Cashflow = 'YES';
couponInfo.EODFlag_Exercise = 'YES';

%resetDate startDate endDate paymentDate
couponInfo.couponScheduleInt = [20190822	20190822	20200222	20200222
20200222	20200222	20200822	20200822
20200822	20200822	20210222	20210222
20210222	20210222	20210822	20210822
20210822	20210822	20220222	20220222
20220222	20220222	20220822	20220822
20220822	20220822	20230222	20230222
20230222	20230222	20230822	20230822
20230822	20230822	20240222	20240222
20240222	20240222	20240822	20240822
20240822	20240822	20250222	20250222
20250222	20250222	20250822	20250822
20250822	20250822	20260222	20260222
20260222	20260222	20260822	20260822
20260822	20260822	20270222	20270222
20270222	20270222	20270822	20270822
20270822	20270822	20280222	20280222
20280222	20280222	20280822	20280822
20280822	20280822	20290222	20290222
20290222	20290222	20290822	20290822
20290822	20290822	20300222	20300222
20300222	20300222	20300822	20300822
20300822	20300822	20310222	20310222
20310222	20310222	20310822	20310822
20310822	20310822	20320222	20320222
20320222	20320222	20320822	20320822
20320822	20320822	20330222	20330222
20330222	20330222	20330822	20330822
20330822	20330822	20340222	20340222
20340222	20340222	20340822	20340822
20340822	20340822	20350222	20350222
20350222	20350222	20350822	20350822
20350822	20350822	20360222	20360222
20360222	20360222	20360822	20360822
20360822	20360822	20370222	20370222
20370222	20370222	20370822	20370822
20370822	20370822	20380222	20380222
20380222	20380222	20380822	20380822
20380822	20380822	20390222	20390222
20390222	20390222	20390822	20390822
];

matu = DateDiff(H_Date(couponInfo.couponScheduleInt(end,4)),H_Date(valueDate))/365.0;

%% coupon info end

%% callable info start
callableInfo = struct;

%eventDate  paymentDate
callableInfo.isCalledFlag = false;

callableInfo.callScheduleInt = [20200222	20200222
20200822	20200822
20210222	20210222
20210822	20210822
20220222	20220222
20220822	20220822
20230222	20230222
20230822	20230822
20240222	20240222
20240822	20240822
20250222	20250222
20250822	20250822
20260222	20260222
20260822	20260822
20270222	20270222
20270822	20270822
20280222	20280222
20280822	20280822
20290222	20290222
20290822	20290822
20300222	20300222
20300822	20300822
20310222	20310222
20310822	20310822
20320222	20320222
20320822	20320822
20330222	20330222
20330822	20330822
20340222	20340222
20340822	20340822
20350222	20350222
20350822	20350822
20360222	20360222
20360822	20360822
20370222	20370222
20370822	20370822
20380222	20380222
20380822	20380822
20390222	20390222
];

%% callable info end

rangeAccrualInfo =  struct('nominal',nominal,'couponInfo',couponInfo,'callableInfo',callableInfo);
rangeAccrualInfo.manualScheduleYN = true;
rangeAccrualParams.params = rangeAccrualInfo;
rangeAccrual = CDAndCDKTBSpreadDualRA(rangeAccrualParams);


%% KRW Model building

%% import and construct a zeroCurve
    rawData = [0.0027397260	0.999965207
0.25	0.997437854
0.5	0.995315228
0.75	0.993228420
1	0.991297775
1.25	0.989191929
1.5	0.987109180
1.75	0.984747589
2	0.982295180
2.25	0.979944770
2.5	0.977566711
2.75	0.975159575
3	0.972723607
3.25	0.970071902
3.5	0.967357316
3.75	0.964586255
4	0.961759249
4.25	0.959142460
4.5	0.956506712
4.75	0.953846630
5	0.951162457
5.25	0.948795255
5.5	0.946442980
5.75	0.944098310
6	0.941761217
6.25	0.939431675
6.5	0.937109655
6.75	0.934795130
7	0.932488074
7.25	0.929973295
7.5	0.927447269
7.75	0.924914754
8	0.922375842
8.25	0.919830622
8.5	0.917279186
8.75	0.914721625
9	0.912158028
9.25	0.909588488
9.5	0.907013095
9.75	0.904431939
10	0.901845112
10.25	0.900331739
10.5	0.898898049
10.75	0.897507973
11	0.896161318
11.25	0.894857892
11.5	0.893597514
11.75	0.892380007
12	0.891205201
12.25	0.889975494
12.5	0.888779890
12.75	0.887622622
13	0.886503542
13.25	0.885422511
13.5	0.884379393
13.75	0.883374056
14	0.882406376
14.25	0.881476229
14.5	0.880583501
14.75	0.879728078
15	0.878909855
15.25	0.878195952
15.5	0.877525101
15.75	0.876893376
16	0.876300692
16.25	0.875746971
16.5	0.875232141
16.75	0.874756132
17	0.874318881
17.25	0.873920331
17.5	0.873560429
17.75	0.873239128
18	0.872956384
18.25	0.872712161
18.5	0.872506426
18.75	0.872339152
19	0.872210317
19.25	0.872119904
19.5	0.872067901
19.75	0.872054301
20	0.872079103
20.25	0.871027661
20.5	0.869894747
20.75	0.868771850
21	0.867658935
21.25	0.866555968
21.5	0.865462914
21.75	0.864379739
22	0.863306409
22.25	0.862242892
22.5	0.861189154
22.75	0.860145162
23	0.859110885
23.25	0.858086289
23.5	0.857071345
23.75	0.856066019
24	0.855070281
24.25	0.854084101
24.5	0.853107448
24.75	0.852140291
25	0.851182602
25.25	0.849988058
25.5	0.848771493
25.75	0.847559611
26	0.846352395
26.25	0.845149831
26.5	0.843951903
26.75	0.842758597
27	0.841569899
27.25	0.840385792
27.5	0.839206264
27.75	0.838031298
28	0.836860881
28.25	0.835694997
28.5	0.834533634
28.75	0.833376775
29	0.832224408
29.25	0.831076517
29.5	0.829933089
29.75	0.828794110
30	0.827659565
30.25	0.826528247
30.5	0.825398487
30.75	0.824270271
31	0.823143598
    ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    KRWAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = [0.0027397260	0.999965207
0.25	0.997708200
0.5	0.995484788
0.75	0.993334408
1	0.991236719
1.5	0.986095536
2	0.980450266
2.5	0.975326487
3	0.970118248
3.5	0.963762405
4	0.957013219
4.5	0.949894519
5	0.942415275
5.5	0.935560501
6	0.928536103
6.5	0.921336979
7	0.913968023
7.5	0.906434217
8	0.898740623
8.5	0.890892379
9	0.882894694
9.5	0.874752840
10	0.866472145
10.5	0.860118896
11	0.853832395
11.5	0.847579492
12	0.841360124
12.5	0.835174225
13	0.829021728
13.5	0.822902567
14	0.816816672
14.5	0.810763974
15	0.804744403
15.5	0.798757888
16	0.792804356
16.5	0.786883735
17	0.780995950
17.5	0.775140928
18	0.769318592
18.5	0.763528865
19	0.757771671
19.5	0.752046931
20	0.746354566
20.5	0.740953873
21	0.735604629
21.5	0.730296334
22	0.725028659
22.5	0.719801277
23	0.714613864
23.5	0.709466100
24	0.704357667
24.5	0.699288247
25	0.694257529
25.5	0.689265202
26	0.684310958
26.5	0.679394492
27	0.674515501
27.5	0.669673686
28	0.664868748
28.5	0.660100392
29	0.655368325
29.5	0.650672258
30	0.646011903
    ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);

    KRWAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurfaceNormal

% GIROPremiumToNormalVol From LIB
rawDataSwaption = [0	1.0000000000	2.000000	3.000000	4.000000	5.000000	7.000000	10.000000	12.000000	15	20
1	0.0048259352	0.005420	0.006029	0.006638	0.007250	0.007395	0.007599	0.007640	0.007694	0.007788
2	0.0053000318	0.005667	0.006024	0.006396	0.006753	0.006950	0.007252	0.007284	0.007341	0.007429
3	0.0055179685	0.005747	0.006000	0.006236	0.006475	0.006594	0.006777	0.006786	0.006802	0.006827
4	0.0057937750	0.005834	0.005880	0.005907	0.005938	0.005999	0.006090	0.006096	0.006104	0.006116
5	0.0061020337	0.005927	0.005761	0.005577	0.005401	0.005402	0.005400	0.005399	0.005401	0.005400
7	0.0061319629	0.005943	0.005783	0.005624	0.005467	0.005466	0.005464	0.005456	0.005436	0.005407
10	0.0060983785	0.005987	0.005836	0.005695	0.005561	0.005561	0.005563	0.005534	0.005492	0.005421
15	0.0060948645	0.005966	0.005823	0.005685	0.005553	0.005550	0.005549	0.005514	0.005462	0.005374
20	0.0060557415	0.005931	0.005810	0.005669	0.005531	0.005530	0.005527	0.005487	0.005426	0.005327
    ];

% % GIROPremiumToNormalVol From LIB & 1st order approximate KTB Swaption VOL
% 
% rawDataSwaption = [0	1	2	3	4	5	7	10	12	15	20
% 1	0.005789142	0.006173099	0.007003145	0.008104496	0.009226212	0.010020414	0.011170237	0.012113923	0.013752219	0.017068843
% 2	0.005734791	0.006488872	0.007394064	0.008244949	0.009092437	0.00979971	0.011328463	0.012294993	0.014145189	0.017045492
% 3	0.006621989	0.007426878	0.008121912	0.008792091	0.009299298	0.009824072	0.011415124	0.012408155	0.014392994	0.016555839
% 4	0.008041698	0.008390107	0.008755239	0.008873627	0.009035604	0.00967005	0.011150672	0.012242308	0.014402052	0.015763867
% 5	0.009119261	0.009164457	0.008897959	0.008689081	0.008519449	0.009392282	0.010743853	0.011953962	0.014333918	0.014729291
% 7	0.009455046	0.009330113	0.009238346	0.009631022	0.009985893	0.01110643	0.013270019	0.015183871	0.016217419	0.015963108
% 10	0.014188358	0.014849572	0.015037887	0.015430327	0.01595195	0.018601299	0.024617296	0.022536835	0.021054264	0.018623334
% 15	0.030688487	0.034170146	0.038608676	0.044712056	0.053610343	0.032425713	0.025701428	0.02258064	0.020144729	0.017530051
% 20	0.01731069	0.017025236	0.016872084	0.016686362	0.01651753	0.015740884	0.015295188	0.014905057	0.01426661	0.013573141
%     ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    KRWAllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a KRWBlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(KRWAllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = KRWBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= KRWBlack.NormalATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
   %% create a CK1FRisky Model with initial modelParams
    vol_r.tenor = [1;2;3;4;5;7;10;15;20];
    
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;
    
    
    joint_swap_tenor = 1;

    targetSize = 20;
    
    freq = 4;
    
    modelParams = containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'},...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});

    %% fixing info -> let it be the modelParams !                        
    fixingTicker = 'rateIndex_KRW_3M';

    tempDate = rangeAccrualParams.startDate;
    rateIndex1Fixing = containers.Map({StrDate(tempDate,'str')},{0.0125});
    for idxhong=1:8000
        tempDate = AddDate(tempDate,1,'day');
        rateIndex1Fixing(StrDate(tempDate,'str')) = 0.0125;

    end


    fixings = containers.Map({fixingTicker},{rateIndex1Fixing});

    modelParams('fixings') = fixings;

    
    KRWCK1FRisky = CK1FRisky(IRModelRisky(Model(KRWAllMktData, modelParams)));
    
    tic

    out = KRWCK1FRisky.CalibrateToATMDiagSwaptionNormal(KRWBlack,'global');
    toc
   
    %% CK1FRisky model building end
    %% KRW model building end

    %% KTB model building start
    %% import and construct a zeroCurve
    rawData = [0.0027397260	0.999965207
0.25	0.997708200
0.5	0.995484788
0.75	0.993334408
1	0.991236719
1.5	0.986095536
2	0.980450266
2.5	0.975326487
3	0.970118248
3.5	0.963762405
4	0.957013219
4.5	0.949894519
5	0.942415275
5.5	0.935560501
6	0.928536103
6.5	0.921336979
7	0.913968023
7.5	0.906434217
8	0.898740623
8.5	0.890892379
9	0.882894694
9.5	0.874752840
10	0.866472145
10.5	0.860118896
11	0.853832395
11.5	0.847579492
12	0.841360124
12.5	0.835174225
13	0.829021728
13.5	0.822902567
14	0.816816672
14.5	0.810763974
15	0.804744403
15.5	0.798757888
16	0.792804356
16.5	0.786883735
17	0.780995950
17.5	0.775140928
18	0.769318592
18.5	0.763528865
19	0.757771671
19.5	0.752046931
20	0.746354566
20.5	0.740953873
21	0.735604629
21.5	0.730296334
22	0.725028659
22.5	0.719801277
23	0.714613864
23.5	0.709466100
24	0.704357667
24.5	0.699288247
25	0.694257529
25.5	0.689265202
26	0.684310958
26.5	0.679394492
27	0.674515501
27.5	0.669673686
28	0.664868748
28.5	0.660100392
29	0.655368325
29.5	0.650672258
30	0.646011903
    ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','USDZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    USDAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = [0.0027397260	0.999965207
0.25	0.997708200
0.5	0.995484788
0.75	0.993334408
1	0.991236719
1.5	0.986095536
2	0.980450266
2.5	0.975326487
3	0.970118248
3.5	0.963762405
4	0.957013219
4.5	0.949894519
5	0.942415275
5.5	0.935560501
6	0.928536103
6.5	0.921336979
7	0.913968023
7.5	0.906434217
8	0.898740623
8.5	0.890892379
9	0.882894694
9.5	0.874752840
10	0.866472145
10.5	0.860118896
11	0.853832395
11.5	0.847579492
12	0.841360124
12.5	0.835174225
13	0.829021728
13.5	0.822902567
14	0.816816672
14.5	0.810763974
15	0.804744403
15.5	0.798757888
16	0.792804356
16.5	0.786883735
17	0.780995950
17.5	0.775140928
18	0.769318592
18.5	0.763528865
19	0.757771671
19.5	0.752046931
20	0.746354566
20.5	0.740953873
21	0.735604629
21.5	0.730296334
22	0.725028659
22.5	0.719801277
23	0.714613864
23.5	0.709466100
24	0.704357667
24.5	0.699288247
25	0.694257529
25.5	0.689265202
26	0.684310958
26.5	0.679394492
27	0.674515501
27.5	0.669673686
28	0.664868748
28.5	0.660100392
29	0.655368325
29.5	0.650672258
30	0.646011903
    ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);
    USDAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
% GIROPremiumToNormalVol From LIB
rawDataSwaption = [0	1.0000000000	2.000000	3.000000	4.000000	5.000000	7.000000	10.000000	12.000000	15	20
1	0.0048259352	0.005420	0.006029	0.006638	0.007250	0.007395	0.007599	0.007640	0.007694	0.007788
2	0.0053000318	0.005667	0.006024	0.006396	0.006753	0.006950	0.007252	0.007284	0.007341	0.007429
3	0.0055179685	0.005747	0.006000	0.006236	0.006475	0.006594	0.006777	0.006786	0.006802	0.006827
4	0.0057937750	0.005834	0.005880	0.005907	0.005938	0.005999	0.006090	0.006096	0.006104	0.006116
5	0.0061020337	0.005927	0.005761	0.005577	0.005401	0.005402	0.005400	0.005399	0.005401	0.005400
7	0.0061319629	0.005943	0.005783	0.005624	0.005467	0.005466	0.005464	0.005456	0.005436	0.005407
10	0.0060983785	0.005987	0.005836	0.005695	0.005561	0.005561	0.005563	0.005534	0.005492	0.005421
15	0.0060948645	0.005966	0.005823	0.005685	0.005553	0.005550	0.005549	0.005514	0.005462	0.005374
20	0.0060557415	0.005931	0.005810	0.005669	0.005531	0.005530	0.005527	0.005487	0.005426	0.005327
    ];
    
    mktData = containers.Map({'rawData','asOfDate','ccy','name'},...
                            {rawDataSwaption,valueDate,'USD','USDSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    USDAllMktData('swaptionVol') = swaptionVolMktData;
    
   %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    USDBlack = IRBlack(IRModel(Model(USDAllMktData,modelParams))); 
    
    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);
    
    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = USDBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= USDBlack.NormalATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = USDBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
    %% create a CK1FModel with initial modelParams
    vol_r.tenor = [1;2;3;4;5;7;10;15;20];
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;

    joint_swap_tenor = 1;
    targetSize = 20;
    freq = 4;
    
    modelParams= containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'}, ...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
    
                        %% fixing info -> let it be the modelParams !                        
    fixingTicker = 'rateIndex_USD_3M';

    tempDate = rangeAccrualParams.startDate;
    rateIndex1Fixing = containers.Map({StrDate(tempDate,'str')},{0.0125});
    for idxhong=1:8000
        tempDate = AddDate(tempDate,1,'day');
        rateIndex1Fixing(StrDate(tempDate,'str')) = 0.0125;

    end


    fixings = containers.Map({fixingTicker},{rateIndex1Fixing});

    modelParams('fixings') = fixings;
                        
    USDCK1F = CK1F(IRModel(Model(USDAllMktData, modelParams)));
    
    tic
    out = USDCK1F.CalibrateToATMDiagSwaptionNormal(USDBlack,'global');
    toc
    %% USD model building end

    %% USDKRW fx model buidling start
    rawData = [0.083333333	0.0
0.25	0.0
0.5	0.0
1	0.0
2	0.0
3	0.0
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','USDKRWATMTermVol'});
    impliedTermVolMktData = MktData(mktData);
%     impliedTermVole = FXVolSurface(impliedTermVolMktData);
%     impliedVolSurface.params('moneyNessType') = 'deltaFwdMoneyNess';
    USDKRWAllMktData =containers.Map('impliedTermVol',impliedTermVolMktData);
%     AllMktData('impliedVolSurface') = impliedTermVolMktData;

    %% create  an FXBlackModel to calculate market PlainVanilla Options
%     modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});
%     USDKRWBlack = FXBlack(FXModel(Model(USDKRWAllMktData,modelParams))); 
    %% USDKRW fx model building end

    %% MultiAssetModel building start
    multiAssetMktData = containers.Map({'KRW','USD','USDKRW'}, ...
                            {KRWAllMktData, USDAllMktData, USDKRWAllMktData});

    % KRW,KTB                    
%     correlationMatrix = ones(2,2);
%     correlationMatrix(1,2) = 0.3;
%     correlationMatrix(2,1) = correlationMatrix(1,2);

    % KRW, USD & USDKRW
    correlationMatrix = ones(3, 3);
    correlationMatrix(1,2) = 0.7;
    correlationMatrix(1,3) = 0.0;
    correlationMatrix(2,3) = 0.0;
    correlationMatrix(2,1) = correlationMatrix(1,2);
    correlationMatrix(3,1) = correlationMatrix(1,3);
    correlationMatrix(3,2) = correlationMatrix(2,3);
    
    multiAssetMktData('correlationMatrix') = correlationMatrix;
    
    modelNameMap = containers.Map({'KRW','USD'},{KRWCK1FRisky, USDCK1F});
    modelParams = containers.Map({'modelNameMap'}, {modelNameMap});
    % correlationNames = {'KRW','USD'};
    modelParams('stochasticModelNames') = {'KRW','USD'};
    modelParams('correlationMatrix') = correlationMatrix;
    modelParams('refModelName') = 'KRW';
    % modelParams('useQuasiRandomYN') = 'true';
    modelParams('useQuasiRandomYN') = 'false';
    modelParams('fxVolCalibrated') = true;
    
    KRWCK1FRiskyUSDCK1F = MultiAssetModelCK1FCK1F(multiAssetMktData, modelParams);
    %% MultiAssetModel building end
   
    %% NumMethod Init Start
    numOfRandom = 40000;
    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                            {'AMC', 7, numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);
    %% NumMethod Init End
    
    % weekly step calculator
    % pricer = Pricer(rangeAccrual,KRWLGM,numMethodAMC);
    % pricingOut = pricer.computePrice(valueDate);
    
    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
    % pricer = DailyWeeklyRiskyPricer_CDKTBSpreadRA(rangeAccrual,KRWLGM1FRiskyUSDLGM1F,numMethodAMC,false);
%     pricer = DailyWeeklyRiskyPricer_CDUSDLibor3MSpreadRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
    pricer = WeeklyRiskyPricer_CDAndCDKTBSpreadDualRA(rangeAccrual, KRWCK1FRiskyUSDCK1F, numMethodAMC, false);
     
%     pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH, 'PSANo');
    pricingOutMC1 = pricer.computePriceMCGenericPricingColumn(valueDateH,'PSA');
    
    pricingOutClosed = pricer.computePriceClosedSimple(valueDateH);
    
%     callablePrice(idx2) = pricingOutMC1.callable.npv;
%     nonCallPrice(idx2) = pricingOutMC1.nonCall.npv;
%     closedPrice(idx2) = pricingOutClosed.pricerInfo.nonCall.npv;
%     
%     for idx3 = 1:length(pricingOutMC1.nonCall.cashflow_npv_e)
%         nonCallPrice_cashflowNPV(idx2, idx3) = pricingOutMC1.nonCall.cashflow_npv_e(idx3);
%         closedPrice_cashflowNPV(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
%     end


aaa =1.0;


In [ ]:
import pandas as pd
import requests
from lxml import html
from tqdm import tqdm

In [ ]:
# 삼성전자
sample_code = '005930'

In [ ]:
# parsing URL
# 우리 컴퓨터 -> [접속] -> 에프앤가이드(Data Source) -> [크롤링/웹 스크래핑] -> 우리 컴퓨터
# client -> request -> [Server] -> response -> client

SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'


In [ ]:
# object -> 데이터 덩어리(추상화)

snap_url = SNAP_URL.format(sample_code)
snap_content = requests.get(snap_url).content # 문자열 binary
snap_tree = html.fromstring(snap_content) # 객체(object)
per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
per = float(per)

In [ ]:
per

8.15

In [ ]:
ratio_url = RATIO_URL.format(sample_code)
ratio_content = requests.get(ratio_url).content
ratio_tree = html.fromstring(ratio_content)
debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
debt_ratio = float(debt_ratio)

In [ ]:
# stockMkt => KOSPI
# kosdaqMkt => KOSDAQ
# konexMkt => KONEX

In [ ]:
def get_stock_list(market):
    market_code = ''
    if market == 'kospi':
        market_code = 'stockMkt'
    elif market == 'kosdaq':
        market_code = 'kosdaqMkt'
    elif market == 'konex':
        market_code = 'konexMkt'
    kind_url = 'https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&pageIndex=1&currentPageSize=3000&comAbbrv=&beginIndex=&orderMode=3&orderStat=D&isurCd=&repIsuSrtCd=&searchCodeType=&marketType={}&searchType=13&industry=&fiscalYearEnd=all&comAbbrvTmp=&location=all'.format(market_code)                                                                 

    return pd.read_html(kind_url, converters={'종목코드':lambda x: str(x)})[0]


In [ ]:
def converter(x):
    return str(x)

# 람다 함수 == 무명(anonymous) 함수 == 일회용 함수
lambda x: str(x)

<function __main__.<lambda>(x)>

In [ ]:
kospi_df = get_stock_list('kospi')
print(kospi_df.shape)

(829, 9)


In [ ]:
kosdaq_df = get_stock_list('kosdaq')
print(kosdaq_df.shape)

(1632, 9)


In [ ]:
# merge -> SQL Join
# append | kospi_df.append([kosdaq_df])
# concatenate(합치다)
stock_list_df = pd.concat([kospi_df, kosdaq_df] )

In [ ]:
print(stock_list_df.shape)

(2461, 9)


In [ ]:
# stock_list_df['종목코드'].dropna()
stock_list_df = stock_list_df[stock_list_df['종목코드'].notnull()]

In [ ]:
stock_list_df = stock_list_df[~stock_list_df['회사명'].str.contains('스팩|리츠')]
print(stock_list_df.shape)

(2364, 9)


In [ ]:
# list comprehension
stock_list_df.index = [x for x in range(len(stock_list_df))]

In [ ]:
stock_list_df.to_csv('kospi_kosdaq_stock_list.csv', encoding='utf-8', index=True)

In [ ]:
code_list = stock_list_df['종목코드']
code_list

0       100090
1       453340
2       452260
3       450140
4       377740
         ...  
2359    013030
2360    019550
2361    019570
2362    019590
2363    006920
Name: 종목코드, Length: 2364, dtype: object

In [ ]:
sample_df = pd.DataFrame(
    {'005930':['삼성전자', 1, 2], '035720':['카카오', 1, 2], '015720':['카카오', 1, 2], '025720':['카카오', 1, 2]}
).transpose()



In [ ]:
sample_df.columns = ['name', 'PER', 'Debt_ratio']

In [ ]:
sample_df

,name,PER,Debt_ratio
005930,삼성전자,1,2
035720,카카오,1,2
015720,카카오,1,2
025720,카카오,1,2


In [ ]:
def FinanceInfoCrawler(li, df):
    result_dict = {}
    error_codes = []
    
    for code in tqdm(li):
        try:
            # Parsing URL setting
            SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
            RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'

            # company name
            company_name = df[df['종목코드'] == code]['회사명'].values[0]

            # Get PER
            snap_url = SNAP_URL.format(code)
            snap_content = requests.get(snap_url).content
            snap_tree = html.fromstring(snap_content)
            per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
            per = float(per)

            # Get Debt ratio
            ratio_url = RATIO_URL.format(code)
            ratio_content = requests.get(ratio_url).content
            ratio_tree = html.fromstring(ratio_content)
            debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
            debt_ratio = float(debt_ratio)
            
            result_dict[code] = [company_name, per, debt_ratio]
            
        except (TypeError, IndexError, AttributeError, ValueError):
            pass
#             print(code)
            error_codes.append(code)
    
    # convert dict to DataFrame
    result_df = pd.DataFrame(result_dict)
    
    # transpose DataFrame
    result_df = result_df.transpose()
    
    # Setting column names
    result_df.columns = ['Name', 'PER', 'Debt_ratio']
    
    return result_df, error_codes

In [ ]:
crawling_result_df = FinanceInfoCrawler(code_list[:50],stock_list_df)

100%|██████████| 50/50 [00:46<00:00,  1.07it/s]


In [ ]:
print(crawling_result_df[0].shape)
crawling_result_df[0].head()

(39, 3)


,Name,PER,Debt_ratio
100090,SK오션플랜트,38.11,132.0
377740,바이오노트,1.89,9.8
446070,유니드비티플러스,104.18,11.2
108320,LX세미콘,7.5,35.7
126720,수산인더스트리,6.3,24.7


In [ ]:
# original data
# crawling_result_df

# copy data
copy_df = crawling_result_df[0].copy()

In [ ]:
# PER 10 이하
# 부채비율 50 이하
# (상위) 20개 종목

final_result_df = copy_df[
    (copy_df['PER'] <= 10)&(copy_df['Debt_ratio'] <= 50)&(copy_df['PER'] > 0)
].sort_values(
    by='PER', ascending=True
).iloc[:20]


In [ ]:
import datetime

# 시간까지 포함한 날짜
now = datetime.datetime.now()

final_result_df.to_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

In [ ]:
pd.read_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

,Unnamed: 0,Name,PER,Debt_ratio
0,377740,바이오노트,1.89,9.8
1,137310,에스디바이오센서,2.39,10.7
2,383800,LX홀딩스,3.90,1.5
3,353200,대덕전자,6.16,39.4
4,363280,티와이홀딩스,6.17,47.6
5,126720,수산인더스트리,6.30,24.7
6,108320,LX세미콘,7.50,35.7
